## Scispacy

Run:

```sh
    pip install scispacy
```

After that pick your models from https://allenai.github.io/scispacy/

Like this:
```sh
    pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz
```

Make sure you do above before you import spacy below, if you want to use scispacy, otherwise switch which model to use.

In [1]:
import scispacy
import spacy
import json
from spacy import displacy
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher
from spacy.util import filter_spans
from tqdm import tqdm

In [2]:
#nlp = spacy.load("en_core_sci_sm")   # The small scispacy model
nlp = spacy.load("en_core_sci_lg")
#nlp = spacy.load("en_core_sci_lg")  # Large scispacy model
#nlp = spacy.load("en_core_web_lg")  # Standard Spacy english model

In [3]:
#dict1 = set(filter(lambda x: len(x) > 0, 
#                   map(str.strip, open("data/manuscript/Supplemental_file1.txt", "r").readlines()
#            )
#        ))

In [4]:
#dict2 = set(filter(lambda x: len(x) > 0, 
#                   map(str.strip, open("data/manuscript/Supplemental_file2.txt", "r").readlines()
#                      )
#        ))

The data below has been derived from the corona repo as above, but to be able to run this jupyter notebook with minimal dependencies.

In [5]:
term_json = """["2019 wuhan seafood market pneumonia virus disorder", "2019new sarscov2 severe respiratory infection", "pneumonia from 2019 novel wuhan seafood market virus", "disorder due to wuhan pneumonia virus", "sars cov 2 2019 severe acute respiratory infection", "2019novel sars cov 2 acute severe disease", "2019 new wuhan coronavirus illness", "2019new severe acute respiratory syndromerelated coronavirus 2 pneumonia", "syndrome from 2019 wuhan seafood market pneumonia virus", "pneumonia caused by 2019new wuhan pneumonia virus", "2019 sars corona virus 2 acute respiratory syndrome", "2019 novel sars coronavirus 2 severe respiratory illness", "pneumonia caused by 2019 novel severe acute respiratory syndromerelated corona virus 2", "2019new sarscov2 acute pneumonia", "2019 novel sars corona virus2 respiratory disease", "chinese coronavirus 2019 acute severe infection", "wuhan seafood market virus 2019 severe acute respiratory illness", "new corona virus acute severe respiratory disease", "illness due to 2019 novel sars cov 2", "infection from 2019 novel chinese coronavirus", "disease from hcov 19", "new coronavirus acute pneumonia", "illness due to 2019 wuhan seafood market pneumonia virus", "2019 new wuhan seafood market virus severe disease", "2019 corona virus severe infection", "infection caused by sars corona virus2 2019", "2019 new sarscov2 respiratory illness", "illness caused by 2019new wuhan corona virus", "pneumonia from sars corona virus2", "2019 novel chinese coronavirus severe pneumonia", "syndrome caused by 2019 new wuhan pneumonia virus", "2019 wuhan corona virus severe acute infection", "2019 novel chinese coronavirus acute disease", "human coronavirus 2019 acute severe illness", "2019 ncov severe acute pneumonia", "infection caused by severe acute respiratory syndromerelated corona virus 2 2019", "2019novel wuhan coronavirus acute severe infection", "2019 sarscov2 acute severe respiratory disorder", "illness caused by chinese coronavirus 2019", "disease caused by sarscov2", "syndrome from 2019novel chinese corona virus", "sars coronavirus 2 acute respiratory disorder", "2019 wuhan seafood market pneumonia virus acute disorder", "syndrome caused by chinese coronavirus 2019", "2019 human corona virus acute respiratory disease", "illness caused by 2019novel seafood market corona virus", "2019novel sars corona virus2 severe acute disorder", "2019 new sars coronavirus 2 severe pneumonia", "illness from severe acute respiratory syndrome corona virus 2", "disease caused by 2019 novel seafood market coronavirus", "2019 seafood market virus severe acute illness", "infection caused by hcov19", "2019 novel wuhan seafood market virus acute severe pneumonia", "2019novel wuhan virus severe acute disease", "sars coronavirus2 syndrome", "pneumonia due to hcov 19", "2019 new wuhan coronavirus severe acute syndrome", "2019new sars cov 2 acute pneumonia", "2019 new seafood market corona virus severe acute pneumonia", "covid19 virus respiratory infection", "2019novel chinese coronavirus acute respiratory syndrome", "2019 new corona virus severe illness", "illness from sarscov2 2019", "2019new sarscov2 acute severe syndrome", "sars corona virus2 2019 severe respiratory syndrome", "disease due to 2019 novel sars coronavirus2", "wuhan corona virus respiratory infection", "2019 new wuhan seafood market pneumonia virus severe acute respiratory infection", "syndrome caused by new coronavirus 2019", "sars corona virus2 acute disorder", "2019 novel wuhancoronavirus disease", "2019 chinese corona virus acute pneumonia", "2019 novel seafood market corona virus severe acute respiratory syndrome", "disease due to 2019 corona virus", "2019novel wuhancoronavirus acute pneumonia", "2019 wuhan corona virus", "sars coronavirus2 severe acute respiratory syndrome", "infection from severe acute respiratory syndrome corona virus2 2019", "2019novel chinese coronavirus severe pneumonia", "2019 novel wuhan virus respiratory illness", "2019 novel wuhan coronavirus severe acute respiratory illness", "2019 new seafood market corona virus severe acute respiratory illness", "2019new seafood market coronavirus acute disorder", "disease due to severe acute respiratory syndromerelated coronavirus 2", "pneumonia due to 2019 new wuhan corona virus", "severe acute respiratory syndrome corona virus2 disease", "chinese coronavirus 2019 severe acute respiratory syndrome", "disease caused by 2019 new wuhan seafood market virus", "seafood market virus 2019 severe respiratory infection", "disorder caused by 2019 new severe acute respiratory syndrome corona virus 2", "disease due to human corona virus 2019", "infection from 2019new severe acute respiratory syndromerelated coronavirus 2", "2019 new seafood market coronavirus respiratory illness", "2019new wuhan coronavirus pneumonia", "2019novel chinese corona virus acute disorder", "2019novel wuhan seafood market pneumonia virus acute severe respiratory syndrome", "2019 new wuhan pneumonia virus acute severe respiratory disease", "pneumonia due to 2019 new wuhancorona virus", "pneumonia caused by 2019 new severe acute respiratory syndrome corona virus2", "2019 novel wuhan corona virus acute pneumonia", "2019novel chinese coronavirus respiratory infection", "virus responsible for covid19 severe acute respiratory syndrome", "2019 novel sars corona virus2 severe acute disease", "sars corona virus2 2019 acute infection", "2019novel sars corona virus 2 severe respiratory syndrome", "severe acute respiratory syndromerelated corona virus2 infection", "2019 novel wuhan seafood market virus severe acute disorder", "2019 new seafood market coronavirus acute severe syndrome", "2019new sars cov 2 severe acute respiratory infection", "sarscov2 severe respiratory disease", "2019 wuhan virus acute severe pneumonia", "2019new wuhan corona virus severe acute respiratory illness", "wuhancorona virus disorder", "wuhan corona virus acute respiratory infection", "2019new chinese corona virus respiratory disease", "2019novel wuhan pneumonia virus severe respiratory syndrome", "2019 wuhancorona virus", "2019new chinese coronavirus severe acute syndrome", "2019 novel sars cov 2 acute severe respiratory illness", "chinese coronavirus acute disease", "disorder from severe acute respiratory syndrome corona virus2 2019", "2019 novel wuhan seafood market pneumonia virus severe disease", "2019 seafood market coronavirus severe acute disease", "2019 human coronavirus severe acute syndrome", "disorder due to wuhan corona virus", "disorder due to seafood market corona virus 2019", "hcov19 severe disorder", "2019 seafood market corona virus acute respiratory syndrome", "2019novel sars cov 2 acute syndrome", "sarscov2 2019 acute pneumonia", "2019 novel sars coronavirus2 severe infection", "sars coronavirus 2 2019 acute disease", "2019new chinese corona virus illness", "infection from 2019new seafood market coronavirus", "2019 wuhan seafood market pneumonia virus acute respiratory disorder", "pneumonia due to 2019 novel severe acute respiratory syndromerelated corona virus2", "2019 seafood market coronavirus severe acute respiratory illness", "2019 novel chinese corona virus respiratory disorder", "wuhan virus severe pneumonia", "2019 human corona virus respiratory disease", "syndrome caused by 2019novel sarscov2", "disorder caused by sars coronavirus2 2019", "disorder from 2019 new wuhan corona virus", "2019novel wuhancoronavirus severe acute respiratory syndrome", "2019 new wuhancorona virus acute disease", "2019 novel chinese coronavirus acute severe respiratory disorder", "wuhan seafood market virus 2019 severe illness", "wuhan pneumonia virus 2019 acute severe syndrome", "2019new sars coronavirus 2 acute severe respiratory disease", "2019 sars coronavirus 2 acute respiratory disorder", "illness due to 2019novel severe acute respiratory syndromerelated coronavirus 2", "syndrome due to hcov 19", "syndrome from wuhan virus 2019", "2019 sars coronavirus2 disorder", "pneumonia due to 2019new severe acute respiratory syndrome coronavirus2", "2019new severe acute respiratory syndrome coronavirus2 disorder", "disorder caused by sars coronavirus2", "2019 severe acute respiratory syndrome corona virus 2 pneumonia", "2019 novel corona virus respiratory disease", "wuhancorona virus 2019 acute severe pneumonia", "novel coronavirus 2019 severe pneumonia", "sarscov2 severe acute disorder", "wuhan seafood market virus severe disorder", "wuhan seafood market pneumonia virus 2019 severe respiratory disorder", "disorder due to severe acute respiratory syndrome corona virus 2 2019", "illness from 2019 new severe acute respiratory syndromerelated corona virus 2", "novel corona virus respiratory illness", "2019 new wuhancoronavirus infection", "2019novel seafood market virus illness", "seafood market severe disease", "2019novel wuhan seafood market pneumonia virus acute severe infection", "seafood market virus 2019 illness", "illness due to 2019new wuhancorona virus", "2019 new chinese coronavirus respiratory infection", "2019new chinese coronavirus syndrome", "sars cov 2 acute severe disease", "new coronavirus 2019 acute respiratory disease", "infection due to 2019new seafood market virus", "wuhancoronavirus pneumonia", "2019 new wuhan seafood market virus acute respiratory syndrome", "wuhan seafood market virus severe acute illness", "2019 new wuhan pneumonia virus acute severe disease", "2019 seafood market virus severe acute respiratory illness", "pneumonia due to 2019 new sarscov2", "pneumonia from 2019 new severe acute respiratory syndrome coronavirus2", "2019new sars corona virus 2", "2019new chinese corona virus acute pneumonia", "2019novel wuhan corona virus respiratory disorder", "2019novel wuhan seafood market pneumonia virus disease", "human corona virus 2019 severe illness", "disease caused by 2019novel severe acute respiratory syndromerelated corona virus2", "2019 new seafood market corona virus acute severe illness", "2019new chinese coronavirus respiratory illness", "2019 new wuhan pneumonia virus", "hcov 19 disease", "seafood market virus severe respiratory illness", "2019novel seafood market coronavirus acute respiratory syndrome", "disorder caused by 2019 new severe acute respiratory syndromerelated corona virus 2", "2019novel severe acute respiratory syndrome corona virus2", "2019 corona virus severe respiratory infection", "2019new chinese corona virus severe respiratory illness", "illness caused by seafood market corona virus 2019", "disease from 2019novel chinese coronavirus", "illness from sars cov 2 2019", "2019 novel sarscov2 disorder", "pneumonia due to 2019 severe acute respiratory syndromerelated corona virus2", "seafood market corona virus 2019 acute respiratory illness", "2019 new sars corona virus 2 severe acute respiratory infection", "2019 new sars coronavirus2 respiratory infection", "2019new sarscov2 severe acute respiratory infection", "infection from 2019 new severe acute respiratory syndrome coronavirus2", "pneumonia caused by seafood market virus 2019", "novel corona virus 2019 severe illness", "2019 novel sars coronavirus 2 respiratory illness", "2019novel wuhancorona virus acute severe syndrome", "2019 new wuhan coronavirus severe respiratory disorder", "2019 wuhan virus acute severe infection", "wuhan coronavirus severe syndrome", "sars cov 2 severe acute pneumonia", "illness caused by severe acute respiratory syndromerelated corona virus 2 2019", "syndrome caused by 2019 sarscov2", "2019new chinese coronavirus acute infection", "disorder due to severe acute respiratory syndromerelated corona virus2", "2019 new seafood market corona virus disorder", "2019new wuhan virus severe respiratory disease", "2019new sars corona virus 2 acute respiratory infection", "2019 new corona virus acute disorder", "illness due to 2019 human coronavirus", "illness from sars cov 2", "seafood market corona virus respiratory disease", "2019new wuhan virus severe acute respiratory disorder", "wuhan coronavirus severe acute disorder", "disease caused by 2019novel chinese corona virus", "2019 novel severe acute respiratory syndrome coronavirus 2 illness", "2019 novel wuhan corona virus severe pneumonia", "2019new chinese coronavirus acute respiratory disorder", "sars coronavirus2 2019 acute infection", "2019novel wuhan seafood market pneumonia virus severe illness", "pneumonia due to sars coronavirus2", "2019 new sars cov 2 respiratory syndrome", "2019novel chinese corona virus severe illness", "2019 new wuhan seafood market pneumonia virus acute pneumonia", "new corona virus severe acute infection", "2019 new wuhancorona virus acute severe respiratory infection", "2019 new sars corona virus2 acute respiratory disease", "novel corona virus 2019 severe syndrome", "2019 wuhan seafood market pneumonia virus severe respiratory syndrome", "2019 wuhan pneumonia virus respiratory disease", "disease caused by 2019 novel chinese corona virus", "2019 seafood market coronavirus disorder", "pneumonia from 2019 severe acute respiratory syndrome coronavirus2", "2019new wuhan pneumonia virus acute severe respiratory syndrome", "2019 chinese coronavirus acute respiratory illness", "wuhan pneumonia virus illness", "2019 new wuhan coronavirus acute infection", "sarscov2 severe disorder", "2019 new wuhancorona virus acute severe illness", "hubei acute severe respiratory illness", "illness from 2019new wuhan virus", "2019new severe acute respiratory syndrome corona virus 2", "wuhan coronavirus respiratory infection", "2019new wuhan seafood market pneumonia virus severe acute respiratory syndrome", "2019 new wuhan pneumonia virus severe illness", "infection from 2019novel severe acute respiratory syndrome coronavirus2", "2019novel wuhan seafood market pneumonia virus pneumonia", "2019new severe acute respiratory syndrome corona virus2 pneumonia", "2019new sars cov 2 asevere acute respiratory disease", "disorder from wuhan coronavirus 2019", "hubei acute respiratory disease", "2019novel chinese coronavirus acute infection", "seafood market virus acute severe respiratory disease", "2019 novel wuhancoronavirus acute syndrome", "2019new wuhan seafood market virus severe respiratory infection", "virus responsible for covid19 asevere acute respiratory disease", "seafood market corona virus 2019 severe disorder", "2019 novel corona virus acute illness", "illness caused by 2019 novel wuhancorona virus", "disorder from sarscov2", "2019 novel wuhan pneumonia virus acute pneumonia", "disorder caused by 2019 novel seafood market corona virus", "wuhancorona virus acute severe respiratory infection", "2019 new sars coronavirus 2 acute severe infection", "wuhancorona virus acute severe illness", "sars corona virus2 severe acute pneumonia", "sars corona virus2 severe respiratory disease", "2019 chinese coronavirus disease", "2019 new chinese coronavirus severe acute disease", "pneumonia due to 2019new severe acute respiratory syndrome coronavirus 2", "hubei acute infection", "seafood market corona virus 2019 acute severe disease", "seafood market coronavirus 2019 respiratory infection", "sars corona virus2 acute severe respiratory infection", "covid19 virus acute respiratory disease", "new corona virus severe respiratory illness", "sars coronavirus2 asevere acute respiratory disease", "seafood market corona virus 2019", "2019new chinese coronavirus pneumonia", "2019 wuhancoronavirus severe acute syndrome", "2019new chinese coronavirus severe acute respiratory syndrome", "2019 new sars corona virus2 severe acute respiratory disorder", "hubei acute severe respiratory infection", "2019 new coronavirus acute severe respiratory disorder", "syndrome due to human corona virus 2019", "2019 novel sars coronavirus 2 respiratory disease", "infection caused by 2019novel severe acute respiratory syndrome coronavirus 2", "2019novel wuhancoronavirus infection", "infection caused by 2019 sars cov 2", "human corona virus 2019 infection", "infection caused by 2019 new seafood market corona virus", "2019 seafood market coronavirus acute respiratory infection", "syndrome from 2019new chinese corona virus", "2019 novel sars cov 2 severe disease", "pneumonia due to severe acute respiratory syndromerelated coronavirus2 2019", "chinese coronavirus acute infection", "2019 new wuhan pneumonia virus severe respiratory infection", "infection due to severe acute respiratory syndrome coronavirus2", "2019new sars cov 2 respiratory disease", "2019 novel sars coronavirus2 severe disorder", "pneumonia due to seafood market corona virus", "pneumonia from 2019 wuhan seafood market pneumonia virus", "disorder from covid19 virus", "2019 wuhan pneumonia virus severe respiratory disease", "2019 new wuhan coronavirus severe acute disorder", "2019 corona virus acute severe respiratory illness", "sarscov2 2019 acute severe respiratory disorder", "2019new wuhan seafood market pneumonia virus severe acute illness", "sars corona virus 2 2019 severe infection", "2019 ncov severe acute respiratory disorder", "seafood market virus severe respiratory disease", "2019novel wuhancoronavirus acute severe illness", "sars corona virus 2 acute severe respiratory infection", "infection caused by 2019 wuhan coronavirus", "syndrome due to sars corona virus2", "sars coronavirus2 respiratory syndrome", "new coronavirus pneumonia", "2019novel seafood market virus severe acute respiratory disorder", "2019 chinese corona virus syndrome", "2019 wuhancoronavirus severe acute pneumonia", "disorder caused by wuhan seafood market pneumonia virus", "2019new sars cov 2 severe infection", "sars corona virus2 severe infection", "disease due to 2019new severe acute respiratory syndromerelated corona virus2", "2019 new chinese coronavirus acute illness", "wuhancorona virus 2019 acute disorder", "wuhan corona virus asevere acute respiratory disease", "disorder due to human coronavirus 2019", "human coronavirus 2019 pneumonia", "2019 wuhancoronavirus acute illness", "infection caused by wuhan seafood market virus 2019", "disease caused by 2019novel sars coronavirus2", "2019new wuhan virus acute severe infection", "syndrome caused by 2019 new wuhan virus", "2019 seafood market coronavirus severe respiratory disease", "2019 human coronavirus severe disorder", "pneumonia caused by 2019 human coronavirus", "2019 novel seafood market virus severe respiratory syndrome", "2019ncov disorder", "wuhan seafood market pneumonia virus 2019 acute illness", "chinese corona virus 2019 acute respiratory disease", "sars coronavirus2 2019 severe acute pneumonia", "pneumonia from 2019 new wuhan pneumonia virus", "2019 new severe acute respiratory syndromerelated corona virus2", "hcov 19 severe illness", "2019 novel wuhan seafood market virus severe syndrome", "2019novel chinese coronavirus severe disease", "sarscov2 2019 acute respiratory illness", "2019novel wuhan virus severe acute syndrome", "wuhan seafood market pneumonia virus 2019 acute severe syndrome", "disorder caused by 2019novel severe acute respiratory syndromerelated coronavirus2", "seafood market severe acute disease", "seafood market severe infection", "seafood market corona virus 2019 pneumonia", "disorder due to sars corona virus2", "disease due to covid 19 virus", "2019 novel sars corona virus 2 respiratory illness", "wuhancorona virus acute illness", "chinese corona virus severe acute respiratory infection", "seafood market virus respiratory infection", "wuhan seafood market pneumonia virus acute severe syndrome", "disorder from 2019 severe acute respiratory syndrome coronavirus2", "disorder due to hcov19", "2019 new sars corona virus 2 acute disease", "2019new wuhan coronavirus severe infection", "seafood market corona virus severe disorder", "2019 novel seafood market coronavirus severe respiratory disorder", "virus responsible for covid19 illness", "2019new seafood market coronavirus pneumonia", "seafood market corona virus", "wuhan seafood market pneumonia virus illness", "illness caused by seafood market virus 2019", "syndrome from 2019new sars corona virus 2", "disease from severe acute respiratory syndromerelated coronavirus2", "wuhancoronavirus", "sars corona virus2 2019 respiratory disease", "2019 novel seafood market virus acute severe respiratory illness", "2019novel sars coronavirus 2 acute severe illness", "new corona virus 2019 severe acute disease", "2019 novel wuhan seafood market pneumonia virus respiratory illness", "syndrome from 2019new sars coronavirus2", "wuhan corona virus severe disease", "2019 new wuhan virus severe respiratory disorder", "2019 novel sars corona virus 2 severe acute disease", "disorder due to 2019 novel corona virus", "2019novel wuhan seafood market pneumonia virus respiratory disorder", "hcov 19 acute respiratory disorder", "2019 novel sars cov 2 syndrome", "virus causing covid19 severe respiratory disease", "pneumonia due to seafood market coronavirus 2019", "sars coronavirus2 acute infection", "disorder caused by 2019 novel severe acute respiratory syndrome coronavirus2", "2019novel wuhan virus severe acute respiratory disorder", "2019 chinese corona virus acute respiratory syndrome", "seafood market virus 2019 acute severe disease", "disease from 2019 novel wuhan pneumonia virus", "wuhancoronavirus infection", "illness from 2019 new severe acute respiratory syndromerelated corona virus2", "2019novel wuhan virus respiratory infection", "2019 sars corona virus2 acute severe respiratory disease", "pneumonia caused by 2019novel severe acute respiratory syndrome coronavirus2", "2019novel sarscov2 severe disease", "hubei severe syndrome", "seafood market corona virus severe acute infection", "2019 seafood market virus acute severe syndrome", "2019 novel wuhancorona virus severe respiratory infection", "2019 novel wuhancoronavirus acute respiratory disorder", "syndrome due to 2019new wuhan seafood market pneumonia virus", "2019 new sarscov2 acute severe pneumonia", "2019novel chinese coronavirus severe acute respiratory disorder", "2019novel sars corona virus2 acute respiratory infection", "2019new wuhan virus pneumonia", "2019novel wuhancoronavirus acute respiratory disease", "seafood market virus 2019 severe acute pneumonia", "illness from wuhan pneumonia virus", "seafood market corona virus infection", "syndrome from seafood market corona virus", "2019novel wuhan pneumonia virus severe acute disorder", "2019 ncov acute severe pneumonia", "2019 corona virus acute respiratory disorder", "syndrome caused by 2019new wuhan seafood market pneumonia virus", "disease from 2019novel sars corona virus2", "2019new wuhan pneumonia virus disorder", "2019novel sars corona virus 2 infection", "seafood market coronavirus acute pneumonia", "seafood market corona virus 2019 respiratory syndrome", "chinese corona virus 2019 acute severe disease", "disease caused by 2019 novel severe acute respiratory syndromerelated corona virus2", "2019 new wuhan corona virus severe respiratory illness", "2019novel sars coronavirus 2 severe acute pneumonia", "infection caused by 2019 new corona virus", "syndrome due to 2019 new wuhan coronavirus", "2019novel sars corona virus2 respiratory infection", "wuhan seafood market pneumonia virus 2019 acute severe disorder", "illness due to 2019new wuhan virus", "2019novel sars coronavirus 2 acute severe pneumonia", "2019 new seafood market coronavirus respiratory syndrome", "illness due to 2019 sars corona virus 2", "2019 new sars corona virus2 acute severe respiratory illness", "2019 novel wuhan coronavirus severe acute pneumonia", "2019 wuhan pneumonia virus disease", "disorder from 2019novel severe acute respiratory syndrome corona virus2", "disorder from sars corona virus 2", "chinese corona virus 2019 severe acute illness", "illness due to 2019 new seafood market virus", "2019new sars cov 2 acute severe respiratory infection", "pneumonia caused by novel coronavirus 2019", "2019new chinese corona virus severe respiratory disease", "syndrome from 2019 novel wuhan coronavirus", "disease caused by 2019 novel sars coronavirus2", "disease from new coronavirus", "2019 wuhan coronavirus disorder", "wuhancorona virus 2019 acute severe disorder", "disease from 2019 novel wuhancoronavirus", "human coronavirus 2019 severe illness", "2019 new coronavirus acute infection", "2019 novel wuhan seafood market virus disorder", "2019new wuhancoronavirus acute illness", "infection caused by 2019 new seafood market virus", "wuhan pneumonia virus 2019 severe respiratory syndrome", "2019new sars coronavirus2 acute severe respiratory infection", "disease due to 2019novel severe acute respiratory syndrome corona virus 2", "2019 wuhan corona virus syndrome", "human corona virus 2019 severe respiratory syndrome", "disorder due to new corona virus", "disease caused by 2019 severe acute respiratory syndromerelated coronavirus2", "new coronavirus 2019 syndrome", "sars corona virus2 acute respiratory disorder", "2019novel sars coronavirus 2 acute severe respiratory infection", "2019 novel wuhancoronavirus infection", "2019 new wuhan virus respiratory disease", "2019 novel seafood market coronavirus severe syndrome", "covid 19 virus acute severe syndrome", "2019 wuhancorona virus acute disease", "2019novel wuhan seafood market virus acute disease", "wuhan seafood market pneumonia virus 2019 acute syndrome", "disorder due to 2019novel seafood market virus", "illness due to 2019novel severe acute respiratory syndrome corona virus2", "infection from 2019novel wuhan seafood market virus", "sars coronavirus2 acute severe syndrome", "2019 novel wuhan coronavirus acute severe pneumonia", "2019new seafood market virus acute severe respiratory disease", "2019 novel seafood market corona virus acute pneumonia", "2019new wuhan pneumonia virus pneumonia", "wuhancorona virus acute respiratory infection", "infection caused by sarscov2", "wuhan acute severe pneumonia", "covid 19 virus acute severe infection", "pneumonia caused by 2019 novel seafood market corona virus", "syndrome from 2019novel sars coronavirus2", "hcov19 asevere acute respiratory disease", "2019 new sars coronavirus2 severe acute respiratory syndrome", "2019 new sarscov2 severe acute disorder", "pneumonia caused by 2019new wuhan seafood market pneumonia virus", "2019 sars coronavirus2 acute severe respiratory disease", "2019 chinese coronavirus severe respiratory syndrome", "2019novel wuhancoronavirus acute disease", "2019new wuhan pneumonia virus acute respiratory syndrome", "pneumonia caused by virus causing covid19", "sars cov 2 acute respiratory disorder", "severe acute respiratory syndromerelated corona virus2 disease", "2019 novel chinese corona virus acute respiratory infection", "2019 ncov severe respiratory illness", "pneumonia due to 2019 novel chinese coronavirus", "infection from 2019new chinese corona virus", "2019 new corona virus acute pneumonia", "sars corona virus 2 2019 acute syndrome", "2019 new seafood market coronavirus severe acute disorder", "disease caused by severe acute respiratory syndromerelated corona virus 2", "pneumonia caused by 2019new severe acute respiratory syndrome coronavirus2", "seafood market corona virus 2019 severe acute disorder", "illness caused by 2019 new chinese coronavirus", "2019novel wuhancoronavirus severe infection", "disorder due to 2019novel severe acute respiratory syndrome coronavirus 2", "2019 human coronavirus severe respiratory disorder", "syndrome due to 2019 novel wuhancorona virus", "2019 seafood market coronavirus respiratory disorder", "illness from 2019novel wuhan seafood market pneumonia virus", "2019new sars cov 2 pneumonia", "2019 novel sars coronavirus 2 severe syndrome", "2019new seafood market corona virus acute severe pneumonia", "illness due to chinese corona virus", "2019new severe acute respiratory syndromerelated corona virus 2 disorder", "2019 new wuhan virus acute severe syndrome", "2019 new sars cov 2 acute severe respiratory illness", "2019 chinese corona virus acute severe respiratory syndrome", "illness due to 2019 new severe acute respiratory syndromerelated coronavirus2", "disease due to 2019 novel wuhan coronavirus", "illness due to wuhan seafood market virus 2019", "virus responsible for covid19 respiratory illness", "2019 wuhancorona virus acute severe respiratory illness", "2019new sars coronavirus2 disorder", "2019new sars coronavirus 2 acute severe pneumonia", "wuhan severe acute respiratory disorder", "syndrome due to 2019new seafood market corona virus", "syndrome due to 2019 sars corona virus2", "pneumonia from virus causing covid19", "2019 new wuhan pneumonia virus acute respiratory disorder", "2019novel wuhancorona virus acute respiratory disorder", "infection due to 2019 new severe acute respiratory syndrome corona virus2", "2019new chinese corona virus acute severe respiratory illness", "wuhan pneumonia virus 2019 illness", "chinese corona virus acute respiratory disease", "2019 novel sarscov2 severe respiratory disease", "2019 human coronavirus severe infection", "2019new seafood market virus acute syndrome", "disease caused by 2019novel seafood market coronavirus", "human corona virus 2019 severe acute infection", "syndrome caused by novel corona virus 2019", "wuhan coronavirus asevere acute respiratory disease", "2019new seafood market virus syndrome", "2019 wuhan corona virus illness", "novel corona virus acute severe respiratory disorder", "disorder caused by new corona virus 2019", "disease due to 2019novel severe acute respiratory syndrome coronavirus2", "sars coronavirus2 acute severe infection", "disorder caused by hcov19", "2019new wuhan virus severe acute respiratory infection", "2019new severe acute respiratory syndrome coronavirus 2", "2019 novel sars cov 2", "2019 novel severe acute respiratory syndromerelated corona virus 2 pneumonia", "disorder from 2019 severe acute respiratory syndrome coronavirus 2", "2019 novel sarscov2 pneumonia", "2019 new sars coronavirus 2 asevere acute respiratory disease", "2019 wuhan pneumonia virus acute disease", "disorder caused by 2019 wuhancorona virus", "disease from wuhan coronavirus", "2019 novel sars corona virus 2 severe acute respiratory infection", "syndrome from 2019novel sarscov2", "disease caused by 2019 new sars corona virus2", "illness due to 2019 sars coronavirus 2", "2019 novel severe acute respiratory syndromerelated corona virus2 infection", "infection from 2019new wuhancorona virus", "2019 new wuhan seafood market virus respiratory infection", "2019 new seafood market coronavirus severe acute infection", "wuhan coronavirus severe acute illness", "hcov 19 respiratory syndrome", "seafood market coronavirus 2019 respiratory disease", "wuhan virus 2019 syndrome", "2019 new chinese corona virus severe acute syndrome", "2019 new sars cov 2 respiratory disease", "wuhancorona virus respiratory disease", "pneumonia due to 2019novel severe acute respiratory syndromerelated corona virus2", "seafood market virus 2019 acute severe respiratory disease", "illness from 2019 human corona virus", "new coronavirus 2019 severe infection", "2019 novel wuhan corona virus", "2019novel wuhan virus acute respiratory illness", "pneumonia caused by 2019 wuhancorona virus", "2019novel wuhan seafood market pneumonia virus respiratory syndrome", "illness caused by 2019new severe acute respiratory syndromerelated corona virus2", "2019novel wuhan coronavirus", "illness due to 2019 new severe acute respiratory syndrome coronavirus2", "pneumonia from 2019 new coronavirus", "2019ncov respiratory disorder", "2019novel wuhan seafood market virus severe infection", "pneumonia caused by novel corona virus 2019", "chinese coronavirus severe acute respiratory infection", "sarscov2 acute severe illness", "2019new wuhan corona virus acute disorder", "2019new wuhan pneumonia virus severe pneumonia", "2019 novel coronavirus severe acute syndrome", "wuhan seafood market pneumonia virus 2019 acute respiratory syndrome", "human coronavirus 2019 severe acute infection", "2019 seafood market coronavirus severe acute infection", "wuhan pneumonia virus 2019 acute respiratory illness", "2019 new wuhancoronavirus respiratory infection", "disorder from 2019novel sarscov2", "2019 coronavirus severe syndrome", "2019ncov severe respiratory disorder", "2019 novel wuhan coronavirus severe infection", "2019 wuhan seafood market pneumonia virus severe disorder", "2019 novel chinese corona virus respiratory infection", "disease caused by 2019 severe acute respiratory syndromerelated corona virus2", "hcov 19 acute severe infection", "2019novel wuhan coronavirus severe acute respiratory infection", "disorder caused by 2019novel wuhan corona virus", "disorder caused by 2019 new sars corona virus2", "2019 sarscov2 asevere acute respiratory disease", "syndrome due to 2019 novel coronavirus", "sars corona virus2 respiratory disorder", "2019 novel wuhan pneumonia virus severe disorder", "illness due to severe acute respiratory syndromerelated coronavirus2", "2019novel sars corona virus2 acute disorder", "pneumonia caused by 2019novel seafood market corona virus", "2019 corona virus acute respiratory illness", "2019 novel wuhan pneumonia virus syndrome", "disease due to 2019 seafood market corona virus", "2019novel severe acute respiratory syndrome corona virus 2 disorder", "2019 novel sarscov2 acute respiratory illness", "2019 wuhan pneumonia virus severe acute respiratory disorder", "2019 human corona virus severe disease", "infection caused by 2019new wuhancoronavirus", "2019 sars coronavirus2 acute severe respiratory disorder", "2019 novel wuhan seafood market pneumonia virus acute severe illness", "2019 new seafood market coronavirus acute severe infection", "seafood market virus respiratory disease", "2019 sars corona virus 2 respiratory illness", "2019novel sarscov2 acute severe respiratory infection", "2019new wuhan coronavirus severe acute disease", "2019new chinese corona virus severe infection", "infection due to sars coronavirus 2 2019", "disorder from 2019new wuhan corona virus", "2019novel wuhan coronavirus severe respiratory disease", "sars 2", "illness from 2019 novel severe acute respiratory syndromerelated coronavirus2", "2019novel seafood market corona virus acute respiratory illness", "disorder from 2019 new chinese coronavirus", "wuhan virus acute severe infection", "disease caused by 2019novel severe acute respiratory syndromerelated coronavirus2", "infection due to 2019new sars cov 2", "2019 new seafood market corona virus acute severe respiratory disorder", "chinese corona virus acute severe respiratory disorder", "human coronavirus 2019 respiratory disease", "2019 novel wuhancorona virus acute disease", "syndrome due to 2019novel sars corona virus 2", "infection due to 2019novel severe acute respiratory syndrome corona virus2", "2019 novel wuhan seafood market pneumonia virus severe respiratory infection", "2019 sars corona virus2 severe acute respiratory infection", "sars coronavirus 2", "2019novel seafood market virus asevere acute respiratory disease", "2019 new sarscov2 acute disease", "2019new wuhan corona virus disease", "2019 new chinese coronavirus severe respiratory syndrome", "infection from 2019 sars cov 2", "pneumonia caused by chinese corona virus", "sars corona virus 2 severe respiratory illness", "pneumonia due to 2019novel sarscov2", "chinese corona virus 2019 acute severe respiratory infection", "2019new wuhan seafood market virus syndrome", "2019 human corona virus acute respiratory illness", "illness caused by 2019 severe acute respiratory syndrome corona virus2", "2019ncov", "new coronavirus severe acute disease", "2019novel seafood market virus severe acute infection", "pneumonia caused by sars corona virus2 2019", "covid19 virus severe respiratory illness", "2019 new sars coronavirus2 asevere acute respiratory disease", "wuhan seafood market pneumonia virus acute respiratory disease", "disease caused by 2019 wuhan corona virus", "2019new chinese coronavirus severe disease", "pneumonia from wuhan coronavirus", "2019novel chinese corona virus infection", "2019 new wuhan coronavirus severe disease", "2019 novel wuhan coronavirus severe acute infection", "wuhan pneumonia virus severe acute syndrome", "human corona virus 2019 respiratory disease", "disorder caused by 2019novel severe acute respiratory syndrome coronavirus2", "new coronavirus acute severe syndrome", "pneumonia caused by 2019 severe acute respiratory syndrome corona virus 2", "severe acute respiratory syndrome corona virus2 2019 disease", "illness from sars coronavirus2 2019", "2019 seafood market corona virus severe disorder", "infection from chinese corona virus 2019", "2019 novel wuhan pneumonia virus respiratory disease", "2019 new chinese corona virus acute respiratory disorder", "pneumonia due to wuhancoronavirus", "illness from 2019new severe acute respiratory syndrome coronavirus2", "2019 new sars coronavirus 2 acute respiratory infection", "2019 novel wuhan virus", "2019 wuhan corona virus acute illness", "2019novel wuhancorona virus acute illness", "2019 new sars corona virus 2 acute respiratory disorder", "human coronavirus 2019 severe disorder", "2019new wuhan virus disease", "2019 new chinese coronavirus acute respiratory disorder", "severe acute respiratory syndromerelated coronavirus 2 2019 disorder", "2019novel sars cov 2 acute respiratory illness", "2019 new corona virus acute severe infection", "2019 new wuhancoronavirus pneumonia", "disorder caused by 2019 coronavirus", "wuhan corona virus acute severe illness", "disorder due to 2019 new corona virus", "illness from 2019 novel wuhan virus", "2019 seafood market corona virus severe disease", "2019new sars corona virus 2 respiratory disorder", "seafood market virus 2019 infection", "sars corona virus 2 severe syndrome", "pneumonia caused by 2019 severe acute respiratory syndromerelated corona virus2", "infection from 2019 novel seafood market corona virus", "disease due to 2019 new sarscov2", "2019 novel wuhan seafood market virus pneumonia", "illness due to sars coronavirus 2 2019", "disorder from 2019new severe acute respiratory syndrome corona virus2", "2019 wuhan coronavirus severe respiratory infection", "sarscov2 2019 acute severe respiratory syndrome", "chinese coronavirus severe illness", "2019 novel wuhan seafood market pneumonia virus acute severe disorder", "new coronavirus 2019 acute infection", "2019novel wuhan coronavirus acute severe respiratory illness", "2019 wuhancoronavirus acute syndrome", "2019 wuhan virus acute severe respiratory disease", "sars cov 2 severe acute infection", "2019 human corona virus severe acute respiratory disorder", "wuhan seafood market virus 2019 infection", "chinese coronavirus 2019 respiratory illness", "2019new wuhan virus respiratory illness", "illness due to 2019new severe acute respiratory syndrome corona virus2", "2019 sars corona virus2 severe acute illness", "pneumonia due to 2019 novel severe acute respiratory syndrome corona virus2", "sars cov 2 2019 severe acute syndrome", "pneumonia caused by hcov19", "2019 novel sars coronavirus 2 acute severe respiratory syndrome", "2019 novel corona virus respiratory infection", "seafood market coronavirus 2019 severe disorder", "wuhan virus acute severe respiratory disease", "infection due to 2019 novel wuhancorona virus", "2019 novel chinese coronavirus acute severe respiratory infection", "disease caused by 2019new wuhan seafood market pneumonia virus", "sars corona virus 2 2019 acute respiratory illness", "hubei acute severe respiratory disease", "2019 novel severe acute respiratory syndromerelated corona virus 2", "2019 novel sars coronavirus 2 acute disease", "infection from 2019 new severe acute respiratory syndrome corona virus 2", "2019new sars coronavirus2 severe respiratory syndrome", "sars coronavirus2 2019 severe acute respiratory infection", "2019 human corona virus disease", "2019 new coronavirus severe pneumonia", "2019 new corona virus acute infection", "novel corona virus pneumonia", "chinese coronavirus 2019 severe respiratory illness", "2019new wuhancorona virus severe acute illness", "infection due to 2019 new wuhan seafood market pneumonia virus", "2019 seafood market virus acute syndrome", "disease from 2019 sars coronavirus2", "disorder caused by 2019 severe acute respiratory syndrome coronavirus2", "2019novel wuhancorona virus acute disorder", "infection due to 2019 new chinese coronavirus", "pneumonia from 2019novel severe acute respiratory syndromerelated coronavirus2", "2019 wuhan seafood market pneumonia virus severe respiratory infection", "2019novel wuhan pneumonia virus severe acute respiratory infection", "chinese coronavirus 2019 acute illness", "2019 new wuhancoronavirus acute respiratory infection", "illness due to 2019new severe acute respiratory syndromerelated coronavirus2", "wuhan seafood market virus severe acute respiratory disorder", "illness caused by 2019 novel seafood market corona virus", "covid 19 virus respiratory infection", "2019new seafood market coronavirus respiratory infection", "2019 ncov severe syndrome", "2019 new wuhan pneumonia virus acute severe pneumonia", "2019novel chinese coronavirus syndrome", "hcov19 severe respiratory illness", "seafood market coronavirus 2019 severe acute syndrome", "2019 novel wuhan coronavirus respiratory syndrome", "infection due to 2019new severe acute respiratory syndromerelated coronavirus 2", "disease caused by 2019 human coronavirus", "2019 sars cov 2 acute severe respiratory disorder", "disease caused by seafood market virus", "2019 sars cov 2 respiratory disorder", "virus causing covid19 severe respiratory syndrome", "2019new wuhan corona virus acute illness", "2019 wuhan seafood market pneumonia virus severe acute disease", "wuhan virus 2019 severe acute disorder", "illness from 2019 novel wuhan seafood market pneumonia virus", "illness caused by 2019novel sars corona virus 2", "2019 wuhan corona virus acute respiratory illness", "virus causing covid19 acute severe respiratory infection", "2019new chinese corona virus severe acute respiratory syndrome", "2019 new wuhan virus severe acute illness", "2019 human coronavirus acute disease", "2019 novel corona virus acute severe infection", "2019 novel corona virus respiratory disorder", "pneumonia caused by 2019novel wuhan seafood market pneumonia virus", "illness caused by 2019 new sars coronavirus 2", "DiseasesDB:60833", "2019new severe acute respiratory syndrome coronavirus 2 disorder", "2019 new sars cov 2 severe infection", "new coronavirus 2019 acute severe illness", "2019new seafood market coronavirus disorder", "2019 sars corona virus2 acute severe disorder", "2019 severe acute respiratory syndrome coronavirus2 infection", "new coronavirus respiratory infection", "2019 novel seafood market coronavirus acute severe infection", "2019 wuhancoronavirus severe disorder", "covid19 virus acute severe pneumonia", "disease caused by 2019 new wuhan pneumonia virus", "disease caused by severe acute respiratory syndrome corona virus2 2019", "pneumonia due to 2019new severe acute respiratory syndromerelated coronavirus 2", "illness due to 2019new seafood market corona virus", "disorder due to 2019new seafood market corona virus", "sars corona virus2 2019 acute severe respiratory disorder", "2019new chinese coronavirus severe acute respiratory infection", "2019novel wuhancoronavirus severe respiratory infection", "2019new sarscov2 acute severe respiratory illness", "pneumonia due to 2019 new sars cov 2", "illness from new corona virus 2019", "2019new sars coronavirus2 severe syndrome", "wuhan coronavirus 2019 severe acute respiratory illness", "syndrome due to virus causing covid19", "new coronavirus 2019 acute respiratory disorder", "2019new wuhancoronavirus severe acute syndrome", "illness due to severe acute respiratory syndromerelated corona virus 2", "pneumonia due to 2019 novel severe acute respiratory syndromerelated corona virus 2", "2019 novel seafood market virus", "2019 new seafood market corona virus", "pneumonia from 2019 novel severe acute respiratory syndrome corona virus2", "sars corona virus 2 acute respiratory illness", "2019new wuhan virus severe respiratory infection", "sars corona virus2 disease", "2019 seafood market corona virus severe syndrome", "hcov 19 severe acute respiratory illness", "disorder caused by 2019novel sars corona virus2", "2019new wuhan pneumonia virus severe respiratory infection", "illness caused by 2019 wuhan pneumonia virus", "hubei severe acute illness", "2019novel wuhancoronavirus severe pneumonia", "pneumonia from sars corona virus2 2019", "sarscov2 2019 respiratory disease", "2019 sars corona virus2 severe acute respiratory syndrome", "infection due to 2019new sars corona virus2", "2019 wuhan coronavirus acute severe respiratory infection", "pneumonia from 2019 coronavirus", "disorder caused by 2019 novel wuhancoronavirus", "2019 new wuhan corona virus acute severe pneumonia", "2019 novel sars coronavirus2 respiratory disease", "2019 wuhan pneumonia virus acute severe infection", "seafood market virus 2019 acute respiratory disorder", "novel corona virus 2019 severe infection", "severe acute respiratory syndromerelated corona virus 2 2019 disorder", "2019 novel coronavirus severe respiratory disease", "disorder caused by covid 19 virus", "seafood market virus 2019 disease", "2019 novel wuhan seafood market virus acute severe illness", "2019 novel sars coronavirus2 respiratory infection", "chinese corona virus acute illness", "2019 new wuhan virus acute severe respiratory disease", "2019 severe acute respiratory syndromerelated corona virus 2 disease", "2019novel sars coronavirus 2 acute severe infection", "disorder due to chinese corona virus 2019", "2019new wuhan pneumonia virus respiratory illness", "2019 wuhan pneumonia virus severe acute infection", "severe acute respiratory syndromerelated coronavirus2 2019 disease", "2019new sarscov2 acute illness", "2019 human coronavirus pneumonia", "sars coronavirus 2 severe disease", "infection due to covid 19 virus", "pneumonia from 2019 new severe acute respiratory syndromerelated corona virus2", "2019new wuhan corona virus acute severe respiratory disorder", "syndrome due to 2019 novel sars coronavirus2", "pneumonia from 2019 chinese corona virus", "2019new wuhancorona virus severe acute infection", "2019novel wuhan virus acute pneumonia", "2019 new chinese corona virus acute respiratory syndrome", "2019novel sars corona virus2 acute severe pneumonia", "infection due to 2019new wuhan coronavirus", "2019 novel sars corona virus 2 severe acute respiratory illness", "infection caused by chinese coronavirus 2019", "2019new severe acute respiratory syndromerelated coronavirus 2 disease", "2019 novel corona virus acute disorder", "infection caused by 2019new severe acute respiratory syndromerelated corona virus2", "wuhancorona virus acute severe respiratory disorder", "disease from 2019novel wuhan pneumonia virus", "seafood market coronavirus acute respiratory infection", "2019novel seafood market coronavirus severe disorder", "2019 new chinese corona virus acute respiratory infection", "pneumonia due to 2019 new severe acute respiratory syndrome corona virus2", "disorder from 2019novel seafood market coronavirus", "sarscov2 2019 severe acute pneumonia", "wuhan seafood market pneumonia virus acute severe respiratory infection", "sars coronavirus2 2019 acute respiratory illness", "2019 novel sars corona virus 2 acute respiratory disease", "disease from severe acute respiratory syndromerelated coronavirus 2", "wuhancoronavirus acute pneumonia", "infection due to hcov19", "2019 sarscov2 severe disease", "2019novel wuhancoronavirus severe respiratory disorder", "2019new wuhancoronavirus acute severe infection", "2019 new wuhan virus severe acute syndrome", "covid19 virus asevere acute respiratory disease", "pneumonia due to severe acute respiratory syndrome coronavirus2", "2019new sars coronavirus 2 severe acute respiratory illness", "2019 chinese coronavirus acute disease", "2019new seafood market corona virus respiratory syndrome", "infection caused by 2019 seafood market coronavirus", "2019 new severe acute respiratory syndrome coronavirus2 disorder", "wuhan virus 2019 severe infection", "sarscov2 acute respiratory illness", "2019novel sars corona virus2 severe respiratory illness", "disease from novel corona virus 2019", "2019novel seafood market corona virus acute severe respiratory disorder", "2019new seafood market virus disorder", "severe acute respiratory syndrome corona virus 2 2019 disorder", "chinese corona virus severe infection", "chinese coronavirus 2019 acute respiratory disease", "hcov19 severe respiratory syndrome", "2019new chinese coronavirus severe acute infection", "sars corona virus 2 2019 severe illness", "wuhan seafood market virus 2019 severe acute disorder", "wuhancorona virus acute severe respiratory disease", "2019 novel wuhan pneumonia virus disorder", "2019new sars coronavirus 2 acute severe syndrome", "illness due to seafood market coronavirus", "2019 new wuhan virus acute respiratory syndrome", "2019 wuhan seafood market pneumonia virus acute respiratory infection", "2019new wuhan seafood market virus severe acute infection", "syndrome from 2019 human corona virus", "sars coronavirus2 severe disorder", "disorder caused by 2019new wuhancoronavirus", "2019 new sars coronavirus 2 pneumonia", "wuhan corona virus acute respiratory disorder", "2019 novel wuhan virus severe acute respiratory syndrome", "disease caused by 2019 new sars coronavirus2", "syndrome due to sars corona virus 2 2019", "disorder due to 2019 novel sars coronavirus2", "wuhancoronavirus 2019 disease", "2019 novel sars corona virus 2 respiratory infection", "2019 human coronavirus syndrome", "2019ncov severe respiratory infection", "2019new sars corona virus2 acute severe infection", "covid19 virus acute severe disease", "wuhan seafood market virus severe acute respiratory syndrome", "illness from 2019novel wuhancoronavirus", "disorder caused by human coronavirus 2019", "2019 new chinese coronavirus acute infection", "pneumonia caused by 2019 novel wuhancorona virus", "2019 new coronavirus acute severe disorder", "2019 wuhancoronavirus", "seafood market corona virus acute respiratory infection", "2019new wuhancorona virus respiratory illness", "2019new sars corona virus 2 acute severe infection", "2019 novel sars cov 2 acute infection", "2019 seafood market virus acute respiratory disease", "wuhan severe acute disorder", "sars corona virus2 severe acute respiratory illness", "disease from 2019 new wuhan virus", "2019 new sars corona virus2 acute severe pneumonia", "disease from sars coronavirus 2 2019", "2019 sars corona virus2 respiratory disorder", "wuhancorona virus 2019 acute infection", "2019 novel sars coronavirus 2 syndrome", "seafood market coronavirus 2019 acute severe illness", "2019new chinese coronavirus infection", "wuhancoronavirus disease", "2019novel seafood market corona virus acute severe respiratory infection", "2019 chinese corona virus asevere acute respiratory disease", "2019 new chinese coronavirus acute severe syndrome", "2019 new wuhan seafood market virus", "2019 ncov pneumonia", "2019 new seafood market corona virus infection", "2019 new seafood market coronavirus disease", "wuhan seafood market virus 2019 severe acute syndrome", "virus causing covid19 severe acute respiratory syndrome", "disorder caused by 2019novel wuhan virus", "disorder caused by 2019novel wuhan pneumonia virus", "2019 new seafood market virus", "2019 wuhan corona virus acute infection", "disease caused by 2019new seafood market coronavirus", "sars corona virus 2 2019 acute respiratory syndrome", "novel corona virus 2019 acute severe respiratory disease", "2019 sars coronavirus 2 acute severe illness", "syndrome caused by wuhancoronavirus 2019", "2019 seafood market virus acute infection", "2019 human coronavirus severe acute pneumonia", "2019novel wuhancoronavirus severe acute respiratory illness", "2019 novel coronavirus acute severe disease", "sars corona virus2 acute severe disease", "2019novel sars cov 2 disorder", "sars coronavirus 2 2019 severe infection", "illness from 2019new chinese coronavirus", "2019 novel sars coronavirus 2 acute severe respiratory disorder", "illness from severe acute respiratory syndrome corona virus 2 2019", "syndrome from 2019 new sarscov2", "seafood market corona virus 2019 severe illness", "2019 novel wuhan corona virus acute respiratory disease", "novel corona virus acute severe respiratory disease", "2019 sars coronavirus2 severe acute disease", "wuhan corona virus acute pneumonia", "2019 new chinese corona virus acute severe respiratory illness", "disorder caused by 2019 novel severe acute respiratory syndrome corona virus2", "pneumonia due to 2019 new severe acute respiratory syndrome coronavirus 2", "hcov19 respiratory disease", "sars cov 2", "disorder from new coronavirus 2019", "2019 wuhan seafood market virus acute severe pneumonia", "sars cov 2 2019 asevere acute respiratory disease", "2019 wuhan seafood market virus acute syndrome", "wuhancoronavirus 2019 severe respiratory infection", "virus responsible for covid19 acute pneumonia", "2019novel wuhancorona virus acute severe respiratory disease", "2019novel wuhan virus severe illness", "2019 human corona virus severe acute illness", "2019new sars corona virus 2 severe acute respiratory syndrome", "2019 novel chinese coronavirus acute severe respiratory syndrome", "2019 new seafood market virus severe disorder", "2019 human corona virus acute severe respiratory illness", "2019 novel wuhancoronavirus severe acute syndrome", "2019 novel wuhan seafood market virus severe acute syndrome", "chinese corona virus 2019 acute illness", "pneumonia caused by 2019 new wuhancorona virus", "2019 wuhan seafood market virus acute severe infection", "2019 seafood market coronavirus severe acute syndrome", "2019new wuhancoronavirus severe illness", "wuhan acute severe respiratory disease", "infection due to 2019 chinese corona virus", "2019 chinese corona virus acute severe respiratory illness", "chinese corona virus severe disease", "2019 novel wuhan coronavirus acute severe respiratory disorder", "infection from 2019 new chinese coronavirus", "illness due to 2019 seafood market coronavirus", "syndrome due to 2019 novel wuhancoronavirus", "2019 new wuhan pneumonia virus acute severe respiratory syndrome", "2019novel sarscov2 severe respiratory disease", "2019 novel seafood market virus severe illness", "disorder from human coronavirus 2019", "covid19 virus severe respiratory syndrome", "2019new wuhan corona virus severe acute pneumonia", "wuhan coronavirus disease", "2019 novel wuhan virus acute disease", "sars corona virus2 2019 acute severe pneumonia", "pneumonia caused by sars corona virus 2", "hcov19 illness", "2019 sars coronavirus 2 acute syndrome", "wuhancoronavirus 2019 asevere acute respiratory disease", "pneumonia caused by 2019 new severe acute respiratory syndromerelated corona virus 2", "seafood market acute severe pneumonia", "2019 new wuhan corona virus severe acute pneumonia", "syndrome caused by new corona virus", "2019 wuhancorona virus infection", "2019 wuhan seafood market pneumonia virus severe respiratory disease", "2019 wuhancoronavirus acute severe infection", "2019 new wuhan coronavirus acute severe illness", "sars cov 2 severe syndrome", "chinese coronavirus acute respiratory disorder", "disease caused by 2019 new severe acute respiratory syndrome coronavirus 2", "infection due to wuhan seafood market pneumonia virus 2019", "syndrome due to 2019 new wuhan corona virus", "2019new sars coronavirus2 acute severe infection", "severe acute respiratory syndrome coronavirus2", "2019 novel wuhan seafood market virus severe infection", "illness due to 2019 novel severe acute respiratory syndromerelated corona virus2", "pneumonia due to 2019 human corona virus", "new corona virus 2019 acute severe respiratory disorder", "sarscov2 2019 severe acute syndrome", "infection due to 2019 chinese coronavirus", "2019novel chinese corona virus severe respiratory disease", "disease caused by 2019 novel wuhan coronavirus", "wuhan seafood market virus acute severe infection", "disease caused by wuhan seafood market virus 2019", "disease from 2019 new chinese coronavirus", "disorder from 2019 novel corona virus", "disorder from severe acute respiratory syndrome corona virus 2 2019", "2019novel wuhancorona virus respiratory disorder", "new coronavirus 2019 acute severe disorder", "2019 novel corona virus acute severe respiratory infection", "2019 new wuhancoronavirus severe pneumonia", "wuhan seafood market virus 2019", "wuhan seafood market virus acute severe syndrome", "pneumonia caused by severe acute respiratory syndromerelated coronavirus 2 2019", "wuhan seafood market pneumonia virus respiratory disease", "infection caused by 2019new seafood market virus", "pneumonia from wuhan corona virus", "disease caused by sars coronavirus 2 2019", "2019novel severe acute respiratory syndromerelated coronavirus2 disorder", "2019 new wuhan corona virus severe acute syndrome", "2019novel sarscov2 respiratory disease", "2019 new coronavirus severe respiratory infection", "2019new wuhan pneumonia virus acute respiratory disease", "2019novel chinese coronavirus severe acute syndrome", "new coronavirus 2019 severe respiratory illness", "2019new sarscov2 acute severe pneumonia", "2019 sars coronavirus2 pneumonia", "new corona virus 2019 acute respiratory illness", "disease caused by 2019 novel sarscov2", "infection due to 2019 new coronavirus", "2019new sars corona virus2 severe acute disease", "infection due to seafood market corona virus 2019", "2019 new seafood market corona virus severe respiratory illness", "2019 new wuhan corona virus respiratory disease", "disease caused by 2019 wuhan seafood market pneumonia virus", "2019novel sars coronavirus2 severe infection", "new corona virus 2019 respiratory syndrome", "chinese corona virus severe acute respiratory illness", "2019new sars corona virus2 acute severe respiratory infection", "sars corona virus2 acute severe disorder", "2019novel sars coronavirus2 acute severe pneumonia", "wuhan coronavirus 2019 severe infection", "2019 novel chinese corona virus severe respiratory infection", "disease due to wuhancoronavirus", "pneumonia due to 2019novel chinese corona virus", "2019novel sars corona virus2 respiratory syndrome", "2019 new sars corona virus2 acute severe respiratory disease", "sars corona virus2 respiratory syndrome", "pneumonia due to sarscov2", "2019new wuhan virus acute severe respiratory illness", "2019 novel wuhan virus acute severe syndrome", "sars corona virus 2 2019 acute severe syndrome", "infection due to wuhan corona virus", "sars corona virus 2 severe respiratory disease", "disorder from severe acute respiratory syndrome coronavirus 2", "2019 ncov", "illness from severe acute respiratory syndrome coronavirus2 2019", "hubei severe illness", "2019new sars corona virus2 acute pneumonia", "2019new wuhan coronavirus acute disease", "pneumonia from severe acute respiratory syndromerelated coronavirus 2", "hcov19", "2019 novel chinese corona virus respiratory illness", "novel corona virus 2019 severe respiratory disorder", "novel corona virus 2019 illness", "2019 wuhan virus severe syndrome", "virus responsible for covid19 severe disorder", "2019 novel wuhan pneumonia virus severe acute pneumonia", "wuhan pneumonia virus 2019 acute severe respiratory illness", "syndrome from 2019new sars cov 2", "seafood market coronavirus 2019 severe acute respiratory disorder", "wuhan virus respiratory syndrome", "2019 sarscov2 acute respiratory syndrome", "2019 new wuhan seafood market virus severe respiratory disease", "infection caused by 2019 sars corona virus2", "pneumonia caused by 2019 new severe acute respiratory syndromerelated coronavirus 2", "2019 sarscov2 acute respiratory disease", "2019 novel wuhancoronavirus acute respiratory disease", "hubei severe disease", "2019 new seafood market coronavirus severe respiratory illness", "2019 novel wuhancoronavirus severe syndrome", "syndrome caused by 2019 seafood market corona virus", "wuhan seafood market virus severe respiratory illness", "disorder from 2019 novel seafood market coronavirus", "pneumonia from 2019novel wuhan seafood market virus", "pneumonia due to wuhan corona virus", "seafood market corona virus 2019 respiratory illness", "2019ncov syndrome", "2019new wuhan corona virus severe acute syndrome", "2019novel wuhan seafood market virus severe acute disorder", "2019novel sars corona virus 2 respiratory disorder", "disorder from 2019 ncov", "chinese coronavirus acute severe respiratory illness", "novel coronavirus severe acute illness", "wuhan seafood market virus 2019 acute severe disease", "illness caused by new coronavirus 2019", "disease from sars corona virus 2 2019", "disease due to 2019 wuhan corona virus", "wuhan corona virus acute severe respiratory syndrome", "pneumonia due to 2019 novel wuhancorona virus", "2019 novel wuhan virus acute infection", "2019 new sars cov 2 acute respiratory syndrome", "illness from 2019 novel severe acute respiratory syndromerelated coronavirus 2", "2019novel severe acute respiratory syndrome corona virus2 disorder", "chinese coronavirus acute illness", "disease due to 2019novel severe acute respiratory syndromerelated coronavirus 2", "2019 novel wuhan coronavirus respiratory illness", "2019 new sars coronavirus2 respiratory disorder", "2019 new chinese corona virus severe disease", "2019new seafood market virus severe respiratory syndrome", "sars coronavirus2 2019 severe infection", "2019 novel coronavirus severe acute pneumonia", "infection from 2019 wuhan virus", "pneumonia caused by 2019 seafood market virus", "wuhan corona virus 2019 acute severe respiratory illness", "disease due to wuhan seafood market virus 2019", "2019new chinese corona virus acute infection", "2019novel sars coronavirus2 acute respiratory disease", "2019 sars coronavirus2 acute severe syndrome", "disease due to 2019new wuhancorona virus", "wuhan coronavirus severe acute disease", "2019 seafood market virus severe acute disorder", "disease caused by 2019 severe acute respiratory syndromerelated coronavirus 2", "2019 new seafood market corona virus severe syndrome", "disease due to seafood market coronavirus", "2019 novel sars cov 2 acute severe infection", "2019 novel sars corona virus 2 severe illness", "2019 novel seafood market coronavirus respiratory infection", "2019 new coronavirus severe respiratory syndrome", "2019 wuhan coronavirus severe acute respiratory syndrome", "2019novel wuhan pneumonia virus disease", "wuhan seafood market virus severe respiratory disease", "novel corona virus severe respiratory disorder", "disorder caused by 2019 seafood market coronavirus", "illness due to wuhan coronavirus", "2019 novel sars coronavirus 2 severe acute disorder", "seafood market corona virus severe respiratory illness", "pneumonia caused by 2019 sars coronavirus2", "new corona virus acute respiratory disorder", "syndrome caused by 2019 seafood market coronavirus", "2019 sars coronavirus2 severe syndrome", "illness caused by 2019 wuhancorona virus", "novel corona virus 2019 syndrome", "disorder from severe acute respiratory syndromerelated corona virus2 2019", "seafood market coronavirus severe acute respiratory infection", "syndrome from seafood market coronavirus 2019", "sarscov2 acute severe syndrome", "novel coronavirus disease", "infection caused by 2019 novel sars coronavirus2", "virus responsible for covid19 severe acute infection", "2019ncov illness", "2019new sars corona virus2 severe acute syndrome", "wuhancoronavirus 2019", "wuhan coronavirus acute severe respiratory disease", "2019 sars coronavirus 2 respiratory infection", "2019novel chinese coronavirus acute disease", "wuhan coronavirus acute severe infection", "syndrome from wuhan pneumonia virus 2019", "2019new sars coronavirus2 acute respiratory syndrome", "2019 new wuhan seafood market pneumonia virus acute severe illness", "2019 wuhan corona virus severe acute respiratory illness", "2019 novel seafood market coronavirus severe respiratory syndrome", "2019new seafood market coronavirus acute severe respiratory infection", "2019novel chinese corona virus respiratory infection", "2019 new sars cov 2 severe acute respiratory syndrome", "2019 new sars corona virus2 acute severe disorder", "syndrome due to wuhancoronavirus 2019", "pneumonia from 2019novel chinese coronavirus", "sars corona virus2 respiratory illness", "infection caused by wuhan seafood market virus", "wuhan coronavirus 2019 acute severe disorder", "2019 seafood market coronavirus acute severe syndrome", "disease from 2019new wuhan virus", "hcov19 acute severe illness", "2019new wuhan seafood market virus respiratory syndrome", "2019novel sars corona virus 2 severe respiratory illness", "new corona virus acute syndrome", "2019new wuhan pneumonia virus severe acute respiratory infection", "illness caused by hcov19", "new coronavirus acute severe respiratory disorder", "2019new wuhan coronavirus severe acute respiratory disorder", "2019new wuhan corona virus severe respiratory disorder", "infection caused by 2019 novel wuhancoronavirus", "syndrome from 2019novel seafood market corona virus", "2019novel wuhan corona virus", "2019new seafood market virus acute respiratory disease", "2019 new sars coronavirus 2 severe respiratory disorder", "illness from 2019 seafood market virus", "seafood market disorder", "2019 new wuhancoronavirus acute respiratory syndrome", "2019 novel seafood market coronavirus severe illness", "2019 new sarscov2 acute disorder", "disease caused by 2019 new wuhan virus", "infection due to new corona virus", "disorder due to sars corona virus2 2019", "2019new severe acute respiratory syndrome coronavirus2", "2019 ncov infection", "seafood market corona virus 2019 severe acute respiratory illness", "2019 chinese corona virus severe syndrome", "2019novel wuhan seafood market virus severe acute respiratory illness", "infection from 2019 coronavirus", "2019 new seafood market corona virus acute respiratory syndrome", "sars coronavirus 2 acute respiratory disease", "sars cov 2 asevere acute respiratory disease", "2019 new chinese coronavirus acute respiratory disease", "virus responsible for covid19 severe acute respiratory illness", "2019 sars coronavirus 2 severe respiratory syndrome", "pneumonia due to 2019 wuhan corona virus", "disease caused by 2019 sars corona virus 2", "pneumonia from 2019new severe acute respiratory syndromerelated coronavirus2", "2019novel sars cov 2 respiratory disorder", "disorder due to 2019 wuhancoronavirus", "sars coronavirus2 acute syndrome", "pneumonia caused by wuhan corona virus", "syndrome due to sarscov2 2019", "infection from 2019 wuhan seafood market virus", "sars corona virus 2 2019 acute disease", "pneumonia from 2019 new severe acute respiratory syndrome corona virus 2", "covid19 virus respiratory disorder", "wuhancoronavirus respiratory infection", "illness caused by 2019novel chinese coronavirus", "2019novel sars coronavirus2 disorder", "infection caused by 2019 novel wuhan virus", "disease from 2019novel severe acute respiratory syndrome corona virus2", "2019 novel coronavirus severe acute infection", "severe acute respiratory syndromerelated corona virus2 2019 disease", "2019novel wuhan coronavirus respiratory disorder", "2019novel chinese corona virus acute severe disease", "syndrome caused by sars coronavirus 2", "wuhan pneumonia virus severe disease", "2019 novel wuhancorona virus syndrome", "2019 new sarscov2 severe disease", "2019novel sarscov2 acute severe disease", "sars coronavirus 2 severe acute illness", "new corona virus acute infection", "pneumonia from 2019novel sars coronavirus 2", "novel coronavirus 2019 acute severe syndrome", "2019 seafood market coronavirus severe respiratory disorder", "infection due to 2019 new wuhan seafood market virus", "2019 wuhan corona virus severe respiratory illness", "sars coronavirus 2 2019 acute severe disorder", "2019 new sars cov 2 severe acute respiratory disorder", "infection caused by 2019 new chinese coronavirus", "disorder due to 2019novel sarscov2", "2019 novel sars cov 2 severe acute respiratory infection", "2019 wuhancoronavirus severe pneumonia", "syndrome from 2019 new wuhan seafood market pneumonia virus", "infection from novel corona virus 2019", "2019 new sars corona virus 2 acute severe illness", "2019 new wuhancorona virus disease", "2019 new wuhan seafood market virus disorder", "new corona virus disease", "illness from 2019 novel coronavirus", "2019new seafood market virus acute infection", "2019 novel sarscov2 severe respiratory infection", "pneumonia due to wuhan virus 2019", "2019new seafood market virus respiratory syndrome", "2019 novel sars corona virus2 acute respiratory illness", "2019novel sarscov2", "2019 novel seafood market virus severe syndrome", "2019 human corona virus acute severe disorder", "2019 new sars corona virus 2 acute infection", "2019new severe acute respiratory syndrome corona virus2 illness", "wuhan seafood market virus respiratory illness", "2019 new wuhan coronavirus severe respiratory illness", "2019new sars corona virus2 acute illness", "pneumonia from 2019new seafood market coronavirus", "virus causing covid19 acute pneumonia", "2019 wuhan seafood market virus acute severe respiratory disease", "disorder from 2019novel seafood market virus", "2019 wuhan seafood market pneumonia virus severe acute pneumonia", "infection due to 2019new wuhan seafood market pneumonia virus", "pneumonia due to 2019novel wuhancoronavirus", "2019new sars coronavirus 2 severe acute disease", "chinese corona virus asevere acute respiratory disease", "disease caused by wuhan seafood market pneumonia virus", "2019novel sars coronavirus2 severe acute disorder", "2019novel seafood market virus severe acute respiratory infection", "2019 sars cov 2 illness", "wuhan corona virus 2019 respiratory syndrome", "syndrome due to 2019 wuhan coronavirus", "infection due to chinese corona virus", "2019 new seafood market coronavirus asevere acute respiratory disease", "disorder due to 2019 novel sars corona virus 2", "2019new seafood market corona virus disease", "2019 novel wuhan seafood market pneumonia virus respiratory disease", "2019new wuhan seafood market virus acute respiratory illness", "novel coronavirus 2019 asevere acute respiratory disease", "infection due to sarscov2 2019", "2019 wuhan seafood market pneumonia virus acute infection", "pneumonia caused by 2019 novel severe acute respiratory syndromerelated coronavirus2", "illness due to severe acute respiratory syndrome corona virus2 2019", "disease caused by 2019new wuhan pneumonia virus", "2019 sars corona virus2 respiratory illness", "disorder from 2019 sars cov 2", "2019 new wuhan coronavirus acute respiratory infection", "seafood market coronavirus acute respiratory syndrome", "wuhan virus 2019 severe acute illness", "sars coronavirus2 2019 severe respiratory infection", "2019novel sars coronavirus 2 severe acute disease", "2019novel chinese coronavirus severe acute disease", "severe acute respiratory syndrome coronavirus2 illness", "2019 novel chinese corona virus severe respiratory syndrome", "disease caused by 2019 novel sars coronavirus 2", "sars coronavirus2 infection", "2019 sarscov2 acute respiratory infection", "illness from sarscov2", "sars cov 2 acute severe illness", "2019 novel seafood market coronavirus acute syndrome", "2019new seafood market virus asevere acute respiratory disease", "novel coronavirus acute severe disorder", "infection due to 2019 wuhan seafood market pneumonia virus", "2019new wuhan virus asevere acute respiratory disease", "2019 wuhan seafood market virus acute illness", "2019new wuhan seafood market virus acute syndrome", "2019novel chinese corona virus respiratory syndrome", "2019 new coronavirus severe acute infection", "disorder due to wuhan virus 2019", "syndrome from 2019 new wuhan seafood market virus", "chinese corona virus 2019 acute respiratory infection", "novel corona virus severe acute respiratory syndrome", "2019 novel wuhan seafood market virus severe acute respiratory disorder", "2019 new severe acute respiratory syndromerelated coronavirus 2 illness", "2019 novel chinese corona virus severe acute respiratory infection", "syndrome due to 2019novel seafood market virus", "illness due to human corona virus 2019", "2019new wuhan seafood market pneumonia virus respiratory disorder", "2019novel wuhan seafood market virus", "2019new seafood market coronavirus severe acute respiratory disorder", "chinese coronavirus 2019 acute disease", "2019new sars cov 2 severe respiratory disorder", "seafood market virus acute pneumonia", "2019 novel corona virus acute severe syndrome", "2019 novel seafood market virus severe disease", "2019new wuhancorona virus acute severe respiratory illness", "covid 19 virus severe acute illness", "disorder caused by 2019new wuhan seafood market pneumonia virus", "2019 new corona virus syndrome", "2019 novel wuhan virus disorder", "pneumonia due to severe acute respiratory syndrome corona virus2", "2019 new wuhancoronavirus severe respiratory illness", "2019 novel wuhan corona virus severe respiratory disease", "novel corona virus illness", "hubei pneumonia", "disease from human corona virus 2019", "2019 new seafood market virus pneumonia", "disorder from 2019 new wuhancoronavirus", "2019new sars coronavirus2 severe disorder", "pneumonia due to 2019new severe acute respiratory syndromerelated corona virus 2", "disorder caused by sarscov2 2019", "seafood market corona virus respiratory syndrome", "2019 wuhan corona virus severe respiratory disease", "2019 new wuhancoronavirus severe respiratory syndrome", "2019novel chinese corona virus acute infection", "2019novel wuhancoronavirus severe acute disease", "syndrome caused by 2019 new sars cov 2", "2019 new sars corona virus2 acute severe syndrome", "wuhan severe acute respiratory syndrome", "sarscov2 acute severe disorder", "infection due to 2019 novel severe acute respiratory syndromerelated corona virus 2", "pneumonia caused by 2019new severe acute respiratory syndrome corona virus 2", "2019 wuhan pneumonia virus syndrome", "2019new chinese corona virus respiratory disorder", "sars cov 2 severe acute respiratory disorder", "2019 novel wuhan corona virus disease", "novel coronavirus 2019 respiratory disorder", "2019 novel seafood market virus severe acute respiratory illness", "severe acute respiratory syndromerelated corona virus 2 pneumonia", "seafood market virus 2019 acute respiratory infection", "2019novel seafood market corona virus severe acute illness", "2019novel chinese coronavirus severe acute pneumonia", "illness from 2019ncov", "2019novel wuhancoronavirus disease", "infection caused by seafood market virus", "2019 novel chinese corona virus acute severe illness", "2019new wuhan seafood market pneumonia virus acute severe infection", "severe acute respiratory syndrome corona virus2 pneumonia", "2019ncov severe illness", "2019 sars coronavirus2 severe acute respiratory infection", "seafood market corona virus severe acute respiratory syndrome", "2019new wuhan pneumonia virus severe acute infection", "2019 chinese coronavirus illness", "virus responsible for covid19 acute severe illness", "2019 seafood market virus acute severe pneumonia", "2019novel seafood market virus acute syndrome", "2019 novel sars cov 2 severe respiratory syndrome", "2019 novel sars cov 2 severe acute respiratory illness", "2019 novel coronavirus severe disease", "2019novel sars corona virus2 severe acute disease", "2019 new sars corona virus2 severe acute disease", "2019 novel wuhan corona virus acute infection", "seafood market virus infection", "disorder from 2019 human corona virus", "infection caused by 2019 novel sars cov 2", "2019 novel wuhan seafood market virus severe acute disease", "illness due to 2019new severe acute respiratory syndrome coronavirus2", "2019 coronavirus acute respiratory syndrome", "2019 new chinese corona virus acute respiratory illness", "pneumonia caused by new corona virus 2019", "2019novel wuhan seafood market pneumonia virus acute severe illness", "infection caused by 2019 novel severe acute respiratory syndrome coronavirus 2", "2019new wuhan seafood market pneumonia virus severe acute syndrome", "wuhan coronavirus 2019 severe syndrome", "2019 novel seafood market corona virus severe respiratory infection", "pneumonia due to 2019 new wuhan coronavirus", "2019 new sarscov2 severe syndrome", "2019novel seafood market virus acute severe disease", "illness due to 2019 new sars coronavirus 2", "2019 novel wuhancorona virus severe disease", "2019 novel seafood market coronavirus acute severe disease", "infection from virus responsible for covid19", "seafood market corona virus 2019 severe acute illness", "2019 novel severe acute respiratory syndromerelated coronavirus 2 pneumonia", "chinese coronavirus 2019 acute respiratory syndrome", "syndrome caused by 2019 new seafood market coronavirus", "2019 novel chinese corona virus acute respiratory syndrome", "2019new wuhan seafood market pneumonia virus acute severe disorder", "infection from 2019 novel sars corona virus2", "2019novel wuhan seafood market virus acute illness", "disorder due to 2019novel sars coronavirus2", "syndrome from 2019new sars coronavirus 2", "2019 wuhan coronavirus severe acute respiratory illness", "2019 wuhan virus syndrome", "2019 wuhan pneumonia virus severe acute respiratory syndrome", "infection from 2019new sarscov2", "2019 sars coronavirus2 acute infection", "sars corona virus2 2019 severe acute disease", "syndrome from chinese corona virus", "2019new wuhan corona virus asevere acute respiratory disease", "disorder caused by 2019 seafood market virus", "2019 new seafood market virus severe illness", "pneumonia from new coronavirus 2019", "wuhan seafood market virus 2019 acute disorder", "infection caused by 2019novel seafood market virus", "illness due to 2019 wuhan seafood market virus", "2019 sars cov 2 acute respiratory syndrome", "illness from 2019 new wuhan pneumonia virus", "sars coronavirus2 2019 respiratory syndrome", "infection due to 2019 novel wuhan pneumonia virus", "sars corona virus2 asevere acute respiratory disease", "2019 new wuhan coronavirus acute severe disorder", "2019 new chinese coronavirus severe acute disorder", "disorder due to severe acute respiratory syndrome coronavirus 2", "syndrome caused by 2019 novel wuhan coronavirus", "2019 new sars cov 2 acute severe syndrome", "syndrome caused by chinese corona virus 2019", "disorder due to 2019new severe acute respiratory syndrome corona virus 2", "pneumonia due to 2019 novel sars cov 2", "disorder from 2019new sars coronavirus 2", "2019novel wuhancoronavirus acute severe respiratory disease", "chinese coronavirus acute severe pneumonia", "sars coronavirus2 severe respiratory illness", "2019novel sars corona virus 2 severe pneumonia", "wuhan coronavirus acute severe respiratory syndrome", "2019 seafood market corona virus severe illness", "disorder caused by chinese coronavirus", "infection due to 2019 new severe acute respiratory syndromerelated coronavirus2", "sars coronavirus 2 severe acute syndrome", "2019novel sarscov2 acute disease", "2019 novel chinese corona virus respiratory disease", "2019 new wuhancoronavirus acute disorder", "chinese corona virus 2019 acute infection", "2019 novel sars cov 2 severe acute pneumonia", "disease from wuhan pneumonia virus 2019", "2019novel seafood market corona virus illness", "chinese coronavirus 2019 acute respiratory illness", "disease caused by 2019novel sars cov 2", "pneumonia caused by wuhan coronavirus", "novel coronavirus acute illness", "disorder from 2019new severe acute respiratory syndromerelated coronavirus2", "wuhancorona virus pneumonia", "pneumonia from 2019 novel wuhan corona virus", "2019 wuhan virus acute severe illness", "2019novel sars corona virus2 acute respiratory disease", "2019 sars corona virus 2", "wuhan seafood market virus acute severe respiratory infection", "disorder from wuhan seafood market pneumonia virus 2019", "seafood market severe acute illness", "2019 new sars coronavirus 2 syndrome", "2019new wuhan pneumonia virus disease", "wuhancoronavirus 2019 acute respiratory infection", "2019 seafood market corona virus severe respiratory illness", "virus causing covid19 acute illness", "disorder from 2019 coronavirus", "disease caused by 2019new severe acute respiratory syndrome corona virus2", "disease caused by 2019 novel wuhan corona virus", "2019novel chinese corona virus acute respiratory illness", "2019new wuhan seafood market pneumonia virus infection", "2019novel sars corona virus2 severe pneumonia", "2019 new wuhan corona virus severe illness", "disorder caused by 2019 new severe acute respiratory syndrome corona virus2", "2019 wuhancoronavirus acute respiratory syndrome", "wuhan pneumonia virus 2019 asevere acute respiratory disease", "chinese corona virus 2019 severe infection", "seafood market corona virus acute disease", "seafood market asevere acute respiratory disease", "disorder due to new coronavirus 2019", "wuhan virus 2019 acute severe illness", "disease caused by human corona virus 2019", "illness from novel corona virus", "2019novel chinese corona virus severe respiratory illness", "2019 sars corona virus 2 severe infection", "pneumonia from 2019new wuhan coronavirus", "2019 corona virus acute severe disease", "disorder from 2019 novel chinese coronavirus", "2019 new sars coronavirus 2 infection", "2019 coronavirus acute respiratory illness", "2019 wuhan seafood market virus acute severe respiratory disorder", "2019 novel wuhancorona virus respiratory infection", "2019 novel sars cov 2 severe acute disease", "disease due to 2019novel sars coronavirus2", "2019 new sars corona virus2 acute syndrome", "2019 new sars corona virus 2 severe acute respiratory illness", "2019 chinese coronavirus respiratory syndrome", "2019new sars cov 2 syndrome", "2019 new wuhan seafood market virus severe acute illness", "2019 new seafood market coronavirus severe illness", "2019novel sars corona virus 2", "sars coronavirus2 2019 severe syndrome", "virus responsible for covid19 severe infection", "2019 novel sars corona virus2 acute respiratory infection", "sars corona virus2 severe respiratory disorder", "covid19 virus illness", "2019 sars corona virus2 acute illness", "2019novel wuhan coronavirus severe illness", "infection caused by wuhancoronavirus", "2019ncov severe respiratory illness", "pneumonia due to chinese coronavirus", "2019 novel sarscov2 acute syndrome", "disease due to wuhan pneumonia virus 2019", "2019 novel chinese corona virus syndrome", "seafood market severe acute pneumonia", "wuhan virus 2019 acute respiratory syndrome", "2019 new wuhan seafood market pneumonia virus severe acute disorder", "seafood market corona virus 2019 severe infection", "2019 wuhan corona virus severe acute disease", "infection caused by 2019 novel sars corona virus 2", "wuhan virus 2019 severe acute respiratory illness", "2019 novel sars corona virus 2 respiratory disorder", "2019 wuhan corona virus pneumonia", "disorder due to severe acute respiratory syndrome corona virus2", "disorder caused by 2019 new severe acute respiratory syndrome coronavirus 2", "disease from 2019 novel seafood market corona virus", "disease caused by chinese coronavirus", "2019 new sars coronavirus 2 acute severe pneumonia", "novel coronavirus severe acute disease", "disease from 2019novel severe acute respiratory syndrome coronavirus 2", "2019 novel wuhancoronavirus respiratory illness", "infection caused by wuhan pneumonia virus", "pneumonia caused by 2019 new sars coronavirus2", "2019novel seafood market virus disorder", "2019 wuhan seafood market virus acute disorder", "sarscov2 2019 severe infection", "2019 new seafood market corona virus acute disease", "2019new sarscov2 acute syndrome", "wuhancorona virus 2019 disorder", "disease from 2019 seafood market corona virus", "disorder caused by 2019 new wuhancoronavirus", "seafood market respiratory infection", "2019 seafood market corona virus acute severe respiratory infection", "2019novel seafood market corona virus infection", "seafood market virus 2019 severe disease", "wuhan pneumonia virus 2019 severe acute respiratory illness", "illness due to severe acute respiratory syndrome corona virus 2 2019", "disease due to 2019 new chinese coronavirus", "2019 novel wuhan seafood market pneumonia virus severe disorder", "wuhancoronavirus 2019 severe acute pneumonia", "2019new sars cov 2 acute severe syndrome", "2019 sars cov 2 severe respiratory syndrome", "2019 human corona virus severe respiratory disorder", "2019 novel seafood market virus acute severe respiratory syndrome", "2019novel seafood market virus severe acute respiratory syndrome", "infection due to 2019 new sars cov 2", "chinese coronavirus 2019 pneumonia", "2019 sars corona virus 2 severe acute respiratory illness", "infection from 2019 novel wuhan coronavirus", "2019new wuhan seafood market virus severe acute respiratory syndrome", "novel coronavirus acute severe pneumonia", "2019 novel sarscov2 acute respiratory syndrome", "2019 new severe acute respiratory syndromerelated corona virus 2 illness", "2019novel wuhancorona virus asevere acute respiratory disease", "disorder caused by 2019 new sars coronavirus2", "illness from wuhan pneumonia virus 2019", "disease from 2019 new sarscov2", "2019new seafood market corona virus acute disease", "illness caused by seafood market coronavirus", "chinese coronavirus asevere acute respiratory disease", "2019novel wuhan pneumonia virus acute disease", "2019 sars corona virus2 syndrome", "2019 new wuhan seafood market pneumonia virus acute syndrome", "2019 novel wuhan pneumonia virus acute severe pneumonia", "2019 new sars cov 2 severe acute respiratory illness", "2019 sars corona virus2 severe illness", "syndrome caused by sars corona virus 2 2019", "2019new seafood market virus severe illness", "seafood market virus acute respiratory infection", "2019new seafood market coronavirus severe acute respiratory infection", "2019 new chinese corona virus severe pneumonia", "wuhan coronavirus 2019 acute severe disease", "disorder from 2019 novel wuhancoronavirus", "2019new wuhan virus severe acute infection", "2019novel wuhancoronavirus severe acute disorder", "2019 new sars coronavirus 2 acute severe respiratory disorder", "2019novel sars cov 2 respiratory illness", "2019 new sars corona virus2 severe acute syndrome", "syndrome caused by wuhan pneumonia virus", "wuhan coronavirus 2019 acute severe syndrome", "disorder from 2019 wuhan coronavirus", "2019 novel coronavirus acute respiratory infection", "severe acute respiratory syndrome coronavirus2 2019 pneumonia", "2019 human coronavirus severe respiratory disease", "2019 novel sarscov2 severe acute pneumonia", "new corona virus severe acute respiratory infection", "2019novel wuhan pneumonia virus severe acute respiratory illness", "2019novel sars corona virus 2 acute severe respiratory infection", "illness caused by 2019 sars cov 2", "novel coronavirus 2019 severe respiratory infection", "2019 seafood market virus severe acute infection", "2019 sars corona virus 2 severe pneumonia", "sarscov2 severe respiratory infection", "2019 seafood market corona virus severe respiratory infection", "2019new sars coronavirus2 acute severe respiratory disorder", "severe acute respiratory syndrome 2019", "2019 new chinese corona virus respiratory infection", "sars cov 2 2019 infection", "wuhancorona virus severe pneumonia", "wuhan virus respiratory disease", "wuhan pneumonia virus disorder", "wuhan pneumonia virus acute respiratory infection", "illness from 2019 new severe acute respiratory syndrome corona virus2", "2019 novel sars corona virus2 severe disorder", "2019 novel wuhan pneumonia virus acute severe disorder", "2019 wuhan corona virus acute severe respiratory illness", "2019new wuhancorona virus infection", "chinese coronavirus 2019 severe acute disorder", "sars cov 2 2019", "2019 seafood market corona virus acute respiratory illness", "covid19 virus severe acute respiratory syndrome", "disorder due to 2019 coronavirus", "syndrome caused by 2019 novel seafood market corona virus", "seafood market illness", "syndrome from chinese corona virus 2019", "virus responsible for covid19 acute respiratory disorder", "2019 novel coronavirus syndrome", "hubei acute severe illness", "2019 new corona virus acute severe disorder", "2019 sars corona virus 2 severe syndrome", "2019 new sars coronavirus2 severe respiratory disorder", "covid 19", "2019 new sars coronavirus 2 disease", "wuhan acute severe illness", "new corona virus 2019 severe acute syndrome", "2019new seafood market coronavirus severe respiratory infection", "syndrome caused by 2019 new wuhan seafood market pneumonia virus", "disease due to 2019 severe acute respiratory syndrome corona virus2", "pneumonia due to novel coronavirus 2019", "seafood market coronavirus severe acute respiratory illness", "2019 novel corona virus respiratory illness", "2019new sars coronavirus 2 respiratory infection", "2019 seafood market coronavirus acute severe illness", "2019 new chinese coronavirus asevere acute respiratory disease", "novel coronavirus respiratory infection", "2019 new wuhan corona virus respiratory illness", "wuhan seafood market pneumonia virus severe acute respiratory illness", "infection caused by 2019 new seafood market coronavirus", "infection from 2019new seafood market corona virus", "infection from severe acute respiratory syndromerelated corona virus 2 2019", "pneumonia from wuhan seafood market pneumonia virus 2019", "2019novel seafood market virus acute illness", "wuhan coronavirus respiratory disease", "human coronavirus 2019 acute severe respiratory illness", "illness from 2019 seafood market coronavirus", "chinese coronavirus syndrome", "syndrome due to 2019 novel sars corona virus 2", "2019 sarscov2 severe acute infection", "2019novel sars coronavirus 2 respiratory syndrome", "2019 new wuhan coronavirus acute severe respiratory infection", "infection from 2019 new seafood market coronavirus", "syndrome from novel coronavirus 2019", "wuhancoronavirus 2019 respiratory syndrome", "2019 new coronavirus disorder", "disease caused by wuhan pneumonia virus 2019", "infection due to 2019novel severe acute respiratory syndromerelated coronavirus 2", "disease due to sars coronavirus2", "disorder due to 2019novel wuhan coronavirus", "hubei illness", "2019 novel wuhancoronavirus acute respiratory syndrome", "2019new sarscov2 severe acute respiratory disorder", "2019 wuhancorona virus pneumonia", "2019novel wuhan coronavirus respiratory syndrome", "pneumonia due to 2019 seafood market virus", "seafood market virus severe respiratory infection", "2019 chinese coronavirus severe acute disease", "sars cov 2 2019 acute disorder", "novel coronavirus acute severe syndrome", "2019novel sars cov 2 acute severe disorder", "seafood market corona virus 2019 infection", "2019 new wuhan virus acute disease", "2019 severe acute respiratory syndromerelated coronavirus2 disorder", "2019 new coronavirus acute respiratory infection", "pneumonia from 2019novel wuhan corona virus", "wuhan corona virus acute illness", "2019novel chinese corona virus respiratory disorder", "2019 novel severe acute respiratory syndromerelated coronavirus2 disease", "disorder caused by 2019 severe acute respiratory syndrome corona virus2", "2019 corona virus syndrome", "2019 wuhan coronavirus disease", "2019novel sars corona virus 2 acute disorder", "wuhan seafood market virus syndrome", "new coronavirus 2019 severe acute respiratory infection", "illness caused by 2019new sars corona virus2", "sars coronavirus 2 2019 respiratory illness", "2019 wuhan virus severe respiratory disease", "pneumonia caused by 2019novel wuhancoronavirus", "wuhan corona virus 2019 acute disorder", "syndrome caused by new coronavirus", "pneumonia due to 2019 new wuhancoronavirus", "pneumonia due to 2019 sars corona virus 2", "2019 novel seafood market coronavirus acute severe respiratory disease", "2019 corona virus disease", "wuhan virus severe illness", "2019new wuhancoronavirus severe disease", "2019new wuhan virus acute severe respiratory infection", "syndrome from 2019 novel wuhan seafood market virus", "wuhan pneumonia virus 2019 severe respiratory infection", "2019new seafood market virus severe acute infection", "disease from 2019 severe acute respiratory syndromerelated coronavirus 2", "2019 corona virus acute severe infection", "seafood market corona virus severe respiratory disease", "2019novel chinese coronavirus severe acute respiratory illness", "disease caused by 2019 wuhancorona virus", "seafood market virus acute respiratory illness", "pneumonia caused by wuhancoronavirus", "human corona virus 2019 severe respiratory infection", "2019 coronavirus disorder", "disease caused by 2019 new severe acute respiratory syndrome coronavirus2", "wuhan coronavirus 2019 severe respiratory syndrome", "2019 novel sars cov 2 respiratory disease", "wuhan virus 2019 acute severe respiratory disease", "new corona virus 2019 severe respiratory syndrome", "disease from covid 19 virus", "wuhan virus 2019 severe acute respiratory infection", "syndrome caused by 2019new chinese coronavirus", "chinese coronavirus severe acute respiratory disorder", "2019novel sarscov2 acute respiratory syndrome", "2019 new chinese coronavirus acute severe disease", "infection due to wuhancorona virus", "2019 novel wuhancoronavirus acute severe respiratory infection", "2019novel wuhan corona virus severe disorder", "disorder from 2019 novel severe acute respiratory syndromerelated corona virus2", "2019 novel severe acute respiratory syndrome coronavirus 2 pneumonia", "2019novel chinese coronavirus acute respiratory disorder", "novel corona virus severe acute illness", "2019novel sarscov2 acute respiratory disorder", "novel corona virus acute respiratory illness", "2019 new wuhan seafood market virus acute respiratory disease", "disease caused by 2019novel seafood market corona virus", "sars coronavirus 2 acute disorder", "2019new wuhan seafood market pneumonia virus respiratory disease", "2019 new seafood market corona virus respiratory disease", "2019 new seafood market coronavirus pneumonia", "covid19 virus acute respiratory infection", "wuhan coronavirus 2019 severe acute respiratory syndrome", "2019novel seafood market corona virus syndrome", "disease due to new corona virus 2019", "2019novel wuhan seafood market pneumonia virus acute respiratory illness", "wuhan seafood market virus severe respiratory syndrome", "syndrome caused by sarscov2 2019", "2019new wuhan seafood market virus disease", "syndrome caused by novel corona virus", "syndrome from 2019 new seafood market virus", "sarscov2 acute syndrome", "2019 novel wuhan corona virus respiratory disease", "2019 novel seafood market virus acute severe respiratory disorder", "2019 new sars coronavirus 2 acute severe disorder", "syndrome caused by 2019 novel wuhan pneumonia virus", "wuhan seafood market virus severe pneumonia", "wuhan severe acute infection", "disorder caused by 2019novel sarscov2", "2019 new seafood market virus infection", "2019 seafood market virus disease", "2019 wuhan virus", "2019new sars coronavirus 2 severe acute syndrome", "syndrome due to 2019new wuhan pneumonia virus", "2019 new wuhan seafood market pneumonia virus syndrome", "2019new chinese corona virus acute respiratory disorder", "sars cov 2 2019 acute severe respiratory disease", "syndrome from 2019 novel coronavirus", "2019new sarscov2 acute severe infection", "disorder from 2019 sarscov2", "2019 novel sars corona virus2 severe acute respiratory illness", "2019 novel seafood market coronavirus severe pneumonia", "2019 new wuhan pneumonia virus severe acute infection", "syndrome caused by 2019novel chinese corona virus", "2019 new seafood market virus respiratory syndrome", "disorder caused by wuhan virus 2019", "infection from 2019novel wuhan pneumonia virus", "syndrome from 2019 novel seafood market virus", "wuhan coronavirus severe disorder", "disorder from wuhan seafood market pneumonia virus", "syndrome from sars cov 2 2019", "novel coronavirus 2019 severe illness", "2019 coronavirus respiratory disease", "illness caused by human coronavirus 2019", "wuhan virus 2019 acute respiratory infection", "2019new wuhan seafood market pneumonia virus acute severe disease", "wuhan seafood market virus acute severe pneumonia", "new coronavirus", "pneumonia caused by covid 19 virus", "illness from 2019novel severe acute respiratory syndromerelated corona virus2", "wuhancorona virus acute syndrome", "disorder from 2019 new severe acute respiratory syndromerelated corona virus 2", "2019 new wuhan coronavirus respiratory syndrome", "chinese corona virus 2019 acute severe syndrome", "2019 novel sars coronavirus2 severe acute pneumonia", "pneumonia due to 2019new chinese coronavirus", "infection from severe acute respiratory syndrome corona virus2", "2019novel seafood market corona virus acute severe illness", "human corona virus 2019 severe respiratory disease", "new coronavirus 2019 severe acute syndrome", "pneumonia due to 2019new wuhan coronavirus", "illness from 2019new severe acute respiratory syndrome coronavirus 2", "2019new sarscov2 severe respiratory syndrome", "2019 new coronavirus acute severe illness", "disease caused by 2019 new corona virus", "wuhan virus 2019 severe illness", "syndrome from wuhan seafood market pneumonia virus", "disorder due to 2019 novel seafood market coronavirus", "disease caused by wuhan corona virus", "sars corona virus 2 severe illness", "syndrome due to wuhan seafood market pneumonia virus 2019", "2019novel sars corona virus 2 severe illness", "disease from 2019 novel severe acute respiratory syndromerelated coronavirus2", "2019novel wuhan corona virus acute severe respiratory syndrome", "hcov19 severe acute infection", "2019 new sarscov2 severe respiratory syndrome", "2019 novel corona virus severe acute respiratory syndrome", "infection caused by 2019new severe acute respiratory syndrome coronavirus 2", "infection from 2019 novel sarscov2", "2019 chinese coronavirus acute respiratory disease", "new coronavirus 2019 severe acute disease", "2019new wuhancorona virus acute respiratory illness", "2019novel sars coronavirus2 severe acute pneumonia", "2019 novel seafood market virus illness", "wuhan virus 2019 pneumonia", "wuhan pneumonia virus 2019 acute respiratory infection", "severe acute respiratory syndrome coronavirus 2 illness", "infection from 2019 novel sars coronavirus 2", "2019 novel wuhan coronavirus severe acute disease", "2019 new wuhan virus severe infection", "2019novel wuhan seafood market virus acute severe illness", "chinese coronavirus 2019 severe acute disease", "2019novel wuhan pneumonia virus severe respiratory disease", "2019 sars coronavirus2 severe acute respiratory disorder", "wuhan corona virus acute severe respiratory infection", "2019 chinese corona virus disease", "2019 novel wuhan seafood market pneumonia virus severe acute disease", "infection caused by 2019novel sars corona virus 2", "disease due to 2019novel wuhancorona virus", "2019novel wuhan seafood market virus severe pneumonia", "hcov19 severe respiratory infection", "2019new wuhan seafood market pneumonia virus acute respiratory disorder", "syndrome from 2019 new sars corona virus2", "illness caused by chinese corona virus", "disorder caused by virus causing covid19", "pneumonia caused by 2019 novel chinese corona virus", "covid 19 virus severe acute respiratory illness", "new corona virus severe respiratory syndrome", "human corona virus 2019 respiratory infection", "2019 new wuhan seafood market pneumonia virus acute respiratory syndrome", "2019 new wuhan virus severe acute infection", "infection from covid 19 virus", "2019 new wuhancoronavirus acute severe infection", "2019 new sars corona virus2 acute infection", "2019new sars cov 2 severe illness", "wuhan seafood market pneumonia virus 2019 syndrome", "2019new wuhan coronavirus severe respiratory syndrome", "2019 novel chinese coronavirus acute severe respiratory illness", "novel coronavirus syndrome", "2019novel chinese coronavirus acute severe respiratory syndrome", "2019novel sars coronavirus2 infection", "2019 new wuhan seafood market virus respiratory illness", "2019 new sars coronavirus2 severe illness", "2019new seafood market corona virus respiratory disease", "2019novel wuhan seafood market virus acute respiratory infection", "sars corona virus2 2019 severe respiratory disease", "infection due to 2019novel wuhan seafood market virus", "illness caused by 2019 wuhan virus", "seafood market coronavirus severe disease", "novel corona virus 2019 acute disorder", "disease from 2019 new severe acute respiratory syndromerelated coronavirus2", "disease caused by severe acute respiratory syndromerelated coronavirus 2", "sars coronavirus2 2019 respiratory infection", "wuhancorona virus acute respiratory disease", "2019novel wuhan seafood market virus acute severe pneumonia", "disease from 2019new wuhancoronavirus", "sars coronavirus2 severe acute respiratory illness", "2019novel chinese coronavirus acute respiratory illness", "2019new sars coronavirus 2 respiratory disorder", "2019novel chinese coronavirus acute severe disease", "illness caused by sars coronavirus2 2019", "illness from 2019 new sarscov2", "2019new wuhan seafood market pneumonia virus severe acute disease", "pneumonia due to 2019 new sars corona virus2", "2019novel wuhan corona virus infection", "disease from 2019new severe acute respiratory syndromerelated coronavirus2", "illness due to 2019new wuhancoronavirus", "disease from 2019 wuhancoronavirus", "2019new wuhan virus acute respiratory infection", "2019 novel severe acute respiratory syndromerelated corona virus2 illness", "2019new seafood market corona virus acute severe respiratory syndrome", "disorder due to virus causing covid19", "wuhan seafood market pneumonia virus acute respiratory syndrome", "pneumonia due to severe acute respiratory syndromerelated coronavirus2", "disorder from seafood market coronavirus", "2019 new sars coronavirus 2 severe acute respiratory syndrome", "2019 new seafood market virus severe respiratory illness", "2019new sars corona virus 2 acute severe syndrome", "pneumonia caused by severe acute respiratory syndrome coronavirus2 2019", "wuhan seafood market pneumonia virus acute severe respiratory disease", "2019 sarscov2 severe respiratory infection", "illness due to 2019 new chinese coronavirus", "wuhan corona virus acute disease", "2019novel wuhancoronavirus acute syndrome", "pneumonia from 2019new sars coronavirus 2", "2019novel wuhan coronavirus severe acute disease", "disorder due to wuhan corona virus 2019", "2019 novel chinese corona virus severe acute disease", "2019new wuhan seafood market virus acute infection", "disease due to 2019 new severe acute respiratory syndromerelated coronavirus2", "2019 novel seafood market virus acute infection", "disorder due to wuhancorona virus", "covid 19 virus severe respiratory infection", "2019ncov acute disorder", "2019novel sars coronavirus2 severe syndrome", "novel corona virus respiratory disorder", "disorder due to sarscov2", "chinese corona virus 2019 acute disease", "ncp", "sars coronavirus 2 acute disease", "sars coronavirus2 severe acute illness", "wuhancorona virus 2019 severe infection", "2019novel severe acute respiratory syndrome corona virus 2 pneumonia", "2019 sars coronavirus 2 severe acute syndrome", "2019 new sars corona virus2 acute severe respiratory syndrome", "2019 wuhancorona virus acute severe illness", "2019novel sars corona virus2 severe disease", "syndrome caused by wuhan pneumonia virus 2019", "chinese corona virus 2019 acute severe disorder", "syndrome due to 2019new sars corona virus2", "2019 novel seafood market virus acute illness", "illness from virus responsible for covid19", "2019 sars coronavirus 2 severe disease", "2019new wuhan seafood market pneumonia virus severe respiratory illness", "sars corona virus2 2019", "2019 new chinese coronavirus acute disorder", "human coronavirus 2019 severe acute illness", "sars cov 2 2019 respiratory infection", "2019 sars corona virus2 severe respiratory syndrome", "pneumonia due to 2019 new chinese coronavirus", "syndrome caused by 2019novel wuhan virus", "wuhancorona virus 2019 acute severe respiratory infection", "2019novel sars coronavirus 2 acute respiratory infection", "wuhan seafood market pneumonia virus severe syndrome", "illness caused by 2019 new wuhancoronavirus", "2019novel severe acute respiratory syndromerelated coronavirus2 pneumonia", "2019 novel wuhan virus acute severe disorder", "disorder from 2019novel sars corona virus 2", "disorder caused by sars corona virus 2 2019", "pneumonia due to 2019novel sars coronavirus2", "2019 novel seafood market coronavirus severe acute infection", "2019 novel wuhancoronavirus severe acute disorder", "seafood market corona virus severe acute pneumonia", "infection due to severe acute respiratory syndrome coronavirus 2 2019", "2019 wuhan seafood market pneumonia virus acute severe respiratory illness", "disease from 2019new sars cov 2", "infection from sars corona virus 2", "2019 new sars coronavirus 2 respiratory disorder", "2019 novel wuhancoronavirus acute severe pneumonia", "disorder from 2019 new sars cov 2", "2019 new coronavirus acute severe infection", "severe acute respiratory syndrome coronavirus 2 2019 disease", "2019 wuhan coronavirus respiratory syndrome", "seafood market acute respiratory disease", "2019 wuhan pneumonia virus acute respiratory infection", "2019 sars corona virus 2 acute syndrome", "2019 new chinese coronavirus acute severe respiratory illness", "2019 sars coronavirus2 acute respiratory infection", "chinese coronavirus acute respiratory disease", "chinese coronavirus severe acute infection", "2019novel wuhancorona virus severe respiratory disease", "pneumonia due to 2019 new severe acute respiratory syndrome corona virus 2", "2019 new sars cov 2 respiratory illness", "illness caused by severe acute respiratory syndromerelated coronavirus2 2019", "2019ncov acute severe syndrome", "pneumonia from 2019 new sars corona virus 2", "novel corona virus acute severe infection", "2019 new wuhan coronavirus severe acute respiratory infection", "sarscov2 2019 acute respiratory syndrome", "2019 new wuhan coronavirus severe acute illness", "2019 new seafood market coronavirus severe acute disease", "infection due to severe acute respiratory syndromerelated coronavirus2", "seafood market coronavirus acute respiratory disease", "sars coronavirus2 acute severe respiratory syndrome", "sars corona virus2 2019 acute respiratory disease", "2019 wuhancoronavirus disease", "2019novel wuhan pneumonia virus severe acute respiratory disorder", "2019 new wuhan seafood market virus severe infection", "novel coronavirus severe acute pneumonia", "wuhancorona virus 2019 acute severe respiratory illness", "2019 new sars corona virus 2 acute severe respiratory illness", "disorder due to chinese coronavirus", "2019 novel seafood market virus acute disease", "2019 novel wuhancoronavirus severe pneumonia", "2019new seafood market corona virus acute respiratory illness", "2019novel sarscov2 acute infection", "wuhan seafood market pneumonia virus respiratory illness", "infection from 2019novel wuhan virus", "2019 novel wuhan seafood market virus acute disorder", "2019 new sars coronavirus2 acute respiratory infection", "2019new wuhan corona virus respiratory disease", "2019new wuhan pneumonia virus acute severe syndrome", "2019 seafood market coronavirus severe acute respiratory disorder", "virus causing covid19 acute severe pneumonia", "wuhancorona virus acute severe disease", "2019 novel wuhan seafood market pneumonia virus respiratory disorder", "2019 new sars corona virus 2 severe acute illness", "disorder due to 2019 severe acute respiratory syndrome coronavirus 2", "2019novel sars corona virus 2 acute disease", "2019 sars cov 2 severe acute disorder", "human corona virus 2019 acute severe respiratory disease", "2019novel sars coronavirus 2 severe acute infection", "2019novel seafood market corona virus acute severe syndrome", "sars corona virus2 2019 severe pneumonia", "syndrome caused by 2019 ncov", "pneumonia due to novel corona virus 2019", "2019 new wuhan virus severe illness", "2019 novel wuhan virus severe respiratory disease", "2019 sars corona virus2 acute respiratory syndrome", "2019 sarscov2 acute disease", "illness caused by 2019 novel severe acute respiratory syndrome corona virus 2", "illness from 2019new severe acute respiratory syndrome corona virus2", "wuhan seafood market virus disorder", "seafood market coronavirus acute severe respiratory syndrome", "2019 new severe acute respiratory syndrome corona virus2 illness", "2019 novel severe acute respiratory syndromerelated corona virus2 disorder", "disorder due to 2019novel wuhancorona virus", "disorder from 2019 new sars corona virus2", "chinese corona virus respiratory disorder", "2019 novel sars coronavirus2 acute pneumonia", "2019 novel sars coronavirus2 severe acute illness", "2019 new wuhan corona virus severe respiratory infection", "illness caused by novel coronavirus 2019", "2019 seafood market corona virus disease", "2019novel chinese coronavirus severe disorder", "2019 novel wuhancoronavirus acute severe respiratory illness", "disorder due to wuhan seafood market pneumonia virus", "sars coronavirus2 respiratory infection", "2019 new wuhan corona virus acute respiratory disease", "seafood market acute syndrome", "human coronavirus 2019 severe disease", "infection due to sars corona virus2", "2019 novel sarscov2 severe respiratory syndrome", "wuhan seafood market pneumonia virus 2019 severe pneumonia", "2019new wuhan seafood market virus acute severe respiratory disorder", "illness caused by 2019 coronavirus", "sars corona virus2 2019 acute respiratory syndrome", "disorder caused by 2019 sars corona virus 2", "disorder caused by severe acute respiratory syndromerelated corona virus 2", "2019 new wuhan corona virus severe respiratory syndrome", "2019novel wuhancorona virus severe acute respiratory disorder", "2019 new sars corona virus2 acute severe respiratory disorder", "infection caused by seafood market coronavirus", "syndrome due to 2019novel wuhan seafood market virus", "wuhan severe respiratory disorder", "infection due to 2019 novel seafood market coronavirus", "infection caused by severe acute respiratory syndromerelated corona virus2", "2019 severe acute respiratory syndrome corona virus2", "disease from severe acute respiratory syndrome coronavirus 2 2019", "sars corona virus2 2019 acute severe infection", "2019 novel wuhan pneumonia virus severe respiratory syndrome", "wuhan seafood market virus severe disease", "2019 new wuhan seafood market virus acute severe respiratory syndrome", "severe acute respiratory syndrome coronavirus 2 2019 infection", "new coronavirus 2019 acute respiratory illness", "sars corona virus2 2019 severe disease", "2019 sars corona virus 2 respiratory disorder", "2019 human coronavirus severe respiratory infection", "2019novel wuhan pneumonia virus acute severe infection", "2019 novel wuhan coronavirus respiratory infection", "wuhan virus asevere acute respiratory disease", "2019new chinese corona virus severe disease", "2019 severe acute respiratory syndrome corona virus2 infection", "pneumonia due to 2019novel seafood market virus", "infection due to 2019 novel severe acute respiratory syndromerelated coronavirus2", "illness caused by sars corona virus2", "seafood market virus 2019 acute respiratory disease", "disease due to 2019 new severe acute respiratory syndrome corona virus2", "2019 coronavirus acute infection", "virus responsible for covid19 acute syndrome", "2019 novel wuhancorona virus severe respiratory syndrome", "illness due to 2019 corona virus", "illness from 2019 new chinese corona virus", "disorder due to 2019 new chinese corona virus", "hcov19 severe acute pneumonia", "seafood market coronavirus acute disorder", "ICD11:RA01.0", "wuhan coronavirus severe acute respiratory disorder", "pneumonia caused by 2019novel wuhan pneumonia virus", "2019 sars coronavirus 2 severe acute illness", "2019novel wuhan pneumonia virus acute severe respiratory illness", "2019 new coronavirus severe acute syndrome", "2019new seafood market coronavirus acute severe syndrome", "syndrome caused by 2019 novel seafood market coronavirus", "2019novel seafood market coronavirus", "2019 new seafood market virus severe syndrome", "2019novel sarscov2 acute pneumonia", "sars corona virus 2 2019 severe disorder", "2019 new seafood market coronavirus infection", "wuhancoronavirus 2019 severe respiratory disease", "2019 new chinese corona virus severe respiratory syndrome", "human corona virus 2019 severe acute respiratory illness", "2019 sars corona virus2", "2019novel sars cov 2 respiratory infection", "disease due to 2019novel severe acute respiratory syndromerelated coronavirus2", "seafood market virus severe illness", "illness caused by 2019 seafood market virus", "2019novel sars corona virus2 acute severe syndrome", "2019 human corona virus severe respiratory illness", "2019new wuhan corona virus acute respiratory illness", "sars corona virus 2 2019 acute severe respiratory disorder", "syndrome due to 2019 novel wuhan virus", "disorder from 2019 sars coronavirus 2", "2019new severe acute respiratory syndromerelated coronavirus2", "2019 new wuhan virus disease", "illness due to new corona virus", "disease due to 2019 new wuhan seafood market pneumonia virus", "disease due to 2019new severe acute respiratory syndrome coronavirus 2", "syndrome due to 2019 novel seafood market coronavirus", "2019new wuhancorona virus severe disorder", "infection from 2019 new wuhan virus", "2019 new sars coronavirus2 respiratory disease", "2019 novel seafood market corona virus severe infection", "pneumonia due to 2019 new wuhan seafood market pneumonia virus", "infection from novel corona virus", "wuhan corona virus acute severe respiratory disorder", "2019 novel sars cov 2 disorder", "2019new sars corona virus2 acute infection", "2019 new chinese coronavirus severe disorder", "2019 wuhan corona virus acute severe illness", "disease due to 2019 novel severe acute respiratory syndrome coronavirus2", "2019 new chinese coronavirus severe disease", "disorder due to novel coronavirus", "disorder due to wuhan seafood market virus 2019", "infection caused by 2019 novel sars coronavirus 2", "2019 novel severe acute respiratory syndromerelated corona virus 2 disease", "2019 new seafood market coronavirus acute illness", "2019new wuhancoronavirus severe disorder", "wuhan pneumonia virus acute respiratory illness", "2019novel wuhan seafood market virus acute severe respiratory syndrome", "infection from seafood market corona virus 2019", "wuhan seafood market pneumonia virus acute severe infection", "2019 novel sars coronavirus2 respiratory disorder", "2019 wuhan virus acute severe respiratory syndrome", "2019ncov acute severe respiratory infection", "infection caused by 2019novel sars coronavirus2", "2019novel chinese coronavirus acute illness", "infection due to severe acute respiratory syndrome corona virus2", "2019new sars corona virus2 severe pneumonia", "illness from wuhan seafood market virus", "pneumonia from severe acute respiratory syndrome coronavirus2 2019", "wuhan coronavirus acute infection", "2019 wuhancorona virus acute severe respiratory disorder", "pneumonia caused by human coronavirus 2019", "2019 wuhan virus acute respiratory illness", "illness from 2019 severe acute respiratory syndrome corona virus 2", "disorder from 2019new wuhan seafood market virus", "2019 wuhan coronavirus acute respiratory disorder", "2019 new wuhancoronavirus severe acute respiratory infection", "wuhan seafood market pneumonia virus acute syndrome", "2019 novel coronavirus severe pneumonia", "infection due to 2019novel severe acute respiratory syndromerelated coronavirus2", "wuhan virus 2019 severe respiratory disease", "2019 novel sars corona virus 2", "pneumonia from sars coronavirus 2 2019", "sars cov 2 disorder", "2019new sars coronavirus2 severe infection", "virus responsible for covid19 acute respiratory syndrome", "disorder due to 2019new severe acute respiratory syndromerelated corona virus 2", "2019novel sars coronavirus2 acute infection", "2019 novel sars corona virus 2 infection", "hcov19 acute pneumonia", "pneumonia from 2019novel sars cov 2", "syndrome from 2019novel sars cov 2", "pneumonia from 2019 new chinese corona virus", "pneumonia from 2019novel sars corona virus 2", "2019novel sars cov 2 acute severe infection", "2019 sars coronavirus 2 acute respiratory disease", "2019 new sars corona virus2 severe respiratory infection", "infection due to 2019 coronavirus", "novel coronavirus 2019 acute respiratory disease", "chinese coronavirus 2019", "wuhan seafood market virus 2019 severe respiratory disease", "2019 chinese corona virus severe respiratory disorder", "illness due to seafood market virus", "new corona virus 2019 syndrome", "disease due to 2019 human coronavirus", "sars cov 2 severe respiratory disorder", "2019novel sars coronavirus 2 acute severe disorder", "2019new sars cov 2 illness", "illness from wuhan virus 2019", "chinese corona virus 2019 respiratory syndrome", "2019novel seafood market corona virus severe acute infection", "infection caused by 2019 novel seafood market coronavirus", "2019 novel wuhan pneumonia virus acute infection", "2019 new wuhancorona virus severe acute pneumonia", "2019new wuhan virus severe syndrome", "wuhancoronavirus 2019 acute severe infection", "hcov19 acute severe respiratory disorder", "2019 new severe acute respiratory syndromerelated corona virus2 infection", "disease due to 2019 novel coronavirus", "2019new chinese coronavirus acute severe disorder", "2019 novel wuhancorona virus acute severe pneumonia", "pneumonia due to 2019novel wuhan seafood market pneumonia virus", "disease caused by 2019 chinese corona virus", "2019 new wuhan pneumonia virus acute respiratory syndrome", "2019 new wuhan seafood market virus acute respiratory illness", "seafood market coronavirus 2019 pneumonia", "2019new seafood market virus severe respiratory illness", "wuhan virus severe acute respiratory syndrome", "new corona virus severe disorder", "disease from 2019 new wuhan corona virus", "illness due to wuhan coronavirus 2019", "illness due to covid 19 virus", "syndrome due to sarscov2", "2019 new sars corona virus2 severe acute respiratory illness", "pneumonia caused by 2019 novel severe acute respiratory syndrome coronavirus2", "infection from 2019 new sars corona virus 2", "2019novel wuhan seafood market virus severe acute respiratory disorder", "2019new sars coronavirus2 severe respiratory disease", "2019 novel wuhan seafood market pneumonia virus acute severe respiratory illness", "syndrome due to 2019 wuhan pneumonia virus", "disease due to 2019novel chinese corona virus", "2019novel seafood market virus pneumonia", "2019 wuhan pneumonia virus pneumonia", "2019 new wuhan corona virus severe disorder", "sars corona virus2 2019 respiratory disorder", "2019 new wuhancoronavirus severe acute respiratory illness", "disorder caused by 2019novel severe acute respiratory syndromerelated corona virus2", "2019 new chinese corona virus acute respiratory disease", "syndrome due to 2019new wuhan coronavirus", "disorder due to sars corona virus 2 2019", "disease due to wuhan pneumonia virus", "disorder due to 2019 seafood market virus", "infection from 2019novel sars coronavirus 2", "2019 new wuhancoronavirus acute severe disease", "2019 novel seafood market coronavirus illness", "2019 severe acute respiratory syndrome corona virus 2 infection", "disorder from wuhan pneumonia virus", "new corona virus severe infection", "2019new wuhan corona virus respiratory disorder", "chinese corona virus acute severe infection", "2019 new sarscov2 acute severe disease", "2019 wuhancoronavirus acute severe disorder", "2019 novel sarscov2 acute infection", "pneumonia from severe acute respiratory syndromerelated coronavirus2 2019", "2019 new corona virus severe acute respiratory illness", "wuhan corona virus infection", "2019new seafood market corona virus disorder", "2019ncov acute pneumonia", "2019 novel wuhan seafood market virus severe respiratory syndrome", "sars corona virus2 2019 acute pneumonia", "wuhan severe disease", "disorder due to 2019 wuhan seafood market virus", "2019 wuhancorona virus severe respiratory syndrome", "pneumonia from wuhan virus", "pneumonia caused by 2019 severe acute respiratory syndromerelated coronavirus 2", "ICD10: U07.1", "2019novel wuhan corona virus severe infection", "2019 sarscov2 disorder", "2019 new sars corona virus 2 acute disorder", "2019 wuhan corona virus respiratory disorder", "covid 19 virus severe acute respiratory disorder", "2019new wuhan virus acute severe disease", "infection due to 2019 novel corona virus", "2019novel severe acute respiratory syndrome corona virus2 pneumonia", "sars cov 2 severe respiratory infection", "2019ncov severe acute illness", "seafood market severe acute respiratory syndrome", "illness caused by 2019 new wuhan pneumonia virus", "wuhan virus severe respiratory illness", "disorder caused by 2019 new severe acute respiratory syndrome coronavirus2", "syndrome due to 2019 new seafood market corona virus", "2019 novel wuhan pneumonia virus respiratory illness", "wuhan seafood market pneumonia virus acute severe respiratory illness", "disorder from virus responsible for covid19", "2019 novel severe acute respiratory syndrome coronavirus2 infection", "illness due to 2019 new sars coronavirus2", "wuhan virus severe respiratory syndrome", "2019 novel sars cov 2 acute respiratory infection", "2019 seafood market virus acute severe respiratory infection", "2019 novel sars corona virus2 severe acute infection", "2019new chinese corona virus pneumonia", "pneumonia due to wuhan seafood market virus", "seafood market corona virus respiratory disorder", "human corona virus 2019 acute disorder", "2019 sars corona virus 2 acute disease", "2019 novel coronavirus acute infection", "2019 novel sars corona virus2 severe syndrome", "wuhan syndrome", "2019novel sars coronavirus2 severe respiratory disease", "2019new wuhancorona virus severe acute respiratory disorder", "pneumonia caused by virus responsible for covid19", "infection caused by novel coronavirus", "wuhan coronavirus severe acute respiratory infection", "wuhan severe acute disease", "pneumonia from 2019novel sars coronavirus2", "hcov19 respiratory syndrome", "chinese coronavirus 2019 severe acute illness", "sars corona virus 2 respiratory syndrome", "disorder caused by 2019new severe acute respiratory syndromerelated coronavirus 2", "wuhan corona virus 2019 acute severe disorder", "2019new seafood market coronavirus acute syndrome", "2019new wuhan seafood market virus infection", "pneumonia due to severe acute respiratory syndrome corona virus2 2019", "2019 ncov disease", "2019 new severe acute respiratory syndrome corona virus2", "2019 novel wuhan pneumonia virus respiratory disorder", "virus causing covid19 severe acute illness", "2019 novel severe acute respiratory syndrome coronavirus2 illness", "illness from chinese coronavirus 2019", "illness from chinese corona virus 2019", "2019new sars coronavirus2 severe acute infection", "sars corona virus2 2019 acute disease", "2019ncov acute illness", "2019 sars corona virus 2 severe acute infection", "hcov19 acute severe syndrome", "disorder due to 2019novel severe acute respiratory syndromerelated coronavirus 2", "seafood market severe disorder", "2019 new seafood market virus acute respiratory disorder", "sars coronavirus2 2019 severe respiratory illness", "pneumonia from 2019 novel seafood market virus", "2019 new seafood market virus acute severe disorder", "infection due to 2019novel sars corona virus2", "novel corona virus severe illness", "pneumonia due to 2019 new sars corona virus 2", "2019 novel wuhan seafood market pneumonia virus acute disorder", "wuhan seafood market virus 2019 severe acute respiratory syndrome", "covid 19 virus severe respiratory syndrome", "infection due to wuhancoronavirus 2019", "2019novel sars corona virus 2 acute severe syndrome", "2019 new sars corona virus 2 acute respiratory disease", "disease due to virus causing covid19", "seafood market coronavirus illness", "disease caused by 2019 new severe acute respiratory syndromerelated coronavirus 2", "disease from 2019novel seafood market virus", "pneumonia from seafood market corona virus", "pneumonia due to 2019new wuhan corona virus", "illness from 2019 wuhan seafood market pneumonia virus", "sars coronavirus 2 2019 severe acute respiratory disorder", "pneumonia from 2019novel severe acute respiratory syndrome corona virus 2", "2019 novel wuhancoronavirus severe illness", "2019 new seafood market coronavirus acute syndrome", "2019novel wuhan seafood market pneumonia virus severe respiratory infection", "2019novel sarscov2 acute severe respiratory syndrome", "2019 novel sarscov2 acute severe disease", "infection due to 2019novel wuhan seafood market pneumonia virus", "disease from severe acute respiratory syndromerelated corona virus 2", "2019 wuhan seafood market virus asevere acute respiratory disease", "2019 severe acute respiratory syndrome coronavirus 2 pneumonia", "2019new sars cov 2 acute severe respiratory illness", "2019 sars coronavirus2 respiratory disorder", "illness due to 2019 new wuhan corona virus", "disorder due to 2019novel wuhan corona virus", "wuhan seafood market virus 2019 acute severe disorder", "infection from new corona virus", "wuhan coronavirus 2019 acute illness", "2019new wuhancorona virus acute pneumonia", "2019 human coronavirus asevere acute respiratory disease", "2019 novel wuhan corona virus severe acute respiratory syndrome", "wuhan corona virus 2019 severe acute respiratory illness", "disease from 2019new wuhan seafood market virus", "illness from sars coronavirus2", "wuhancoronavirus respiratory disorder", "disorder due to 2019 novel seafood market virus", "infection due to severe acute respiratory syndromerelated corona virus2 2019", "chinese coronavirus severe respiratory infection", "2019new wuhancoronavirus severe infection", "wuhancoronavirus severe syndrome", "pneumonia due to chinese coronavirus 2019", "2019 new wuhan seafood market virus respiratory disease", "illness from 2019novel wuhan virus", "disease due to severe acute respiratory syndrome coronavirus 2 2019", "2019 novel wuhan pneumonia virus", "disease from 2019 new wuhancoronavirus", "2019 novel chinese coronavirus acute respiratory syndrome", "2019new chinese corona virus respiratory syndrome", "2019 novel wuhan seafood market virus respiratory disorder", "2019new seafood market coronavirus acute severe respiratory disorder", "pneumonia caused by 2019 sars corona virus2", "disease from 2019 new seafood market virus", "2019 new corona virus infection", "2019 novel severe acute respiratory syndrome coronavirus2", "2019 new corona virus acute respiratory infection", "2019 novel chinese corona virus disease", "2019 new wuhan coronavirus acute severe pneumonia", "2019 new wuhan coronavirus", "2019 new sars coronavirus2 illness", "2019novel sars coronavirus2", "seafood market coronavirus 2019 disease", "2019new sars corona virus 2 acute illness", "2019novel wuhan virus severe acute infection", "2019 wuhancoronavirus severe acute respiratory syndrome", "wuhan virus 2019 severe disease", "2019 novel sars corona virus 2 severe acute syndrome", "2019 new chinese corona virus severe acute pneumonia", "covid19 virus", "infection due to 2019novel sars coronavirus2", "pneumonia caused by 2019novel severe acute respiratory syndromerelated coronavirus2", "disorder caused by severe acute respiratory syndrome corona virus 2 2019", "2019 novel sars coronavirus2 severe respiratory disorder", "human corona virus 2019", "disease due to 2019 novel wuhancoronavirus", "disorder caused by novel coronavirus 2019", "syndrome caused by 2019 new wuhan coronavirus", "2019 new coronavirus syndrome", "illness caused by severe acute respiratory syndrome coronavirus2", "2019 novel chinese coronavirus", "2019novel wuhancorona virus", "covid 19 virus severe disease", "wuhancoronavirus severe respiratory syndrome", "2019 novel sars coronavirus2 acute severe respiratory syndrome", "2019 wuhancorona virus acute severe respiratory disease", "infection caused by 2019 novel severe acute respiratory syndrome corona virus2", "2019 new sars cov 2 acute infection", "illness caused by 2019novel wuhan coronavirus", "novel coronavirus 2019 acute severe illness", "wuhan seafood market pneumonia virus acute severe respiratory disorder", "2019 novel wuhan seafood market pneumonia virus severe acute infection", "pneumonia due to 2019ncov", "2019 sars corona virus 2 pneumonia", "2019 novel wuhancorona virus severe acute disorder", "wuhancoronavirus severe pneumonia", "2019 novel sars cov 2 acute respiratory syndrome", "2019 novel wuhan coronavirus acute infection", "2019 new wuhan corona virus severe respiratory disease", "2019 chinese coronavirus acute severe respiratory syndrome", "sars corona virus2 acute respiratory syndrome", "new coronavirus severe respiratory disorder", "2019 new wuhan virus severe disease", "wuhancoronavirus severe acute respiratory infection", "2019 severe acute respiratory syndromerelated corona virus2 illness", "2019 new seafood market virus severe respiratory disorder", "sars cov 2 severe acute disorder", "seafood market virus disorder", "2019novel sars corona virus 2 acute illness", "hcov 19 severe respiratory disorder", "2019 sars corona virus2 severe disease", "2019novel wuhan seafood market virus severe acute infection", "disease caused by wuhancoronavirus 2019", "2019 novel seafood market coronavirus respiratory illness", "2019 corona virus severe acute syndrome", "2019 new chinese coronavirus severe acute illness", "2019 sars coronavirus2 acute respiratory disease", "seafood market corona virus respiratory infection", "2019new sars coronavirus2 acute respiratory illness", "new coronavirus acute respiratory disease", "disease caused by wuhan virus", "2019 novel sars cov 2 acute syndrome", "pneumonia caused by 2019new chinese coronavirus", "syndrome from 2019 novel wuhan seafood market pneumonia virus", "wuhancoronavirus acute severe respiratory infection", "2019 new corona virus severe acute pneumonia", "disease from 2019novel sars cov 2", "2019 new wuhan pneumonia virus respiratory disorder", "disorder from severe acute respiratory syndromerelated coronavirus 2", "2019new wuhan virus respiratory infection", "infection from sars coronavirus2 2019", "covid19 virus severe acute pneumonia", "disorder caused by 2019 new severe acute respiratory syndromerelated corona virus2", "2019new sars corona virus2 severe disease", "wuhancorona virus 2019 acute severe disease", "2019 chinese corona virus severe respiratory infection", "2019 seafood market corona virus acute severe disorder", "2019new seafood market corona virus severe acute respiratory disorder", "2019novel wuhan virus acute severe disease", "sarscov2 2019 severe respiratory disorder", "sars coronavirus2 severe acute disorder", "sars coronavirus 2 illness", "2019novel wuhan pneumonia virus severe infection", "2019 new sarscov2 severe respiratory disorder", "2019new wuhan virus acute disorder", "2019new sars coronavirus 2 acute disorder", "chinese coronavirus 2019 acute severe respiratory infection", "2019novel sars coronavirus 2 severe infection", "chinese coronavirus 2019 acute severe illness", "illness caused by 2019new wuhan virus", "infection caused by 2019 new severe acute respiratory syndrome corona virus 2", "2019 new seafood market virus respiratory disorder", "infection from 2019 new sars cov 2", "wuhancoronavirus 2019 disorder", "seafood market virus acute severe infection", "disorder from chinese corona virus", "2019 sars coronavirus 2 severe acute disorder", "2019 sars coronavirus2 illness", "2019 novel sars corona virus 2 severe respiratory disease", "2019 ncov acute respiratory infection", "pneumonia from 2019 seafood market corona virus", "2019 new wuhancorona virus severe acute respiratory illness", "2019 wuhan seafood market pneumonia virus syndrome", "wuhan seafood market virus severe syndrome", "wuhan corona virus acute syndrome", "2019 novel seafood market coronavirus acute respiratory disease", "wuhan pneumonia virus 2019 severe respiratory illness", "2019 novel sarscov2 severe acute respiratory disorder", "2019novel sars corona virus 2 severe acute syndrome", "2019novel sars corona virus2 acute respiratory disorder", "2019 wuhancoronavirus acute respiratory infection", "infection caused by severe acute respiratory syndromerelated coronavirus2 2019", "2019 wuhan corona virus severe acute illness", "pneumonia caused by 2019 new sars cov 2", "seafood market virus 2019 acute severe respiratory infection", "2019 new seafood market virus severe pneumonia", "2019 novel wuhan seafood market pneumonia virus acute disease", "wuhan pneumonia virus 2019 acute severe disease", "infection from seafood market corona virus", "illness from 2019 novel sars corona virus2", "2019 sars corona virus 2 acute severe respiratory syndrome", "pneumonia from 2019 severe acute respiratory syndrome corona virus 2", "seafood market virus acute infection", "pneumonia from 2019new sars corona virus2", "2019novel wuhan seafood market virus pneumonia", "human corona virus 2019 severe syndrome", "sarscov2 2019 severe acute respiratory disorder", "2019 corona virus illness", "2019 corona virus severe disorder", "disease caused by 2019novel sarscov2", "sars corona virus 2 2019 severe respiratory disorder", "2019new seafood market virus severe acute respiratory illness", "2019 novel sars corona virus 2 acute respiratory infection", "infection from wuhan coronavirus", "2019novel sars corona virus2 severe acute respiratory infection", "2019novel wuhan seafood market virus acute severe disorder", "2019 novel chinese coronavirus severe acute respiratory disorder", "pneumonia due to 2019new seafood market coronavirus", "2019 new seafood market virus acute syndrome", "2019new seafood market virus acute severe illness", "pneumonia due to 2019 wuhan coronavirus", "2019new seafood market corona virus acute severe syndrome", "2019 sars corona virus 2 acute severe respiratory disorder", "sars corona virus2 2019 acute severe illness", "illness due to novel coronavirus", "infection from 2019 chinese corona virus", "pneumonia due to 2019new sarscov2", "2019 new severe acute respiratory syndromerelated corona virus 2 disease", "illness from severe acute respiratory syndromerelated corona virus 2 2019", "2019 new sarscov2 severe respiratory disease", "2019 wuhan pneumonia virus severe illness", "2019novel sars corona virus2 severe acute respiratory illness", "2019 novel seafood market virus severe acute disease", "wuhan seafood market virus illness", "2019new sars cov 2 acute severe disease", "disorder caused by severe acute respiratory syndromerelated coronavirus2 2019", "2019 wuhancoronavirus severe acute respiratory disorder", "2019novel severe acute respiratory syndromerelated corona virus2 pneumonia", "2019novel sars coronavirus 2 severe disorder", "disorder from new corona virus 2019", "infection from sars corona virus 2 2019", "2019 novel wuhan seafood market virus acute respiratory infection", "seafood market pneumonia", "2019novel wuhan seafood market pneumonia virus acute respiratory disease", "2019 sarscov2 severe syndrome", "2019 ncov acute severe respiratory disorder", "2019 severe acute respiratory syndromerelated corona virus2 disorder", "virus responsible for covid19 severe respiratory disorder", "2019 novel wuhan seafood market pneumonia virus respiratory syndrome", "infection caused by severe acute respiratory syndromerelated corona virus 2", "pneumonia caused by 2019new sars corona virus2", "2019new sarscov2 severe acute respiratory syndrome", "wuhan corona virus 2019 acute severe respiratory infection", "covid19 virus acute severe respiratory syndrome", "illness caused by 2019 novel coronavirus", "2019novel wuhan seafood market pneumonia virus severe acute infection", "2019 novel wuhan corona virus illness", "sarscov2 2019 acute disease", "sars corona virus2 2019 respiratory illness", "hubei respiratory disease", "2019 novel wuhancorona virus severe acute pneumonia", "disease caused by novel coronavirus 2019", "wuhancoronavirus 2019 severe disease", "sars corona virus 2 respiratory illness", "novel coronavirus acute severe respiratory disease", "2019novel wuhan pneumonia virus acute severe respiratory disease", "wuhancoronavirus acute illness", "sars coronavirus2 2019 acute severe respiratory disorder", "hcov 19 disorder", "illness caused by 2019 novel sars coronavirus2", "seafood market corona virus 2019 respiratory disease", "2019 severe acute respiratory syndromerelated corona virus2 infection", "2019new wuhan pneumonia virus severe infection", "syndrome caused by 2019 new sars corona virus 2", "pneumonia due to 2019 novel sars corona virus 2", "novel corona virus 2019 disorder", "2019 new sars cov 2 severe respiratory disease", "syndrome due to 2019 novel wuhan corona virus", "wuhan pneumonia virus 2019 acute respiratory syndrome", "sars corona virus 2 illness", "2019 novel sarscov2 acute severe infection", "2019novel wuhan seafood market virus acute severe respiratory illness", "disorder caused by seafood market coronavirus", "infection from 2019 new wuhan seafood market pneumonia virus", "hubei acute illness", "wuhancorona virus infection", "2019 new wuhan coronavirus syndrome", "2019 new seafood market virus acute severe respiratory disease", "2019 novel wuhan corona virus severe acute respiratory infection", "2019 novel wuhan coronavirus acute severe illness", "pneumonia from new corona virus 2019", "2019 new chinese coronavirus acute disease", "infection from 2019 new severe acute respiratory syndromerelated corona virus2", "chinese coronavirus respiratory disorder", "2019new chinese coronavirus acute illness", "infection due to 2019 new seafood market virus", "sars coronavirus2 2019 acute severe respiratory infection", "severe acute respiratory syndromerelated coronavirus 2 2019 pneumonia", "2019 coronavirus acute syndrome", "2019 wuhan seafood market virus severe acute disease", "2019 new wuhancoronavirus disease", "human corona virus 2019 acute illness", "2019 wuhan virus severe acute respiratory infection", "sars coronavirus 2 severe respiratory syndrome", "sars coronavirus 2 2019 severe acute disorder", "2019new chinese corona virus severe acute syndrome", "human coronavirus 2019 severe acute respiratory disorder", "disorder caused by 2019novel severe acute respiratory syndrome corona virus2", "syndrome from 2019new sarscov2", "chinese corona virus severe illness", "2019 wuhancorona virus severe acute syndrome", "2019 novel sars corona virus 2 severe acute disorder", "2019novel severe acute respiratory syndromerelated corona virus 2 infection", "infection caused by 2019novel seafood market corona virus", "2019 novel chinese corona virus severe respiratory disorder", "disease caused by sars corona virus2 2019", "pneumonia due to 2019novel seafood market corona virus", "2019new sars corona virus2 severe illness", "seafood market corona virus 2019 acute severe respiratory disorder", "syndrome due to 2019 new wuhan seafood market pneumonia virus", "disorder from 2019new seafood market corona virus", "pneumonia due to 2019 novel chinese corona virus", "2019 wuhan virus severe acute respiratory illness", "seafood market coronavirus acute severe infection", "sars coronavirus 2 2019 acute pneumonia", "wuhan coronavirus acute respiratory syndrome", "sarscov2 2019 severe acute illness", "2019novel wuhan corona virus acute severe illness", "2019novel seafood market corona virus acute severe pneumonia", "2019 sars coronavirus 2 severe infection", "2019 novel sarscov2 acute respiratory disorder", "disorder caused by 2019 novel severe acute respiratory syndrome coronavirus 2", "chinese coronavirus", "pneumonia caused by 2019new chinese corona virus", "novel corona virus 2019 acute syndrome", "2019 novel sars cov 2 asevere acute respiratory disease", "2019 new wuhan coronavirus severe disorder", "illness due to 2019 chinese corona virus", "2019 wuhan seafood market virus", "2019 seafood market corona virus acute severe respiratory disease", "hubei severe respiratory infection", "infection from chinese coronavirus", "2019 sars cov 2 severe respiratory disorder", "chinese coronavirus 2019 respiratory disease", "disease due to wuhan virus", "human corona virus 2019 acute infection", "illness caused by 2019novel sars corona virus2", "infection caused by 2019new seafood market coronavirus", "human coronavirus 2019 severe respiratory disorder", "syndrome from wuhan seafood market virus", "illness from 2019 novel seafood market coronavirus", "2019novel sars corona virus2 severe acute respiratory disorder", "disease due to 2019novel seafood market coronavirus", "2019 new sars corona virus 2 acute severe pneumonia", "disorder caused by 2019 sars cov 2", "infection caused by 2019novel sarscov2", "pneumonia from severe acute respiratory syndromerelated corona virus2", "2019 chinese coronavirus severe acute disorder", "2019 wuhancoronavirus acute disorder", "2019 ncov severe disease", "2019new wuhan virus severe respiratory disorder", "infection from 2019 novel coronavirus", "illness due to 2019 new sars corona virus2", "wuhan pneumonia virus severe respiratory disease", "syndrome caused by 2019 new sars corona virus2", "disease from 2019 novel wuhan coronavirus", "2019novel sarscov2 infection", "wuhan corona virus 2019 severe acute respiratory infection", "2019 seafood market virus acute severe respiratory illness", "disease due to severe acute respiratory syndromerelated coronavirus2 2019", "2019new seafood market virus acute pneumonia", "sars coronavirus2 severe acute infection", "2019 wuhan coronavirus severe acute pneumonia", "wuhancoronavirus 2019 severe acute syndrome", "2019 novel seafood market virus severe acute disorder", "2019new wuhan pneumonia virus severe respiratory disorder", "syndrome from 2019 coronavirus", "2019 wuhan coronavirus acute severe respiratory syndrome", "2019new wuhancorona virus acute respiratory infection", "2019novel wuhancorona virus acute severe disorder", "syndrome due to 2019 new seafood market coronavirus", "syndrome from sars corona virus 2", "2019 sars corona virus 2 acute severe disease", "disorder due to 2019 severe acute respiratory syndromerelated corona virus2", "2019new sars cov 2 acute severe respiratory disorder", "2019 novel seafood market corona virus severe respiratory illness", "hcov19 disorder", "2019novel seafood market coronavirus acute respiratory disorder", "human coronavirus 2019 severe respiratory illness", "2019novel seafood market corona virus severe acute respiratory illness", "sars coronavirus2 2019 acute severe infection", "2019new severe acute respiratory syndrome coronavirus 2 illness", "2019 new chinese coronavirus acute severe infection", "2019 new wuhan corona virus severe acute respiratory illness", "disease due to 2019 wuhancoronavirus", "2019 new wuhancorona virus acute pneumonia", "2019 chinese corona virus severe acute respiratory syndrome", "sars coronavirus 2 severe acute respiratory infection", "2019 new sars corona virus2 severe respiratory disorder", "2019 corona virus acute syndrome", "sars coronavirus2 acute pneumonia", "2019 coronavirus acute severe respiratory syndrome", "disorder caused by 2019 new wuhan seafood market pneumonia virus", "2019new wuhan pneumonia virus severe syndrome", "disease from 2019 wuhan virus", "2019 novel wuhancorona virus severe acute syndrome", "human coronavirus 2019 disease", "2019new seafood market virus severe syndrome", "2019 wuhancoronavirus severe acute illness", "hcov19 acute respiratory infection", "2019novel severe acute respiratory syndrome corona virus2 illness", "2019novel sars corona virus 2 acute severe respiratory illness", "2019new wuhancoronavirus acute severe disorder", "2019 seafood market corona virus acute illness", "2019 coronavirus severe acute respiratory disorder", "disorder due to human corona virus 2019", "2019novel wuhan corona virus respiratory syndrome", "pneumonia from 2019 novel severe acute respiratory syndromerelated corona virus2", "2019novel seafood market coronavirus severe pneumonia", "2019novel wuhan virus acute respiratory disease", "wuhan pneumonia virus acute severe pneumonia", "severe acute respiratory syndromerelated coronavirus2 disease", "2019 sars cov 2 severe syndrome", "sars cov 2 2019 severe disorder", "novel corona virus 2019 severe acute infection", "2019 novel wuhancorona virus acute severe respiratory disorder", "2019 seafood market coronavirus severe infection", "illness caused by 2019new chinese coronavirus", "2019new wuhan pneumonia virus severe acute disorder", "2019 novel seafood market virus severe respiratory infection", "pneumonia caused by 2019 novel wuhan virus", "pneumonia caused by severe acute respiratory syndromerelated corona virus2 2019", "pneumonia from novel corona virus 2019", "2019 novel wuhan seafood market pneumonia virus acute respiratory disorder", "2019novel severe acute respiratory syndromerelated corona virus2 illness", "pneumonia from 2019new sars coronavirus2", "illness due to 2019 new corona virus", "sarscov2 2019 severe respiratory infection", "2019 new severe acute respiratory syndromerelated coronavirus 2 infection", "seafood market coronavirus acute severe disease", "2019 new sarscov2 respiratory disease", "pneumonia caused by chinese corona virus 2019", "infection from 2019 new severe acute respiratory syndrome corona virus2", "virus responsible for covid19 severe respiratory infection", "2019new sars cov 2 acute respiratory disease", "infection caused by 2019 new sars coronavirus 2", "2019new chinese corona virus respiratory illness", "sars coronavirus 2 2019 severe acute respiratory infection", "infection from 2019new wuhan pneumonia virus", "2019 novel wuhan corona virus disorder", "2019 novel wuhan coronavirus disorder", "2019 chinese coronavirus severe respiratory illness", "syndrome caused by 2019new wuhan pneumonia virus", "2019new wuhan corona virus severe acute illness", "disorder caused by chinese corona virus 2019", "wuhancorona virus 2019 severe acute syndrome", "2019 novel coronavirus acute pneumonia", "2019new wuhan virus severe disease", "2019novel wuhan pneumonia virus asevere acute respiratory disease", "2019new sars cov 2 infection", "2019 novel severe acute respiratory syndromerelated corona virus2", "disorder from severe acute respiratory syndromerelated corona virus 2 2019", "syndrome from 2019 seafood market coronavirus", "sars coronavirus 2 2019 syndrome", "wuhan coronavirus 2019 acute infection", "2019new wuhan seafood market virus respiratory disorder", "2019new sars corona virus 2 severe acute disease", "virus responsible for covid19 severe disease", "disease due to 2019 new sars coronavirus2", "2019new wuhancoronavirus illness", "human corona virus 2019 acute severe pneumonia", "disease from 2019 wuhan seafood market virus", "sars coronavirus 2 2019 severe acute illness", "hcov 19 severe acute respiratory disorder", "2019 novel sars coronavirus2 severe disease", "2019novel sars corona virus2 acute respiratory syndrome", "wuhan seafood market virus 2019 acute severe respiratory syndrome", "disease due to 2019 new sars coronavirus 2", "2019 wuhan virus acute severe respiratory illness", "sars coronavirus 2 2019 infection", "wuhancorona virus 2019 pneumonia", "2019 wuhan corona virus severe disorder", "2019 novel wuhancorona virus severe syndrome", "disorder caused by novel coronavirus", "2019new seafood market virus acute illness", "2019 novel wuhan corona virus severe acute respiratory illness", "2019novel wuhan seafood market pneumonia virus severe infection", "2019 sars cov 2 severe acute syndrome", "illness due to 2019 novel severe acute respiratory syndrome coronavirus 2", "2019 sars cov 2 acute pneumonia", "2019 wuhan corona virus severe acute pneumonia", "2019new wuhan virus acute pneumonia", "seafood market coronavirus 2019 severe acute disorder", "2019novel sars coronavirus2 asevere acute respiratory disease", "2019 novel sars corona virus 2 severe syndrome", "2019 new seafood market coronavirus acute severe respiratory illness", "covid 19 virus asevere acute respiratory disease", "covid 19 virus acute severe pneumonia", "2019 new wuhan corona virus severe acute disorder", "2019novel seafood market corona virus respiratory disorder", "wuhan seafood market virus 2019 respiratory disease", "disease caused by 2019new seafood market corona virus", "2019 novel wuhan pneumonia virus severe acute infection", "2019 novel wuhan corona virus acute severe syndrome", "pneumonia from human coronavirus 2019", "sars coronavirus 2 respiratory disorder", "new corona virus 2019 disease", "2019 new wuhancoronavirus acute respiratory disorder", "syndrome from 2019new wuhancoronavirus", "2019novel sars corona virus 2 severe acute pneumonia", "2019novel severe acute respiratory syndrome corona virus 2", "2019novel sars coronavirus2 acute respiratory disorder", "disorder from covid 19 virus", "2019novel wuhan seafood market virus acute severe syndrome", "2019 new sars corona virus 2 severe acute infection", "2019 new sarscov2 severe pneumonia", "sars corona virus2 2019 acute disorder", "infection caused by 2019 new sarscov2", "2019 new sars corona virus2 acute illness", "pneumonia from 2019new severe acute respiratory syndromerelated corona virus2", "sars corona virus 2 2019 acute severe respiratory illness", "2019novel wuhan coronavirus acute severe disorder", "disorder from 2019 new wuhan seafood market pneumonia virus", "2019 new coronavirus asevere acute respiratory disease", "2019 new wuhan seafood market virus severe acute respiratory disorder", "2019novel wuhan corona virus respiratory illness", "2019 new chinese corona virus respiratory illness", "2019novel sars coronavirus 2 acute respiratory illness", "wuhan corona virus illness", "2019novel wuhancorona virus respiratory disease", "2019 wuhan pneumonia virus acute severe syndrome", "covid 19 virus acute severe respiratory infection", "wuhancorona virus syndrome", "infection due to 2019 severe acute respiratory syndrome coronavirus2", "wuhan virus acute severe respiratory syndrome", "new corona virus asevere acute respiratory disease", "wuhan acute severe infection", "new coronavirus 2019 severe respiratory disorder", "2019new wuhan virus acute severe pneumonia", "pneumonia due to novel corona virus", "2019 chinese corona virus", "2019 new wuhancoronavirus severe acute infection", "illness from 2019 human coronavirus", "2019new seafood market coronavirus acute illness", "virus causing covid19 pneumonia", "2019ncov severe acute syndrome", "2019novel wuhan coronavirus respiratory infection", "disease caused by severe acute respiratory syndromerelated coronavirus2 2019", "2019 seafood market virus acute severe respiratory syndrome", "2019novel wuhan seafood market pneumonia virus severe respiratory syndrome", "2019novel sarscov2 acute syndrome", "2019 sars cov 2 syndrome", "2019novel wuhan corona virus acute respiratory disorder", "seafood market coronavirus 2019 respiratory syndrome", "2019 new wuhancoronavirus severe respiratory disease", "sars corona virus 2 severe acute respiratory illness", "infection caused by wuhan corona virus 2019", "disorder caused by 2019 human coronavirus", "sars coronavirus 2 acute infection", "illness from 2019 novel chinese coronavirus", "2019 novel seafood market corona virus respiratory syndrome", "2019 new seafood market coronavirus severe respiratory infection", "2019 new wuhan coronavirus acute severe syndrome", "disease caused by 2019new wuhan seafood market virus", "disorder due to severe acute respiratory syndrome corona virus2 2019", "2019 novel wuhancorona virus severe acute disease", "2019new wuhancoronavirus acute respiratory infection", "2019 novel sars corona virus 2 severe pneumonia", "infection due to 2019 novel wuhan coronavirus", "2019novel sars coronavirus2 severe respiratory infection", "2019 novel sars corona virus2 severe respiratory infection", "wuhan pneumonia virus 2019 severe acute respiratory infection", "2019 wuhan corona virus acute pneumonia", "2019 new seafood market coronavirus severe respiratory disease", "2019 wuhancoronavirus acute respiratory disease", "wuhan virus 2019 severe respiratory disorder", "2019 novel sars corona virus2 acute severe illness", "infection caused by severe acute respiratory syndrome coronavirus2", "wuhan corona virus 2019 acute infection", "2019 new sars coronavirus 2 acute pneumonia", "chinese corona virus 2019 severe respiratory infection", "infection caused by 2019new wuhan seafood market pneumonia virus", "syndrome caused by 2019 chinese coronavirus", "virus responsible for covid19 respiratory syndrome", "2019new chinese coronavirus severe acute disorder", "pneumonia due to covid19 virus", "sars cov 2 2019 disease", "2019 corona virus severe acute respiratory illness", "wuhan virus 2019 severe acute pneumonia", "infection caused by severe acute respiratory syndromerelated coronavirus 2", "2019 novel sars corona virus2 acute respiratory disorder", "pneumonia caused by 2019 novel corona virus", "virus responsible for covid19 severe acute disorder", "2019 chinese corona virus acute illness", "2019 corona virus pneumonia", "2019 sars cov 2 pneumonia", "syndrome caused by 2019 new sars coronavirus2", "pneumonia caused by 2019 novel sarscov2", "wuhan pneumonia virus acute severe respiratory disease", "2019new sars coronavirus2 acute severe disease", "2019novel wuhan corona virus acute respiratory syndrome", "2019 ncov severe respiratory syndrome", "chinese corona virus 2019 respiratory illness", "2019 coronavirus respiratory illness", "sars corona virus 2 2019 respiratory syndrome", "2019 human corona virus acute respiratory infection", "2019 chinese corona virus severe acute illness", "2019 new sars corona virus2 acute respiratory syndrome", "sars cov 2 severe respiratory illness", "2019novel seafood market coronavirus pneumonia", "sars coronavirus2 2019 pneumonia", "seafood market virus 2019 severe acute illness", "sars corona virus 2 severe acute respiratory infection", "2019novel chinese coronavirus acute severe respiratory disorder", "2019 wuhan seafood market virus severe disorder", "2019new sars coronavirus2 respiratory syndrome", "virus responsible for covid19 acute disease", "2019novel wuhan corona virus acute severe disease", "disorder caused by 2019new sars corona virus 2", "2019 wuhan corona virus severe respiratory disorder", "2019 novel wuhan coronavirus severe illness", "2019new sars cov 2 severe acute infection", "pneumonia due to 2019new chinese corona virus", "2019 seafood market coronavirus acute disease", "2019new seafood market virus acute respiratory infection", "2019 new sars corona virus 2 disorder", "2019 novel seafood market corona virus severe pneumonia", "chinese coronavirus severe respiratory syndrome", "2019 seafood market coronavirus severe syndrome", "2019new sars corona virus2 severe acute pneumonia", "syndrome from 2019 human coronavirus", "2019 novel wuhancorona virus severe respiratory disorder", "2019novel sars corona virus 2 acute respiratory disorder", "2019new severe acute respiratory syndromerelated coronavirus2 disorder", "sars corona virus 2 2019 severe acute respiratory syndrome", "sars cov 2 2019 acute respiratory illness", "2019novel seafood market coronavirus acute severe syndrome", "hcov19 acute respiratory illness", "wuhan seafood market virus 2019 acute respiratory illness", "2019novel wuhancoronavirus acute severe disorder", "syndrome from 2019 new wuhan pneumonia virus", "severe acute respiratory syndrome coronavirus 2 pneumonia", "syndrome from virus causing covid19", "illness due to seafood market virus 2019", "chinese coronavirus acute syndrome", "2019 novel sars coronavirus 2 severe acute syndrome", "2019 sars coronavirus 2 asevere acute respiratory disease", "2019 new seafood market corona virus acute disorder", "2019 sars coronavirus2 severe acute syndrome", "wuhancorona virus acute severe disorder", "2019 new wuhancoronavirus respiratory illness", "2019 sars corona virus 2 respiratory disease", "wuhan virus severe acute respiratory illness", "2019new chinese corona virus severe respiratory disorder", "2019 new wuhan pneumonia virus severe acute pneumonia", "infection from 2019 severe acute respiratory syndrome coronavirus2", "seafood market virus severe disease", "covid19 virus severe disease", "2019 new wuhan seafood market pneumonia virus acute severe infection", "2019 novel sars coronavirus2 acute severe infection", "chinese coronavirus disorder", "2019 sarscov2 acute illness", "2019 coronavirus acute disorder", "infection from severe acute respiratory syndromerelated coronavirus2", "2019 new sars corona virus 2 severe illness", "2019 chinese corona virus respiratory syndrome", "2019 sars corona virus 2 severe disease", "wuhan coronavirus acute severe respiratory disorder", "illness caused by sars cov 2", "2019 sars coronavirus 2 severe illness", "disorder caused by 2019novel severe acute respiratory syndromerelated corona virus 2", "syndrome caused by 2019new sarscov2", "2019novel wuhan seafood market virus acute respiratory disease", "pneumonia due to 2019 novel seafood market corona virus", "2019 wuhan coronavirus severe infection", "novel coronavirus 2019 acute severe disorder", "2019 sarscov2 acute disorder", "2019 new seafood market coronavirus acute respiratory illness", "2019 new corona virus acute respiratory syndrome", "2019 wuhancoronavirus severe acute respiratory illness", "2019 novel sars corona virus 2 severe respiratory illness", "illness caused by 2019 sars corona virus 2", "syndrome due to wuhan pneumonia virus 2019", "2019 new sars cov 2 severe illness", "sars coronavirus 2 2019 severe respiratory illness", "syndrome due to 2019 new chinese coronavirus", "2019 human corona virus respiratory illness", "2019 sars corona virus2 respiratory disease", "disease from 2019 severe acute respiratory syndromerelated coronavirus2", "2019novel severe acute respiratory syndromerelated coronavirus 2 pneumonia", "2019 new wuhancoronavirus severe syndrome", "2019 new chinese corona virus severe acute respiratory infection", "seafood market coronavirus 2019 illness", "2019 corona virus acute disease", "2019new sarscov2 disorder", "wuhancoronavirus severe respiratory disease", "2019 new wuhan pneumonia virus severe pneumonia", "disease caused by chinese corona virus", "2019new severe acute respiratory syndrome corona virus2 disorder", "pneumonia caused by 2019 coronavirus", "infection due to human coronavirus 2019", "2019novel wuhan virus acute severe respiratory disorder", "infection from 2019new seafood market virus", "disease caused by 2019novel wuhancorona virus", "2019new chinese corona virus disease", "2019 wuhan seafood market virus severe acute respiratory syndrome", "sars coronavirus 2 severe respiratory illness", "infection caused by 2019 novel wuhan corona virus", "2019 sarscov2 severe acute respiratory disorder", "wuhan corona virus severe acute respiratory disorder", "new coronavirus severe disorder", "2019new sarscov2 acute respiratory illness", "pneumonia from 2019 wuhan corona virus", "infection due to 2019 seafood market corona virus", "chinese coronavirus 2019 severe infection", "infection from 2019 novel severe acute respiratory syndromerelated coronavirus 2", "2019new sars corona virus2 disorder", "infection due to seafood market corona virus", "2019 novel coronavirus severe infection", "2019 novel wuhan coronavirus acute illness", "2019 new sars cov 2 acute severe disorder", "illness from 2019 severe acute respiratory syndrome coronavirus 2", "2019 new wuhancorona virus severe acute infection", "2019 sars coronavirus 2 acute severe respiratory infection", "2019 new wuhancorona virus severe respiratory disorder", "seafood market coronavirus severe acute disorder", "2019novel sars coronavirus 2 disease", "wuhan corona virus acute infection", "pneumonia from 2019 novel wuhan pneumonia virus", "wuhancorona virus severe respiratory infection", "human coronavirus 2019 acute severe respiratory infection", "illness due to 2019 sars coronavirus2", "new coronavirus severe respiratory syndrome", "wuhan virus 2019 respiratory disease", "infection due to 2019 sars coronavirus2", "illness from 2019novel wuhan pneumonia virus", "2019 new sarscov2 severe respiratory illness", "2019novel seafood market corona virus respiratory illness", "2019new wuhan seafood market pneumonia virus respiratory illness", "wuhan corona virus 2019 severe acute respiratory syndrome", "disorder caused by sars cov 2 2019", "2019 new wuhan pneumonia virus severe acute illness", "covid19 virus disorder", "infection caused by 2019new wuhan coronavirus", "syndrome caused by seafood market corona virus", "wuhan virus severe acute syndrome", "2019new chinese coronavirus acute severe illness", "virus causing covid19 acute severe illness", "wuhancoronavirus 2019 severe acute respiratory syndrome", "2019 novel severe acute respiratory syndrome corona virus2 infection", "disease caused by 2019 novel seafood market virus", "2019 new wuhancoronavirus disorder", "2019novel severe acute respiratory syndrome coronavirus 2 disease", "chinese corona virus severe acute respiratory disorder", "2019novel sars corona virus 2 syndrome", "wuhan seafood market pneumonia virus acute disease", "wuhan virus 2019 severe pneumonia", "2019 novel sars corona virus2 respiratory disorder", "2019ncov acute respiratory disorder", "2019 seafood market coronavirus acute severe pneumonia", "2019new wuhan virus acute severe respiratory disorder", "illness caused by 2019 novel sars cov 2", "2019novel wuhancoronavirus severe acute syndrome", "syndrome due to 2019new sars cov 2", "disorder from sarscov2 2019", "infection from 2019 severe acute respiratory syndrome corona virus2", "disorder due to novel corona virus 2019", "pneumonia caused by hcov 19", "2019 novel sars cov 2 pneumonia", "wuhan coronavirus severe respiratory syndrome", "sars cov 2 acute respiratory illness", "2019new seafood market coronavirus acute severe respiratory disease", "disease from 2019 new severe acute respiratory syndrome corona virus 2", "2019new sars cov 2 acute severe respiratory syndrome", "illness caused by 2019novel wuhan pneumonia virus", "2019 novel wuhan corona virus acute severe respiratory disorder", "2019novel wuhan corona virus acute disorder", "disorder caused by 2019 novel chinese coronavirus", "2019 wuhan coronavirus acute severe disease", "wuhan corona virus severe acute respiratory illness", "2019new sars coronavirus2 severe acute respiratory syndrome", "severe acute respiratory syndromerelated corona virus 2 2019", "2019 novel sars coronavirus 2 acute respiratory disease", "disorder caused by seafood market virus", "disease due to 2019 wuhan pneumonia virus", "wuhan pneumonia virus 2019 acute disease", "2019new seafood market corona virus respiratory disorder", "2019novel wuhancoronavirus severe syndrome", "2019 corona virus respiratory syndrome", "severe acute respiratory syndromerelated coronavirus 2 pneumonia", "2019new wuhan virus illness", "2019 novel wuhan pneumonia virus acute illness", "new coronavirus 2019 respiratory disorder", "virus responsible for covid19 acute severe syndrome", "infection caused by 2019 severe acute respiratory syndrome coronavirus2", "2019 novel corona virus severe acute pneumonia", "2019 new wuhan seafood market pneumonia virus acute severe pneumonia", "chinese corona virus acute severe respiratory syndrome", "infection due to severe acute respiratory syndromerelated coronavirus 2", "wuhan corona virus acute severe syndrome", "infection from sars coronavirus2", "seafood market virus illness", "2019 novel wuhan seafood market virus respiratory disease", "2019 novel wuhan seafood market virus acute pneumonia", "sars corona virus 2 severe acute respiratory syndrome", "wuhan seafood market virus 2019 severe acute pneumonia", "2019 new chinese coronavirus severe respiratory illness", "wuhan coronavirus 2019 severe acute respiratory disorder", "sars coronavirus2 respiratory disorder", "infection due to sars corona virus 2 2019", "sarscov2 2019 illness", "wuhan seafood market virus 2019 severe respiratory illness", "2019new wuhan seafood market virus severe pneumonia", "2019novel wuhancorona virus syndrome", "2019new seafood market corona virus severe respiratory illness", "wuhan seafood market pneumonia virus 2019 acute severe disease", "seafood market severe respiratory disorder", "disorder due to 2019new sars corona virus2", "pneumonia due to 2019 wuhan seafood market pneumonia virus", "2019 novel chinese coronavirus acute illness", "2019 new chinese corona virus acute illness", "pneumonia due to 2019 new sars coronavirus2", "2019 new chinese coronavirus acute severe disorder", "sars coronavirus 2 2019 severe respiratory disorder", "disease caused by 2019 new wuhancoronavirus", "2019new wuhancoronavirus acute disorder", "2019 coronavirus severe acute disorder", "2019 new seafood market coronavirus syndrome", "2019new sars coronavirus2 acute severe illness", "disease from 2019 novel corona virus", "2019new wuhancorona virus acute severe illness", "2019 novel wuhan pneumonia virus illness", "new coronavirus severe acute disorder", "2019novel sars corona virus 2 severe acute illness", "illness due to 2019 novel coronavirus", "2019novel wuhancorona virus severe acute illness", "virus causing covid19 severe respiratory disorder", "wuhan seafood market virus 2019 severe acute disease", "2019 novel seafood market coronavirus severe acute disorder", "seafood market corona virus acute pneumonia", "illness caused by sars corona virus 2 2019", "2019novel sars coronavirus 2 acute severe respiratory disorder", "hcov 19 severe acute respiratory infection", "disease caused by 2019 new seafood market virus", "illness due to 2019 new severe acute respiratory syndrome coronavirus 2", "2019 wuhan virus severe acute disorder", "syndrome from 2019 new wuhan virus", "illness caused by 2019novel severe acute respiratory syndrome coronavirus 2", "2019new sarscov2 severe acute illness", "2019 new sars corona virus 2 respiratory disorder", "wuhancoronavirus 2019 acute severe respiratory infection", "2019novel chinese coronavirus acute syndrome", "2019 new seafood market virus severe acute syndrome", "novel corona virus respiratory syndrome", "2019 novel sars corona virus2 disorder", "infection from 2019 new wuhan corona virus", "2019new sars coronavirus 2 acute severe illness", "illness due to sars corona virus2", "illness caused by 2019 new seafood market coronavirus", "syndrome due to 2019 novel seafood market corona virus", "2019new seafood market virus acute severe respiratory disorder", "sars corona virus2 2019 acute illness", "wuhan pneumonia virus severe respiratory illness", "sars cov 2 severe pneumonia", "2019novel wuhan seafood market pneumonia virus asevere acute respiratory disease", "2019 coronavirus acute severe disorder", "2019new seafood market corona virus severe syndrome", "sars cov 2 acute severe pneumonia", "2019 novel wuhan corona virus severe acute syndrome", "wuhancorona virus 2019 severe respiratory infection", "2019new sars cov 2 acute severe infection", "2019 wuhancoronavirus severe respiratory disease", "2019 novel sars corona virus2 severe acute disorder", "2019new sars coronavirus 2 severe illness", "seafood market corona virus 2019 severe syndrome", "novel corona virus respiratory infection", "2019 coronavirus acute severe respiratory illness", "2019 new seafood market virus respiratory disease", "sars coronavirus2 2019 acute illness", "disease from wuhan seafood market pneumonia virus 2019", "severe acute respiratory syndrome coronavirus2 2019 illness", "2019 sarscov2 acute syndrome", "wuhancorona virus severe respiratory disease", "severe acute respiratory syndrome corona virus 2 2019 pneumonia", "infection caused by 2019novel severe acute respiratory syndrome corona virus2", "disease due to novel corona virus", "wuhancoronavirus 2019 acute respiratory illness", "sars corona virus 2 2019 severe respiratory infection", "2019novel sars coronavirus2 acute severe disorder", "2019 new severe acute respiratory syndrome corona virus2 infection", "chinese corona virus 2019 illness", "2019 corona virus acute respiratory disease", "2019new wuhan virus", "infection due to severe acute respiratory syndrome coronavirus2 2019", "2019 novel wuhan corona virus acute severe pneumonia", "2019novel seafood market corona virus acute severe respiratory disease", "syndrome caused by virus causing covid19", "2019new wuhan coronavirus severe disorder", "new coronavirus respiratory disease", "2019novel sars corona virus 2 acute severe pneumonia", "illness caused by 2019 wuhan coronavirus", "2019new wuhan coronavirus severe acute pneumonia", "disease due to 2019 seafood market coronavirus", "illness due to 2019novel seafood market corona virus", "sars coronavirus 2 2019 disorder", "2019 wuhan coronavirus acute pneumonia", "2019 wuhan coronavirus illness", "infection caused by 2019new seafood market corona virus", "pneumonia due to 2019new sars corona virus 2", "hcov19 acute severe infection", "2019 novel coronavirus acute syndrome", "2019new sars corona virus2 acute disease", "seafood market corona virus 2019 severe disease", "2019 novel sarscov2 severe acute respiratory syndrome", "pneumonia caused by 2019 new severe acute respiratory syndromerelated corona virus2", "disorder from 2019 novel severe acute respiratory syndrome corona virus2", "2019 novel sars cov 2 severe disorder", "2019novel sars coronavirus2 illness", "2019new seafood market coronavirus infection", "2019 new sars corona virus2 syndrome", "2019novel wuhan seafood market virus severe disease", "wuhan pneumonia virus 2019 disease", "syndrome from novel coronavirus", "2019new sars coronavirus 2 asevere acute respiratory disease", "disorder from 2019 wuhan corona virus", "2019ncov respiratory illness", "2019 new coronavirus acute severe disease", "wuhancoronavirus respiratory disease", "2019 severe acute respiratory syndrome corona virus 2 disease", "2019new wuhan coronavirus acute disorder", "infection from 2019 wuhancorona virus", "2019new sars corona virus 2 acute pneumonia", "wuhancorona virus acute respiratory illness", "2019 seafood market virus acute severe respiratory disease", "2019 chinese corona virus severe respiratory illness", "sarscov2 2019 acute severe respiratory illness", "pneumonia caused by sars coronavirus 2", "disorder caused by sars coronavirus 2 2019", "sars cov 2 2019 acute severe respiratory disorder", "seafood market virus 2019 acute severe illness", "disease from 2019 seafood market coronavirus", "2019 novel corona virus acute respiratory infection", "2019 sars cov 2 severe respiratory disease", "2019 novel wuhancoronavirus severe acute respiratory illness", "disorder from 2019 human coronavirus", "2019 novel wuhancoronavirus acute respiratory infection", "disorder caused by 2019 new sars coronavirus 2", "wuhancorona virus respiratory illness", "2019new wuhan pneumonia virus acute respiratory infection", "sars cov 2 acute syndrome", "syndrome due to 2019 wuhan seafood market virus", "2019 novel sars coronavirus2 disease", "2019 wuhancoronavirus acute respiratory disorder", "novel corona virus 2019 severe acute respiratory syndrome", "2019 new sars corona virus 2 severe infection", "2019 new wuhan seafood market virus infection", "2019 novel seafood market coronavirus severe acute disease", "2019 ncov severe acute respiratory syndrome", "2019 ncov acute pneumonia", "seafood market acute severe respiratory infection", "disorder from 2019 new seafood market corona virus", "2019novel sars coronavirus 2", "pneumonia from severe acute respiratory syndromerelated coronavirus 2 2019", "2019 wuhan seafood market virus respiratory disorder", "sars corona virus2 severe disorder", "2019 sars corona virus 2 severe acute respiratory infection", "2019new chinese coronavirus acute severe respiratory illness", "sarscov2 2019 acute severe disease", "novel coronavirus acute disease", "2019 seafood market virus severe acute respiratory infection", "syndrome due to 2019 novel sarscov2", "wuhan virus 2019 acute disorder", "2019 novel sars cov 2 severe pneumonia", "2019new sars coronavirus 2 severe respiratory infection", "2019 new wuhancorona virus acute severe infection", "pneumonia caused by 2019 new sars corona virus 2", "2019novel wuhancoronavirus illness", "2019 new wuhan corona virus acute severe respiratory infection", "2019 novel sars coronavirus 2 acute respiratory infection", "2019novel wuhan corona virus severe acute pneumonia", "infection due to 2019novel severe acute respiratory syndrome corona virus 2", "new coronavirus 2019 disease", "new corona virus acute severe respiratory infection", "illness from 2019 sars corona virus 2", "2019novel wuhan coronavirus severe disease", "2019new wuhancoronavirus acute severe respiratory infection", "2019new wuhan seafood market virus acute severe infection", "2019 new sars corona virus 2 severe acute disorder", "2019 severe acute respiratory syndrome coronavirus2 illness", "2019new wuhan seafood market virus severe disorder", "2019 new sars coronavirus2 severe acute respiratory disorder", "pneumonia caused by 2019 severe acute respiratory syndrome coronavirus2", "pneumonia due to severe acute respiratory syndrome coronavirus 2 2019", "2019 new seafood market virus acute respiratory infection", "disorder from 2019 novel wuhancorona virus", "covid 19 virus severe acute infection", "severe acute respiratory syndrome corona virus2", "2019 novel corona virus severe disease", "disorder due to 2019 new severe acute respiratory syndromerelated corona virus 2", "new corona virus 2019 acute respiratory syndrome", "seafood market virus 2019 severe acute respiratory syndrome", "2019 novel wuhan coronavirus acute disorder", "disorder caused by severe acute respiratory syndrome coronavirus2", "chinese coronavirus acute severe illness", "infection caused by 2019 novel corona virus", "2019 novel corona virus acute disease", "2019new wuhan virus disorder", "syndrome due to 2019new seafood market coronavirus", "2019novel severe acute respiratory syndromerelated coronavirus 2 disorder", "sars coronavirus2 2019 severe acute syndrome", "pneumonia due to sars coronavirus2 2019", "syndrome due to seafood market coronavirus", "2019new sars coronavirus 2 severe acute respiratory syndrome", "disease caused by 2019 novel severe acute respiratory syndromerelated coronavirus 2", "infection due to 2019novel wuhan pneumonia virus", "disorder due to 2019novel severe acute respiratory syndrome corona virus 2", "wuhan respiratory syndrome", "seafood market acute severe respiratory syndrome", "wuhan pneumonia virus 2019 severe acute pneumonia", "illness caused by severe acute respiratory syndrome coronavirus2 2019", "syndrome caused by 2019novel chinese coronavirus", "2019novel sars cov 2 severe respiratory illness", "disorder due to 2019 wuhan pneumonia virus", "2019novel seafood market coronavirus severe acute syndrome", "2019novel sars coronavirus2 respiratory disease", "2019new sars corona virus 2 severe acute pneumonia", "syndrome due to 2019 novel chinese coronavirus", "2019new sars corona virus2 acute severe pneumonia", "2019 novel wuhan seafood market virus acute syndrome", "infection caused by 2019 novel severe acute respiratory syndromerelated corona virus 2", "2019 new wuhancorona virus respiratory syndrome", "2019 new wuhan virus acute respiratory disease", "disorder due to severe acute respiratory syndromerelated coronavirus 2", "2019 novel seafood market coronavirus severe disorder", "sarscov2 2019 acute respiratory infection", "disease from chinese coronavirus 2019", "2019new sars coronavirus 2 severe respiratory disease", "2019novel seafood market coronavirus severe acute illness", "2019novel wuhan seafood market virus acute respiratory disorder", "disorder caused by 2019 new wuhan coronavirus", "disease due to 2019 new wuhan coronavirus", "wuhan seafood market virus severe acute pneumonia", "syndrome from 2019 wuhancoronavirus", "sars cov 2 respiratory syndrome", "illness caused by novel corona virus 2019", "disease due to 2019 ncov", "virus responsible for covid19 severe respiratory syndrome", "2019 new coronavirus severe respiratory disorder", "2019 novel coronavirus acute severe illness", "disorder from 2019 sars corona virus2", "sars corona virus 2 acute respiratory infection", "syndrome caused by wuhan coronavirus", "sars corona virus2 acute severe infection", "2019new wuhan pneumonia virus syndrome", "2019novel severe acute respiratory syndromerelated coronavirus2 illness", "seafood market virus 2019 acute severe respiratory syndrome", "disease caused by 2019novel wuhan coronavirus", "sars corona virus 2 acute pneumonia", "seafood market acute severe disease", "wuhan virus acute disease", "infection caused by 2019 novel sarscov2", "2019new sars corona virus 2 disorder", "2019novel sars cov 2 severe infection", "new coronavirus illness", "2019new wuhan virus respiratory disease", "wuhan seafood market virus acute respiratory disorder", "2019 sars corona virus2 respiratory syndrome", "2019 chinese corona virus severe acute pneumonia", "seafood market virus 2019 severe respiratory illness", "2019 novel wuhan corona virus respiratory disorder", "virus responsible for covid19 acute severe respiratory disorder", "2019 novel chinese corona virus acute disease", "wuhan coronavirus 2019 severe acute disease", "2019novel sars coronavirus 2 severe respiratory disorder", "wuhan seafood market virus asevere acute respiratory disease", "2019 sarscov2 severe acute respiratory syndrome", "sarscov2 2019 severe acute respiratory illness", "wuhancorona virus 2019 severe acute disorder", "syndrome from 2019 novel seafood market corona virus", "2019 novel sars corona virus2 severe respiratory disorder", "2019 novel sars coronavirus2 syndrome", "new corona virus severe acute respiratory illness", "infection due to 2019new wuhancorona virus", "2019 new sars cov 2 acute respiratory disorder", "2019 novel coronavirus severe acute respiratory illness", "wuhan seafood market pneumonia virus severe disease", "2019novel chinese coronavirus acute severe pneumonia", "2019novel sars coronavirus 2 severe acute disorder", "2019new seafood market virus acute severe respiratory infection", "sars corona virus 2 acute severe respiratory disorder", "new coronavirus 2019 acute severe pneumonia", "novel coronavirus severe acute respiratory syndrome", "2019new chinese coronavirus", "2019new seafood market coronavirus acute respiratory disease", "2019 new chinese coronavirus acute severe respiratory disorder", "disorder caused by severe acute respiratory syndrome corona virus 2", "disorder due to 2019novel sars coronavirus 2", "sarscov2 2019 acute respiratory disorder", "illness due to 2019 novel wuhan seafood market pneumonia virus", "illness due to sars corona virus 2", "sarscov2 acute disease", "2019 novel chinese corona virus severe disorder", "2019novel sarscov2 acute severe disorder", "2019new wuhancoronavirus severe respiratory illness", "2019 new seafood market virus acute severe syndrome", "2019 wuhancoronavirus severe disease", "sars coronavirus 2 2019 acute disorder", "chinese coronavirus 2019 respiratory syndrome", "2019novel wuhancoronavirus acute severe disease", "2019 new wuhan virus severe syndrome", "2019 novel sars corona virus2 acute severe respiratory illness", "syndrome due to seafood market virus 2019", "2019novel wuhan pneumonia virus severe acute syndrome", "2019 novel chinese corona virus severe acute respiratory disorder", "wuhan pneumonia virus acute disorder", "wuhan virus 2019 disorder", "wuhan coronavirus severe disease", "syndrome caused by wuhan seafood market virus", "2019novel wuhancorona virus severe respiratory disorder", "2019 new wuhan corona virus severe acute infection", "2019 human coronavirus respiratory illness", "2019 new wuhan seafood market virus acute severe syndrome", "2019 seafood market virus respiratory infection", "novel corona virus 2019 acute respiratory syndrome", "2019 novel coronavirus severe acute respiratory syndrome", "wuhan corona virus 2019 respiratory disease", "2019 new wuhancorona virus acute infection", "wuhan coronavirus severe respiratory disease", "infection due to 2019 new wuhancoronavirus", "infection from hcov 19", "2019 wuhan coronavirus acute infection", "2019 new wuhan seafood market pneumonia virus respiratory disease", "illness due to 2019 severe acute respiratory syndromerelated corona virus2", "2019 sars cov 2 severe acute disease", "2019novel sars corona virus2 respiratory disorder", "infection from 2019novel sarscov2", "disease caused by severe acute respiratory syndromerelated corona virus2 2019", "disease from 2019new severe acute respiratory syndrome coronavirus2", "2019 chinese coronavirus respiratory infection", "disorder due to 2019new chinese coronavirus", "2019 new sars coronavirus 2 severe respiratory disease", "2019 new sars corona virus2 acute severe respiratory infection", "pneumonia due to 2019 chinese coronavirus", "wuhan pneumonia virus 2019 respiratory disorder", "seafood market coronavirus 2019 severe infection", "2019 new seafood market virus severe acute disorder", "2019novel seafood market coronavirus severe acute disease", "wuhan seafood market virus 2019 severe syndrome", "2019 novel sars coronavirus 2 severe respiratory infection", "infection from novel coronavirus", "disorder from seafood market virus", "illness caused by 2019novel severe acute respiratory syndromerelated corona virus 2", "2019 seafood market virus respiratory disorder", "wuhan corona virus 2019 asevere acute respiratory disease", "disease caused by wuhancorona virus 2019", "virus responsible for covid19 pneumonia", "covid 19 virus severe infection", "2019 new sars coronavirus2 acute infection", "chinese coronavirus 2019 infection", "new coronavirus severe respiratory illness", "new coronavirus acute severe respiratory infection", "infection from 2019 new severe acute respiratory syndromerelated coronavirus2", "disease from chinese coronavirus", "2019 novel sars coronavirus2", "2019new seafood market coronavirus illness", "2019 chinese corona virus acute infection", "novel coronavirus pneumonia", "2019 novel wuhan coronavirus acute severe infection", "disease caused by severe acute respiratory syndrome coronavirus 2", "2019 corona virus severe pneumonia", "2019 new wuhancoronavirus severe respiratory infection", "2019novel seafood market virus severe acute pneumonia", "wuhan coronavirus 2019 severe respiratory infection", "pneumonia due to 2019novel sars corona virus 2", "2019 novel sarscov2 respiratory disorder", "seafood market coronavirus 2019 acute respiratory syndrome", "human corona virus 2019 severe acute respiratory syndrome", "chinese corona virus 2019 severe acute respiratory disorder", "2019novel wuhan pneumonia virus disorder", "2019 sars coronavirus 2 acute severe syndrome", "2019 new chinese coronavirus severe infection", "2019 ncov acute severe respiratory illness", "sars coronavirus 2 acute severe respiratory disorder", "sars coronavirus 2 acute severe respiratory disease", "2019novel sars coronavirus 2 severe syndrome", "syndrome due to seafood market corona virus 2019", "2019novel seafood market coronavirus disorder", "2019 new sars cov 2 severe acute illness", "2019 human corona virus acute severe syndrome", "2019novel chinese coronavirus severe respiratory illness", "2019 new wuhan coronavirus respiratory disorder", "2019new wuhan virus severe acute respiratory illness", "disorder from 2019 novel wuhan seafood market virus", "sars coronavirus 2 2019 acute severe respiratory infection", "2019 new corona virus acute severe respiratory disorder", "2019new sars cov 2 acute respiratory illness", "disorder due to 2019 human coronavirus", "syndrome from 2019 wuhan seafood market virus", "hcov 19 acute severe respiratory syndrome", "wuhan pneumonia virus 2019 respiratory disease", "seafood market virus acute respiratory syndrome", "2019 novel sars coronavirus2 acute severe disease", "infection from 2019 novel sars cov 2", "sars coronavirus 2 2019 respiratory infection", "syndrome caused by 2019 coronavirus", "infection caused by novel corona virus 2019", "disease from 2019 novel sars corona virus 2", "2019 novel wuhancoronavirus acute severe syndrome", "2019 novel sarscov2 severe syndrome", "2019novel wuhan pneumonia virus acute severe respiratory disorder", "2019novel chinese coronavirus severe acute respiratory infection", "wuhan corona virus 2019 acute severe illness", "seafood market coronavirus 2019 severe disease", "2019novel sarscov2 severe acute respiratory illness", "2019 wuhancorona virus severe acute respiratory disorder", "human coronavirus 2019 severe pneumonia", "pneumonia from 2019 wuhancoronavirus", "sars corona virus 2 2019 disorder", "2019 new wuhan coronavirus asevere acute respiratory disease", "pneumonia due to 2019novel severe acute respiratory syndromerelated corona virus 2", "pneumonia from 2019 novel sars corona virus 2", "illness due to 2019novel wuhancorona virus", "2019 seafood market virus illness", "sars corona virus2 2019 severe illness", "2019 new wuhan corona virus acute respiratory illness", "2019 novel wuhan virus acute illness", "2019new wuhancoronavirus acute respiratory disease", "sars corona virus 2 acute severe respiratory illness", "disorder due to 2019 novel sars cov 2", "2019 novel sarscov2 severe disorder", "wuhancorona virus 2019 asevere acute respiratory disease", "2019 human corona virus severe acute disorder", "2019 severe acute respiratory syndrome corona virus2 disorder", "sars corona virus2 acute severe illness", "2019 new seafood market virus asevere acute respiratory disease", "2019new sars coronavirus 2 respiratory syndrome", "2019 new sars cov 2 severe syndrome", "2019 wuhancoronavirus acute severe syndrome", "2019novel sars corona virus2 acute severe illness", "new coronavirus 2019 acute severe respiratory illness", "2019 wuhancoronavirus acute severe pneumonia", "covid 19 virus severe pneumonia", "2019 new seafood market coronavirus severe respiratory syndrome", "2019 novel wuhan seafood market pneumonia virus respiratory infection", "2019 novel coronavirus acute illness", "2019novel wuhancorona virus acute respiratory infection", "2019 sars corona virus 2 severe illness", "2019 wuhan pneumonia virus acute severe respiratory illness", "pneumonia caused by 2019 corona virus", "disease from 2019 human corona virus", "illness from 2019 chinese coronavirus", "2019 novel wuhan virus acute pneumonia", "chinese coronavirus 2019 acute respiratory infection", "2019 coronavirus severe pneumonia", "2019 new chinese coronavirus severe acute respiratory disorder", "2019novel sarscov2 disease", "2019novel wuhan seafood market virus infection", "infection caused by 2019 novel wuhan seafood market pneumonia virus", "disease due to 2019 novel wuhan virus", "2019new chinese corona virus severe syndrome", "2019novel wuhan coronavirus acute infection", "sars corona virus2 2019 severe disorder", "2019 novel chinese coronavirus severe acute pneumonia", "2019 new wuhan seafood market virus acute severe pneumonia", "pneumonia caused by 2019ncov", "2019new wuhan seafood market virus acute pneumonia", "sars cov 2 pneumonia", "wuhancoronavirus acute syndrome", "disease due to 2019novel sars cov 2", "infection due to sars corona virus 2", "2019 novel sars coronavirus 2 respiratory syndrome", "human corona virus 2019 disease", "2019 novel wuhan seafood market pneumonia virus acute severe respiratory infection", "seafood market coronavirus severe acute illness", "2019 seafood market corona virus severe acute respiratory syndrome", "2019 new wuhancoronavirus acute severe respiratory illness", "seafood market coronavirus 2019 syndrome", "sars cov 2 acute infection", "2019 new wuhancoronavirus severe respiratory disorder", "syndrome caused by 2019new wuhan virus", "wuhan severe syndrome", "2019 novel chinese coronavirus respiratory syndrome", "sarscov2 syndrome", "2019 novel sars corona virus2 severe respiratory illness", "syndrome due to sars corona virus 2", "chinese coronavirus 2019 illness", "2019new sarscov2 acute severe disorder", "2019novel sars cov 2 acute infection", "2019new sars corona virus2 severe acute disorder", "disorder due to severe acute respiratory syndromerelated corona virus2 2019", "disorder due to sars coronavirus 2 2019", "2019novel severe acute respiratory syndromerelated corona virus 2", "chinese corona virus 2019 severe acute syndrome", "2019novel wuhan coronavirus acute respiratory disease", "2019 new severe acute respiratory syndromerelated coronavirus2 disease", "2019novel seafood market virus severe respiratory infection", "infection caused by 2019new wuhan seafood market virus", "illness caused by 2019new wuhan seafood market virus", "disorder due to 2019new seafood market coronavirus", "2019 novel sars coronavirus 2 severe acute infection", "2019 wuhan coronavirus severe acute syndrome", "pneumonia due to severe acute respiratory syndromerelated corona virus 2", "2019 novel wuhancorona virus severe illness", "infection due to 2019 new sars corona virus 2", "2019 wuhancoronavirus respiratory infection", "infection from wuhan pneumonia virus 2019", "disorder due to 2019novel wuhan seafood market virus", "pneumonia from sars coronavirus2", "2019novel wuhan seafood market pneumonia virus acute severe disease", "2019novel sars cov 2 acute illness", "illness caused by 2019new wuhan pneumonia virus", "disease due to 2019new sars cov 2", "2019 novel coronavirus disorder", "disease caused by 2019new sars coronavirus 2", "2019 new wuhan corona virus acute severe respiratory syndrome", "2019 sars coronavirus2 severe respiratory disorder", "2019 novel chinese coronavirus severe acute respiratory syndrome", "illness from 2019 wuhan seafood market virus", "disease caused by severe acute respiratory syndrome coronavirus2", "pneumonia due to 2019 severe acute respiratory syndromerelated corona virus 2", "disease due to wuhan corona virus 2019", "infection due to wuhan pneumonia virus 2019", "infection caused by chinese coronavirus", "sars cov 2 2019 severe acute infection", "pneumonia caused by severe acute respiratory syndrome corona virus 2 2019", "2019new seafood market virus respiratory infection", "sars corona virus 2 2019 severe syndrome", "2019 novel severe acute respiratory syndrome corona virus 2 pneumonia", "syndrome from covid 19 virus", "syndrome caused by wuhancorona virus 2019", "disease due to 2019novel sars corona virus2", "pneumonia due to 2019 new severe acute respiratory syndromerelated coronavirus 2", "2019new severe acute respiratory syndrome corona virus 2 infection", "2019 seafood market virus acute severe infection", "new corona virus 2019 severe syndrome", "covid 19 virus severe acute disease", "sars coronavirus2 acute respiratory syndrome", "seafood market corona virus asevere acute respiratory disease", "disease from 2019new seafood market coronavirus", "2019new sars corona virus 2 respiratory illness", "2019 new wuhan coronavirus severe acute disease", "wuhancoronavirus severe acute disorder", "2019new wuhancorona virus respiratory disease", "new coronavirus 2019 acute severe respiratory disease", "2019novel seafood market coronavirus acute disease", "2019 novel sars cov 2 severe acute disorder", "human coronavirus 2019 respiratory infection", "2019new sars coronavirus 2 severe disease", "2019 novel wuhan seafood market virus acute severe syndrome", "infection from 2019 sars coronavirus2", "virus responsible for covid19 acute severe disease", "infection caused by 2019 new severe acute respiratory syndrome corona virus2", "2019 novel sars coronavirus2 acute disease", "2019 chinese corona virus acute respiratory infection", "chinese corona virus 2019 severe respiratory illness", "2019 novel sars coronavirus 2 severe respiratory syndrome", "2019 new chinese coronavirus disease", "disease due to new coronavirus", "infection from 2019novel sars corona virus 2", "2019novel seafood market coronavirus severe respiratory syndrome", "illness from wuhan seafood market pneumonia virus 2019", "2019novel chinese corona virus severe disease", "2019novel chinese corona virus severe pneumonia", "2019new wuhan seafood market virus severe infection", "wuhan seafood market pneumonia virus 2019 acute severe pneumonia", "2019 wuhan seafood market pneumonia virus severe illness", "2019 new wuhan corona virus acute severe illness", "disorder caused by 2019new sars coronavirus 2", "2019 sars coronavirus 2 acute disorder", "2019 novel seafood market coronavirus disease", "virus causing covid19 acute disorder", "2019 new wuhan seafood market virus acute respiratory disorder", "pneumonia caused by wuhan pneumonia virus 2019", "disorder due to 2019novel wuhan virus", "infection caused by sars corona virus 2 2019", "infection from 2019 chinese coronavirus", "2019 novel wuhan seafood market pneumonia virus severe pneumonia", "2019 sars cov 2 acute illness", "pneumonia caused by wuhan coronavirus 2019", "infection from 2019novel seafood market coronavirus", "sarscov2 2019 infection", "wuhan corona virus 2019 severe respiratory disease", "pneumonia caused by 2019 sarscov2", "infection from severe acute respiratory syndromerelated corona virus2 2019", "2019 novel chinese coronavirus acute severe infection", "illness from sars coronavirus 2", "seafood market corona virus severe acute respiratory infection", "sars corona virus2 2019 acute respiratory disorder", "2019 novel coronavirus acute severe infection", "illness from 2019novel severe acute respiratory syndromerelated coronavirus 2", "disease due to 2019 new wuhancorona virus", "illness caused by 2019 ncov", "seafood market corona virus acute disorder", "2019new seafood market virus severe acute respiratory syndrome", "sarscov2 2019 asevere acute respiratory disease", "2019new sars corona virus2 asevere acute respiratory disease", "new corona virus 2019 respiratory illness", "syndrome from wuhan pneumonia virus", "wuhan virus acute severe respiratory illness", "2019 novel wuhancorona virus asevere acute respiratory disease", "infection caused by 2019 novel severe acute respiratory syndrome corona virus 2", "pneumonia from 2019 novel corona virus", "syndrome due to chinese coronavirus 2019", "new corona virus severe syndrome", "wuhan coronavirus 2019 acute syndrome", "2019 new seafood market corona virus severe acute infection", "sars corona virus2 2019 severe respiratory disorder", "2019 novel wuhan seafood market virus respiratory infection", "chinese corona virus 2019 respiratory disease", "2019novel wuhan corona virus severe acute respiratory infection", "2019new wuhan pneumonia virus respiratory disease", "2019 seafood market corona virus acute severe respiratory illness", "2019 new wuhan seafood market pneumonia virus acute severe respiratory disorder", "2019 novel chinese coronavirus pneumonia", "wuhan seafood market pneumonia virus acute respiratory illness", "disease from 2019new wuhan coronavirus", "wuhan corona virus acute severe infection", "2019novel wuhan seafood market virus severe acute illness", "disorder from 2019 new chinese corona virus", "seafood market corona virus acute infection", "wuhancorona virus 2019 acute severe illness", "2019 novel sars coronavirus 2 acute severe respiratory disease", "2019 seafood market coronavirus acute illness", "pneumonia caused by sarscov2", "infection due to 2019new seafood market coronavirus", "infection due to severe acute respiratory syndrome coronavirus 2", "syndrome due to 2019 wuhan virus", "disorder from 2019novel severe acute respiratory syndrome coronavirus2", "2019novel wuhan seafood market virus severe respiratory infection", "wuhan virus 2019 respiratory illness", "hcov 19 acute respiratory illness", "2019novel seafood market virus acute respiratory disorder", "wuhancorona virus severe respiratory illness", "2019 sars cov 2 acute severe respiratory infection", "syndrome due to 2019 ncov", "novel corona virus 2019 acute severe infection", "2019 seafood market coronavirus respiratory illness", "2019new wuhan pneumonia virus severe respiratory syndrome", "2019novel seafood market coronavirus severe respiratory infection", "2019 novel wuhan virus severe respiratory infection", "hcov 19 acute severe pneumonia", "2019 new wuhan virus acute respiratory disorder", "2019 novel sarscov2 severe acute disease", "2019novel wuhan pneumonia virus severe syndrome", "2019 new wuhancorona virus acute respiratory disorder", "illness due to 2019 severe acute respiratory syndromerelated corona virus 2", "sars corona virus 2 acute respiratory syndrome", "novel corona virus 2019 acute infection", "2019 new corona virus acute severe respiratory disease", "seafood market coronavirus 2019 severe acute respiratory infection", "2019new sars coronavirus2 syndrome", "novel coronavirus 2019 severe acute disease", "pneumonia caused by 2019 new seafood market corona virus", "wuhancoronavirus severe disorder", "2019 new seafood market virus severe acute illness", "2019 new wuhan coronavirus acute illness", "human coronavirus 2019 acute infection", "sars corona virus2 respiratory disease", "2019 novel wuhan pneumonia virus infection", "covid19 virus respiratory syndrome", "covid19 virus severe respiratory disease", "disease due to 2019novel wuhan virus", "disorder due to 2019 novel wuhan pneumonia virus", "wuhan pneumonia virus severe respiratory infection", "virus responsible for covid19 severe respiratory disease", "2019new seafood market virus acute severe infection", "wuhancorona virus severe acute respiratory infection", "2019new wuhan coronavirus severe acute respiratory illness", "pneumonia caused by 2019novel wuhan coronavirus", "2019 wuhan corona virus severe disease", "2019 novel sars coronavirus2 respiratory illness", "wuhan virus respiratory disorder", "2019novel wuhan corona virus severe acute infection", "2019 novel chinese coronavirus severe infection", "2019 novel wuhan seafood market virus acute severe disorder", "2019 novel wuhan seafood market virus acute disease", "2019 novel sarscov2 acute respiratory infection", "wuhan severe infection", "2019novel severe acute respiratory syndromerelated corona virus2 disorder", "wuhan seafood market virus infection", "2019 novel wuhan seafood market pneumonia virus pneumonia", "2019novel sars corona virus 2 severe disease", "disorder due to 2019novel severe acute respiratory syndrome coronavirus2", "2019 coronavirus acute respiratory infection", "2019 novel wuhancorona virus acute respiratory infection", "illness due to 2019 new severe acute respiratory syndromerelated corona virus 2", "2019 novel severe acute respiratory syndrome corona virus 2 disorder", "2019 novel sars corona virus2 respiratory illness", "2019 new sars corona virus 2 severe disease", "illness caused by 2019novel wuhan seafood market virus", "chinese corona virus severe acute disease", "2019 new severe acute respiratory syndromerelated corona virus2 pneumonia", "2019 novel sars coronavirus2 severe respiratory syndrome", "illness from 2019 new severe acute respiratory syndrome coronavirus 2", "2019 novel sars coronavirus 2", "virus causing covid19 severe illness", "illness from 2019new wuhan pneumonia virus", "novel corona virus acute severe respiratory syndrome", "2019novel seafood market coronavirus respiratory illness", "illness caused by severe acute respiratory syndrome corona virus 2", "wuhan virus 2019 severe acute infection", "wuhan pneumonia virus 2019 acute severe respiratory infection", "2019 new seafood market corona virus acute severe pneumonia", "disease due to wuhan seafood market pneumonia virus", "seafood market virus acute severe disorder", "pneumonia due to 2019 wuhan seafood market virus", "2019new sars coronavirus2 severe acute respiratory illness", "2019 novel sars coronavirus2 severe pneumonia", "chinese coronavirus severe infection", "2019 wuhan seafood market virus acute severe respiratory syndrome", "sars coronavirus 2 respiratory illness", "2019 wuhan seafood market pneumonia virus acute severe syndrome", "2019novel wuhan corona virus acute disease", "2019novel chinese corona virus severe acute illness", "2019 severe acute respiratory syndromerelated coronavirus2 pneumonia", "2019novel sars coronavirus 2 respiratory illness", "2019 human coronavirus acute respiratory syndrome", "2019novel chinese corona virus acute severe infection", "disorder from 2019 new severe acute respiratory syndrome corona virus2", "2019 novel sars coronavirus 2 acute disorder", "wuhan corona virus 2019 severe infection", "syndrome from 2019novel wuhan corona virus", "pneumonia from 2019new severe acute respiratory syndrome coronavirus2", "infection caused by 2019new sars coronavirus2", "new coronavirus disease", "syndrome from 2019 new sars coronavirus2", "2019 seafood market corona virus severe acute disorder", "2019new sars coronavirus2 respiratory infection", "chinese corona virus 2019 severe acute pneumonia", "2019novel sars corona virus 2 severe acute respiratory disorder", "severe acute respiratory syndrome corona virus 2 illness", "wuhan virus respiratory infection", "chinese corona virus 2019 acute respiratory syndrome", "2019 new seafood market corona virus acute severe disorder", "syndrome from 2019 new sars corona virus 2", "2019 novel sars corona virus 2 acute pneumonia", "disease due to 2019 seafood market virus", "infection caused by 2019 new wuhan pneumonia virus", "disease due to 2019new sars coronavirus 2", "2019 new seafood market corona virus acute severe disease", "2019novel wuhancoronavirus severe acute illness", "2019 wuhan seafood market pneumonia virus respiratory disorder", "illness from seafood market coronavirus 2019", "disease due to human coronavirus 2019", "disease from 2019 new sars coronavirus2", "2019novel sars coronavirus 2 severe acute respiratory disorder", "illness from human coronavirus 2019", "infection from 2019novel severe acute respiratory syndrome corona virus 2", "2019 new wuhan corona virus disease", "2019new sars coronavirus 2 acute severe respiratory disorder", "novel coronavirus 2019 respiratory syndrome", "2019novel wuhan seafood market pneumonia virus acute severe disorder", "2019new wuhan seafood market virus respiratory disease", "sars corona virus 2 severe acute disease", "2019new seafood market corona virus severe pneumonia", "disorder due to sarscov2 2019", "2019 new chinese corona virus infection", "2019 sars coronavirus 2 respiratory disease", "chinese coronavirus acute severe respiratory syndrome", "infection due to 2019novel sars corona virus 2", "disorder caused by 2019 novel severe acute respiratory syndrome corona virus 2", "sars corona virus2 2019 acute severe disorder", "wuhancorona virus acute respiratory syndrome", "seafood market coronavirus 2019 severe syndrome", "2019 new coronavirus respiratory infection", "seafood market coronavirus acute infection", "2019 new severe acute respiratory syndrome corona virus2 disorder", "2019new wuhan seafood market virus acute severe respiratory infection", "infection caused by 2019novel severe acute respiratory syndromerelated corona virus 2", "syndrome from 2019 sars corona virus 2", "2019 novel sarscov2 acute disease", "2019 novel sars cov 2 respiratory illness", "2019 new sarscov2 severe acute infection", "hcov19 acute severe respiratory illness", "seafood market severe respiratory infection", "seafood market virus", "infection due to seafood market virus 2019", "2019 novel wuhan seafood market pneumonia virus severe respiratory illness", "new corona virus 2019 asevere acute respiratory disease", "2019novel wuhan corona virus acute infection", "wuhancoronavirus acute disorder", "2019 ncov severe acute respiratory illness", "seafood market coronavirus acute severe disorder", "2019 sars coronavirus2 severe acute respiratory illness", "2019 chinese corona virus severe acute respiratory infection", "wuhan coronavirus acute pneumonia", "illness from 2019novel sarscov2", "sarscov2 2019 severe illness", "sars coronavirus2 severe respiratory infection", "pneumonia from 2019 sars corona virus 2", "2019 new wuhan virus syndrome", "2019novel wuhan virus acute syndrome", "sars corona virus 2 acute severe respiratory disease", "disorder caused by sarscov2", "2019 chinese corona virus severe illness", "novel coronavirus severe disease", "2019 new seafood market corona virus severe acute respiratory syndrome", "illness caused by 2019 wuhan seafood market pneumonia virus", "seafood market coronavirus 2019 severe acute disease", "disease due to severe acute respiratory syndrome corona virus 2", "severe acute respiratory syndromerelated corona virus2", "sars coronavirus 2 2019 severe acute infection", "illness due to 2019novel severe acute respiratory syndromerelated corona virus2", "2019novel wuhan pneumonia virus acute severe illness", "2019 sars corona virus 2 severe respiratory disease", "disease caused by new coronavirus", "infection caused by 2019novel wuhan virus", "hcov19 acute syndrome", "2019novel seafood market corona virus severe infection", "disorder from 2019 chinese corona virus", "sars cov 2 2019 acute respiratory disorder", "disorder caused by new coronavirus", "2019new severe acute respiratory syndrome corona virus2", "2019 new seafood market coronavirus acute pneumonia", "seafood market corona virus acute severe syndrome", "2019new wuhancorona virus pneumonia", "sars coronavirus 2 2019 severe syndrome", "2019 novel chinese corona virus disorder", "2019ncov acute severe disease", "disorder due to 2019 new coronavirus", "wuhancoronavirus 2019 severe respiratory syndrome", "infection from novel coronavirus 2019", "2019novel wuhan corona virus syndrome", "seafood market corona virus acute severe respiratory disorder", "2019novel chinese coronavirus respiratory disease", "2019 novel sars cov 2 infection", "disorder caused by 2019novel wuhan seafood market pneumonia virus", "2019novel wuhan coronavirus disorder", "2019 wuhan virus acute severe disease", "2019 seafood market corona virus acute disorder", "illness due to 2019 wuhan corona virus", "seafood market virus acute respiratory disease", "2019 novel wuhan corona virus severe respiratory syndrome", "virus causing covid19 respiratory infection", "wuhan virus acute disorder", "sars cov 2 2019 severe respiratory disease", "infection due to 2019 human coronavirus", "disorder due to severe acute respiratory syndromerelated corona virus 2", "wuhan virus severe acute illness", "2019new wuhancoronavirus syndrome", "2019 new sars coronavirus 2 disorder", "disorder caused by 2019 human corona virus", "2019 chinese coronavirus acute syndrome", "2019 new chinese coronavirus severe illness", "2019new wuhan seafood market pneumonia virus severe disorder", "2019new seafood market coronavirus severe illness", "sars cov 2 severe disorder", "2019 novel wuhan corona virus severe acute infection", "2019 novel seafood market corona virus disorder", "2019new sars corona virus2 acute severe respiratory disease", "2019 new wuhan coronavirus acute disorder", "pneumonia caused by 2019 new seafood market coronavirus", "2019novel sars cov 2 acute respiratory syndrome", "2019 new sars coronavirus 2 severe respiratory syndrome", "disorder due to 2019new seafood market virus", "sars cov 2 2019 pneumonia", "pneumonia caused by covid19 virus", "sars cov 2 acute pneumonia", "wuhan seafood market virus acute pneumonia", "2019 novel wuhan pneumonia virus severe acute disease", "sars coronavirus2 severe pneumonia", "wuhan seafood market pneumonia virus 2019 severe acute syndrome", "seafood market virus severe syndrome", "2019new sars corona virus2 pneumonia", "2019 novel seafood market coronavirus respiratory disorder", "2019novel wuhancorona virus severe acute respiratory syndrome", "illness caused by 2019new sars coronavirus 2", "disorder caused by wuhan pneumonia virus", "covid 19 virus acute severe respiratory syndrome", "2019new sars corona virus2 severe acute respiratory disorder", "disease due to 2019 novel wuhancorona virus", "disorder due to 2019 new wuhan corona virus", "2019 sars coronavirus2 disease", "2019 novel chinese coronavirus acute severe syndrome", "syndrome from chinese coronavirus 2019", "wuhancorona virus 2019 severe acute respiratory syndrome", "chinese corona virus respiratory infection", "seafood market coronavirus 2019 severe acute illness", "pneumonia due to severe acute respiratory syndrome coronavirus 2", "pneumonia from 2019novel severe acute respiratory syndromerelated corona virus2", "wuhan coronavirus acute respiratory disease", "2019 new sars cov 2 acute severe infection", "sars corona virus2 2019 acute severe syndrome", "2019 novel wuhan seafood market pneumonia virus severe respiratory disease", "2019novel wuhan corona virus acute severe pneumonia", "new corona virus 2019 severe respiratory disease", "sarscov2 respiratory illness", "2019 new wuhan virus severe acute respiratory syndrome", "2019 new wuhan seafood market virus acute disease", "2019new chinese coronavirus acute syndrome", "2019new wuhan seafood market virus acute respiratory infection", "illness due to severe acute respiratory syndromerelated corona virus 2 2019", "pneumonia due to 2019 novel wuhan pneumonia virus", "syndrome due to 2019 novel sars corona virus2", "illness from 2019 novel wuhan coronavirus", "disorder from 2019 new coronavirus", "illness due to 2019 new seafood market corona virus", "2019new sars corona virus 2 acute respiratory disease", "infection from 2019 novel seafood market virus", "2019 severe acute respiratory syndromerelated corona virus 2", "2019novel wuhan coronavirus severe acute respiratory disorder", "pneumonia caused by wuhancorona virus 2019", "2019 new wuhan seafood market pneumonia virus acute infection", "severe acute respiratory syndrome corona virus2 2019 disorder", "2019novel sars coronavirus 2 severe disease", "hcov 19", "2019new wuhancorona virus severe respiratory infection", "2019novel wuhan virus acute severe infection", "infection caused by sars coronavirus 2", "infection from wuhan seafood market pneumonia virus 2019", "2019 chinese corona virus acute severe disorder", "seafood market corona virus 2019 acute disorder", "2019 new wuhan seafood market virus syndrome", "human coronavirus 2019 acute respiratory disease", "2019 chinese corona virus severe acute respiratory disorder", "2019 new corona virus severe acute respiratory infection", "severe acute respiratory syndrome coronavirus 2 2019", "2019 wuhan seafood market pneumonia virus acute illness", "infection caused by 2019novel wuhancoronavirus", "illness from 2019novel wuhan seafood market virus", "2019 new sarscov2", "infection due to severe acute respiratory syndrome corona virus2 2019", "disorder due to 2019 new sars coronavirus2", "pneumonia due to chinese corona virus", "hcov19 severe disease", "new coronavirus severe acute infection", "2019 wuhan seafood market pneumonia virus severe acute respiratory syndrome", "infection due to 2019new sars corona virus 2", "2019 new seafood market corona virus severe acute disease", "2019new sars coronavirus 2 syndrome", "disease caused by 2019 new coronavirus", "illness caused by 2019 wuhan corona virus", "disorder from severe acute respiratory syndrome coronavirus 2 2019", "2019 coronavirus pneumonia", "2019 wuhancorona virus respiratory syndrome", "illness from 2019 severe acute respiratory syndromerelated corona virus2", "illness due to wuhancoronavirus", "wuhan seafood market pneumonia virus acute disorder", "2019 sars cov 2 acute severe illness", "illness caused by 2019 novel severe acute respiratory syndrome coronavirus 2", "2019 new corona virus severe acute disorder", "2019 sars corona virus 2 severe disorder", "disorder due to 2019 new severe acute respiratory syndrome coronavirus2", "virus causing covid19 severe syndrome", "2019 novel wuhan seafood market pneumonia virus acute infection", "sars coronavirus2 2019 severe acute respiratory illness", "illness from 2019novel severe acute respiratory syndrome corona virus 2", "2019 sars coronavirus2 severe respiratory infection", "2019 novel seafood market virus severe respiratory illness", "2019novel wuhan seafood market pneumonia virus severe acute pneumonia", "2019novel sars coronavirus 2 acute syndrome", "disorder caused by 2019 new wuhan pneumonia virus", "2019 novel seafood market corona virus acute severe respiratory infection", "2019 chinese corona virus severe acute infection", "2019novel sars corona virus 2 severe acute infection", "wuhan coronavirus severe respiratory infection", "2019 wuhancoronavirus infection", "infection from severe acute respiratory syndromerelated coronavirus 2", "illness caused by 2019 human corona virus", "disorder from 2019 wuhancoronavirus", "infection from 2019novel severe acute respiratory syndrome coronavirus 2", "pneumonia from 2019 new wuhan virus", "sars coronavirus2 severe respiratory syndrome", "2019novel wuhancoronavirus severe respiratory illness", "2019new sarscov2 asevere acute respiratory disease", "disorder due to 2019novel wuhan pneumonia virus", "2019new seafood market corona virus acute severe infection", "2019 novel coronavirus severe acute disease", "wuhancoronavirus 2019 acute disorder", "disorder caused by new coronavirus 2019", "2019 novel wuhancoronavirus respiratory infection", "2019novel wuhancoronavirus respiratory disorder", "2019 novel seafood market corona virus acute severe disorder", "novel corona virus 2019 severe disease", "illness caused by 2019 novel sars corona virus 2", "disorder from 2019 novel sars coronavirus 2", "2019novel seafood market coronavirus severe acute respiratory illness", "seafood market respiratory syndrome", "2019 novel sars corona virus 2 disease", "illness caused by 2019 new seafood market virus", "2019 novel sars cov 2 severe acute infection", "2019 novel sars cov 2 acute respiratory disorder", "disease due to chinese coronavirus", "chinese coronavirus 2019 acute pneumonia", "disorder from 2019 seafood market coronavirus", "wuhan virus severe acute infection", "2019new wuhancorona virus severe respiratory disorder", "2019novel wuhan corona virus severe acute syndrome", "2019 novel wuhancoronavirus illness", "wuhancorona virus 2019 acute respiratory syndrome", "2019 new wuhancorona virus severe acute illness", "2019 wuhan seafood market virus severe acute pneumonia", "2019 wuhan pneumonia virus", "2019novel wuhancorona virus acute severe respiratory illness", "syndrome due to 2019new wuhan seafood market virus", "2019 chinese corona virus severe disorder", "wuhan severe pneumonia", "disease due to 2019 novel wuhan corona virus", "2019 novel sars coronavirus2 acute respiratory disease", "2019new wuhancorona virus illness", "syndrome from 2019novel wuhan virus", "2019 wuhan seafood market virus severe respiratory disorder", "2019novel wuhancorona virus acute severe illness", "2019 novel wuhan pneumonia virus severe syndrome", "seafood market coronavirus severe respiratory syndrome", "human corona virus 2019 severe respiratory disorder", "wuhan virus 2019 acute severe disease", "sars corona virus 2 acute illness", "virus responsible for covid19 disease", "sars coronavirus 2 2019 severe respiratory disease", "disorder caused by 2019 novel severe acute respiratory syndromerelated corona virus2", "2019novel wuhancorona virus acute respiratory disease", "wuhan virus 2019 acute respiratory disorder", "2019novel wuhancoronavirus syndrome", "2019new sars corona virus 2 severe acute illness", "2019 sars corona virus 2 severe respiratory infection", "2019 new sars coronavirus 2 acute respiratory disease", "2019 sars cov 2 severe infection", "disease from 2019ncov", "2019new sars corona virus2 disease", "syndrome due to 2019 wuhancoronavirus", "illness caused by wuhan seafood market pneumonia virus", "disease due to 2019 novel severe acute respiratory syndromerelated coronavirus 2", "2019novel wuhan coronavirus syndrome", "2019new wuhan seafood market virus pneumonia", "2019 novel seafood market virus severe respiratory disorder", "2019new sars corona virus 2 severe pneumonia", "2019novel seafood market corona virus acute respiratory disorder", "2019 sars corona virus 2 acute severe respiratory infection", "disorder caused by covid19 virus", "infection from 2019 severe acute respiratory syndrome corona virus 2", "2019novel severe acute respiratory syndromerelated corona virus2 disease", "2019 wuhan seafood market virus respiratory illness", "2019 new wuhancorona virus acute severe pneumonia", "seafood market corona virus 2019 syndrome", "2019 sars corona virus 2 respiratory syndrome", "hcov 19 acute respiratory infection", "2019novel wuhan corona virus severe illness", "2019 new wuhan virus respiratory illness", "2019 wuhancorona virus acute severe infection", "new corona virus 2019 severe acute pneumonia", "wuhan seafood market virus acute severe disease", "sars coronavirus 2 2019 severe respiratory syndrome", "2019novel severe acute respiratory syndromerelated corona virus 2 pneumonia", "2019novel sarscov2 severe acute disease", "disorder due to 2019 novel sars corona virus2", "2019new wuhan seafood market virus severe acute respiratory illness", "disease from 2019 novel severe acute respiratory syndromerelated corona virus2", "2019 novel wuhancorona virus severe disorder", "2019 new wuhancorona virus severe respiratory disease", "disease from 2019 new seafood market coronavirus", "2019 wuhancoronavirus acute severe respiratory disease", "2019 novel sars coronavirus2 severe acute disease", "2019 novel coronavirus acute respiratory disease", "2019 novel wuhan corona virus acute severe disease", "2019 seafood market virus syndrome", "2019 seafood market virus severe illness", "2019novel wuhan seafood market virus respiratory infection", "seafood market virus severe acute respiratory illness", "disease caused by 2019 wuhancoronavirus", "disease caused by severe acute respiratory syndrome coronavirus2 2019", "2019new wuhan corona virus severe respiratory infection", "new coronavirus acute respiratory illness", "2019novel wuhan seafood market virus disorder", "chinese coronavirus acute respiratory illness", "2019new wuhan coronavirus acute severe syndrome", "2019 wuhan virus pneumonia", "new corona virus 2019 severe disorder", "pneumonia due to severe acute respiratory syndrome coronavirus2 2019", "2019new seafood market virus acute respiratory disorder", "2019 seafood market virus severe infection", "severe acute respiratory syndromerelated coronavirus 2 2019 infection", "2019 new corona virus severe respiratory infection", "sars cov 2 acute disease", "2019 seafood market coronavirus acute severe infection", "seafood market virus 2019 severe acute infection", "2019novel wuhan virus severe acute illness", "2019new sars corona virus2 severe syndrome", "infection caused by 2019novel severe acute respiratory syndrome corona virus 2", "disorder caused by seafood market corona virus 2019", "pneumonia caused by 2019 novel sars cov 2", "2019 novel sars cov 2 severe respiratory illness", "wuhan acute severe respiratory syndrome", "2019 new severe acute respiratory syndrome corona virus 2 infection", "syndrome from 2019 new coronavirus", "illness due to 2019 novel wuhancoronavirus", "2019new wuhancorona virus respiratory syndrome", "2019new sars cov 2 severe acute illness", "disorder due to 2019new wuhan pneumonia virus", "virus causing covid19 acute severe disease", "2019 novel corona virus severe respiratory illness", "wuhan coronavirus 2019 acute severe respiratory syndrome", "severe acute respiratory syndromerelated corona virus 2 2019 illness", "disorder caused by severe acute respiratory syndromerelated corona virus2 2019", "disorder due to 2019 novel severe acute respiratory syndrome corona virus2", "pneumonia due to sars cov 2 2019", "wuhancorona virus acute severe syndrome", "novel corona virus severe syndrome", "2019 wuhan seafood market virus acute respiratory illness", "2019 new wuhan seafood market virus acute severe disease", "2019 new wuhan corona virus severe acute illness", "wuhan seafood market pneumonia virus 2019 severe acute infection", "2019novel wuhancoronavirus severe illness", "novel coronavirus 2019 severe acute disorder", "disorder from new coronavirus", "illness due to sars corona virus2 2019", "human corona virus 2019 acute respiratory disease", "2019 novel wuhan coronavirus severe acute syndrome", "2019new seafood market corona virus severe respiratory disorder", "wuhan seafood market virus disease", "2019new wuhan seafood market pneumonia virus acute severe respiratory illness", "2019 human coronavirus acute severe disease", "sars cov 2 2019 severe acute respiratory illness", "infection caused by 2019 severe acute respiratory syndromerelated coronavirus2", "wuhan coronavirus 2019 acute severe respiratory infection", "sars coronavirus 2 acute severe infection", "pneumonia caused by 2019 new corona virus", "syndrome due to 2019 new wuhan seafood market virus", "disorder due to 2019 novel wuhan coronavirus", "severe acute respiratory syndromerelated coronavirus 2 illness", "2019new sars coronavirus 2 illness", "2019 wuhan pneumonia virus severe acute respiratory infection", "2019new chinese corona virus severe acute disorder", "illness due to 2019 new wuhan virus", "2019new severe acute respiratory syndrome coronavirus 2 pneumonia", "seafood market virus 2019 respiratory infection", "2019 new sars cov 2 acute severe respiratory disorder", "sars cov 2 syndrome", "2019 new wuhan pneumonia virus severe acute syndrome", "2019novel sars coronavirus 2 acute illness", "2019novel seafood market virus acute severe disorder", "seafood market corona virus acute illness", "new corona virus severe acute pneumonia", "disorder caused by 2019 novel wuhan virus", "2019novel sarscov2 acute severe illness", "2019new chinese coronavirus disease", "2019novel sars cov 2 acute severe respiratory disease", "2019novel severe acute respiratory syndrome corona virus 2 illness", "disorder caused by wuhan virus", "novel coronavirus 2019", "infection due to chinese corona virus 2019", "2019 sars coronavirus2 respiratory disease", "sars corona virus 2 acute respiratory disease", "sars coronavirus 2 severe acute respiratory syndrome", "wuhan pneumonia virus 2019 acute severe respiratory syndrome", "disorder due to new corona virus 2019", "2019 novel wuhancorona virus severe acute respiratory infection", "2019novel wuhan virus respiratory disorder", "covid19 virus acute respiratory illness", "syndrome from new corona virus 2019", "hcov 19 syndrome", "2019 novel wuhan virus severe acute pneumonia", "syndrome caused by 2019new sars cov 2", "2019novel wuhan corona virus illness", "2019 novel wuhan corona virus respiratory infection", "chinese corona virus severe pneumonia", "syndrome caused by 2019novel sars coronavirus 2", "pneumonia caused by 2019 new chinese corona virus", "2019novel chinese coronavirus respiratory disorder", "2019novel severe acute respiratory syndrome coronavirus2 disorder", "new coronavirus 2019 severe acute disorder", "2019 novel chinese coronavirus severe acute disorder", "pneumonia caused by 2019 new wuhan coronavirus", "disease from 2019 new seafood market corona virus", "2019 wuhancoronavirus acute infection", "illness due to 2019new chinese coronavirus", "2019 corona virus respiratory illness", "seafood market virus 2019 acute infection", "seafood market virus acute disorder", "pneumonia caused by 2019novel severe acute respiratory syndromerelated corona virus 2", "2019 new sars corona virus2 respiratory illness", "illness from 2019 new severe acute respiratory syndrome corona virus 2", "disease from 2019 coronavirus", "new corona virus 2019 respiratory disease", "disease caused by 2019 new sars corona virus 2", "2019 chinese coronavirus acute disorder", "2019novel chinese corona virus acute syndrome", "2019 coronavirus acute severe illness", "2019novel chinese coronavirus severe respiratory infection", "illness caused by wuhan virus 2019", "syndrome caused by 2019new wuhancoronavirus", "illness due to wuhan seafood market pneumonia virus 2019", "sars coronavirus 2 2019 acute respiratory illness", "illness due to wuhan seafood market virus", "2019 seafood market coronavirus severe respiratory infection", "2019new sars cov 2 severe respiratory syndrome", "2019new seafood market corona virus acute severe disease", "infection due to 2019new chinese coronavirus", "infection from 2019new wuhancoronavirus", "2019novel chinese coronavirus severe infection", "disorder due to 2019 new sars corona virus 2", "2019new severe acute respiratory syndromerelated coronavirus 2 illness", "new coronavirus 2019 respiratory illness", "illness caused by 2019 severe acute respiratory syndrome coronavirus2", "pneumonia due to severe acute respiratory syndrome corona virus 2", "2019new sars cov 2 acute severe disorder", "illness caused by severe acute respiratory syndromerelated coronavirus 2", "2019novel severe acute respiratory syndrome coronavirus 2", "2019novel wuhan seafood market pneumonia virus severe acute respiratory syndrome", "pneumonia from 2019 wuhan seafood market virus", "chinese corona virus acute respiratory illness", "2019 new corona virus illness", "2019novel sarscov2 acute severe infection", "syndrome caused by 2019 novel chinese corona virus", "chinese corona virus 2019 severe respiratory disease", "severe acute respiratory syndromerelated coronavirus2 disorder", "2019 novel sarscov2 respiratory syndrome", "2019 new sars coronavirus2 severe pneumonia", "2019 human coronavirus illness", "2019novel wuhan corona virus severe respiratory illness", "2019new sars coronavirus2 acute disorder", "disease caused by novel corona virus", "illness from 2019 ncov", "sars corona virus 2 severe pneumonia", "2019 wuhancorona virus severe respiratory disease", "2019novel seafood market virus acute severe respiratory illness", "2019novel wuhan virus acute severe respiratory illness", "2019new seafood market corona virus acute severe respiratory illness", "2019 chinese coronavirus acute illness", "seafood market corona virus severe disease", "2019new seafood market corona virus severe acute illness", "human coronavirus 2019 severe respiratory disease", "wuhan virus 2019 acute respiratory illness", "2019new sars corona virus 2 acute severe respiratory illness", "infection from 2019new sars coronavirus 2", "2019new wuhancorona virus disease", "2019 wuhancoronavirus illness", "2019 novel chinese corona virus severe acute illness", "2019new wuhan pneumonia virus severe acute respiratory syndrome", "2019novel sars coronavirus 2 asevere acute respiratory disease", "pneumonia due to 2019 novel coronavirus", "wuhan severe illness", "sars cov 2 2019 severe acute illness", "wuhancorona virus 2019 severe acute respiratory illness", "2019new chinese corona virus acute severe infection", "disease caused by sars cov 2 2019", "disease caused by 2019 novel wuhancoronavirus", "infection due to 2019 new wuhancorona virus", "disease due to sars corona virus2 2019", "syndrome due to 2019novel chinese corona virus", "2019 new corona virus disease", "2019novel sarscov2 acute severe respiratory disease", "wuhan virus acute severe respiratory infection", "seafood market coronavirus 2019 acute severe disorder", "2019 new wuhancorona virus disorder", "syndrome caused by 2019 corona virus", "2019 wuhan seafood market pneumonia virus acute severe pneumonia", "2019 novel seafood market coronavirus severe acute illness", "infection caused by 2019new sars corona virus 2", "2019new chinese coronavirus severe syndrome", "sars corona virus2 2019 asevere acute respiratory disease", "sars coronavirus2 2019 severe disorder", "2019 wuhan pneumonia virus respiratory illness", "2019 severe acute respiratory syndromerelated corona virus 2 disorder", "syndrome due to 2019new sarscov2", "2019new seafood market corona virus pneumonia", "2019 new sars coronavirus2 acute severe respiratory disorder", "2019 wuhan seafood market virus severe acute respiratory disorder", "2019 sars corona virus2 acute respiratory disorder", "2019 wuhan seafood market pneumonia virus severe acute illness", "2019new wuhancoronavirus acute respiratory syndrome", "pneumonia from seafood market virus", "illness from wuhan virus", "2019 novel wuhan virus severe illness", "new coronavirus acute severe infection", "2019 novel coronavirus acute respiratory disorder", "wuhan seafood market pneumonia virus severe acute infection", "sars coronavirus 2 2019 severe disease", "2019 new sars corona virus2 severe acute respiratory syndrome", "2019 novel severe acute respiratory syndrome corona virus 2 infection", "infection due to novel coronavirus", "2019novel chinese coronavirus respiratory illness", "infection caused by severe acute respiratory syndrome corona virus2 2019", "syndrome due to 2019 novel sars coronavirus 2", "disorder from human corona virus 2019", "human corona virus 2019 severe disease", "disorder from 2019novel wuhan coronavirus", "2019 wuhan pneumonia virus disorder", "2019 coronavirus acute severe infection", "2019 new chinese corona virus severe respiratory infection", "2019new sars corona virus 2 severe acute respiratory illness", "2019 new coronavirus severe respiratory illness", "hubei acute severe pneumonia", "disorder from 2019 novel wuhan virus", "infection from 2019 novel severe acute respiratory syndromerelated corona virus2", "2019 chinese corona virus severe disease", "2019 seafood market virus severe pneumonia", "2019 wuhan seafood market virus severe respiratory disease", "covid 19 virus syndrome", "2019 wuhancoronavirus acute severe illness", "2019new chinese corona virus asevere acute respiratory disease", "disease from seafood market coronavirus", "2019 seafood market coronavirus severe acute pneumonia", "disorder due to wuhan seafood market virus", "2019 sars cov 2 acute severe respiratory illness", "seafood market virus severe acute respiratory syndrome", "2019new wuhan corona virus acute infection", "syndrome caused by 2019 human coronavirus", "2019 wuhan seafood market virus respiratory disease", "2019 new sars corona virus2 severe acute infection", "illness caused by 2019novel severe acute respiratory syndrome coronavirus2", "wuhancoronavirus severe acute infection", "2019novel sars coronavirus2 acute severe respiratory infection", "2019new wuhan pneumonia virus acute infection", "pneumonia due to 2019 seafood market coronavirus", "2019 new coronavirus acute disease", "2019 wuhancoronavirus acute severe respiratory syndrome", "sars coronavirus2 acute severe pneumonia", "2019novel wuhancorona virus acute syndrome", "pneumonia caused by wuhan pneumonia virus", "covid19 virus severe acute respiratory infection", "2019novel sars corona virus2 acute severe respiratory illness", "infection from 2019 novel seafood market coronavirus", "2019 new wuhan corona virus acute disease", "wuhan corona virus 2019 severe respiratory disorder", "illness due to 2019 novel sars corona virus2", "2019new seafood market corona virus respiratory illness", "wuhancorona virus 2019 acute syndrome", "2019novel wuhan seafood market virus acute pneumonia", "wuhancorona virus acute pneumonia", "seafood market coronavirus 2019 acute syndrome", "wuhancorona virus 2019 acute respiratory illness", "coronavirus disease19", "human corona virus 2019 severe acute pneumonia", "2019new sarscov2 severe disease", "hubei severe infection", "2019 new wuhan seafood market pneumonia virus asevere acute respiratory disease", "new corona virus severe acute illness", "illness from new coronavirus", "2019 new wuhan pneumonia virus severe infection", "wuhancorona virus 2019 severe illness", "2019ncov pneumonia", "syndrome due to chinese corona virus 2019", "sars coronavirus2 2019 severe illness", "disorder caused by 2019novel chinese corona virus", "illness from 2019new wuhan corona virus", "disorder due to 2019novel seafood market coronavirus", "wuhan seafood market virus 2019 respiratory illness", "2019 wuhan corona virus acute respiratory disorder", "disorder due to 2019 novel wuhancorona virus", "2019 wuhancorona virus acute respiratory infection", "2019new chinese corona virus severe disorder", "disease from 2019 severe acute respiratory syndrome corona virus2", "pneumonia from 2019ncov", "2019 novel wuhan virus syndrome", "2019 wuhan corona virus severe acute syndrome", "2019novel sars cov 2 acute disorder", "2019 novel wuhan coronavirus severe pneumonia", "new coronavirus acute severe respiratory disease", "disorder due to 2019 novel severe acute respiratory syndromerelated coronavirus2", "2019 wuhancorona virus severe disorder", "2019new wuhan coronavirus acute severe illness", "2019novel sars coronavirus2 severe pneumonia", "seafood market coronavirus 2019 acute severe respiratory syndrome", "chinese coronavirus severe acute respiratory syndrome", "2019 new seafood market virus disorder", "2019new chinese coronavirus severe acute illness", "illness caused by 2019new severe acute respiratory syndrome coronavirus 2", "illness due to chinese coronavirus 2019", "2019new wuhan seafood market virus acute severe disease", "2019 chinese coronavirus disorder", "2019novel sars cov 2 severe acute respiratory disorder", "disease caused by 2019new sars corona virus 2", "illness caused by 2019new seafood market coronavirus", "illness caused by new coronavirus", "novel coronavirus severe syndrome", "infection from new corona virus 2019", "2019 new wuhan virus acute disorder", "illness from hcov19", "new corona virus acute disorder", "illness caused by 2019 sars corona virus2", "infection from 2019new wuhan corona virus", "2019 severe acute respiratory syndromerelated coronavirus2", "pneumonia caused by 2019novel sars coronavirus2", "novel coronavirus 2019 acute infection", "disease caused by 2019 new wuhan coronavirus", "2019novel sars corona virus 2 disorder", "2019 novel corona virus infection", "2019new seafood market virus acute severe respiratory illness", "2019novel sarscov2 severe illness", "2019 new wuhancoronavirus acute syndrome", "2019 novel wuhan virus disease", "2019 wuhan virus severe acute syndrome", "2019 novel sars corona virus 2 acute severe respiratory disorder", "syndrome caused by new corona virus 2019", "new coronavirus 2019 acute severe infection", "2019novel sars corona virus 2 asevere acute respiratory disease", "2019 novel sars corona virus 2 acute severe respiratory syndrome", "2019new wuhan seafood market pneumonia virus severe acute respiratory infection", "wuhan seafood market pneumonia virus 2019 asevere acute respiratory disease", "sars corona virus2 acute respiratory illness", "2019 novel coronavirus pneumonia", "hubei acute severe infection", "2019new wuhan seafood market virus severe acute syndrome", "seafood market virus 2019 severe acute syndrome", "2019 novel wuhan pneumonia virus severe acute respiratory infection", "2019 wuhan coronavirus severe pneumonia", "wuhan seafood market virus acute syndrome", "2019new wuhan virus severe pneumonia", "2019novel wuhan corona virus respiratory infection", "2019 novel sars coronavirus2 severe acute infection", "2019 novel wuhan seafood market virus acute respiratory disorder", "2019new wuhan seafood market virus acute severe respiratory disease", "2019 new sarscov2 acute severe disorder", "infection caused by 2019new sars coronavirus 2", "syndrome caused by 2019 human corona virus", "pneumonia due to sars corona virus2", "2019new wuhan virus severe respiratory illness", "wikidata:Q84263196", "illness caused by hcov 19", "2019 novel wuhan pneumonia virus acute severe respiratory disease", "2019 new severe acute respiratory syndromerelated coronavirus 2 pneumonia", "2019 wuhan seafood market pneumonia virus acute pneumonia", "2019 new wuhan pneumonia virus severe respiratory disease", "wuhan seafood market virus acute severe respiratory disease", "2019 new sars coronavirus2 acute severe disorder", "2019new wuhan coronavirus respiratory infection", "2019new seafood market coronavirus severe respiratory disease", "2019new sars corona virus2 acute disorder", "disorder caused by 2019 novel sars corona virus 2", "infection due to 2019ncov", "disorder caused by novel corona virus", "2019 new chinese corona virus respiratory disease", "2019 new seafood market corona virus severe illness", "2019novel wuhan virus acute disease", "new corona virus 2019 severe pneumonia", "2019 novel corona virus severe respiratory disorder", "2019new wuhan coronavirus severe respiratory infection", "2019 new wuhan pneumonia virus acute severe respiratory disorder", "2019 ncov acute respiratory illness", "2019novel chinese corona virus respiratory disease", "new coronavirus severe acute respiratory illness", "syndrome caused by wuhancoronavirus", "disorder caused by wuhancorona virus", "disorder from 2019 wuhan seafood market pneumonia virus", "disorder from 2019new wuhancoronavirus", "disorder from 2019new sars cov 2", "2019 sars cov 2", "chinese coronavirus severe disease", "novel corona virus acute illness", "pneumonia from wuhan pneumonia virus", "2019 new severe acute respiratory syndromerelated coronavirus2", "infection from severe acute respiratory syndrome coronavirus 2 2019", "2019novel seafood market virus acute pneumonia", "2019 sars corona virus 2 syndrome", "seafood market coronavirus disorder", "2019 wuhancoronavirus pneumonia", "2019novel wuhan corona virus severe disease", "2019new wuhan seafood market virus acute severe respiratory syndrome", "disorder from 2019novel sars coronavirus 2", "syndrome caused by sars cov 2 2019", "illness due to 2019 wuhan virus", "disorder due to 2019 new sars coronavirus 2", "infection caused by 2019 new wuhancoronavirus", "human corona virus 2019 severe acute respiratory disorder", "illness due to severe acute respiratory syndrome coronavirus2 2019", "infection from 2019 novel severe acute respiratory syndromerelated corona virus 2", "2019novel chinese coronavirus respiratory syndrome", "disease from 2019 novel coronavirus", "wuhancorona virus 2019 infection", "2019 wuhan seafood market pneumonia virus illness", "disease caused by 2019 novel wuhan virus", "sars coronavirus2 2019 acute severe illness", "disease from sars corona virus2 2019", "2019novel chinese coronavirus illness", "2019novel sars cov 2 acute pneumonia", "seafood market corona virus 2019 illness", "pneumonia from seafood market virus 2019", "new coronavirus respiratory disorder", "sars coronavirus2 2019 acute respiratory infection", "2019 novel wuhan virus acute severe respiratory disease", "disease due to 2019 new severe acute respiratory syndrome corona virus 2", "sars coronavirus 2 2019 respiratory disorder", "syndrome caused by seafood market coronavirus", "disease due to wuhan coronavirus", "2019new chinese corona virus acute respiratory illness", "2019new seafood market corona virus severe acute respiratory syndrome", "novel corona virus 2019 acute severe respiratory illness", "2019 wuhan pneumonia virus acute severe pneumonia", "disease from 2019new severe acute respiratory syndromerelated corona virus2", "2019new sars corona virus 2 acute respiratory disorder", "2019 sars coronavirus2 acute severe disorder", "infection due to 2019novel sars coronavirus 2", "disorder caused by 2019 novel chinese corona virus", "2019 chinese coronavirus acute severe illness", "2019 novel wuhan seafood market pneumonia virus infection", "2019 new wuhan seafood market pneumonia virus infection", "novel corona virus severe respiratory illness", "wuhancorona virus severe acute illness", "2019novel sarscov2 severe infection", "2019novel sars coronavirus2 acute severe respiratory disease", "2019 novel wuhan virus acute severe respiratory syndrome", "disease due to 2019 novel seafood market virus", "pneumonia from 2019novel sars corona virus2", "human coronavirus 2019 syndrome", "wuhan coronavirus 2019 severe acute illness", "2019new sarscov2 severe acute disease", "sars corona virus 2 2019 acute severe infection", "wuhan pneumonia virus severe syndrome", "sars cov 2 2019 severe acute respiratory disorder", "infection caused by severe acute respiratory syndrome coronavirus2 2019", "2019new wuhan seafood market pneumonia virus acute severe pneumonia", "wuhancoronavirus severe acute illness", "pneumonia caused by 2019 novel wuhan coronavirus", "2019 wuhan pneumonia virus severe pneumonia", "2019new wuhan coronavirus acute respiratory syndrome", "infection caused by 2019new severe acute respiratory syndrome coronavirus2", "syndrome due to sars corona virus2 2019", "chinese corona virus 2019 disease", "severe acute respiratory syndromerelated corona virus2 2019 infection", "2019new sars coronavirus2 acute respiratory disorder", "2019 wuhan seafood market pneumonia virus disease", "seafood market corona virus 2019 severe respiratory infection", "new corona virus acute severe respiratory syndrome", "novel coronavirus 2019 acute severe respiratory syndrome", "pneumonia caused by 2019novel sars coronavirus 2", "novel coronavirus 2019 severe acute respiratory infection", "infection due to 2019new severe acute respiratory syndrome corona virus 2", "infection due to wuhan pneumonia virus", "illness caused by 2019 new severe acute respiratory syndrome corona virus 2", "2019 novel wuhan coronavirus acute respiratory illness", "novel coronavirus 2019 infection", "infection from 2019 new wuhan coronavirus", "2019 sars corona virus2 infection", "2019novel sarscov2 acute respiratory infection", "disorder due to 2019 new sarscov2", "infection due to 2019 sarscov2", "chinese corona virus severe acute respiratory syndrome", "2019 new coronavirus severe acute pneumonia", "2019 new seafood market virus severe acute respiratory syndrome", "illness from 2019 chinese corona virus", "2019 novel sars coronavirus 2 acute respiratory illness", "2019 wuhancoronavirus severe respiratory illness", "2019novel sars coronavirus 2 severe illness", "2019 wuhan seafood market virus severe acute disorder", "sarscov2 2019 syndrome", "2019 chinese coronavirus acute severe respiratory disease", "2019novel wuhan seafood market pneumonia virus acute disease", "2019 novel wuhan virus severe acute infection", "new coronavirus 2019 acute severe respiratory syndrome", "2019novel sars coronavirus2 acute respiratory illness", "2019 new sarscov2 acute severe infection", "2019 novel chinese corona virus acute syndrome", "2019 corona virus severe syndrome", "illness from 2019new sars corona virus 2", "2019novel wuhancorona virus severe acute disease", "2019 novel wuhan seafood market virus syndrome", "2019 wuhan coronavirus acute respiratory infection", "2019new wuhan coronavirus illness", "2019 new sarscov2 syndrome", "new corona virus severe respiratory disease", "syndrome caused by covid 19 virus", "2019 wuhancorona virus acute severe respiratory infection", "pneumonia from 2019novel wuhancorona virus", "human corona virus 2019 illness", "2019 new wuhancorona virus severe pneumonia", "2019 novel wuhan coronavirus acute severe respiratory disease", "chinese corona virus acute severe syndrome", "2019novel wuhan pneumonia virus severe respiratory illness", "pneumonia caused by 2019novel seafood market coronavirus", "disease due to 2019 severe acute respiratory syndromerelated corona virus 2", "sars corona virus2 acute severe respiratory illness", "infection from 2019 novel wuhancoronavirus", "2019 seafood market virus severe disorder", "seafood market coronavirus severe respiratory disorder", "novel corona virus acute respiratory syndrome", "2019 wuhan coronavirus severe acute infection", "wuhan seafood market virus acute severe disorder", "2019 novel seafood market coronavirus severe infection", "2019novel wuhan coronavirus severe respiratory illness", "infection caused by 2019 novel seafood market corona virus", "wuhan coronavirus acute disease", "2019 wuhancorona virus respiratory illness", "syndrome caused by chinese corona virus", "2019 human coronavirus acute severe illness", "2019 new wuhan coronavirus acute respiratory syndrome", "2019novel sars cov 2 severe pneumonia", "wuhan virus acute severe illness", "2019 new wuhan coronavirus acute severe respiratory disorder", "2019new severe acute respiratory syndrome corona virus2 disease", "2019novel sars coronavirus 2 severe respiratory infection", "wuhancoronavirus 2019 infection", "2019 novel sars coronavirus2 illness", "wuhancoronavirus 2019 acute severe illness", "2019 new sars coronavirus2 severe acute syndrome", "novel corona virus 2019 infection", "illness caused by 2019 new severe acute respiratory syndromerelated coronavirus 2", "seafood market infection", "2019novel seafood market coronavirus severe disease", "2019new sars corona virus 2 severe respiratory disorder", "2019 wuhancoronavirus disorder", "novel coronavirus respiratory disorder", "illness due to 2019novel seafood market coronavirus", "illness due to 2019novel sars coronavirus 2", "disease caused by 2019 new sars coronavirus 2", "disorder from wuhancorona virus 2019", "novel corona virus disorder", "syndrome from 2019 sars coronavirus 2", "illness caused by 2019 sars coronavirus2", "2019new wuhancoronavirus severe pneumonia", "2019 new wuhan seafood market virus respiratory disorder", "2019 new wuhan coronavirus acute pneumonia", "pneumonia due to sarscov2 2019", "2019 new corona virus acute severe respiratory illness", "2019 new seafood market coronavirus acute respiratory infection", "sars coronavirus 2 acute respiratory infection", "seafood market corona virus 2019 acute severe syndrome", "2019 sars corona virus2 acute severe respiratory disorder", "new coronavirus asevere acute respiratory disease", "2019 ncov severe disorder", "wuhan virus disease", "virus causing covid19 severe acute respiratory infection", "2019 novel corona virus acute severe pneumonia", "2019novel wuhan coronavirus severe acute disorder", "2019new wuhan seafood market virus severe acute respiratory disorder", "wuhan seafood market virus 2019 acute severe infection", "disease caused by seafood market corona virus", "2019 sars corona virus 2 asevere acute respiratory disease", "2019 new chinese corona virus severe acute disease", "syndrome caused by 2019 new sars coronavirus 2", "illness due to 2019new seafood market virus", "infection due to 2019new severe acute respiratory syndrome coronavirus2", "2019 new wuhancorona virus acute respiratory infection", "2019novel chinese corona virus pneumonia", "2019novel wuhancoronavirus acute severe respiratory syndrome", "sarscov2 severe acute respiratory illness", "infection caused by 2019 sars corona virus 2", "disease from novel coronavirus 2019", "2019 new wuhan seafood market virus asevere acute respiratory disease", "pneumonia caused by wuhan seafood market virus", "disease from 2019 new severe acute respiratory syndrome corona virus2", "seafood market severe pneumonia", "2019 new wuhancoronavirus severe acute disorder", "2019 new sars corona virus 2 disease", "2019 novel wuhan pneumonia virus acute disease", "wuhan corona virus severe illness", "2019new wuhan corona virus acute severe disease", "sars coronavirus 2 acute severe pneumonia", "2019 wuhancorona virus severe syndrome", "2019 wuhan seafood market pneumonia virus acute disease", "2019novel wuhancoronavirus respiratory illness", "2019new seafood market virus acute disease", "2019novel wuhan seafood market virus acute disorder", "covid19 virus acute pneumonia", "covid 19 virus acute respiratory syndrome", "illness from 2019 seafood market corona virus", "2019new severe acute respiratory syndromerelated corona virus2 pneumonia", "2019 novel corona virus", "2019 new severe acute respiratory syndromerelated corona virus2 disorder", "novel coronavirus acute disorder", "2019 coronavirus acute severe pneumonia", "syndrome caused by novel coronavirus", "pneumonia caused by 2019 novel seafood market coronavirus", "sars corona virus2 2019 acute severe respiratory disease", "2019novel sars corona virus 2 severe disorder", "new coronavirus 2019 severe acute infection", "2019 coronavirus respiratory disorder", "disorder due to 2019new chinese corona virus", "disease caused by 2019 new seafood market corona virus", "disease due to 2019 new severe acute respiratory syndromerelated corona virus 2", "new coronavirus acute severe pneumonia", "seafood market virus 2019 acute severe respiratory illness", "2019new severe acute respiratory syndromerelated corona virus 2 disease", "2019 novel wuhan seafood market virus severe disease", "2019new wuhan seafood market virus severe acute pneumonia", "infection due to 2019 wuhan corona virus", "infection caused by sars cov 2 2019", "disorder from wuhancoronavirus", "2019new severe acute respiratory syndrome corona virus 2 pneumonia", "illness due to 2019 novel wuhan corona virus", "disease due to 2019 new wuhan corona virus", "disorder due to novel corona virus", "hubei acute respiratory syndrome", "hcov19 severe respiratory disease", "illness caused by 2019 new corona virus", "syndrome from 2019 chinese corona virus", "disorder from 2019new severe acute respiratory syndrome coronavirus 2", "2019 sars corona virus 2 severe acute disorder", "2019 new seafood market virus acute severe respiratory infection", "disease caused by human coronavirus 2019", "2019novel sars corona virus 2 pneumonia", "seafood market virus 2019 acute respiratory syndrome", "2019 new seafood market coronavirus acute respiratory disorder", "sars coronavirus 2 acute respiratory syndrome", "disease from 2019 new sars corona virus2", "2019 wuhan seafood market virus severe acute infection", "infection caused by 2019 new severe acute respiratory syndromerelated corona virus2", "syndrome caused by seafood market corona virus 2019", "2019novel seafood market coronavirus infection", "2019 new wuhan coronavirus severe illness", "2019new seafood market virus acute severe disorder", "2019 novel wuhan seafood market virus asevere acute respiratory disease", "2019new wuhan virus severe acute disease", "2019 wuhan seafood market virus infection", "disorder due to 2019novel chinese corona virus", "syndrome due to sars coronavirus2 2019", "wuhan seafood market pneumonia virus 2019 respiratory syndrome", "hcov19 acute respiratory syndrome", "sars corona virus 2 2019 respiratory illness", "2019 sars cov 2 acute disorder", "2019 novel seafood market virus pneumonia", "illness due to 2019 wuhancorona virus", "sars coronavirus2 2019 severe acute respiratory disorder", "2019 coronavirus", "2019 wuhan seafood market virus disorder", "novel coronavirus severe infection", "syndrome due to 2019novel chinese coronavirus", "wuhancoronavirus 2019 severe acute infection", "syndrome from 2019new seafood market virus", "pneumonia from 2019 new severe acute respiratory syndromerelated coronavirus 2", "novel coronavirus 2019 disorder", "2019new seafood market coronavirus respiratory disorder", "wuhan seafood market pneumonia virus 2019 respiratory disease", "sars cov 2 2019 illness", "2019new sars corona virus 2 syndrome", "pneumonia from 2019novel seafood market virus", "disorder from sars cov 2 2019", "infection from severe acute respiratory syndrome coronavirus 2", "2019 human coronavirus acute pneumonia", "pneumonia due to covid 19 virus", "2019 sars corona virus 2 severe acute disease", "pneumonia caused by 2019 new severe acute respiratory syndromerelated coronavirus2", "2019new sars corona virus2 respiratory infection", "pneumonia caused by 2019novel severe acute respiratory syndrome coronavirus 2", "2019new wuhan seafood market pneumonia virus respiratory syndrome", "2019 new wuhan seafood market pneumonia virus acute respiratory disease", "novel coronavirus 2019 acute illness", "2019 wuhancorona virus acute severe pneumonia", "disease caused by 2019 new severe acute respiratory syndromerelated corona virus 2", "2019new chinese coronavirus acute respiratory disease", "syndrome due to 2019new wuhan virus", "2019new chinese corona virus severe acute infection", "disease from severe acute respiratory syndrome coronavirus2 2019", "2019new sars corona virus2 respiratory syndrome", "2019novel wuhan corona virus severe pneumonia", "2019novel wuhan seafood market virus severe illness", "wuhan seafood market pneumonia virus severe acute disorder", "2019 wuhancoronavirus severe infection", "novel coronavirus 2019 acute severe respiratory infection", "2019new sars cov 2 severe acute disorder", "hcov 19 illness", "syndrome due to wuhan seafood market pneumonia virus", "2019 sars corona virus 2 acute respiratory disease", "wuhan coronavirus 2019 acute respiratory syndrome", "wuhan seafood market pneumonia virus acute severe illness", "2019novel chinese corona virus severe acute disorder", "infection caused by 2019new sars cov 2", "disorder caused by 2019novel seafood market virus", "hcov19 respiratory illness", "seafood market virus 2019 severe acute respiratory disorder", "disease caused by 2019novel severe acute respiratory syndrome coronavirus2", "2019 wuhan seafood market pneumonia virus respiratory syndrome", "2019new sars cov 2 acute respiratory infection", "2019 novel wuhan seafood market pneumonia virus acute pneumonia", "sars coronavirus 2 acute respiratory illness", "hcov 19 acute infection", "2019 chinese coronavirus asevere acute respiratory disease", "2019 new wuhan seafood market pneumonia virus", "2019novel wuhan seafood market virus acute severe respiratory disorder", "syndrome from 2019 new sars cov 2", "hubei severe acute infection", "syndrome caused by wuhan seafood market pneumonia virus 2019", "wuhan virus severe acute respiratory disorder", "2019 new wuhan coronavirus acute syndrome", "new coronavirus severe illness", "covid19 virus severe pneumonia", "2019 novel wuhancorona virus acute severe respiratory disease", "2019new wuhan coronavirus infection", "illness caused by 2019 new seafood market corona virus", "novel coronavirus severe pneumonia", "syndrome from 2019 novel sars coronavirus2", "wuhan pneumonia virus acute severe disorder", "txid2697049", "human corona virus 2019 acute severe syndrome", "sars cov 2 severe respiratory disease", "seafood market coronavirus 2019 infection", "2019 new chinese coronavirus severe syndrome", "2019new wuhancoronavirus respiratory disease", "2019 new sars coronavirus2 acute severe respiratory syndrome", "2019 novel wuhancorona virus severe pneumonia", "2019 new coronavirus respiratory syndrome", "new corona virus 2019 acute severe respiratory syndrome", "disorder due to seafood market virus 2019", "syndrome from 2019 novel wuhancorona virus", "2019 novel wuhan pneumonia virus severe pneumonia", "2019 novel coronavirus asevere acute respiratory disease", "2019 novel chinese coronavirus severe respiratory infection", "2019 new wuhan corona virus acute respiratory disorder", "2019novel wuhan seafood market virus illness", "2019new chinese coronavirus acute disease", "disorder due to novel coronavirus 2019", "virus causing covid19 acute respiratory disease", "chinese coronavirus severe disorder", "2019new sars corona virus 2 respiratory syndrome", "sars coronavirus 2 2019 acute illness", "2019new wuhan corona virus severe acute disorder", "2019new seafood market coronavirus asevere acute respiratory disease", "2019 novel seafood market corona virus severe respiratory syndrome", "infection from 2019 new wuhancoronavirus", "pneumonia due to 2019 novel severe acute respiratory syndrome coronavirus2", "2019novel wuhancorona virus severe disease", "pneumonia caused by seafood market corona virus 2019", "2019 human coronavirus severe respiratory syndrome", "wuhancoronavirus disorder", "new coronavirus acute illness", "illness caused by 2019novel seafood market virus", "sars coronavirus 2 severe disorder", "novel corona virus acute severe disorder", "wuhan seafood market pneumonia virus 2019 severe disease", "seafood market corona virus 2019 acute disease", "syndrome from wuhan virus", "severe acute respiratory syndromerelated corona virus 2 2019 disease", "2019novel seafood market coronavirus acute severe respiratory syndrome", "infection due to 2019 new seafood market coronavirus", "2019new sars corona virus2 severe respiratory infection", "2019 novel sars corona virus2 acute severe infection", "sarscov2 severe acute pneumonia", "sars coronavirus2 2019", "2019new wuhan seafood market virus severe respiratory disorder", "wuhan virus acute illness", "disorder caused by 2019novel seafood market coronavirus", "2019novel seafood market coronavirus disease", "disorder caused by 2019novel seafood market corona virus", "disorder from 2019 novel sars cov 2", "wuhan pneumonia virus", "2019 human coronavirus severe illness", "2019 new wuhan pneumonia virus severe acute disease", "2019 novel sars corona virus 2 acute severe respiratory infection", "2019new wuhan corona virus respiratory illness", "pneumonia caused by wuhan seafood market pneumonia virus 2019", "sars coronavirus 2 severe respiratory disease", "sars coronavirus2 2019 severe acute illness", "illness from sars coronavirus 2 2019", "2019 new seafood market corona virus severe acute disorder", "2019 novel wuhan virus acute severe illness", "hubei severe acute respiratory illness", "2019new seafood market coronavirus severe acute illness", "sars coronavirus2 severe acute respiratory infection", "syndrome caused by human corona virus 2019", "wuhan corona virus severe respiratory syndrome", "pneumonia caused by 2019 new wuhan seafood market pneumonia virus", "2019novel wuhan seafood market virus respiratory disease", "sars cov 2 acute severe respiratory illness", "2019novel seafood market virus severe acute disease", "2019new sars coronavirus 2 infection", "2019new seafood market virus severe disorder", "2019new chinese coronavirus respiratory infection", "sarscov2 disease", "2019novel seafood market corona virus severe respiratory disorder", "2019 seafood market coronavirus acute severe respiratory illness", "infection caused by wuhan coronavirus", "wuhan corona virus 2019 disorder", "hubei severe pneumonia", "syndrome caused by virus responsible for covid19", "infection from wuhan seafood market virus", "wuhan corona virus severe acute infection", "infection from wuhancorona virus 2019", "2019 novel seafood market coronavirus acute severe illness", "illness caused by 2019 severe acute respiratory syndrome corona virus 2", "infection from 2019new chinese coronavirus", "2019 novel wuhancorona virus severe respiratory illness", "wuhan pneumonia virus acute severe infection", "2019novel seafood market coronavirus acute severe disease", "disorder from seafood market corona virus 2019", "2019 novel wuhancoronavirus acute severe respiratory disease", "2019 sarscov2 severe respiratory syndrome", "syndrome from 2019new wuhan pneumonia virus", "pneumonia from chinese corona virus", "wuhan seafood market virus acute respiratory illness", "2019new wuhan corona virus pneumonia", "illness due to 2019 new sarscov2", "2019 wuhan coronavirus severe syndrome", "2019 new wuhan seafood market pneumonia virus severe disorder", "illness caused by 2019novel wuhancoronavirus", "wuhan corona virus severe respiratory disorder", "2019 novel severe acute respiratory syndrome corona virus2", "pneumonia due to 2019 chinese corona virus", "2019new sars cov 2 severe acute respiratory illness", "2019 wuhancoronavirus acute severe respiratory illness", "seafood market virus severe acute syndrome", "novel corona virus 2019 severe pneumonia", "2019 novel wuhan pneumonia virus acute respiratory disorder", "disease due to 2019 new severe acute respiratory syndrome coronavirus2", "2019 novel wuhan pneumonia virus severe illness", "2019new wuhan seafood market virus acute respiratory disorder", "2019new wuhancoronavirus severe acute infection", "2019 wuhancorona virus respiratory infection", "2019new wuhan seafood market pneumonia virus syndrome", "wuhan seafood market pneumonia virus severe acute respiratory syndrome", "chinese coronavirus respiratory disease", "2019 novel chinese coronavirus acute respiratory illness", "pneumonia due to 2019 wuhan pneumonia virus", "2019 new wuhan coronavirus acute severe disease", "2019 sars coronavirus 2 severe acute respiratory infection", "pneumonia caused by 2019 sars coronavirus 2", "disorder from 2019new chinese coronavirus", "2019 new wuhan pneumonia virus severe acute disorder", "novel coronavirus 2019 respiratory illness", "illness caused by 2019 seafood market coronavirus", "2019 seafood market coronavirus severe acute respiratory syndrome", "pneumonia from human corona virus 2019", "2019novel wuhan pneumonia virus acute respiratory disorder", "illness caused by 2019 severe acute respiratory syndrome coronavirus 2", "disease caused by 2019novel wuhan corona virus", "2019 new seafood market coronavirus severe disease", "2019 novel sarscov2 acute severe pneumonia", "2019new sarscov2 severe infection", "wuhan seafood market virus 2019 acute respiratory infection", "seafood market virus 2019 acute severe syndrome", "2019 new wuhancorona virus severe acute syndrome", "disorder from severe acute respiratory syndromerelated coronavirus2", "2019 new wuhan seafood market pneumonia virus severe acute respiratory illness", "infection caused by 2019 wuhan seafood market pneumonia virus", "2019 chinese coronavirus acute infection", "2019novel seafood market corona virus severe acute disease", "pneumonia due to 2019 wuhancorona virus", "2019novel sars cov 2 disease", "2019novel wuhancorona virus acute severe infection", "2019 novel sars coronavirus2 severe acute respiratory illness", "2019 seafood market virus acute respiratory infection", "infection due to 2019 new chinese corona virus", "infection from wuhancoronavirus", "2019 new wuhancoronavirus severe illness", "disease from wuhan corona virus", "syndrome caused by 2019 novel coronavirus", "sarscov2 2019 disease", "2019 novel wuhan coronavirus syndrome", "disorder due to 2019novel severe acute respiratory syndromerelated corona virus2", "wuhan virus acute infection", "sars coronavirus 2 acute syndrome", "2019 new sars corona virus2 severe pneumonia", "2019 new wuhan coronavirus infection", "pneumonia caused by severe acute respiratory syndromerelated coronavirus2 2019", "pneumonia from 2019 sars corona virus2", "2019novel wuhan seafood market pneumonia virus", "2019 novel sars coronavirus 2 acute severe illness", "2019 new coronavirus respiratory illness", "novel corona virus 2019 severe acute respiratory infection", "new coronavirus 2019 acute severe syndrome", "infection due to 2019new severe acute respiratory syndrome coronavirus 2", "chinese corona virus infection", "2019 coronavirus severe acute illness", "wuhancoronavirus acute severe respiratory disease", "2019 new chinese coronavirus severe pneumonia", "2019new wuhan corona virus infection", "2019 wuhan pneumonia virus illness", "illness due to severe acute respiratory syndrome coronavirus 2", "2019novel wuhan virus acute respiratory infection", "sarscov2 2019 acute illness", "pneumonia due to 2019 new seafood market corona virus", "2019new sars coronavirus2 acute severe syndrome", "pneumonia caused by 2019new seafood market coronavirus", "2019 new sars corona virus2 infection", "2019 new wuhancorona virus respiratory illness", "human corona virus 2019 severe infection", "syndrome due to 2019new sars coronavirus 2", "illness due to 2019 wuhan coronavirus", "virus causing covid19 respiratory illness", "2019 new seafood market coronavirus", "sars cov 2 2019 acute respiratory disease", "new coronavirus 2019 severe disorder", "wuhan pneumonia virus 2019 severe illness", "2019 new seafood market coronavirus acute respiratory disease", "2019new chinese coronavirus severe respiratory syndrome", "2019 new severe acute respiratory syndrome coronavirus 2 illness", "2019 new sarscov2 severe acute respiratory illness", "sars corona virus 2 severe respiratory infection", "disease due to sars cov 2", "2019 new seafood market corona virus asevere acute respiratory disease", "2019 wuhan coronavirus acute respiratory disease", "illness from novel coronavirus", "2019 novel wuhan seafood market virus disease", "wuhan acute syndrome", "2019novel sars corona virus2 severe respiratory syndrome", "infection from 2019new severe acute respiratory syndromerelated corona virus2", "2019 severe acute respiratory syndromerelated coronavirus 2 disease", "2019novel sars corona virus 2 illness", "sars corona virus 2 2019 severe acute syndrome", "2019 novel wuhancoronavirus acute disorder", "disorder caused by 2019 new wuhan corona virus", "2019new sars corona virus2 acute respiratory illness", "human coronavirus 2019 acute severe disorder", "infection caused by 2019novel severe acute respiratory syndromerelated coronavirus2", "wuhan seafood market pneumonia virus respiratory infection", "2019new sars cov 2 severe respiratory disease", "hcov 19 acute syndrome", "wuhancorona virus acute severe respiratory illness", "pneumonia caused by 2019novel severe acute respiratory syndromerelated corona virus2", "2019 novel wuhan pneumonia virus acute respiratory syndrome", "infection caused by 2019novel sars cov 2", "infection due to 2019 novel chinese corona virus", "disorder from 2019new sars corona virus 2", "2019 sars corona virus2 acute severe respiratory syndrome", "disorder from sars corona virus2", "2019 novel sars coronavirus 2 acute illness", "2019new seafood market corona virus severe disease", "disorder from wuhan seafood market virus 2019", "seafood market corona virus severe respiratory syndrome", "pneumonia from sars cov 2", "2019novel sarscov2 acute respiratory illness", "2019new seafood market corona virus acute severe respiratory infection", "syndrome from wuhan seafood market virus 2019", "2019new wuhan seafood market pneumonia virus disease", "illness from severe acute respiratory syndromerelated coronavirus 2 2019", "disease from 2019 new wuhan seafood market virus", "wuhan corona virus severe respiratory infection", "2019 human coronavirus severe disease", "2019 new coronavirus illness", "chinese corona virus 2019 acute syndrome", "covid 19 virus acute illness", "2019 new sars coronavirus 2 severe disorder", "chinese coronavirus 2019 severe pneumonia", "2019 sars cov 2 severe respiratory illness", "2019new wuhan virus acute severe disorder", "syndrome from 2019new seafood market corona virus", "disorder from seafood market coronavirus 2019", "2019 new coronavirus severe acute respiratory illness", "2019 human coronavirus acute severe respiratory syndrome", "2019 new wuhan virus severe acute disorder", "wuhan seafood market pneumonia virus acute severe disorder", "disease from seafood market virus", "syndrome from 2019 wuhan virus", "2019 sars corona virus 2 acute disorder", "disease from sars coronavirus2", "wuhan asevere acute respiratory disease", "novel coronavirus acute severe disease", "covid19 virus severe acute syndrome", "2019 new wuhan corona virus", "disorder due to 2019 wuhan seafood market pneumonia virus", "infection from 2019 novel wuhan seafood market virus", "chinese corona virus 2019 pneumonia", "disease from 2019 novel wuhan virus", "infection caused by 2019 new wuhan seafood market virus", "2019 human corona virus respiratory infection", "pneumonia caused by seafood market coronavirus 2019", "seafood market virus severe acute pneumonia", "2019new wuhancoronavirus severe acute disorder", "2019novel seafood market virus severe infection", "2019 novel sars cov 2 severe acute illness", "seafood market severe acute syndrome", "sars cov 2 respiratory illness", "2019ncov severe respiratory syndrome", "2019 wuhan coronavirus acute disorder", "2019 novel wuhan virus severe respiratory illness", "sars coronavirus 2 acute severe respiratory infection", "illness caused by covid19 virus", "2019 novel sars corona virus2 severe illness", "2019 new sars corona virus 2 acute severe respiratory disease", "2019novel seafood market corona virus respiratory infection", "2019 new chinese corona virus severe acute respiratory illness", "infection caused by 2019 severe acute respiratory syndromerelated corona virus2", "2019 new sars cov 2 severe pneumonia", "wuhan virus 2019 severe respiratory infection", "pneumonia from 2019 new sars corona virus2", "2019 new sars coronavirus 2", "2019new seafood market corona virus acute severe disorder", "2019 novel chinese corona virus asevere acute respiratory disease", "2019new wuhan virus acute illness", "chinese corona virus acute severe respiratory infection", "disorder from seafood market virus 2019", "2019novel wuhan virus acute respiratory syndrome", "2019novel sars coronavirus2 severe acute infection", "2019new severe acute respiratory syndromerelated corona virus 2 illness", "2019 new severe acute respiratory syndromerelated coronavirus 2 disease", "2019novel wuhan seafood market pneumonia virus acute syndrome", "chinese corona virus acute severe disorder", "chinese corona virus 2019 severe acute respiratory syndrome", "sars coronavirus2 2019 respiratory disease", "2019novel seafood market virus respiratory disorder", "2019 wuhan pneumonia virus severe syndrome", "2019 novel sars corona virus 2 syndrome", "2019novel wuhancoronavirus disorder", "2019 novel wuhan seafood market virus acute illness", "infection due to 2019novel severe acute respiratory syndromerelated corona virus2", "seafood market virus syndrome", "seafood market corona virus 2019 severe acute syndrome", "2019 wuhancorona virus asevere acute respiratory disease", "infection from 2019 sars coronavirus 2", "pneumonia caused by new coronavirus 2019", "pneumonia caused by wuhan corona virus 2019", "2019new wuhan seafood market pneumonia virus acute respiratory syndrome", "disorder caused by 2019new wuhan coronavirus", "2019 novel seafood market corona virus acute severe pneumonia", "2019novel seafood market corona virus severe illness", "sars corona virus 2 2019 severe acute illness", "illness from seafood market corona virus", "2019 sars corona virus2 acute severe syndrome", "disorder caused by 2019 new chinese corona virus", "infection due to 2019novel wuhancoronavirus", "disease due to 2019new wuhan corona virus", "disease due to 2019 wuhancorona virus", "wuhancorona virus 2019 severe respiratory syndrome", "disease due to wuhancorona virus", "2019novel severe acute respiratory syndrome coronavirus2 infection", "illness from chinese coronavirus", "virus responsible for covid19 acute infection", "2019novel sars coronavirus2 severe respiratory disorder", "2019 coronavirus severe acute respiratory syndrome", "sarscov2 acute disorder", "pneumonia from new coronavirus", "2019new seafood market corona virus", "new coronavirus 2019 respiratory disease", "2019 human corona virus acute severe infection", "2019 novel sars corona virus 2 acute respiratory syndrome", "wuhan corona virus 2019", "syndrome from 2019 new chinese corona virus", "sars cov 2 severe acute syndrome", "infection due to 2019 novel severe acute respiratory syndrome coronavirus2", "2019 new wuhan seafood market pneumonia virus severe respiratory syndrome", "illness due to 2019 novel severe acute respiratory syndrome corona virus 2", "2019novel sarscov2 severe disorder", "2019 novel corona virus severe syndrome", "infection due to seafood market coronavirus", "pneumonia due to 2019novel severe acute respiratory syndrome coronavirus2", "disease caused by 2019new wuhancoronavirus", "pneumonia from 2019 novel severe acute respiratory syndromerelated coronavirus2", "2019 novel wuhan coronavirus disease", "wuhancorona virus acute severe pneumonia", "2019 sars cov 2 disease", "disorder due to 2019 new seafood market coronavirus", "syndrome caused by 2019 novel wuhan seafood market pneumonia virus", "seafood market virus severe respiratory syndrome", "2019novel wuhancoronavirus severe disorder", "disorder caused by 2019novel wuhan coronavirus", "wuhan corona virus 2019 severe pneumonia", "sarscov2 severe illness", "sars cov 2 severe infection", "infection caused by 2019 chinese corona virus", "2019 novel sars corona virus2 acute syndrome", "infection caused by sars coronavirus2", "2019 new sars coronavirus2 severe respiratory infection", "new coronavirus 2019 severe respiratory syndrome", "disorder caused by new corona virus", "disease due to seafood market coronavirus 2019", "2019 new sars coronavirus 2 severe acute syndrome", "illness caused by 2019 novel chinese corona virus", "disease from sarscov2 2019", "2019 novel seafood market corona virus asevere acute respiratory disease", "seafood market severe respiratory syndrome", "illness from hcov 19", "2019 novel wuhancorona virus severe acute respiratory syndrome", "wuhancorona virus 2019 severe acute illness", "disease caused by 2019new severe acute respiratory syndromerelated corona virus2", "2019 chinese corona virus severe acute respiratory illness", "2019new wuhan seafood market virus acute severe respiratory illness", "novel corona virus 2019 acute illness", "disorder caused by 2019 novel seafood market virus", "infection from severe acute respiratory syndrome coronavirus2 2019", "2019 wuhancorona virus syndrome", "2019 novel sarscov2 acute severe disorder", "novel coronavirus 2019 severe respiratory disorder", "syndrome due to chinese coronavirus", "wuhan pneumonia virus acute respiratory syndrome", "2019 new wuhancoronavirus illness", "chinese corona virus 2019 severe pneumonia", "new corona virus acute disease", "2019 new wuhan virus severe acute respiratory infection", "pneumonia from 2019new seafood market corona virus", "wuhancorona virus acute severe infection", "pneumonia due to sars corona virus 2", "2019 new coronavirus acute disorder", "2019 new chinese coronavirus respiratory disorder", "2019 human corona virus acute severe respiratory disorder", "2019 new sars coronavirus2 acute pneumonia", "syndrome from novel corona virus 2019", "2019novel sars coronavirus2 acute illness", "novel coronavirus acute infection", "2019 coronavirus severe disorder", "pneumonia due to 2019novel severe acute respiratory syndrome corona virus2", "new coronavirus severe acute syndrome", "disorder caused by 2019 new coronavirus", "2019 wuhan pneumonia virus severe respiratory illness", "illness caused by human corona virus 2019", "disorder from 2019novel wuhan seafood market virus", "infection due to 2019novel chinese coronavirus", "virus causing covid19 respiratory syndrome", "2019new seafood market virus acute respiratory illness", "2019new chinese coronavirus severe illness", "2019 wuhancorona virus severe disease", "2019 chinese corona virus severe pneumonia", "wuhan corona virus severe acute disorder", "2019novel sarscov2 severe acute respiratory disorder", "2019new wuhancorona virus severe respiratory syndrome", "disorder from sars corona virus2 2019", "2019 new coronavirus acute severe respiratory syndrome", "2019 sars coronavirus2 acute illness", "covid19 virus acute severe respiratory disease", "2019 seafood market virus asevere acute respiratory disease", "2019novel sars coronavirus2 severe illness", "2019novel wuhancoronavirus acute disorder", "pneumonia due to wuhancorona virus", "2019new sars coronavirus 2 severe acute pneumonia", "sars coronavirus 2 2019 acute syndrome", "novel corona virus 2019 acute severe respiratory infection", "new coronavirus severe syndrome", "illness due to 2019 chinese coronavirus", "2019 new wuhancoronavirus acute respiratory disease", "2019novel seafood market coronavirus acute illness", "2019 new wuhancorona virus severe disease", "wuhan virus severe respiratory infection", "2019novel wuhan seafood market pneumonia virus severe acute respiratory illness", "2019 new wuhan coronavirus severe acute respiratory disorder", "2019 novel severe acute respiratory syndromerelated coronavirus2 disorder", "2019new seafood market coronavirus acute respiratory syndrome", "sarscov2 acute severe respiratory disease", "2019new seafood market coronavirus acute infection", "pneumonia due to 2019 coronavirus", "2019 corona virus acute infection", "sarscov2 severe acute illness", "syndrome from 2019 novel wuhan corona virus", "pneumonia from 2019new severe acute respiratory syndromerelated coronavirus 2", "2019 novel sars cov 2 severe acute syndrome", "illness due to 2019new severe acute respiratory syndromerelated corona virus 2", "2019 new sars cov 2 respiratory infection", "2019 ncov respiratory disease", "virus responsible for covid19 severe acute disease", "sars corona virus2 2019 severe syndrome", "novel coronavirus severe illness", "2019 wuhan virus severe acute respiratory disorder", "disease due to 2019novel wuhan coronavirus", "sars corona virus 2 acute severe respiratory syndrome", "2019novel chinese coronavirus acute severe respiratory illness", "illness caused by 2019new severe acute respiratory syndrome corona virus 2", "wuhan corona virus 2019 severe acute illness", "wuhan corona virus severe acute syndrome", "chinese coronavirus 2019 respiratory disorder", "2019 new sars corona virus 2 severe disorder", "pneumonia caused by novel corona virus", "seafood market corona virus acute severe infection", "infection due to 2019 novel coronavirus", "pneumonia due to 2019 new severe acute respiratory syndromerelated corona virus2", "illness from 2019 sars corona virus2", "2019 new sars cov 2 acute respiratory illness", "sarscov2 2019 acute syndrome", "wuhan corona virus 2019 acute respiratory syndrome", "disorder caused by 2019 corona virus", "2019 sars coronavirus 2 acute pneumonia", "pneumonia due to 2019 novel wuhan coronavirus", "wuhan virus 2019 severe disorder", "wuhan pneumonia virus acute illness", "2019new sars cov 2 respiratory syndrome", "2019 new seafood market corona virus severe infection", "2019 new sars cov 2 severe disorder", "2019new wuhan pneumonia virus respiratory syndrome", "hcov19 severe acute respiratory disorder", "hubei severe disorder", "pneumonia caused by severe acute respiratory syndrome corona virus 2", "pneumonia caused by 2019 new wuhancoronavirus", "2019ncov severe acute pneumonia", "2019 new chinese coronavirus illness", "2019 novel severe acute respiratory syndrome coronavirus 2 disorder", "disorder from 2019ncov", "2019new sars cov 2 severe disorder", "2019 sars coronavirus2 respiratory syndrome", "2019 sars corona virus2 acute severe respiratory illness", "disease caused by 2019new chinese coronavirus", "2019new sarscov2 acute disorder", "chinese coronavirus 2019 acute severe respiratory illness", "2019 new wuhan virus acute pneumonia", "2019 novel severe acute respiratory syndromerelated coronavirus2", "sarscov2 2019 severe respiratory disease", "human coronavirus 2019 severe acute respiratory infection", "2019 novel seafood market virus respiratory illness", "pneumonia from 2019new wuhancoronavirus", "pneumonia from 2019 novel wuhancoronavirus", "2019novel wuhan seafood market pneumonia virus severe acute disorder", "2019 wuhan seafood market pneumonia virus severe acute respiratory infection", "virus causing covid19 acute respiratory infection", "2019novel sars corona virus 2 acute syndrome", "new coronavirus severe acute illness", "disorder due to 2019 novel severe acute respiratory syndromerelated corona virus 2", "seafood market coronavirus 2019 disorder", "human corona virus 2019 severe pneumonia", "infection caused by 2019 novel severe acute respiratory syndromerelated corona virus2", "2019new wuhan seafood market pneumonia virus severe respiratory disease", "wuhan coronavirus 2019 severe acute infection", "novel corona virus acute severe respiratory infection", "infection caused by 2019 new severe acute respiratory syndrome coronavirus2", "disorder caused by 2019new seafood market corona virus", "disorder due to 2019 severe acute respiratory syndromerelated corona virus 2", "2019novel wuhan virus disease", "sarscov2 2019 acute disorder", "sars corona virus2 2019 respiratory infection", "2019 novel sarscov2 severe acute infection", "pneumonia from 2019 corona virus", "2019 sars coronavirus2 acute respiratory syndrome", "2019novel seafood market corona virus acute illness", "seafood market coronavirus 2019 acute respiratory disorder", "wuhan corona virus acute severe respiratory illness", "sars cov 2 2019 acute syndrome", "2019 novel wuhan coronavirus severe respiratory syndrome", "disease from wuhan coronavirus 2019", "novel coronavirus 2019 severe acute respiratory illness", "pneumonia due to 2019 new corona virus", "seafood market coronavirus 2019 severe illness", "2019novel wuhan pneumonia virus acute severe respiratory syndrome", "infection due to 2019new severe acute respiratory syndromerelated coronavirus2", "2019novel wuhancorona virus respiratory infection", "2019 wuhancorona virus severe respiratory illness", "2019 new wuhan seafood market virus acute severe illness", "infection from 2019 human coronavirus", "seafood market corona virus 2019 severe respiratory disease", "2019new severe acute respiratory syndrome coronavirus 2 infection", "wuhan pneumonia virus severe acute pneumonia", "2019 new seafood market corona virus severe respiratory infection", "disorder due to 2019 new chinese coronavirus", "2019new seafood market coronavirus acute respiratory illness", "hcov19 acute severe respiratory syndrome", "seafood market corona virus severe respiratory disorder", "2019 novel severe acute respiratory syndrome corona virus 2 disease", "2019novel wuhan coronavirus severe respiratory syndrome", "infection caused by 2019 severe acute respiratory syndromerelated corona virus 2", "wuhan seafood market pneumonia virus 2019 severe respiratory illness", "seafood market virus 2019 syndrome", "chinese corona virus severe acute infection", "2019new wuhancorona virus acute disorder", "infection due to 2019 wuhancorona virus", "wuhancoronavirus acute severe illness", "2019novel wuhan corona virus acute severe syndrome", "illness from 2019 severe acute respiratory syndromerelated coronavirus 2", "2019 chinese coronavirus severe syndrome", "disorder caused by 2019novel severe acute respiratory syndromerelated coronavirus 2", "2019 new sars coronavirus2 severe acute respiratory infection", "novel coronavirus 2019 severe acute pneumonia", "2019 novel wuhan seafood market virus acute infection", "infection from 2019new sars corona virus 2", "seafood market virus 2019 respiratory illness", "pneumonia due to seafood market virus 2019", "sars corona virus 2 2019 severe acute respiratory infection", "novel coronavirus acute pneumonia", "pneumonia due to sars coronavirus 2", "2019new seafood market corona virus severe illness", "novel coronavirus 2019 acute severe pneumonia", "severe acute respiratory syndrome coronavirus2 2019 disorder", "2019 new sars coronavirus2 acute severe disease", "disorder due to 2019 new severe acute respiratory syndrome corona virus2", "2019 new wuhan pneumonia virus severe acute respiratory illness", "illness from 2019 new wuhan corona virus", "2019 sarscov2 acute severe respiratory infection", "2019 seafood market corona virus illness", "2019novel sars coronavirus 2 disorder", "2019 novel seafood market coronavirus severe acute respiratory infection", "pneumonia due to hcov19", "pneumonia from 2019new wuhancorona virus", "2019 seafood market corona virus severe acute illness", "sars corona virus2 2019 acute severe respiratory illness", "2019 wuhancorona virus acute syndrome", "2019 ncov acute severe disease", "novel corona virus 2019 severe acute disorder", "syndrome caused by 2019 wuhan seafood market pneumonia virus", "illness due to 2019 novel severe acute respiratory syndrome corona virus2", "pneumonia due to 2019 sars coronavirus2", "2019novel severe acute respiratory syndromerelated corona virus 2 illness", "disease due to novel coronavirus 2019", "2019 new wuhan pneumonia virus acute respiratory illness", "syndrome from wuhancoronavirus", "infection caused by 2019new wuhan pneumonia virus", "disorder due to wuhan pneumonia virus 2019", "disease due to 2019novel wuhan pneumonia virus", "wuhan corona virus acute severe pneumonia", "disorder due to 2019 sars corona virus2", "wuhan pneumonia virus severe acute illness", "2019 novel sars corona virus2 acute severe syndrome", "infection caused by 2019 coronavirus", "2019 new wuhan seafood market pneumonia virus pneumonia", "chinese corona virus severe respiratory syndrome", "2019 sars coronavirus2 severe disorder", "2019 new seafood market virus acute disease", "2019 new sars coronavirus2 infection", "2019novel sarscov2 respiratory infection", "syndrome caused by 2019 novel wuhancorona virus", "2019 novel wuhan virus acute respiratory disease", "human corona virus 2019 acute severe illness", "disorder due to 2019 novel chinese corona virus", "2019new seafood market corona virus severe acute infection", "2019 new sarscov2 acute severe respiratory illness", "seafood market virus acute illness", "2019 human coronavirus disease", "syndrome due to 2019 new corona virus", "2019novel seafood market coronavirus illness", "disease caused by 2019 wuhan coronavirus", "illness due to 2019 novel severe acute respiratory syndromerelated coronavirus2", "disorder due to 2019 new sars corona virus2", "2019novel chinese corona virus disease", "novel corona virus 2019 acute respiratory disorder", "chinese corona virus 2019 acute severe infection", "pneumonia due to 2019 new wuhan pneumonia virus", "2019new wuhan corona virus severe disorder", "2019 novel corona virus disorder", "2019 new wuhan pneumonia virus respiratory illness", "2019 chinese coronavirus severe pneumonia", "illness due to 2019novel severe acute respiratory syndrome corona virus 2", "syndrome due to 2019novel sars cov 2", "seafood market coronavirus 2019 acute respiratory illness", "2019 novel corona virus acute severe disorder", "seafood market corona virus severe infection", "illness caused by wuhan pneumonia virus 2019", "2019 novel sars corona virus2 disease", "novel coronavirus 2019 syndrome", "2019 new wuhan seafood market virus severe respiratory illness", "sars cov 2 acute respiratory syndrome", "2019new chinese coronavirus severe acute pneumonia", "illness due to sars coronavirus2", "2019 seafood market virus severe acute respiratory disorder", "infection due to 2019 novel wuhan seafood market pneumonia virus", "disorder from wuhan virus 2019", "hcov 19 acute pneumonia", "virus responsible for covid19 severe illness", "pneumonia from sars corona virus 2 2019", "2019new wuhancoronavirus severe respiratory disorder", "sars corona virus 2 severe acute pneumonia", "2019novel sars coronavirus2 acute disease", "disease caused by severe acute respiratory syndromerelated coronavirus2", "2019 human corona virus acute respiratory syndrome", "infection caused by 2019novel wuhan coronavirus", "2019new chinese coronavirus disorder", "2019 wuhan coronavirus asevere acute respiratory disease", "hcov 19 severe acute pneumonia", "disease from 2019new severe acute respiratory syndrome corona virus2", "2019new seafood market coronavirus", "wuhancoronavirus acute severe infection", "infection due to wuhan virus", "2019 new wuhancoronavirus syndrome", "severe acute respiratory syndromerelated corona virus2 illness", "wuhan seafood market virus 2019 severe respiratory syndrome", "2019 new wuhan pneumonia virus acute severe respiratory illness", "illness due to 2019 novel wuhancorona virus", "seafood market acute severe disorder", "syndrome due to sars coronavirus 2", "2019 new sars coronavirus 2 severe disease", "novel coronavirus 2019 acute severe respiratory illness", "sars corona virus 2 2019 respiratory disorder", "sars corona virus 2 2019 pneumonia", "illness from sars corona virus 2", "disorder from 2019new seafood market virus", "illness from 2019new severe acute respiratory syndromerelated corona virus2", "2019novel severe acute respiratory syndromerelated coronavirus 2", "2019 ncov severe infection", "virus causing covid19 acute severe respiratory disorder", "pneumonia due to seafood market coronavirus", "2019novel sars cov 2 severe respiratory syndrome", "2019 corona virus infection", "sars coronavirus2 2019 asevere acute respiratory disease", "disease from 2019 novel severe acute respiratory syndromerelated coronavirus 2", "infection from wuhan seafood market virus 2019", "2019 novel corona virus acute severe illness", "2019new wuhan virus acute disease", "pneumonia caused by 2019novel sarscov2", "2019novel chinese corona virus severe syndrome", "novel corona virus 2019 severe acute disease", "2019 novel wuhancorona virus acute disorder", "2019 novel seafood market coronavirus acute disorder", "2019 new sarscov2 acute respiratory disorder", "new coronavirus 2019 respiratory infection", "2019 novel chinese corona virus severe pneumonia", "2019novel seafood market virus acute severe infection", "infection due to 2019novel seafood market virus", "illness due to 2019 severe acute respiratory syndrome corona virus2", "disorder caused by 2019new sars coronavirus2", "syndrome due to 2019novel seafood market corona virus", "illness due to 2019novel chinese corona virus", "ICD10:U07.1", "sarscov2", "sars coronavirus 2 respiratory infection", "infection caused by chinese corona virus 2019", "disorder caused by 2019 seafood market corona virus", "human corona virus 2019 acute pneumonia", "2019new wuhan virus respiratory disorder", "2019 novel seafood market virus acute severe respiratory disease", "2019new seafood market virus", "wuhan coronavirus respiratory disorder", "2019 novel sars corona virus2 acute severe disorder", "virus causing covid19 acute severe respiratory syndrome", "2019novel seafood market corona virus disorder", "seafood market virus acute severe pneumonia", "pneumonia caused by seafood market corona virus", "sars corona virus2 acute severe respiratory syndrome", "syndrome from sars coronavirus2", "2019novel wuhan seafood market pneumonia virus severe syndrome", "wuhan coronavirus", "2019 ncov acute severe infection", "disorder caused by 2019ncov", "wuhancorona virus 2019 respiratory infection", "2019new wuhancoronavirus acute syndrome", "pneumonia from 2019 human corona virus", "2019 novel chinese coronavirus acute severe disorder", "2019new wuhancorona virus severe acute respiratory syndrome", "2019novel seafood market corona virus severe respiratory illness", "disease caused by 2019new wuhan virus", "2019 new wuhan coronavirus pneumonia", "wuhancorona virus 2019 respiratory disorder", "sars coronavirus2 acute respiratory illness", "2019 corona virus severe illness", "illness caused by 2019 wuhancoronavirus", "2019new sars corona virus 2 acute severe respiratory infection", "covid 19 virus severe respiratory disorder", "2019novel seafood market virus severe syndrome", "sarscov2 acute severe infection", "illness due to 2019 new severe acute respiratory syndromerelated corona virus2", "disease from 2019new seafood market virus", "2019 wuhancorona virus respiratory disease", "disease due to severe acute respiratory syndrome coronavirus 2", "2019 new wuhan seafood market virus illness", "sars coronavirus 2 2019 pneumonia", "hcov 19 severe respiratory infection", "wuhancorona virus 2019 illness", "syndrome caused by 2019novel sars corona virus2", "2019 novel sars coronavirus 2 severe respiratory disorder", "2019new seafood market corona virus acute respiratory disorder", "severe acute respiratory syndromerelated corona virus 2 2019 pneumonia", "novel coronavirus 2019 acute severe infection", "syndrome caused by 2019 sars corona virus 2", "infection caused by 2019 wuhan virus", "2019new sarscov2 acute severe illness", "disease from 2019 chinese coronavirus", "seafood market coronavirus 2019 severe respiratory disorder", "2019 corona virus respiratory disease", "2019 new seafood market virus acute severe illness", "seafood market coronavirus acute severe syndrome", "pneumonia from wuhan corona virus 2019", "new coronavirus acute respiratory infection", "2019 new sars cov 2 severe acute syndrome", "sars coronavirus 2 acute severe illness", "2019 novel sars cov 2 illness", "syndrome caused by 2019 novel sars coronavirus 2", "2019 novel sars corona virus 2 acute infection", "chinese coronavirus severe acute disorder", "2019 sars corona virus 2 severe acute respiratory syndrome", "disorder caused by novel corona virus 2019", "2019 wuhancoronavirus acute severe respiratory infection", "syndrome from 2019 new chinese coronavirus", "infection from 2019 novel corona virus", "seafood market virus 2019 severe syndrome", "hubei severe acute respiratory syndrome", "2019new severe acute respiratory syndromerelated coronavirus 2 infection", "disease caused by 2019 sars corona virus2", "2019 chinese corona virus respiratory disease", "infection due to sars corona virus2 2019", "illness due to sars corona virus 2 2019", "virus responsible for covid19 acute severe pneumonia", "2019 novel wuhancorona virus respiratory disease", "seafood market corona virus severe acute disease", "2019 novel seafood market virus asevere acute respiratory disease", "disease due to 2019new severe acute respiratory syndrome corona virus 2", "illness caused by 2019 new sars coronavirus2", "new corona virus severe acute respiratory disorder", "2019 coronavirus acute severe respiratory disorder", "2019 sars cov 2 severe pneumonia", "2019novel sars corona virus 2 severe acute respiratory syndrome", "2019 wuhan seafood market pneumonia virus acute severe disease", "illness due to 2019new sars corona virus2", "disease due to chinese corona virus 2019", "pneumonia caused by 2019 novel wuhan seafood market virus", "virus responsible for covid19 severe acute respiratory infection", "2019 wuhan seafood market pneumonia virus acute respiratory disease", "2019 wuhan virus disorder", "wuhancoronavirus severe acute respiratory syndrome", "sars coronavirus 2 2019 acute respiratory infection", "hcov19 acute illness", "2019new seafood market virus acute severe syndrome", "disease from 2019 corona virus", "syndrome due to 2019 wuhan corona virus", "infection caused by 2019 novel wuhan seafood market virus", "2019 wuhan coronavirus acute severe illness", "2019 novel sars corona virus2 asevere acute respiratory disease", "disorder due to 2019 novel sars coronavirus 2", "new corona virus 2019 acute infection", "pneumonia caused by 2019novel severe acute respiratory syndromerelated coronavirus 2", "2019 sars coronavirus2 acute respiratory disorder", "2019novel chinese coronavirus acute respiratory infection", "2019novel wuhan corona virus severe acute disorder", "2019 sars cov 2 disorder", "hcov 19 severe disorder", "novel corona virus 2019 severe acute respiratory illness", "2019 new coronavirus severe disorder", "2019 new chinese coronavirus severe respiratory infection", "chinese corona virus severe respiratory infection", "2019 sars corona virus2 severe respiratory illness", "2019novel wuhancoronavirus severe acute infection", "disorder from 2019 new wuhan virus", "wurs", "new coronavirus severe acute pneumonia", "2019 human coronavirus severe acute respiratory illness", "2019 novel wuhan coronavirus acute severe syndrome", "2019 new seafood market corona virus acute respiratory infection", "disease from hcov19", "2019 seafood market virus severe acute pneumonia", "disease caused by 2019 new chinese corona virus", "pneumonia caused by sars coronavirus2", "2019new sars cov 2 acute syndrome", "2019new seafood market corona virus acute severe respiratory disorder", "2019novel seafood market corona virus acute pneumonia", "illness from 2019 wuhan corona virus", "sars coronavirus2 2019 severe acute infection", "2019 new sars corona virus 2 severe respiratory illness", "2019 novel sars coronavirus 2 disease", "disorder caused by severe acute respiratory syndromerelated coronavirus 2 2019", "2019novel wuhancorona virus disorder", "2019 novel wuhancoronavirus severe acute respiratory infection", "illness caused by 2019 new wuhan seafood market pneumonia virus", "disease due to 2019novel seafood market virus", "syndrome caused by 2019new seafood market virus", "wuhan virus acute pneumonia", "new coronavirus severe respiratory disease", "hubei infection", "2019new wuhan coronavirus respiratory disorder", "syndrome from wuhancoronavirus 2019", "2019 novel chinese corona virus acute respiratory disease", "sars cov 2 acute respiratory infection", "2019new wuhan coronavirus", "2019 corona virus asevere acute respiratory disease", "seafood market acute respiratory illness", "infection caused by 2019new wuhancorona virus", "wuhan coronavirus 2019 pneumonia", "2019novel wuhan virus acute severe disorder", "chinese corona virus 2019 severe acute disease", "syndrome due to 2019 novel wuhan seafood market virus", "2019new seafood market coronavirus respiratory disease", "2019 novel sars corona virus 2 acute severe pneumonia", "2019 seafood market corona virus severe respiratory disease", "2019 severe acute respiratory syndrome coronavirus2 disorder", "hubei respiratory illness", "pneumonia due to 2019novel severe acute respiratory syndrome coronavirus 2", "2019 chinese corona virus severe acute syndrome", "syndrome from sars coronavirus2 2019", "disease from 2019 wuhan pneumonia virus", "2019 new seafood market corona virus acute severe respiratory infection", "2019 new coronavirus acute pneumonia", "2019 seafood market virus acute disease", "seafood market coronavirus respiratory syndrome", "2019new wuhan corona virus acute severe respiratory illness", "2019new sars corona virus 2 acute disorder", "novel corona virus severe respiratory syndrome", "wuhan virus acute respiratory infection", "2019novel wuhan pneumonia virus acute severe disorder", "wuhancoronavirus severe illness", "novel corona virus 2019 respiratory disorder", "wuhan seafood market virus acute respiratory disease", "2019 novel seafood market coronavirus asevere acute respiratory disease", "2019 new wuhancorona virus respiratory disorder", "2019new sars coronavirus2 severe acute pneumonia", "2019 wuhan virus respiratory disorder", "2019novel sars cov 2 severe acute illness", "sars cov 2 illness", "2019novel wuhancorona virus pneumonia", "DOID:0080600", "seafood market virus severe pneumonia", "2019 seafood market corona virus respiratory disease", "2019 sars coronavirus 2 severe acute respiratory disorder", "disorder due to wuhan coronavirus", "2019new sars corona virus 2 acute syndrome", "pneumonia from 2019 severe acute respiratory syndromerelated corona virus 2", "2019 novel wuhan corona virus acute severe respiratory infection", "2019 new seafood market coronavirus severe syndrome", "2019 novel wuhan coronavirus severe respiratory disorder", "wuhan respiratory illness", "disease caused by 2019 seafood market virus", "wuhan corona virus 2019 severe acute disorder", "disease due to 2019 new severe acute respiratory syndromerelated corona virus2", "2019 wuhan virus severe disorder", "wuhan corona virus acute severe disease", "2019novel sarscov2 severe respiratory syndrome", "2019novel wuhancorona virus acute respiratory illness", "novel corona virus 2019 acute pneumonia", "chinese coronavirus acute severe respiratory infection", "2019novel sarscov2 pneumonia", "disorder due to 2019new wuhancoronavirus", "disease caused by 2019 seafood market coronavirus", "hubei acute respiratory illness", "2019novel seafood market corona virus severe pneumonia", "2019 sars corona virus2 acute disease", "hubei acute disorder", "2019 wuhan coronavirus respiratory disease", "seafood market corona virus 2019 acute severe respiratory illness", "2019 novel wuhan pneumonia virus severe respiratory illness", "2019 new sars coronavirus2 disease", "2019new sars corona virus 2 severe disorder", "wuhan virus severe acute disorder", "2019 new severe acute respiratory syndromerelated corona virus 2", "2019 novel seafood market virus acute severe disorder", "2019 new wuhan seafood market pneumonia virus acute respiratory infection", "disease from sars corona virus 2", "2019ncov acute respiratory syndrome", "2019 new seafood market corona virus respiratory infection", "illness from 2019new severe acute respiratory syndromerelated coronavirus2", "2019 new sars cov 2 acute disorder", "wuhan virus severe disease", "infection due to 2019 wuhan coronavirus", "2019 novel wuhan seafood market pneumonia virus acute respiratory syndrome", "infection from severe acute respiratory syndromerelated coronavirus 2 2019", "disease from 2019 novel sars cov 2", "2019 novel seafood market coronavirus acute pneumonia", "novel coronavirus illness", "2019 sars coronavirus 2 severe acute pneumonia", "2019novel wuhan corona virus severe respiratory infection", "2019 wuhan seafood market pneumonia virus severe acute respiratory illness", "syndrome due to wuhan coronavirus", "2019 new seafood market coronavirus severe disorder", "pneumonia caused by wuhan virus 2019", "syndrome caused by 2019 wuhan virus", "illness from chinese corona virus", "2019new wuhan coronavirus respiratory syndrome", "2019 new wuhan pneumonia virus syndrome", "2019 novel wuhancorona virus disorder", "2019 corona virus respiratory disorder", "2019 new seafood market corona virus respiratory disorder", "2019 wuhan coronavirus acute severe respiratory disorder", "2019 new sars corona virus 2 infection", "sars corona virus 2 2019 disease", "disorder from 2019novel sars corona virus2", "2019 wuhan corona virus asevere acute respiratory disease", "hubei respiratory infection", "2019 new wuhan coronavirus disorder", "2019novel chinese corona virus asevere acute respiratory disease", "wuhancoronavirus respiratory illness", "chinese coronavirus acute severe disorder", "2019 chinese corona virus acute severe disease", "infection from seafood market coronavirus", "2019 novel coronavirus acute severe respiratory disorder", "wuhancoronavirus 2019 respiratory infection", "disease caused by 2019new severe acute respiratory syndromerelated corona virus 2", "2019ncov severe infection", "2019 novel wuhan corona virus acute disorder", "2019 new wuhancorona virus acute respiratory syndrome", "new corona virus severe respiratory disorder", "pneumonia due to 2019novel severe acute respiratory syndrome corona virus 2", "wuhan corona virus 2019 acute respiratory infection", "chinese coronavirus severe respiratory illness", "pneumonia from 2019novel wuhan pneumonia virus", "disorder due to 2019new severe acute respiratory syndromerelated coronavirus2", "disorder from 2019new severe acute respiratory syndromerelated corona virus2", "2019 new wuhan pneumonia virus acute pneumonia", "disease caused by 2019novel sars coronavirus 2", "seafood market acute severe syndrome", "2019 sars corona virus2 severe acute disease", "pneumonia due to 2019 sars cov 2", "2019 wuhan seafood market pneumonia virus severe disease", "infection caused by 2019 new wuhan coronavirus", "2019novel wuhan virus acute illness", "infection from severe acute respiratory syndrome corona virus 2", "2019novel sars cov 2 acute severe pneumonia", "2019new wuhancorona virus acute severe syndrome", "2019 new sars cov 2 disease", "disorder from 2019 novel severe acute respiratory syndromerelated coronavirus2", "2019 novel chinese corona virus pneumonia", "2019novel seafood market coronavirus syndrome", "2019 novel wuhan pneumonia virus acute severe respiratory disorder", "sars cov 2 acute respiratory disease", "disorder from 2019novel chinese corona virus", "disease caused by 2019 novel severe acute respiratory syndrome coronavirus 2", "2019new wuhan corona virus acute severe illness", "2019 novel coronavirus severe acute illness", "2019novel chinese corona virus acute severe respiratory illness", "syndrome from 2019 sarscov2", "2019 new wuhan pneumonia virus acute severe infection", "hcov 19 severe syndrome", "illness caused by 2019new severe acute respiratory syndrome coronavirus2", "2019 sars cov 2 severe acute respiratory disorder", "wuhan seafood market virus severe infection", "2019 chinese coronavirus acute pneumonia", "2019 new wuhancorona virus acute severe disease", "2019 new chinese coronavirus acute respiratory syndrome", "syndrome caused by 2019novel wuhancorona virus", "2019 new sars corona virus2 acute severe infection", "2019novel chinese coronavirus severe acute illness", "infection due to seafood market virus", "2019novel sars corona virus2 acute severe respiratory disorder", "2019new seafood market virus acute severe respiratory syndrome", "2019 new sars cov 2 acute pneumonia", "infection due to 2019new wuhancoronavirus", "2019 novel sarscov2 severe pneumonia", "2019 new sarscov2 pneumonia", "sars coronavirus 2 severe pneumonia", "2019 novel chinese corona virus acute severe pneumonia", "2019new sars corona virus2 syndrome", "2019novel chinese coronavirus disorder", "2019new wuhancorona virus", "illness from severe acute respiratory syndromerelated corona virus 2", "hcov 19 severe acute syndrome", "2019 seafood market coronavirus severe illness", "illness from 2019 novel sars corona virus 2", "syndrome due to wuhan virus", "pneumonia due to 2019 novel wuhan virus", "2019 new coronavirus infection", "2019 new chinese coronavirus infection", "human coronavirus 2019 acute disease", "2019 corona virus acute disorder", "2019 novel chinese corona virus severe infection", "2019 wuhan virus acute disorder", "2019new sars cov 2 acute severe illness", "infection due to 2019 new severe acute respiratory syndromerelated coronavirus 2", "human corona virus 2019 acute respiratory illness", "2019 new sars cov 2 asevere acute respiratory disease", "chinese coronavirus 2019 severe acute syndrome", "2019novel chinese coronavirus acute severe infection", "human coronavirus 2019 illness", "wuhan disease", "2019novel sars corona virus2 disease", "seafood market coronavirus severe acute syndrome", "wuhan corona virus severe respiratory illness", "disease caused by 2019 sars coronavirus 2", "disorder caused by wuhan seafood market virus 2019", "disease caused by 2019 novel severe acute respiratory syndromerelated corona virus 2", "pneumonia caused by 2019novel sars corona virus2", "infection due to 2019 corona virus", "wuhan illness", "2019new sars corona virus2 severe infection", "wuhancoronavirus 2019 severe acute disease", "2019new sars corona virus2 severe acute illness", "2019 seafood market virus acute severe disorder", "virus causing covid19 severe acute respiratory disorder", "2019 novel wuhan virus severe disorder", "2019novel seafood market virus acute severe respiratory syndrome", "2019new wuhan corona virus illness", "2019 seafood market corona virus acute severe disease", "2019novel sars corona virus 2 severe acute respiratory infection", "novel corona virus respiratory disease", "2019 new wuhan seafood market pneumonia virus severe respiratory disease", "2019 novel sars coronavirus2 severe syndrome", "disorder due to 2019 novel severe acute respiratory syndromerelated coronavirus 2", "disorder due to 2019 new seafood market corona virus", "disease from 2019 novel wuhan corona virus", "illness caused by virus causing covid19", "2019 human coronavirus acute severe respiratory illness", "novel corona virus 2019 respiratory illness", "2019new seafood market coronavirus severe respiratory syndrome", "2019 seafood market virus disorder", "2019 sars corona virus2 acute infection", "2019 novel sarscov2 acute severe respiratory disorder", "2019novel sars corona virus2 severe acute illness", "disease caused by sarscov2 2019", "syndrome from wuhan coronavirus 2019", "2019new wuhancorona virus acute severe respiratory disease", "2019 wuhan corona virus acute severe pneumonia", "seafood market virus 2019 respiratory disease", "illness due to seafood market corona virus 2019", "2019 novel seafood market corona virus pneumonia", "covid19 virus acute respiratory disorder", "2019 coronavirus severe acute disease", "2019 new sars corona virus2 acute disease", "seafood market coronavirus 2019 severe respiratory disease", "2019 novel corona virus acute infection", "wuhan pneumonia virus 2019 severe pneumonia", "syndrome from 2019novel wuhan seafood market virus", "2019 novel seafood market corona virus severe respiratory disease", "severe acute respiratory syndromerelated corona virus 2 disorder", "2019 new seafood market coronavirus severe acute illness", "sars cov 2 2019 respiratory illness", "2019 wuhancorona virus acute pneumonia", "2019 seafood market corona virus severe acute syndrome", "wuhan corona virus severe acute respiratory syndrome", "covid19 virus severe acute infection", "2019 new seafood market corona virus severe disorder", "illness caused by 2019new wuhan coronavirus", "new corona virus respiratory disease", "2019 chinese corona virus severe respiratory syndrome", "2019 new wuhan pneumonia virus severe acute respiratory infection", "2019 sars corona virus 2 severe respiratory syndrome", "2019novel seafood market corona virus severe respiratory infection", "2019ncov severe acute respiratory syndrome", "2019 novel wuhan corona virus respiratory syndrome", "2019novel sars corona virus 2 acute severe illness", "2019 new sars coronavirus 2 acute disorder", "novel coronavirus 2019 acute disorder", "2019novel wuhan corona virus acute respiratory disease", "syndrome from sars coronavirus 2", "disorder due to 2019 new wuhan virus", "2019 wuhancoronavirus acute severe respiratory disorder", "2019 new wuhan seafood market virus acute severe respiratory disorder", "2019new wuhan pneumonia virus acute disorder", "2019novel sars corona virus2 disorder", "2019 novel sars coronavirus 2 severe acute illness", "2019 new sars cov 2 severe respiratory disorder", "chinese corona virus severe acute illness", "hubei severe acute pneumonia", "sars corona virus 2 2019 respiratory disease", "sars cov 2 2019 acute infection", "2019 ncov acute syndrome", "disorder caused by 2019novel severe acute respiratory syndrome coronavirus 2", "2019 novel wuhan corona virus acute respiratory illness", "2019 new wuhan corona virus syndrome", "2019 new severe acute respiratory syndromerelated coronavirus2 pneumonia", "sars coronavirus2 2019 respiratory disorder", "wuhan corona virus severe acute disease", "novel coronavirus 2019 respiratory disease", "2019 human corona virus syndrome", "seafood market severe acute respiratory illness", "2019 wuhan seafood market virus acute respiratory disease", "2019 ncov respiratory disorder", "2019 novel wuhan pneumonia virus acute severe respiratory syndrome", "2019novel wuhan pneumonia virus acute pneumonia", "disorder due to 2019 severe acute respiratory syndromerelated coronavirus2", "2019novel wuhan pneumonia virus", "2019new severe acute respiratory syndromerelated corona virus2 infection", "seafood market virus 2019 respiratory disorder", "pneumonia from 2019 new severe acute respiratory syndromerelated corona virus 2", "chinese coronavirus 2019 acute severe disease", "2019novel sars coronavirus2 syndrome", "2019 new sars corona virus2 respiratory disease", "2019 sars cov 2 acute severe infection", "disease from 2019 novel sarscov2", "disease due to 2019 new seafood market coronavirus", "2019novel wuhancoronavirus", "chinese coronavirus acute severe respiratory disease", "2019 novel sars cov 2 severe illness", "disorder due to severe acute respiratory syndromerelated coronavirus2", "2019novel sars coronavirus2 acute severe disease", "2019 novel seafood market corona virus acute respiratory illness", "wuhan acute severe respiratory disorder", "pneumonia caused by 2019new seafood market virus", "covid 19 virus acute respiratory illness", "2019 sars coronavirus2 severe pneumonia", "2019new wuhan corona virus respiratory syndrome", "sars coronavirus2 acute severe respiratory disease", "2019 sarscov2 acute severe illness", "sars corona virus2 severe acute respiratory disorder", "pneumonia from 2019 chinese coronavirus", "2019new seafood market coronavirus severe acute disorder", "syndrome caused by novel coronavirus 2019", "seafood market corona virus severe acute disorder", "2019 new wuhan seafood market pneumonia virus acute severe respiratory illness", "disease due to 2019novel chinese coronavirus", "disease from 2019novel severe acute respiratory syndrome corona virus 2", "seafood market virus 2019 severe respiratory disease", "new corona virus acute respiratory infection", "2019 new seafood market coronavirus acute severe disease", "2019novel wuhan seafood market pneumonia virus acute disorder", "2019 new sars coronavirus 2 severe acute respiratory disorder", "2019new wuhan seafood market virus illness", "2019 sars cov 2 severe disease", "2019new chinese corona virus severe respiratory infection", "new corona virus acute illness", "pneumonia caused by new coronavirus", "seafood market virus 2019 severe respiratory disorder", "disease caused by severe acute respiratory syndrome corona virus 2 2019", "sarscov2 severe acute syndrome", "infection from 2019 sarscov2", "2019new seafood market virus severe infection", "disease from 2019 novel wuhan seafood market virus", "2019 new wuhancoronavirus", "2019 novel wuhancoronavirus severe acute respiratory disorder", "2019 sars cov 2 acute infection", "seafood market disease", "2019new wuhan corona virus acute disease", "disease caused by 2019 human corona virus", "2019 sars coronavirus2 severe illness", "disease due to seafood market corona virus", "syndrome due to wuhan seafood market virus", "wuhan seafood market pneumonia virus severe respiratory syndrome", "2019 novel wuhancorona virus acute severe illness", "2019 novel seafood market virus respiratory disorder", "2019 wuhan corona virus acute severe respiratory disorder", "infection due to chinese coronavirus 2019", "disease from severe acute respiratory syndrome corona virus2 2019", "2019 new sars coronavirus2 respiratory syndrome", "infection caused by 2019 sarscov2", "pneumonia due to 2019 human coronavirus", "sarscov2 2019 severe acute infection", "hcov19 acute severe respiratory infection", "sars cov 2 2019 acute severe respiratory illness", "severe acute respiratory syndrome coronavirus 2 2019 pneumonia", "2019 coronavirus acute pneumonia", "wuhan respiratory disorder", "pneumonia due to chinese corona virus 2019", "pneumonia from seafood market corona virus 2019", "2019 novel sars coronavirus2 severe respiratory infection", "2019novel wuhan coronavirus severe respiratory disorder", "2019new chinese corona virus infection", "novel coronavirus severe disorder", "sars cov 2 2019 acute respiratory syndrome", "2019 new sarscov2 acute respiratory syndrome", "2019new wuhan corona virus acute respiratory infection", "pneumonia due to severe acute respiratory syndrome corona virus 2 2019", "new coronavirus severe pneumonia", "wuhancoronavirus 2019 acute respiratory disorder", "2019 new corona virus severe syndrome", "2019 corona virus", "2019 novel wuhan virus severe acute respiratory infection", "2019new wuhancoronavirus acute disease", "disorder caused by 2019 severe acute respiratory syndromerelated corona virus 2", "2019 novel sarscov2 disease", "disease from 2019new seafood market corona virus", "2019novel wuhan virus acute infection", "2019 seafood market coronavirus syndrome", "2019new wuhan seafood market virus severe disease", "pneumonia from 2019 wuhan pneumonia virus", "2019new wuhancoronavirus asevere acute respiratory disease", "2019new chinese corona virus severe acute respiratory infection", "wuhancorona virus acute severe respiratory syndrome", "2019 ncov asevere acute respiratory disease", "2019 sars corona virus2 acute severe respiratory infection", "2019 new corona virus severe respiratory syndrome", "2019 novel wuhan coronavirus severe acute respiratory infection", "2019 severe acute respiratory syndrome coronavirus 2 infection", "disease caused by 2019 novel wuhan seafood market pneumonia virus", "2019 novel wuhan seafood market pneumonia virus acute severe respiratory syndrome", "seafood market corona virus severe acute respiratory disorder", "disease due to sars coronavirus 2 2019", "2019 human corona virus pneumonia", "2019novel sars coronavirus 2 severe respiratory disease", "disorder from 2019novel wuhan virus", "2019 wuhancoronavirus acute disease", "wuhan pneumonia virus acute pneumonia", "2019 new sarscov2 acute severe illness", "sars coronavirus2 2019 infection", "2019novel wuhan coronavirus severe syndrome", "infection from 2019 novel severe acute respiratory syndrome coronavirus2", "2019 new wuhan corona virus acute infection", "illness from wuhancorona virus", "2019novel wuhan seafood market virus severe syndrome", "2019 sars coronavirus2 acute disorder", "syndrome from new coronavirus 2019", "chinese corona virus 2019 acute disorder", "disorder due to 2019 severe acute respiratory syndrome corona virus2", "2019new wuhan virus severe acute respiratory syndrome", "2019novel chinese corona virus severe acute disease", "2019 new wuhan seafood market pneumonia virus severe acute disease", "seafood market acute severe respiratory disease", "2019 novel chinese coronavirus disease", "2019 novel wuhan pneumonia virus severe acute respiratory syndrome", "pneumonia from severe acute respiratory syndrome corona virus2 2019", "new coronavirus 2019 asevere acute respiratory disease", "illness due to wuhan virus", "2019novel sars coronavirus2 acute severe respiratory disorder", "2019 novel severe acute respiratory syndrome corona virus2 disorder", "illness from wuhancorona virus 2019", "disorder from 2019new seafood market coronavirus", "pneumonia caused by 2019novel chinese corona virus", "seafood market acute infection", "2019 new wuhan virus acute infection", "wuhan seafood market virus 2019 acute respiratory disease", "2019 wuhan pneumonia virus severe acute disease", "2019 sars coronavirus2 severe acute disorder", "sars corona virus 2 acute respiratory disorder", "2019new sars coronavirus2 severe acute disorder", "seafood market virus 2019 disorder", "disorder from 2019 seafood market virus", "wuhan acute illness", "2019 novel sars coronavirus 2 severe acute respiratory syndrome", "infection from 2019 new wuhan pneumonia virus", "pneumonia from 2019 novel wuhan coronavirus", "2019 novel sarscov2 acute respiratory disease", "disorder due to 2019novel sars corona virus2", "human corona virus 2019 acute severe respiratory illness", "illness caused by 2019novel wuhancorona virus", "2019new chinese coronavirus severe respiratory infection", "2019novel sars coronavirus 2 acute severe disease", "2019 wuhan virus acute severe disorder", "disorder due to 2019new wuhan seafood market pneumonia virus", "sars 2019", "2019 corona virus severe respiratory illness", "2019 new sars cov 2 severe acute disorder", "2019 novel sars corona virus 2 acute severe disease", "2019 wuhan seafood market virus acute severe disorder", "disease caused by 2019 corona virus", "chinese coronavirus infection", "2019 wuhancorona virus severe acute disease", "2019novel sars cov 2 acute severe respiratory syndrome", "sars cov 2 2019 acute severe respiratory infection", "2019 novel wuhancorona virus acute severe respiratory syndrome", "seafood market coronavirus acute respiratory disorder", "2019 wuhan seafood market pneumonia virus severe acute disorder", "2019 novel wuhan corona virus severe acute illness", "pneumonia caused by sarscov2 2019", "syndrome due to 2019 sarscov2", "sars corona virus 2 2019 severe disease", "illness due to 2019 novel corona virus", "covid 19 virus severe syndrome", "2019new chinese coronavirus severe acute respiratory illness", "infection caused by 2019novel wuhancorona virus", "sars cov 2 2019 acute severe disease", "infection from 2019 novel wuhancorona virus", "infection from 2019new wuhan seafood market pneumonia virus", "syndrome from 2019 novel seafood market coronavirus", "2019 novel severe acute respiratory syndrome corona virus 2 illness", "seafood market virus respiratory illness", "2019 chinese coronavirus severe respiratory disease", "disorder from chinese corona virus 2019", "sarscov2 2019 acute infection", "2019 human coronavirus acute respiratory disorder", "sarscov2 severe syndrome", "2019novel seafood market corona virus acute respiratory disease", "disorder due to sars cov 2 2019", "2019 novel sars coronavirus2 pneumonia", "2019 seafood market coronavirus acute pneumonia", "novel corona virus 2019 acute severe respiratory syndrome", "sars corona virus 2 2019 syndrome", "2019 human corona virus acute disorder", "sars corona virus2 infection", "2019 wuhan coronavirus severe acute respiratory disorder", "2019novel sars coronavirus2 acute severe syndrome", "disease caused by 2019new sars corona virus2", "2019new sarscov2 infection", "sarscov2 acute respiratory syndrome", "wuhancorona virus severe infection", "2019 sars cov 2 acute syndrome", "disorder due to 2019 new sars cov 2", "2019new wuhan seafood market pneumonia virus acute disease", "wuhan coronavirus acute disorder", "wuhan seafood market pneumonia virus 2019 acute respiratory illness", "pneumonia from 2019 new wuhan coronavirus", "2019 wuhancorona virus acute severe syndrome", "2019 new wuhan seafood market virus severe acute infection", "2019 novel sars corona virus2 acute severe respiratory disorder", "2019new sars corona virus 2 severe acute infection", "2019new sars corona virus2 acute syndrome", "disease due to wuhancorona virus 2019", "infection caused by 2019new severe acute respiratory syndrome corona virus 2", "2019new sars coronavirus 2 acute severe infection", "2019 novel wuhan virus severe acute respiratory illness", "2019 new sars coronavirus2 severe acute disease", "2019 novel wuhan seafood market pneumonia virus acute respiratory illness", "seafood market coronavirus respiratory disease", "2019 new sarscov2 severe respiratory infection", "new coronavirus respiratory illness", "new corona virus 2019", "chinese corona virus respiratory illness", "wuhan pneumonia virus asevere acute respiratory disease", "2019new wuhan corona virus acute severe infection", "2019new sarscov2 acute infection", "2019novel sars coronavirus2 severe acute respiratory syndrome", "infection due to 2019 seafood market coronavirus", "syndrome due to sars coronavirus2", "syndrome caused by 2019ncov", "2019novel wuhan seafood market pneumonia virus severe disorder", "infection caused by 2019 seafood market virus", "seafood market virus 2019 acute severe infection", "2019novel wuhancoronavirus severe respiratory disease", "2019novel wuhancorona virus infection", "2019novel wuhan seafood market virus respiratory disorder", "wuhan seafood market pneumonia virus 2019 respiratory infection", "2019 new wuhan seafood market pneumonia virus disease", "pneumonia caused by 2019 novel chinese coronavirus", "illness due to sarscov2", "2019new seafood market coronavirus severe acute respiratory illness", "chinese corona virus respiratory syndrome", "disorder caused by 2019 novel sars corona virus2", "wuhan corona virus acute disorder", "sars cov 2 respiratory disorder", "2019novel seafood market virus acute disorder", "2019 novel sars cov 2 severe syndrome", "2019 severe acute respiratory syndromerelated corona virus2", "2019novel chinese corona virus acute respiratory syndrome", "wuhan seafood market virus 2019 acute illness", "novel coronavirus severe respiratory disease", "sars corona virus 2 acute severe disorder", "2019 novel sars coronavirus 2 acute severe disorder", "syndrome from wuhan coronavirus", "infection due to novel corona virus 2019", "human corona virus 2019 respiratory syndrome", "infection caused by 2019 new severe acute respiratory syndrome coronavirus 2", "disorder from 2019 sars coronavirus2", "disorder caused by 2019 novel wuhan pneumonia virus", "seafood market coronavirus 2019 acute severe syndrome", "infection caused by wuhan pneumonia virus 2019", "pneumonia caused by 2019 novel sars coronavirus 2", "2019 new sarscov2 acute severe respiratory disorder", "covid 19 virus acute severe disease", "illness from 2019 new severe acute respiratory syndrome coronavirus2", "2019new sars coronavirus 2 acute severe disorder", "2019 sarscov2 respiratory disease", "2019 novel wuhan virus infection", "infection caused by severe acute respiratory syndromerelated coronavirus2", "sars coronavirus2 severe acute syndrome", "2019 new wuhancoronavirus acute severe syndrome", "2019novel seafood market corona virus acute syndrome", "disorder caused by wuhancoronavirus 2019", "disease from 2019 severe acute respiratory syndrome coronavirus2", "wuhancorona virus 2019 severe acute infection", "2019 novel severe acute respiratory syndrome corona virus2 pneumonia", "disorder due to 2019 novel coronavirus", "2019 new wuhan pneumonia virus acute severe respiratory infection", "chinese corona virus syndrome", "syndrome from 2019new chinese coronavirus", "2019 novel wuhan pneumonia virus severe acute respiratory illness", "infection due to 2019 new sarscov2", "2019novel wuhancorona virus disease", "disease due to new coronavirus 2019", "2019novel chinese coronavirus severe acute infection", "2019 new seafood market virus acute severe infection", "wuhan virus 2019 acute severe disorder", "wuhan pneumonia virus pneumonia", "2019novel chinese corona virus acute pneumonia", "2019 sars cov 2 infection", "2019 novel wuhan seafood market pneumonia virus", "2019new wuhan pneumonia virus acute syndrome", "novel corona virus 2019 severe respiratory disease", "disease from severe acute respiratory syndrome corona virus2", "2019 new seafood market coronavirus severe pneumonia", "2019 ncov severe respiratory disease", "disorder caused by 2019new severe acute respiratory syndromerelated corona virus2", "2019new sars corona virus2 severe acute respiratory illness", "2019 new coronavirus acute respiratory disease", "2019novel wuhan virus illness", "2019new sars coronavirus 2 acute respiratory illness", "illness caused by 2019novel seafood market coronavirus", "2019 novel sars cov 2 acute severe disease", "2019novel wuhan seafood market pneumonia virus acute severe respiratory infection", "severe acute respiratory syndrome corona virus 2 2019 infection", "pneumonia due to 2019new wuhan pneumonia virus", "disease caused by 2019 novel chinese coronavirus", "2019 novel wuhan seafood market pneumonia virus acute severe respiratory disorder", "severe acute respiratory syndrome coronavirus 2 infection", "illness caused by severe acute respiratory syndromerelated corona virus2 2019", "illness caused by 2019new severe acute respiratory syndromerelated coronavirus2", "2019ncov acute severe illness", "illness from severe acute respiratory syndromerelated coronavirus 2", "disorder caused by 2019novel sars corona virus 2", "2019new chinese coronavirus severe disorder", "syndrome from 2019 novel sars coronavirus 2", "illness due to sars coronavirus2 2019", "disease due to virus responsible for covid19", "disorder due to severe acute respiratory syndrome corona virus 2", "sars corona virus2 2019 severe acute syndrome", "human coronavirus 2019 severe acute respiratory syndrome", "infection from 2019novel sars coronavirus2", "2019 chinese corona virus acute severe infection", "2019 novel coronavirus acute disorder", "seafood market respiratory illness", "wuhan seafood market virus acute severe respiratory disorder", "2019 new severe acute respiratory syndrome coronavirus 2", "disease caused by 2019 new wuhancorona virus", "severe acute respiratory syndrome corona virus2 disorder", "2019 new sars coronavirus 2 illness", "2019novel seafood market corona virus acute infection", "2019novel wuhan pneumonia virus respiratory illness", "2019 new sarscov2 acute respiratory illness", "infection caused by virus responsible for covid19", "2019novel wuhan seafood market virus disease", "2019 novel severe acute respiratory syndromerelated corona virus 2 disorder", "novel coronavirus severe acute respiratory illness", "disease from 2019 wuhan coronavirus", "disorder from wuhan pneumonia virus 2019", "2019new wuhan seafood market pneumonia virus severe acute pneumonia", "2019 new wuhan virus severe disorder", "illness from 2019 sars coronavirus2", "sars cov 2 2019 syndrome", "2019 seafood market virus acute illness", "2019 wuhancoronavirus severe acute infection", "syndrome from 2019novel wuhan coronavirus", "2019novel sars cov 2 acute severe respiratory disorder", "2019 new wuhan coronavirus acute disease", "human coronavirus 2019 acute severe respiratory disorder", "2019 new seafood market coronavirus severe acute pneumonia", "novel corona virus 2019 acute respiratory illness", "disease caused by wuhancorona virus", "syndrome caused by 2019novel sars coronavirus2", "seafood market virus 2019 severe infection", "disease from severe acute respiratory syndromerelated coronavirus 2 2019", "syndrome from seafood market virus", "novel coronavirus 2019 severe acute syndrome", "illness caused by 2019new sars corona virus 2", "2019 sars cov 2 acute severe respiratory disease", "wuhan coronavirus 2019 severe acute pneumonia", "2019 wuhancorona virus acute illness", "disorder from severe acute respiratory syndrome corona virus2", "sars coronavirus 2 syndrome", "2019novel chinese coronavirus severe acute respiratory syndrome", "2019 novel seafood market virus severe acute pneumonia", "infection due to 2019new sars coronavirus2", "illness due to seafood market coronavirus 2019", "2019 wuhancorona virus acute respiratory disorder", "2019new wuhan seafood market pneumonia virus severe illness", "2019 wuhan coronavirus severe acute respiratory infection", "2019 new sarscov2 severe acute respiratory disorder", "pneumonia caused by 2019 new wuhan seafood market virus", "infection from 2019 new corona virus", "2019new sars cov 2 acute respiratory disorder", "infection due to 2019novel wuhan corona virus", "virus causing covid19 acute disease", "disorder caused by 2019 novel wuhan corona virus", "2019new chinese coronavirus respiratory disease", "2019 new sars cov 2 severe acute disease", "chinese corona virus acute severe pneumonia", "2019novel sarscov2 acute severe respiratory illness", "2019 novel chinese corona virus acute infection", "2019 sars cov 2 severe disorder", "2019 coronavirus respiratory infection", "syndrome caused by hcov 19", "2019 novel seafood market corona virus severe syndrome", "disorder caused by 2019new seafood market coronavirus", "pneumonia caused by wuhan seafood market virus 2019", "2019new wuhan corona virus severe pneumonia", "2019novel seafood market virus respiratory illness", "2019 new wuhan virus acute severe respiratory illness", "wuhan seafood market virus severe acute infection", "2019new chinese corona virus severe acute respiratory disorder", "2019novel sars cov 2 pneumonia", "2019new sars corona virus 2 severe disease", "disease caused by 2019 sarscov2", "disease due to 2019 novel wuhan seafood market pneumonia virus", "2019new wuhancorona virus severe infection", "virus causing covid19 severe acute disease", "2019novel severe acute respiratory syndrome corona virus2 infection", "2019 severe acute respiratory syndrome corona virus2 disease", "pneumonia from 2019 novel wuhan virus", "disease caused by 2019new wuhan coronavirus", "sars corona virus 2 2019", "2019 new wuhan seafood market virus severe respiratory infection", "sars coronavirus 2 severe acute pneumonia", "2019 new wuhan corona virus pneumonia", "sarscov2 2019 severe acute disease", "covid 19 virus respiratory disease", "wuhancorona virus severe acute syndrome", "2019 novel chinese corona virus acute severe disorder", "disorder from 2019 wuhan virus", "2019novel wuhan seafood market pneumonia virus acute severe respiratory disorder", "new corona virus respiratory syndrome", "2019 wuhan coronavirus acute disease", "wuhan seafood market pneumonia virus 2019 acute infection", "new coronavirus 2019 respiratory syndrome", "pneumonia caused by 2019 wuhan seafood market pneumonia virus", "2019 new wuhan corona virus acute severe respiratory illness", "illness from 2019novel seafood market corona virus", "syndrome from sarscov2 2019", "disease from 2019 new severe acute respiratory syndromerelated coronavirus 2", "2019 new wuhan virus acute illness", "infection from 2019novel chinese coronavirus", "2019novel wuhan virus severe disorder", "disease due to 2019 wuhan coronavirus", "2019 seafood market coronavirus acute infection", "2019 novel chinese coronavirus respiratory illness", "pneumonia caused by 2019novel wuhancorona virus", "pneumonia due to wuhan pneumonia virus", "2019novel wuhan pneumonia virus severe respiratory infection", "2019 new seafood market coronavirus severe acute respiratory syndrome", "pneumonia from 2019 severe acute respiratory syndromerelated coronavirus2", "2019 new corona virus acute severe syndrome", "2019 novel sarscov2 acute severe respiratory syndrome", "2019new wuhan coronavirus acute illness", "illness from 2019novel chinese corona virus", "2019 chinese corona virus acute severe pneumonia", "2019novel sars coronavirus 2 severe acute illness", "novel corona virus 2019 severe acute pneumonia", "2019 human coronavirus acute severe respiratory disease", "disorder due to 2019 new severe acute respiratory syndromerelated corona virus2", "2019novel seafood market coronavirus acute respiratory disease", "2019 seafood market corona virus acute severe respiratory disorder", "2019 corona virus acute severe respiratory disease", "syndrome from 2019novel seafood market coronavirus", "2019novel wuhan corona virus acute pneumonia", "2019novel chinese corona virus severe acute infection", "2019 new coronavirus disease", "pneumonia from 2019 novel severe acute respiratory syndromerelated coronavirus 2", "2019novel sars corona virus2 asevere acute respiratory disease", "2019 new sars coronavirus 2 severe respiratory infection", "disease caused by wuhan corona virus 2019", "2019 new chinese corona virus asevere acute respiratory disease", "wuhan virus disorder", "2019 severe acute respiratory syndromerelated coronavirus2 illness", "2019 novel sars coronavirus 2 severe disorder", "2019novel sarscov2 acute severe syndrome", "2019new wuhancoronavirus severe acute respiratory disorder", "2019 wuhancorona virus acute respiratory disease", "2019 new wuhan coronavirus severe respiratory syndrome", "2019 wuhan seafood market pneumonia virus severe infection", "chinese coronavirus 2019 disorder", "seafood market virus asevere acute respiratory disease", "2019new sars cov 2", "wuhancorona virus acute disease", "sarscov2 2019 severe respiratory illness", "2019 new wuhan corona virus acute respiratory syndrome", "2019 new coronavirus acute severe pneumonia", "2019 human corona virus acute severe respiratory infection", "2019 novel severe acute respiratory syndromerelated coronavirus 2 disorder", "virus causing covid19 disorder", "sars cov 2 2019 acute severe syndrome", "2019new sars coronavirus 2 acute illness", "2019new sarscov2 acute severe disease", "infection from hcov19", "sarscov2 2019 acute severe syndrome", "infection from 2019 novel sars corona virus 2", "2019 novel wuhan coronavirus acute severe respiratory infection", "2019 coronavirus severe acute respiratory infection", "novel corona virus syndrome", "syndrome caused by 2019novel wuhan pneumonia virus", "new corona virus 2019 illness", "disorder due to severe acute respiratory syndrome coronavirus 2 2019", "wuhan coronavirus 2019 respiratory illness", "2019novel seafood market virus disease", "2019new wuhan coronavirus respiratory disease", "2019novel sars corona virus2 acute infection", "2019new sars cov 2 respiratory disorder", "2019 wuhan seafood market pneumonia virus acute severe illness", "2019novel wuhan pneumonia virus severe acute pneumonia", "2019 new chinese corona virus respiratory syndrome", "seafood market coronavirus 2019 acute severe pneumonia", "disorder from 2019 severe acute respiratory syndromerelated corona virus 2", "2019new chinese corona virus acute severe disease", "new corona virus 2019 acute respiratory disorder", "infection from 2019novel severe acute respiratory syndromerelated corona virus 2", "2019 new wuhan virus severe acute respiratory illness", "2019 severe acute respiratory syndromerelated coronavirus 2 infection", "2019new chinese coronavirus acute respiratory infection", "2019 new sarscov2 acute infection", "seafood market coronavirus 2019 severe acute infection", "disease caused by sars corona virus 2", "pneumonia due to 2019 new wuhan virus", "2019novel seafood market corona virus severe acute pneumonia", "2019 novel sars coronavirus2 acute severe syndrome", "pneumonia due to 2019novel wuhan corona virus", "new corona virus", "2019novel sars coronavirus 2 infection", "2019 new chinese coronavirus syndrome", "wuhancoronavirus 2019 acute infection", "infection caused by severe acute respiratory syndromerelated coronavirus 2 2019", "2019 wuhan pneumonia virus acute pneumonia", "2019novel seafood market corona virus acute respiratory infection", "wuhan pneumonia virus acute severe respiratory illness", "2019 seafood market corona virus asevere acute respiratory disease", "wuhan seafood market pneumonia virus syndrome", "syndrome due to 2019 new seafood market virus", "illness caused by 2019 chinese coronavirus", "pneumonia caused by sars cov 2 2019", "illness from wuhan seafood market virus 2019", "2019novel severe acute respiratory syndrome coronavirus 2 disorder", "new corona virus 2019 respiratory infection", "wuhan virus 2019 asevere acute respiratory disease", "2019new wuhancorona virus severe acute syndrome", "disorder caused by 2019 wuhan virus", "2019novel wuhan pneumonia virus severe illness", "2019 novel sars cov 2 acute severe respiratory disease", "wuhancorona virus severe disorder", "severe acute respiratory syndrome coronavirus2 2019 infection", "virus causing covid19 disease", "disorder from 2019new severe acute respiratory syndrome coronavirus2", "wuhan acute respiratory disease", "seafood market corona virus 2019 acute syndrome", "sars corona virus2 acute respiratory infection", "2019 wuhancorona virus illness", "seafood market corona virus 2019 acute illness", "2019 novel wuhan corona virus severe acute disease", "infection from 2019novel wuhancoronavirus", "2019new sars coronavirus 2 severe acute respiratory disorder", "disorder caused by wuhan corona virus", "infection from 2019novel wuhan corona virus", "2019 seafood market corona virus respiratory syndrome", "2019novel wuhan pneumonia virus respiratory disorder", "2019 corona virus severe acute pneumonia", "2019 seafood market virus acute respiratory disorder", "seafood market coronavirus respiratory disorder", "illness due to new coronavirus 2019", "wuhan coronavirus 2019 acute disease", "2019 novel seafood market coronavirus respiratory syndrome", "2019new seafood market virus pneumonia", "wuhan seafood market virus 2019 syndrome", "2019new seafood market virus severe acute syndrome", "2019 new sars corona virus 2 severe acute pneumonia", "disease caused by 2019new sars cov 2", "syndrome caused by 2019 chinese corona virus", "2019 novel seafood market corona virus severe respiratory disorder", "disease from 2019 novel severe acute respiratory syndrome corona virus 2", "2019new seafood market coronavirus acute severe disorder", "2019 novel seafood market coronavirus acute severe respiratory infection", "wuhan pneumonia virus 2019 severe acute respiratory disorder", "sars coronavirus 2 respiratory disease", "infection from 2019 novel wuhan pneumonia virus", "pneumonia caused by 2019new severe acute respiratory syndrome corona virus2", "infection from 2019new severe acute respiratory syndrome corona virus 2", "wuhan corona virus 2019 acute severe respiratory disorder", "wuhan pneumonia virus 2019 infection", "hcov19 severe acute respiratory syndrome", "sars coronavirus2 2019 severe acute disorder", "disease from 2019novel severe acute respiratory syndrome coronavirus2", "2019 severe acute respiratory syndrome corona virus 2", "infection due to 2019new chinese corona virus", "sarscov2 acute severe respiratory syndrome", "disease due to novel coronavirus", "human corona virus 2019 respiratory illness", "2019 seafood market coronavirus infection", "2019 new chinese corona virus acute severe respiratory disease", "2019 sars cov 2 acute severe pneumonia", "2019 novel sars corona virus2 acute disease", "2019 new seafood market corona virus severe respiratory disease", "sars coronavirus 2 disease", "2019 new sars cov 2 severe respiratory illness", "wuhan coronavirus 2019 syndrome", "syndrome from wuhancorona virus", "pneumonia due to 2019new sars coronavirus2", "2019 chinese coronavirus severe respiratory infection", "wuhancorona virus 2019 acute pneumonia", "illness caused by 2019 novel wuhan virus", "2019new seafood market corona virus severe respiratory infection", "disorder caused by sars corona virus2 2019", "disease from 2019 new wuhancorona virus", "2019 new sars corona virus 2 severe syndrome", "2019novel sars coronavirus2 acute severe respiratory syndrome", "chinese corona virus 2019 syndrome", "2019novel wuhancorona virus acute pneumonia", "2019novel wuhan seafood market pneumonia virus severe respiratory disorder", "2019 chinese coronavirus acute respiratory infection", "seafood market corona virus severe acute illness", "chinese corona virus 2019 respiratory disorder", "disorder from wuhan corona virus 2019", "2019 new wuhan seafood market virus acute infection", "illness due to 2019 severe acute respiratory syndrome coronavirus2", "2019novel wuhan coronavirus severe acute respiratory illness", "2019 novel sarscov2 severe acute disorder", "2019ncov respiratory infection", "2019 novel seafood market corona virus acute severe infection", "2019 new wuhancoronavirus acute severe disorder", "pneumonia from 2019 new severe acute respiratory syndrome corona virus2", "2019 wuhan virus illness", "new corona virus 2019 severe acute respiratory syndrome", "syndrome due to 2019 wuhancorona virus", "2019new sars coronavirus 2 severe acute disorder", "disorder due to sars coronavirus 2", "2019novel wuhan seafood market virus acute respiratory syndrome", "novel coronavirus 2019 severe acute respiratory syndrome", "pneumonia due to 2019 novel sars corona virus2", "wuhan coronavirus severe respiratory disorder", "2019 ncov severe acute disorder", "2019 new sars corona virus 2 acute severe syndrome", "2019novel chinese corona virus severe acute respiratory syndrome", "seafood market coronavirus 2019 acute disorder", "2019novel wuhancorona virus respiratory syndrome", "2019 novel wuhan coronavirus severe respiratory illness", "syndrome due to 2019 new coronavirus", "2019 novel corona virus severe respiratory syndrome", "2019new wuhan corona virus acute severe pneumonia", "seafood market virus 2019 severe pneumonia", "pneumonia due to 2019new wuhan virus", "2019 seafood market coronavirus acute respiratory disease", "sars coronavirus2 acute disorder", "illness due to 2019novel wuhan corona virus", "2019 novel seafood market corona virus acute infection", "2019 new seafood market virus acute severe respiratory syndrome", "2019 sars corona virus 2 disorder", "pneumonia from 2019 new wuhan seafood market pneumonia virus", "2019new wuhan seafood market pneumonia virus severe respiratory syndrome", "illness due to 2019 novel chinese corona virus", "2019 new wuhancoronavirus severe acute respiratory syndrome", "2019 new chinese coronavirus acute severe illness", "syndrome caused by 2019novel seafood market virus", "disease from 2019 new wuhan seafood market pneumonia virus", "seafood market acute illness", "syndrome from 2019new seafood market coronavirus", "2019 seafood market coronavirus acute syndrome", "2019 new sars corona virus2", "2019 sars coronavirus2 acute severe disease", "2019 novel wuhan seafood market pneumonia virus acute illness", "seafood market virus severe infection", "2019new sarscov2 severe illness", "2019new chinese coronavirus acute severe respiratory disease", "disease caused by 2019 novel coronavirus", "2019 seafood market coronavirus severe acute respiratory infection", "2019new sars corona virus2 severe acute infection", "disease from severe acute respiratory syndromerelated corona virus2 2019", "pneumonia caused by 2019 novel wuhan seafood market pneumonia virus", "2019 new sars cov 2 severe respiratory infection", "syndrome from 2019novel wuhan seafood market pneumonia virus", "illness caused by 2019new severe acute respiratory syndromerelated coronavirus 2", "2019novel sars coronavirus2 disease", "covid 19 virus respiratory illness", "new corona virus acute severe infection", "sars corona virus 2 2019 severe respiratory disease", "disease due to 2019 novel sarscov2", "seafood market corona virus disorder", "2019 sars coronavirus2 acute severe respiratory syndrome", "disorder caused by 2019new chinese corona virus", "2019 new wuhan virus acute severe respiratory infection", "2019 new wuhan pneumonia virus acute severe syndrome", "2019 novel sarscov2 severe respiratory disorder", "novel corona virus acute severe disease", "2019 novel sars corona virus2 severe acute syndrome", "disease from 2019 severe acute respiratory syndrome corona virus 2", "2019 severe acute respiratory syndromerelated coronavirus 2 disorder", "2019new chinese corona virus acute illness", "2019novel sars coronavirus2 acute severe infection", "illness due to 2019 new wuhan coronavirus", "seafood market acute severe infection", "illness caused by novel coronavirus", "2019 new seafood market corona virus acute severe respiratory disease", "2019new seafood market coronavirus severe syndrome", "disease caused by 2019novel severe acute respiratory syndrome coronavirus 2", "new corona virus 2019 acute severe syndrome", "2019new sars corona virus2 acute severe syndrome", "human coronavirus 2019 acute severe disease", "2019 new chinese corona virus severe acute disorder", "2019 novel chinese coronavirus acute syndrome", "2019 new sarscov2 severe acute respiratory syndrome", "disease from 2019 new severe acute respiratory syndromerelated corona virus 2", "2019 novel corona virus illness", "wuhan pneumonia virus 2019 acute disorder", "disease due to 2019 sars corona virus 2", "infection due to new corona virus 2019", "syndrome caused by 2019 wuhan pneumonia virus", "disease due to 2019 severe acute respiratory syndrome corona virus 2", "seafood market coronavirus 2019 respiratory illness", "2019 novel chinese corona virus acute disorder", "new corona virus 2019 infection", "2019novel sars corona virus 2 acute severe disease", "2019 novel wuhan virus severe infection", "hcov19 respiratory infection", "2019new sars corona virus2 respiratory illness", "2019 wuhan coronavirus acute respiratory illness", "infection from 2019 severe acute respiratory syndromerelated corona virus 2", "2019new seafood market virus severe acute pneumonia", "pneumonia due to 2019new severe acute respiratory syndromerelated corona virus2", "novel corona virus", "sars corona virus2 2019 acute syndrome", "2019 novel wuhancorona virus severe acute infection", "2019 novel sars corona virus 2 severe acute illness", "2019 wuhancoronavirus severe acute disorder", "syndrome caused by 2019novel seafood market corona virus", "2019 novel wuhan virus severe syndrome", "sarscov2 2019 acute severe infection", "illness due to 2019novel sars coronavirus2", "2019 new wuhancoronavirus respiratory syndrome", "chinese corona virus severe syndrome", "severe acute respiratory syndrome corona virus2 2019 pneumonia", "seafood market virus 2019 pneumonia", "2019novel wuhan seafood market virus respiratory syndrome", "2019 novel seafood market corona virus severe acute pneumonia", "2019novel sars corona virus2 pneumonia", "disorder from 2019novel wuhan seafood market pneumonia virus", "disease caused by wuhan seafood market pneumonia virus 2019", "covid 19 virus acute disorder", "syndrome caused by 2019 novel wuhan seafood market virus", "hubei acute syndrome", "illness from novel coronavirus 2019", "pneumonia from 2019new wuhan seafood market virus", "disorder due to 2019 new seafood market virus", "disease from 2019 sarscov2", "2019 human coronavirus acute respiratory disease", "human corona virus 2019 syndrome", "sars coronavirus2 2019 acute syndrome", "infection due to seafood market coronavirus 2019", "2019 novel seafood market virus respiratory infection", "infection due to 2019 novel wuhancoronavirus", "2019new sars coronavirus2 severe acute disease", "syndrome caused by 2019 wuhancoronavirus", "2019new wuhan pneumonia virus severe respiratory disease", "2019 new wuhan seafood market pneumonia virus severe respiratory disorder", "chinese corona virus 2019 severe syndrome", "hcov 19 acute illness", "sars corona virus2 2019 severe respiratory illness", "2019new sars coronavirus2 severe acute syndrome", "new corona virus 2019 severe acute respiratory illness", "disease caused by 2019novel severe acute respiratory syndrome corona virus 2", "infection due to 2019novel wuhan virus", "2019 corona virus severe acute respiratory disorder", "2019novel seafood market coronavirus respiratory disorder", "2019 sars corona virus2 acute respiratory infection", "2019 new seafood market coronavirus acute severe respiratory infection", "2019 human corona virus severe syndrome", "2019 wuhan virus severe respiratory illness", "2019new wuhan virus acute respiratory disease", "disorder from 2019 new sarscov2", "chinese corona virus 2019 severe acute disorder", "disease caused by 2019 wuhan pneumonia virus", "pneumonia from 2019 new chinese coronavirus", "illness due to wuhan virus 2019", "seafood market virus acute severe syndrome", "2019 wuhan corona virus acute respiratory disease", "hcov19 acute severe respiratory disease", "sars coronavirus2 2019 acute disorder", "sars corona virus2 acute pneumonia", "disease from 2019 novel seafood market virus", "wuhan virus 2019 acute illness", "2019 human corona virus severe respiratory syndrome", "2019 novel wuhan virus acute severe disease", "pneumonia due to human corona virus 2019", "severe acute respiratory syndrome coronavirus 2 2019 disorder", "disease caused by 2019novel wuhan pneumonia virus", "2019 wuhan coronavirus acute illness", "wuhan pneumonia virus 2019 severe infection", "2019new wuhan corona virus acute severe disorder", "2019 sars coronavirus 2 respiratory disorder", "2019 new sars coronavirus2 severe acute infection", "sars cov 2 acute severe respiratory disease", "seafood market acute severe respiratory illness", "new corona virus severe acute disease", "2019novel chinese corona virus acute severe disorder", "severe acute respiratory syndromerelated corona virus 2 disease", "2019novel sars coronavirus2 acute respiratory syndrome", "2019 novel wuhancorona virus acute severe respiratory illness", "2019 novel wuhancoronavirus severe infection", "infection caused by 2019 wuhancorona virus", "2019 new wuhan pneumonia virus respiratory infection", "wuhancorona virus severe disease", "2019novel wuhan virus severe respiratory disease", "2019 wuhan virus acute infection", "2019novel wuhan coronavirus acute syndrome", "2019new seafood market corona virus acute respiratory disease", "hcov 19 severe respiratory syndrome", "illness caused by 2019 severe acute respiratory syndromerelated coronavirus2", "2019 new seafood market corona virus acute severe respiratory syndrome", "2019new seafood market virus respiratory disease", "2019 novel sars corona virus2 severe acute respiratory disorder", "infection from sarscov2", "sars coronavirus2 severe acute pneumonia", "2019new sars coronavirus 2 acute severe respiratory syndrome", "disease caused by 2019 novel wuhan seafood market virus", "syndrome from 2019new sars corona virus2", "2019 new sars coronavirus 2 acute disease", "wuhan acute disorder", "2019 new sars coronavirus 2 severe acute pneumonia", "sarscov2 2019 severe acute disorder", "pneumonia from 2019 new seafood market virus", "2019novel wuhan corona virus respiratory disease", "disorder due to 2019 sars cov 2", "2019novel sars corona virus2 severe syndrome", "2019 chinese coronavirus severe acute respiratory infection", "2019 new wuhan seafood market virus acute syndrome", "wuhan pneumonia virus acute infection", "pneumonia caused by severe acute respiratory syndromerelated corona virus 2", "disorder due to seafood market coronavirus 2019", "disorder from 2019 chinese coronavirus", "2019 chinese corona virus acute severe respiratory infection", "2019new sarscov2 severe acute respiratory illness", "disease caused by 2019 sars coronavirus2", "infection due to 2019novel sarscov2", "illness due to 2019 novel wuhan virus", "wuhan seafood market pneumonia virus severe respiratory infection", "2019 novel wuhancoronavirus severe acute pneumonia", "illness due to sars cov 2", "2019new sars corona virus2 severe acute respiratory infection", "sars coronavirus2 acute illness", "2019 human corona virus infection", "severe acute respiratory syndrome corona virus2 illness", "illness caused by 2019new seafood market virus", "new corona virus 2019 severe acute disorder", "2019 human coronavirus respiratory infection", "2019 new wuhancorona virus severe disorder", "2019novel chinese corona virus disorder", "disease caused by new corona virus 2019", "infection caused by 2019novel chinese coronavirus", "illness due to sarscov2 2019", "2019 wuhan seafood market virus severe disease", "pneumonia caused by 2019new severe acute respiratory syndromerelated corona virus 2", "wuhan seafood market virus severe acute disorder", "2019new wuhan coronavirus syndrome", "2019new chinese coronavirus illness", "2019 novel coronavirus acute respiratory syndrome", "2019 novel wuhan seafood market pneumonia virus illness", "chinese corona virus 2019 acute severe pneumonia", "2019new sarscov2 severe acute pneumonia", "2019 novel seafood market corona virus acute disorder", "2019new sars corona virus 2 severe acute respiratory disorder", "disorder due to 2019novel severe acute respiratory syndrome corona virus2", "2019 novel chinese coronavirus acute severe respiratory disease", "2019 new coronavirus acute severe syndrome", "2019novel wuhancorona virus severe acute syndrome", "2019 sars coronavirus 2 severe acute respiratory illness", "new coronavirus 2019 severe illness", "disorder caused by seafood market coronavirus 2019", "disorder caused by 2019new severe acute respiratory syndrome coronavirus 2", "infection from 2019new severe acute respiratory syndrome corona virus2", "2019novel wuhan seafood market pneumonia virus respiratory disease", "sars corona virus 2 2019 acute severe respiratory infection", "2019novel sars corona virus2 illness", "2019 novel sars coronavirus2 acute severe respiratory disorder", "infection from 2019new wuhan seafood market virus", "2019 new wuhancorona virus severe acute respiratory syndrome", "2019 new sars coronavirus2 disorder", "seafood market corona virus severe respiratory infection", "seafood market corona virus 2019 acute severe pneumonia", "disorder caused by 2019 new sarscov2", "wuhan corona virus 2019 pneumonia", "illness from 2019novel wuhan corona virus", "disease caused by 2019 novel wuhancorona virus", "2019 novel wuhancoronavirus acute severe infection", "2019 novel sars corona virus2 acute severe disease", "chinese coronavirus acute severe infection", "2019new chinese corona virus severe illness", "2019novel wuhan coronavirus severe acute pneumonia", "disease caused by severe acute respiratory syndrome coronavirus 2 2019", "disease due to 2019 novel severe acute respiratory syndromerelated corona virus 2", "illness from 2019new wuhancorona virus", "2019novel sarscov2 illness", "2019novel chinese corona virus severe disorder", "disease from 2019 human coronavirus", "syndrome from covid19 virus", "illness from 2019 novel sars coronavirus2", "disease due to severe acute respiratory syndromerelated corona virus2 2019", "2019 novel seafood market coronavirus acute severe pneumonia", "chinese corona virus severe respiratory illness", "2019 wuhan seafood market pneumonia virus acute syndrome", "2019novel wuhan pneumonia virus infection", "severe acute respiratory syndrome corona virus 2 2019 illness", "2019novel wuhan coronavirus severe acute infection", "2019novel wuhancoronavirus severe respiratory syndrome", "seafood market acute respiratory disorder", "disease from 2019 novel wuhan seafood market pneumonia virus", "2019new wuhan pneumonia virus severe disorder", "2019 novel sars coronavirus2 acute severe respiratory disease", "2019ncov acute severe infection", "illness due to 2019new sars coronavirus2", "2019 sars corona virus 2 acute severe illness", "seafood market virus 2019 acute severe disorder", "infection from 2019 new seafood market virus", "wuhan seafood market pneumonia virus 2019 severe respiratory infection", "2019new sars cov 2 acute illness", "disorder from sars coronavirus2", "2019novel severe acute respiratory syndrome corona virus2 disease", "2019 chinese coronavirus pneumonia", "illness due to 2019 severe acute respiratory syndromerelated coronavirus 2", "2019new wuhancoronavirus severe respiratory disease", "2019 novel sars corona virus 2 acute respiratory disorder", "sars coronavirus 2 2019 acute severe pneumonia", "illness caused by 2019novel severe acute respiratory syndrome corona virus 2", "illness caused by 2019 new sarscov2", "pneumonia caused by 2019 novel sars corona virus 2", "2019new severe acute respiratory syndromerelated corona virus 2", "wuhan coronavirus 2019 severe disease", "2019 new chinese coronavirus severe respiratory disorder", "disease due to 2019novel seafood market corona virus", "2019new sars coronavirus 2 severe acute illness", "2019 new wuhan coronavirus acute respiratory disease", "novel coronavirus respiratory syndrome", "2019 sars coronavirus 2 disease", "syndrome from 2019 novel wuhan pneumonia virus", "human corona virus 2019 severe acute illness", "2019 new wuhancorona virus severe respiratory syndrome", "2019 wuhan seafood market virus disease", "disease caused by 2019 chinese coronavirus", "2019new severe acute respiratory syndromerelated corona virus 2 pneumonia", "disease from 2019novel sars coronavirus2", "wuhan virus 2019 infection", "2019new seafood market coronavirus acute severe respiratory illness", "pneumonia due to severe acute respiratory syndromerelated corona virus2 2019", "infection due to 2019new severe acute respiratory syndromerelated corona virus2", "chinese corona virus disease", "2019 novel sars coronavirus 2 severe acute respiratory infection", "2019novel wuhan virus disorder", "2019new wuhan corona virus severe respiratory syndrome", "syndrome due to 2019 sars coronavirus2", "chinese corona virus acute respiratory infection", "infection from 2019novel sars cov 2", "2019 novel wuhan seafood market virus severe respiratory illness", "sarscov2 2019 severe disorder", "2019novel wuhan seafood market pneumonia virus severe acute disease", "2019new severe acute respiratory syndromerelated corona virus2 disorder", "syndrome from 2019 new wuhancorona virus", "infection caused by 2019ncov", "covid19 virus syndrome", "wuhan seafood market pneumonia virus 2019 disease", "2019novel chinese corona virus acute disease", "chinese coronavirus 2019 severe disease", "2019 new seafood market virus acute respiratory syndrome", "2019new wuhan corona virus acute pneumonia", "2019 new severe acute respiratory syndromerelated corona virus2 disease", "2019 new wuhan seafood market virus severe acute disorder", "disorder caused by 2019 sars corona virus2", "disorder due to 2019 new wuhan coronavirus", "seafood market virus 2019 acute syndrome", "2019new sars coronavirus2 acute severe respiratory syndrome", "disease from seafood market corona virus 2019", "2019 new sars corona virus2 acute respiratory disorder", "disease caused by 2019 new sars cov 2", "wuhan corona virus 2019 acute severe pneumonia", "sars cov 2 acute severe respiratory infection", "2019 novel severe acute respiratory syndrome corona virus2 illness", "sarscov2 2019 severe acute respiratory syndrome", "2019novel wuhan coronavirus infection", "wuhan corona virus syndrome", "2019new wuhan pneumonia virus acute illness", "wuhan seafood market virus 2019 severe infection", "wuhan virus 2019 severe syndrome", "sars corona virus2 acute syndrome", "novel coronavirus acute respiratory disease", "2019 wuhancoronavirus respiratory disease", "2019 novel wuhan seafood market pneumonia virus acute severe infection", "syndrome caused by sars corona virus2 2019", "2019 new seafood market coronavirus disorder", "syndrome from 2019 novel wuhan virus", "sars cov 2 2019 disorder", "2019new sars coronavirus 2 severe respiratory disorder", "2019novel sarscov2 acute illness", "2019 wuhan virus respiratory disease", "2019 novel corona virus severe pneumonia", "2019 new chinese corona virus severe infection", "2019novel wuhan coronavirus acute severe respiratory infection", "wuhancoronavirus severe acute respiratory illness", "2019 novel seafood market corona virus severe acute disease", "2019novel chinese coronavirus disease", "virus causing covid19 acute severe infection", "chinese corona virus acute severe disease", "illness from 2019 new wuhan coronavirus", "2019novel sars corona virus 2 acute severe respiratory disease", "new corona virus severe acute respiratory syndrome", "syndrome caused by 2019novel sars cov 2", "2019 novel wuhan seafood market virus infection", "2019 novel wuhancoronavirus severe acute infection", "wuhan coronavirus severe acute respiratory illness", "sars cov 2 infection", "wuhan seafood market pneumonia virus 2019 severe acute respiratory illness", "pneumonia caused by 2019 new severe acute respiratory syndrome coronavirus 2", "sars coronavirus 2 2019 acute severe respiratory disorder", "pneumonia from 2019 novel wuhancorona virus", "2019 novel sars coronavirus 2 severe infection", "pneumonia caused by 2019 severe acute respiratory syndromerelated corona virus 2", "pneumonia from wuhancorona virus", "wuhan acute respiratory disorder", "wuhan corona virus respiratory illness", "2019 ncov severe acute illness", "pneumonia due to 2019novel wuhan seafood market virus", "2019 new corona virus pneumonia", "2019novel seafood market corona virus acute disorder", "infection from wuhan virus", "illness caused by 2019 novel severe acute respiratory syndromerelated corona virus2", "2019 new wuhan corona virus infection", "2019 wuhan coronavirus syndrome", "2019 novel sars corona virus2 illness", "new corona virus severe disease", "2019 seafood market virus severe respiratory disorder", "disease from 2019 novel severe acute respiratory syndrome coronavirus 2", "2019new chinese corona virus severe acute pneumonia", "2019 novel sarscov2 illness", "illness caused by new corona virus 2019", "2019novel seafood market virus acute severe respiratory infection", "wuhan virus respiratory illness", "wuhan severe acute illness", "2019 human coronavirus severe acute disorder", "2019 ncov respiratory infection", "wuhan seafood market virus 2019 illness", "infection due to 2019novel chinese corona virus", "syndrome from 2019 novel chinese coronavirus", "wuhan coronavirus 2019 severe respiratory disorder", "syndrome caused by hcov19", "syndrome from 2019 wuhan corona virus", "chinese coronavirus severe respiratory disorder", "2019new wuhan seafood market virus disorder", "2019 new wuhan seafood market virus severe illness", "2019 novel chinese coronavirus severe acute disease", "chinese coronavirus 2019 severe illness", "2019 new corona virus acute severe respiratory syndrome", "chinese corona virus acute syndrome", "chinese coronavirus 2019 acute respiratory disorder", "2019 new wuhan seafood market virus severe acute respiratory infection", "2019 chinese corona virus acute syndrome", "2019 human coronavirus respiratory syndrome", "2019 novel chinese corona virus", "2019new seafood market virus severe respiratory infection", "2019new sars coronavirus 2 respiratory illness", "2019 wuhan seafood market pneumonia virus severe acute respiratory disorder", "disease caused by wuhan coronavirus 2019", "severe acute respiratory syndrome corona virus2 2019", "2019 sars coronavirus 2 severe pneumonia", "syndrome caused by 2019 new coronavirus", "2019novel sars cov 2 severe acute respiratory syndrome", "wuhan corona virus respiratory disease", "infection caused by seafood market corona virus 2019", "illness from 2019novel seafood market virus", "disease due to 2019new chinese corona virus", "pneumonia due to 2019 severe acute respiratory syndrome corona virus 2", "2019ncov severe acute infection", "pneumonia from wuhancorona virus 2019", "covid19 virus acute severe respiratory infection", "sars corona virus2 2019 acute respiratory illness", "2019novel wuhan seafood market pneumonia virus disorder", "severe acute respiratory syndrome coronavirus2 infection", "2019novel sars coronavirus 2 severe respiratory syndrome", "2019 new seafood market corona virus severe respiratory disorder", "infection due to 2019 seafood market virus", "wuhan severe respiratory disease", "hubei respiratory syndrome", "illness caused by 2019 severe acute respiratory syndromerelated corona virus2", "2019new wuhan seafood market virus severe respiratory syndrome", "U07.1", "disorder caused by wuhan coronavirus 2019", "2019 new severe acute respiratory syndromerelated coronavirus2 disorder", "2019 novel wuhan virus acute severe respiratory infection", "disorder from 2019 novel severe acute respiratory syndromerelated corona virus 2", "2019 new wuhan seafood market pneumonia virus illness", "seafood market corona virus acute severe disorder", "infection from 2019new sars coronavirus2", "illness caused by chinese coronavirus", "disease from sarscov2", "novel corona virus severe acute respiratory illness", "sars corona virus 2 acute severe disease", "wuhan corona virus severe pneumonia", "wuhan coronavirus severe acute respiratory syndrome", "disorder caused by 2019 new sars corona virus 2", "2019new wuhancoronavirus severe respiratory syndrome", "wuhan seafood market pneumonia virus 2019 severe acute disease", "2019 new sarscov2 severe acute syndrome", "new corona virus 2019 severe disease", "2019 novel wuhan virus acute respiratory infection", "wuhan pneumonia virus 2019 acute severe illness", "2019new chinese corona virus severe acute respiratory illness", "2019 wuhan seafood market pneumonia virus respiratory infection", "2019 chinese corona virus severe acute disease", "2019 chinese coronavirus infection", "disease caused by 2019new chinese corona virus", "sars corona virus2 disorder", "2019new sars coronavirus2 illness", "hcov 19 acute severe disease", "2019 sars cov 2 severe acute pneumonia", "2019new sars corona virus2 acute severe respiratory disorder", "wuhan severe acute pneumonia", "2019novel seafood market corona virus severe disease", "virus causing covid19 severe respiratory infection", "illness from 2019 novel wuhancoronavirus", "2019 severe acute respiratory syndrome coronavirus 2 disease", "virus causing covid19 severe disorder", "illness from 2019 novel corona virus", "2019 sars coronavirus 2 illness", "chinese corona virus 2019 infection", "infection caused by human coronavirus 2019", "2019 new seafood market virus severe acute pneumonia", "pneumonia from 2019new severe acute respiratory syndrome corona virus2", "2019 novel chinese coronavirus severe respiratory illness", "pneumonia due to seafood market corona virus 2019", "2019 novel sars cov 2 severe respiratory infection", "2019 new coronavirus severe illness", "disease from 2019new severe acute respiratory syndrome coronavirus 2", "novel coronavirus 2019 severe acute respiratory disorder", "infection caused by 2019 chinese coronavirus", "2019 wuhan seafood market virus acute respiratory disorder", "wuhancorona virus 2019 severe disorder", "2019new wuhan seafood market virus severe acute illness", "disorder from seafood market corona virus", "seafood market corona virus 2019 acute pneumonia", "illness from new coronavirus 2019", "2019novel wuhan corona virus severe respiratory syndrome", "disease due to severe acute respiratory syndromerelated corona virus 2 2019", "2019new wuhan seafood market pneumonia virus acute severe respiratory syndrome", "syndrome due to 2019 new sarscov2", "illness from 2019 novel severe acute respiratory syndromerelated corona virus2", "2019 seafood market coronavirus", "disease due to sars coronavirus 2", "2019novel severe acute respiratory syndromerelated coronavirus2 disease", "2019novel seafood market corona virus severe acute syndrome", "2019new sars corona virus 2 asevere acute respiratory disease", "wuhancoronavirus 2019 severe acute respiratory illness", "2019 new wuhan pneumonia virus acute disorder", "2019 wuhan virus respiratory illness", "2019 sars corona virus 2 severe acute respiratory disorder", "2019 new sars corona virus2 illness", "hcov 19 acute respiratory syndrome", "2019 sars cov 2 acute severe syndrome", "2019 new sars coronavirus 2 acute severe respiratory infection", "infection due to wuhan seafood market pneumonia virus", "2019new chinese coronavirus asevere acute respiratory disease", "2019 new seafood market coronavirus acute disorder", "seafood market corona virus severe pneumonia", "wuhan coronavirus acute severe disease", "wuhan virus 2019 respiratory infection", "wuhan coronavirus 2019 severe pneumonia", "disease from 2019 wuhan corona virus", "wuhan corona virus 2019 acute severe respiratory disease", "2019 sars coronavirus 2 severe syndrome", "2019 wuhancoronavirus syndrome", "severe acute respiratory syndromerelated corona virus2 disorder", "2019 new sars coronavirus 2 acute syndrome", "2019 wuhan corona virus acute severe infection", "2019 ncov acute severe illness", "infection caused by wuhan virus", "2019novel wuhan coronavirus acute severe respiratory disease", "chinese corona virus 2019 acute respiratory disorder", "pneumonia from 2019novel wuhan coronavirus", "2019novel severe acute respiratory syndromerelated corona virus 2 disease", "2019new sarscov2 acute respiratory infection", "sars cov 2 severe acute disease", "pneumonia caused by severe acute respiratory syndrome coronavirus 2", "chinese coronavirus respiratory infection", "2019 human coronavirus acute respiratory illness", "2019 sarscov2 acute severe disorder", "chinese corona virus acute severe respiratory disease", "2019novel seafood market corona virus severe acute respiratory syndrome", "2019novel wuhan coronavirus disease", "2019 novel wuhan seafood market virus acute severe respiratory illness", "2019 new sars cov 2 acute severe pneumonia", "2019novel wuhan coronavirus acute severe pneumonia", "2019 wuhan corona virus acute respiratory syndrome", "wuhan seafood market pneumonia virus severe respiratory illness", "infection caused by severe acute respiratory syndrome corona virus 2 2019", "wuhan virus severe disorder", "2019 new wuhancorona virus severe infection", "2019 novel sars coronavirus 2 illness", "disorder from 2019novel severe acute respiratory syndromerelated coronavirus2", "2019new seafood market coronavirus severe pneumonia", "2019 new wuhan pneumonia virus acute severe disorder", "wuhan virus acute severe respiratory disorder", "2019 novel wuhan corona virus acute severe respiratory disease", "illness due to 2019new wuhan seafood market virus", "2019 sars coronavirus 2 acute severe infection", "seafood market virus severe acute illness", "2019 new sars coronavirus2 acute severe respiratory illness", "sars coronavirus2 2019 acute disease", "2019 new wuhan virus acute severe illness", "infection from 2019ncov", "2019 new seafood market coronavirus severe acute respiratory disorder", "2019novel seafood market virus acute infection", "new coronavirus acute severe respiratory syndrome", "2019 seafood market corona virus acute severe syndrome", "2019 new sars corona virus2 severe acute illness", "2019new seafood market coronavirus severe infection", "2019 novel wuhancorona virus acute respiratory illness", "infection from wuhan virus 2019", "syndrome from 2019 new wuhan coronavirus", "new corona virus illness", "2019 novel sars corona virus 2 severe respiratory syndrome", "2019 novel corona virus severe acute syndrome", "wuhan acute severe disease", "2019new wuhan virus acute severe respiratory disease", "2019 new sarscov2 acute pneumonia", "seafood market coronavirus 2019 acute severe respiratory disorder", "2019 novel sarscov2 severe infection", "pneumonia due to 2019 novel severe acute respiratory syndrome corona virus 2", "2019 chinese corona virus respiratory disorder", "2019new wuhancorona virus asevere acute respiratory disease", "disease caused by 2019 new severe acute respiratory syndromerelated corona virus2", "2019 novel coronavirus severe acute disorder", "2019new sars coronavirus2 acute severe respiratory illness", "2019novel sars coronavirus 2 severe respiratory illness", "2019 wuhan corona virus acute severe disorder", "2019novel sars corona virus 2 disease", "2019 human corona virus severe acute pneumonia", "2019 novel wuhan virus acute severe pneumonia", "2019novel sars coronavirus2 severe disorder", "2019new sars corona virus 2 illness", "severe acute respiratory syndromerelated corona virus2 2019 illness", "2019new wuhan corona virus acute severe respiratory syndrome", "pneumonia due to 2019 sarscov2", "disorder caused by wuhan pneumonia virus 2019", "2019 ncov severe acute respiratory infection", "2019 novel sars corona virus2 severe acute pneumonia", "wuhan coronavirus 2019 acute severe pneumonia", "new corona virus 2019 acute respiratory infection", "disorder due to 2019new sars coronavirus 2", "infection from sars corona virus2 2019", "pneumonia from seafood market coronavirus 2019", "2019novel wuhan seafood market pneumonia virus acute illness", "disease from 2019novel sars coronavirus 2", "2019 wuhan pneumonia virus acute infection", "2019new wuhan seafood market virus acute severe disorder", "wuhan seafood market virus acute illness", "illness from 2019 new chinese coronavirus", "wuhancoronavirus 2019 acute disease", "illness due to wuhancorona virus", "pneumonia from 2019 new seafood market coronavirus", "hcov19 severe respiratory disorder", "2019 novel sars corona virus2 acute severe pneumonia", "2019 novel wuhancorona virus acute respiratory disease", "syndrome due to 2019 new sars corona virus2", "2019 wuhan coronavirus severe acute disorder", "novel coronavirus acute respiratory illness", "disease caused by chinese coronavirus 2019", "2019 novel wuhan seafood market pneumonia virus acute severe syndrome", "2019 seafood market corona virus acute infection", "illness caused by wuhan pneumonia virus", "2019novel seafood market corona virus respiratory disease", "2019 novel seafood market corona virus infection", "2019novel seafood market virus severe acute illness", "2019 novel sars coronavirus2 acute infection", "human corona virus 2019 acute respiratory syndrome", "disorder caused by sars cov 2", "2019 new chinese coronavirus severe acute respiratory syndrome", "syndrome due to 2019 novel chinese corona virus", "2019 sarscov2 acute severe syndrome", "2019novel wuhancoronavirus respiratory disease", "sarscov2 asevere acute respiratory disease", "infection due to severe acute respiratory syndromerelated corona virus 2", "wuhan corona virus 2019 acute respiratory illness", "2019 sars coronavirus2 severe disease", "2019 wuhan pneumonia virus acute illness", "disorder due to seafood market corona virus", "2019 new sars corona virus2 acute severe illness", "wuhan seafood market pneumonia virus pneumonia", "2019novel sars corona virus 2 acute infection", "wuhancorona virus asevere acute respiratory disease", "2019 novel wuhancoronavirus acute infection", "syndrome from wuhancorona virus 2019", "wuhan seafood market pneumonia virus 2019 severe infection", "infection caused by 2019 wuhan corona virus", "2019new severe acute respiratory syndrome corona virus2 infection", "seafood market corona virus pneumonia", "2019 sarscov2 severe acute illness", "2019new chinese corona virus severe respiratory syndrome", "seafood market coronavirus asevere acute respiratory disease", "2019 novel coronavirus severe respiratory syndrome", "disease due to severe acute respiratory syndromerelated corona virus2", "2019 novel corona virus acute respiratory disease", "2019 novel chinese coronavirus respiratory infection", "2019new severe acute respiratory syndrome coronavirus2 infection", "2019 novel wuhan virus asevere acute respiratory disease", "2019 human corona virus severe acute respiratory syndrome", "wuhan seafood market virus 2019 respiratory disorder", "2019 novel sars coronavirus2 severe acute disorder", "hubei disorder", "disorder due to 2019 seafood market corona virus", "pneumonia due to wuhancorona virus 2019", "2019novel sars corona virus2 respiratory illness", "infection caused by 2019new severe acute respiratory syndrome corona virus2", "illness caused by 2019novel wuhan virus", "hcov 19 asevere acute respiratory disease", "2019novel sars coronavirus2 respiratory illness", "disorder from sars corona virus 2 2019", "wuhan pneumonia virus 2019 severe acute infection", "seafood market coronavirus severe acute respiratory syndrome", "new corona virus 2019 acute severe disorder", "2019new seafood market coronavirus severe acute respiratory syndrome", "2019 new wuhancoronavirus asevere acute respiratory disease", "2019 novel corona virus severe infection", "2019novel seafood market corona virus acute severe disorder", "novel corona virus 2019 acute respiratory infection", "2019 novel chinese corona virus acute respiratory disorder", "sars corona virus 2 severe disease", "2019 novel wuhan coronavirus respiratory disorder", "disorder due to seafood market coronavirus", "2019 chinese coronavirus severe acute syndrome", "disorder due to 2019 novel wuhan seafood market virus", "new corona virus severe acute disorder", "disease due to 2019novel sars corona virus 2", "2019new wuhan seafood market virus severe syndrome", "disease caused by 2019new severe acute respiratory syndromerelated coronavirus2", "2019 novel wuhan coronavirus severe disease", "2019 novel wuhan pneumonia virus acute respiratory illness", "wuhancoronavirus 2019 severe acute respiratory disorder", "2019new seafood market virus disease", "2019 new seafood market corona virus acute respiratory disease", "disease due to seafood market corona virus 2019", "syndrome due to 2019 sars corona virus 2", "2019 seafood market virus severe respiratory illness", "2019 wuhan seafood market virus severe acute illness", "infection from wuhan pneumonia virus", "novel coronavirus disorder", "human coronavirus 2019 acute pneumonia", "2019 novel sars corona virus2 severe respiratory syndrome", "2019 wuhan pneumonia virus severe infection", "pneumonia caused by 2019 new coronavirus", "2019 novel seafood market coronavirus pneumonia", "2019novel wuhan pneumonia virus severe acute disease", "disorder due to 2019new severe acute respiratory syndrome coronavirus2", "sars coronavirus 2 acute pneumonia", "disorder caused by 2019new severe acute respiratory syndrome corona virus 2", "2019new sars coronavirus2 severe respiratory illness", "illness from severe acute respiratory syndrome coronavirus2", "wuhan corona virus 2019 disease", "seafood market corona virus acute severe respiratory illness", "infection caused by severe acute respiratory syndrome coronavirus 2 2019", "2019new sars cov 2 respiratory illness", "new corona virus acute respiratory disease", "sarscov2 2019 acute severe disorder", "disorder caused by severe acute respiratory syndrome coronavirus 2 2019", "wuhan pneumonia virus severe acute respiratory syndrome", "pneumonia caused by 2019 novel wuhancoronavirus", "2019 novel corona virus syndrome", "2019 new wuhan seafood market pneumonia virus severe pneumonia", "2019novel wuhan virus asevere acute respiratory disease", "2019 novel wuhan seafood market pneumonia virus severe acute respiratory syndrome", "wuhan corona virus 2019 acute severe infection", "2019 ncov syndrome", "2019 new chinese coronavirus acute severe respiratory syndrome", "2019new wuhan seafood market pneumonia virus acute respiratory infection", "seafood market coronavirus acute severe respiratory disorder", "disease due to covid19 virus", "2019 novel sars corona virus 2 disorder", "disorder from severe acute respiratory syndromerelated coronavirus2 2019", "2019 new wuhan corona virus severe acute disease", "chinese coronavirus 2019 severe acute respiratory disorder", "2019 novel sarscov2 infection", "2019 sars corona virus 2 acute illness", "2019 novel seafood market coronavirus infection", "illness due to 2019novel wuhancoronavirus", "disease from 2019novel seafood market coronavirus", "sars cov 2 2019 severe disease", "2019 new wuhan pneumonia virus respiratory syndrome", "disorder due to 2019 novel wuhan seafood market pneumonia virus", "2019 new coronavirus acute severe respiratory illness", "seafood market virus 2019 acute disorder", "2019novel wuhan virus severe respiratory disorder", "infection due to 2019new seafood market corona virus", "infection due to 2019 novel sars corona virus2", "2019 new wuhan pneumonia virus acute syndrome", "2019 new sars corona virus 2 acute illness", "severe acute respiratory syndrome coronavirus 2 2019 illness", "2019 wuhan seafood market pneumonia virus pneumonia", "pneumonia from 2019 new wuhancorona virus", "wuhan coronavirus 2019 severe acute respiratory infection", "2019 new wuhan corona virus acute severe respiratory disorder", "disorder caused by severe acute respiratory syndromerelated coronavirus 2", "disorder from 2019 new severe acute respiratory syndrome corona virus 2", "2019 new coronavirus acute syndrome", "2019 novel seafood market virus disorder", "2019 sarscov2", "wuhan respiratory disease", "2019novel sars coronavirus2 acute respiratory infection", "syndrome from 2019novel chinese coronavirus", "illness caused by 2019new sars coronavirus2", "2019 sars coronavirus 2 acute severe respiratory syndrome", "2019 new seafood market coronavirus respiratory disorder", "illness due to 2019novel wuhan coronavirus", "disease from 2019novel wuhan virus", "illness from 2019 novel wuhan corona virus", "2019 novel wuhan coronavirus pneumonia", "seafood market virus severe acute infection", "pneumonia due to 2019novel sars corona virus2", "2019new sarscov2 respiratory syndrome", "2019 novel wuhancoronavirus severe acute respiratory syndrome", "disease due to 2019novel wuhan corona virus", "wuhan coronavirus respiratory illness", "sars corona virus 2 2019 severe acute infection", "illness caused by wuhan corona virus", "novel coronavirus 2019 acute disease", "2019 new wuhan virus acute respiratory illness", "2019 human coronavirus severe respiratory illness", "2019 seafood market coronavirus disease", "2019 novel seafood market virus acute pneumonia", "human coronavirus 2019 infection", "sars coronavirus2 2019 syndrome", "pneumonia from 2019 novel severe acute respiratory syndrome coronavirus2", "syndrome due to 2019 new wuhancorona virus", "2019 wuhan coronavirus", "infection caused by 2019 novel coronavirus", "pneumonia caused by 2019novel wuhan seafood market virus", "wuhan seafood market pneumonia virus acute infection", "infection from severe acute respiratory syndromerelated corona virus2", "chinese coronavirus 2019 acute severe respiratory disease", "2019novel sars coronavirus2 acute disorder", "2019 novel severe acute respiratory syndromerelated coronavirus2 pneumonia", "disease due to 2019new wuhan pneumonia virus", "illness due to 2019 novel seafood market virus", "2019new sars corona virus 2 severe infection", "2019 new sars coronavirus 2 severe syndrome", "infection caused by 2019 novel wuhan pneumonia virus", "2019 new wuhan seafood market pneumonia virus acute severe respiratory disease", "illness from 2019novel severe acute respiratory syndrome corona virus2", "sars coronavirus2 2019 acute severe disease", "2019 novel coronavirus severe syndrome", "2019 new wuhan virus acute respiratory infection", "seafood market corona virus severe acute syndrome", "syndrome due to new corona virus 2019", "2019 novel wuhan pneumonia virus asevere acute respiratory disease", "2019new sars coronavirus2 severe acute respiratory infection", "2019 novel wuhan pneumonia virus severe acute disorder", "2019 new sars corona virus 2 acute severe respiratory disorder", "syndrome caused by 2019 new chinese coronavirus", "2019 new sars corona virus2 severe illness", "severe acute respiratory syndrome corona virus 2 disease", "2019 sars corona virus 2 acute infection", "2019 novel coronavirus respiratory illness", "2019ncov respiratory syndrome", "2019 novel wuhan corona virus respiratory illness", "disorder caused by 2019 novel sars cov 2", "2019 novel chinese coronavirus respiratory disease", "human coronavirus 2019 severe acute disorder", "disorder due to sars corona virus 2", "infection from new coronavirus 2019", "illness due to novel coronavirus 2019", "2019novel wuhancorona virus acute severe respiratory infection", "disease caused by novel corona virus 2019", "sars corona virus 2 pneumonia", "2019 human coronavirus respiratory disease", "infection from 2019 new wuhancorona virus", "hubei severe acute respiratory infection", "infection from 2019 new sarscov2", "2019 new wuhan coronavirus acute severe respiratory syndrome", "2019novel wuhancorona virus severe acute respiratory illness", "2019 new sars corona virus2 respiratory infection", "new corona virus severe illness", "infection caused by new coronavirus", "2019 sars coronavirus2 respiratory illness", "syndrome caused by sars coronavirus2", "2019 novel seafood market coronavirus syndrome", "sars coronavirus2 respiratory illness", "2019 wuhan seafood market virus acute severe respiratory illness", "disorder from 2019 severe acute respiratory syndrome corona virus 2", "2019 new wuhancoronavirus acute severe pneumonia", "disease due to 2019 novel sars corona virus 2", "2019 ncov acute illness", "MeSH:M000679178", "2019novel wuhan coronavirus acute disorder", "sars corona virus 2 2019 acute severe disease", "infection due to sars coronavirus 2", "pneumonia due to 2019 novel sars coronavirus2", "2019new wuhancorona virus acute infection", "2019 novel corona virus acute severe respiratory disorder", "2019 new severe acute respiratory syndrome coronavirus2 pneumonia", "2019 new sars corona virus 2 acute respiratory infection", "wuhan seafood market virus 2019 disorder", "2019 novel wuhan coronavirus acute respiratory disease", "infection caused by 2019 novel seafood market virus", "disorder due to 2019new sars cov 2", "2019 novel chinese coronavirus acute severe illness", "2019 novel sarscov2 respiratory illness", "wuhan coronavirus 2019 severe respiratory illness", "2019 chinese coronavirus acute severe syndrome", "2019new sars corona virus 2 acute severe pneumonia", "seafood market corona virus 2019 severe respiratory illness", "disorder from wuhancoronavirus 2019", "2019 novel sars coronavirus 2 acute severe syndrome", "2019new wuhan corona virus severe infection", "2019 severe acute respiratory syndromerelated corona virus2 disease", "wuhan virus severe acute pneumonia", "severe acute respiratory syndromerelated corona virus 2 infection", "2019 novel corona virus acute respiratory syndrome", "wuhan corona virus acute severe disorder", "2019new sars coronavirus 2 acute syndrome", "2019 novel sars corona virus2 acute illness", "syndrome due to 2019novel wuhan seafood market pneumonia virus", "infection from seafood market virus", "infection caused by 2019new severe acute respiratory syndromerelated corona virus 2", "disorder due to 2019 sarscov2", "illness from 2019 severe acute respiratory syndrome coronavirus2", "2019new seafood market virus severe disease", "disorder caused by 2019 new severe acute respiratory syndromerelated coronavirus 2", "2019new wuhan seafood market virus acute respiratory syndrome", "seafood market coronavirus acute disease", "infection due to wuhan seafood market virus 2019", "wuhan seafood market pneumonia virus severe respiratory disease", "sarscov2 disorder", "covid19 virus severe infection", "infection caused by severe acute respiratory syndrome corona virus 2", "2019new wuhan pneumonia virus acute severe respiratory disorder", "illness due to 2019 novel sars coronavirus2", "hcov 19 pneumonia", "2019 novel sars corona virus 2 severe acute pneumonia", "syndrome caused by seafood market coronavirus 2019", "2019 new sars coronavirus2 severe infection", "new coronavirus 2019 acute severe respiratory disorder", "2019novel wuhancorona virus severe pneumonia", "2019 novel chinese coronavirus severe respiratory disease", "2019novel wuhan virus syndrome", "pneumonia due to 2019novel wuhan coronavirus", "2019 sarscov2 illness", "sarscov2 acute respiratory infection", "2019 novel sars cov 2 acute severe respiratory syndrome", "disease caused by virus responsible for covid19", "2019 chinese coronavirus respiratory disorder", "2019 wuhan pneumonia virus respiratory syndrome", "2019new seafood market coronavirus syndrome", "2019 human corona virus illness", "2019 seafood market coronavirus respiratory disease", "2019 human coronavirus acute severe respiratory disorder", "2019 new severe acute respiratory syndromerelated corona virus 2 disorder", "2019 new coronavirus severe acute respiratory infection", "illness caused by 2019 new wuhan corona virus", "2019 human corona virus severe illness", "sars corona virus2 2019 severe acute pneumonia", "syndrome caused by wuhan seafood market virus 2019", "2019 novel sars cov 2 acute severe syndrome", "2019new sars corona virus2 severe respiratory disease", "2019novel seafood market virus acute severe illness", "infection due to 2019 new wuhan pneumonia virus", "2019 seafood market corona virus acute severe illness", "disease caused by chinese corona virus 2019", "virus causing covid19 acute severe respiratory illness", "syndrome due to 2019 new sars corona virus 2", "seafood market coronavirus 2019 acute respiratory disease", "2019 sars corona virus2 asevere acute respiratory disease", "disease due to seafood market virus 2019", "infection from sars cov 2", "infection caused by 2019 new wuhancorona virus", "disorder from wuhancorona virus", "2019 wuhan pneumonia virus severe respiratory disorder", "wuhan seafood market pneumonia virus acute severe respiratory syndrome", "2019 new corona virus severe respiratory disease", "illness from 2019new severe acute respiratory syndrome corona virus 2", "disorder caused by seafood market corona virus", "2019 human corona virus acute disease", "sars coronavirus2 severe acute disease", "2019 novel corona virus severe acute illness", "wuhancoronavirus 2019 severe illness", "2019 novel sars coronavirus 2 severe acute disease", "pneumonia caused by wuhan seafood market pneumonia virus", "2019novel seafood market corona virus", "virus causing covid19 severe disease", "2019novel severe acute respiratory syndromerelated corona virus2", "wuhan pneumonia virus 2019 respiratory infection", "2019 novel sarscov2 acute disorder", "2019 novel chinese coronavirus acute severe pneumonia", "pneumonia due to 2019 wuhancoronavirus", "2019 new chinese corona virus acute disorder", "disorder due to 2019 wuhan virus", "pneumonia from 2019 novel severe acute respiratory syndrome corona virus 2", "sars corona virus 2 2019 acute respiratory disorder", "illness caused by 2019novel severe acute respiratory syndromerelated coronavirus 2", "illness from 2019new sars coronavirus 2", "novel corona virus severe disorder", "2019 novel wuhan corona virus acute severe illness", "disease due to 2019 new wuhancoronavirus", "syndrome from hcov19", "2019new chinese corona virus acute respiratory infection", "2019new wuhan seafood market virus acute severe syndrome", "covid19 virus acute severe disorder", "2019 novel sars cov 2 acute severe pneumonia", "illness from 2019 wuhan pneumonia virus", "2019 new wuhancorona virus", "2019 new wuhan pneumonia virus severe disease", "disorder caused by 2019 new seafood market corona virus", "syndrome caused by wuhan corona virus 2019", "infection due to 2019 new severe acute respiratory syndrome coronavirus2", "2019novel wuhan seafood market pneumonia virus infection", "2019 new seafood market coronavirus severe acute respiratory illness", "infection due to 2019 novel wuhan seafood market virus", "2019novel wuhancorona virus acute infection", "illness caused by 2019 novel severe acute respiratory syndromerelated coronavirus 2", "illness caused by wuhan seafood market virus 2019", "2019 novel chinese corona virus acute severe respiratory infection", "syndrome caused by 2019 novel corona virus", "disease caused by virus causing covid19", "syndrome from 2019new wuhan coronavirus", "2019 new coronavirus severe infection", "2019 novel seafood market coronavirus acute infection", "sarscov2 respiratory disease", "wuhan coronavirus 2019 acute respiratory infection", "2019novel seafood market coronavirus acute severe disorder", "wuhan virus", "2019novel wuhan coronavirus acute severe respiratory syndrome", "disorder from 2019 novel sars corona virus 2", "2019 sars cov 2 acute severe disease", "2019 wuhan coronavirus acute severe respiratory illness", "illness from severe acute respiratory syndromerelated corona virus2 2019", "severe acute respiratory syndrome coronavirus 2", "chinese coronavirus 2019 respiratory infection", "2019 novel severe acute respiratory syndrome coronavirus 2 disease", "pneumonia from 2019new wuhan virus", "wuhan virus pneumonia", "sars cov 2 severe acute respiratory syndrome", "2019 human coronavirus severe acute illness", "new coronavirus infection", "2019 novel wuhancorona virus respiratory syndrome", "2019 severe acute respiratory syndromerelated corona virus 2 illness", "pneumonia caused by 2019novel sars corona virus 2", "sars cov 2 acute severe syndrome", "infection from new coronavirus", "illness from wuhancoronavirus", "pneumonia due to 2019 new sars coronavirus 2", "sars coronavirus 2 2019 disease", "2019 sars corona virus 2 severe acute syndrome", "illness caused by seafood market coronavirus 2019", "2019new sars cov 2 acute severe pneumonia", "chinese coronavirus 2019 acute severe disorder", "2019novel wuhan pneumonia virus syndrome", "2019new sars coronavirus 2 acute infection", "2019 novel wuhan virus acute severe respiratory illness", "sars corona virus2 acute infection", "2019 novel seafood market corona virus disease", "2019 wuhan virus acute disease", "illness from 2019novel wuhan coronavirus", "syndrome due to novel coronavirus 2019", "human corona virus 2019 severe respiratory illness", "2019 ncov acute respiratory disease", "2019 severe acute respiratory syndrome corona virus 2 illness", "sars cov 2 disease", "2019 novel sars corona virus 2 acute disorder", "wuhan seafood market virus respiratory disorder", "2019novel severe acute respiratory syndromerelated corona virus 2 disorder", "new coronavirus acute disease", "2019 novel coronavirus acute severe respiratory disease", "wuhancorona virus severe acute disorder", "2019 wuhan coronavirus pneumonia", "2019novel sars cov 2 severe syndrome", "2019 new seafood market corona virus severe respiratory syndrome", "2019 new wuhancorona virus acute severe respiratory disease", "seafood market corona virus acute severe respiratory disease", "infection caused by virus causing covid19", "pneumonia from sars coronavirus2 2019", "2019 new wuhancoronavirus severe acute respiratory disorder", "disorder caused by 2019 sars coronavirus 2", "2019 wuhan virus acute respiratory disorder", "disorder from virus causing covid19", "2019novel sars corona virus2 severe acute infection", "sarscov2 2019 disorder", "wuhan acute infection", "wuhan pneumonia virus syndrome", "chinese coronavirus respiratory syndrome", "illness from covid 19 virus", "seafood market virus 2019 acute severe respiratory disorder", "infection caused by 2019 seafood market corona virus", "2019 new seafood market virus severe acute disease", "disease due to 2019 novel wuhan pneumonia virus", "2019 wuhancorona virus severe infection", "disease due to sarscov2 2019", "2019 novel chinese coronavirus disorder", "illness from wuhan coronavirus 2019", "2019 new sars corona virus 2 acute severe respiratory syndrome", "pneumonia due to 2019novel wuhan virus", "virus causing covid19 severe respiratory illness", "infection caused by 2019new wuhan virus", "2019 ncov respiratory illness", "2019 sars coronavirus 2 acute illness", "2019 chinese coronavirus acute respiratory disorder", "2019 seafood market corona virus acute severe pneumonia", "2019novel seafood market virus acute respiratory syndrome", "wuhan seafood market pneumonia virus 2019", "disease caused by 2019 new chinese coronavirus", "2019 chinese coronavirus", "2019novel wuhan seafood market pneumonia virus severe acute respiratory disorder", "disorder due to 2019 wuhan corona virus", "2019 seafood market virus acute pneumonia", "disease from 2019 severe acute respiratory syndrome coronavirus 2", "2019 new coronavirus respiratory disorder", "novel corona virus 2019 acute respiratory disease", "2019novel sars cov 2 severe acute respiratory infection", "infection caused by sars cov 2", "sarscov2 severe disease", "2019novel seafood market virus severe pneumonia", "2019 new wuhan corona virus illness", "2019 new coronavirus acute respiratory disorder", "pneumonia caused by wuhancorona virus", "2019ncov acute disease", "syndrome due to 2019new wuhancoronavirus", "2019 novel wuhan corona virus acute severe respiratory syndrome", "2019 seafood market corona virus severe respiratory disorder", "disease from new coronavirus 2019", "2019 sars coronavirus2 infection", "chinese corona virus 2019 severe acute infection", "2019 novel seafood market coronavirus disorder", "illness due to 2019 sars cov 2", "2019 new wuhan pneumonia virus infection", "syndrome from 2019 new wuhan corona virus", "severe acute respiratory syndromerelated coronavirus 2", "sars corona virus 2 2019 acute disorder", "wuhan virus 2019 severe respiratory syndrome", "syndrome from 2019 novel wuhancoronavirus", "2019 novel chinese coronavirus severe acute respiratory infection", "sars cov 2 acute disorder", "2019 new sars coronavirus2 severe acute pneumonia", "sars coronavirus 2 severe acute infection", "2019 novel sars corona virus2 acute respiratory syndrome", "wuhancoronavirus 2019 severe respiratory illness", "wuhancoronavirus asevere acute respiratory disease", "2019 novel sarscov2 acute illness", "2019 new sars coronavirus2 acute disorder", "2019 corona virus severe respiratory disease", "illness due to 2019new sarscov2", "2019novel seafood market virus acute severe syndrome", "2019new wuhan coronavirus severe acute illness", "2019new wuhan virus acute severe syndrome", "2019novel wuhan pneumonia virus acute severe syndrome", "syndrome caused by 2019 novel wuhan virus", "disease caused by severe acute respiratory syndromerelated corona virus2", "wuhancoronavirus acute severe syndrome", "2019 novel sarscov2 asevere acute respiratory disease", "sars coronavirus 2 2019 acute severe infection", "2019 novel wuhan virus severe respiratory disorder", "infection caused by 2019 wuhan seafood market virus", "sars corona virus2 2019 syndrome", "2019new seafood market virus severe pneumonia", "syndrome from 2019new wuhan seafood market virus", "2019new sars coronavirus 2 pneumonia", "2019novel wuhan seafood market virus syndrome", "disease from severe acute respiratory syndrome coronavirus2", "2019 wuhan corona virus severe acute respiratory disorder", "2019new seafood market corona virus severe respiratory disease", "infection due to 2019 severe acute respiratory syndromerelated corona virus 2", "seafood market coronavirus severe syndrome", "infection due to 2019 novel seafood market corona virus", "2019new wuhancorona virus acute severe disease", "2019 seafood market virus severe acute disease", "illness from new corona virus", "2019new seafood market corona virus acute infection", "2019 novel corona virus acute respiratory disorder", "2019new wuhan virus severe disorder", "2019 new coronavirus severe acute respiratory syndrome", "illness caused by 2019 novel wuhancoronavirus", "disease due to 2019 chinese corona virus", "infection caused by 2019novel seafood market coronavirus", "pneumonia caused by 2019 novel sars corona virus2", "2019 severe acute respiratory syndromerelated coronavirus 2 pneumonia", "2019 novel seafood market coronavirus acute disease", "2019 novel wuhancorona virus infection", "2019 novel wuhancorona virus acute pneumonia", "2019new seafood market virus acute respiratory syndrome", "2019new sars cov 2 severe syndrome", "2019 new wuhancoronavirus respiratory disease", "wuhan seafood market virus 2019 disease", "2019 new sars corona virus2 severe acute pneumonia", "2019 human corona virus respiratory syndrome", "2019novel seafood market coronavirus acute respiratory infection", "2019 new wuhancorona virus pneumonia", "2019 sars corona virus 2 acute respiratory illness", "disorder due to seafood market virus", "wuhan seafood market pneumonia virus severe acute pneumonia", "2019new sars coronavirus2 severe acute illness", "syndrome due to wuhan virus 2019", "human coronavirus 2019 acute severe respiratory syndrome", "2019novel sars coronavirus2 respiratory disorder", "2019novel wuhan virus severe infection", "pneumonia from sars corona virus 2", "chinese corona virus acute infection", "disease due to 2019new severe acute respiratory syndromerelated coronavirus2", "2019 novel sarscov2 acute severe respiratory disease", "2019 corona virus acute severe respiratory infection", "illness caused by 2019novel wuhan corona virus", "disease due to sars cov 2 2019", "pneumonia from 2019 new sars cov 2", "2019new wuhan virus acute syndrome", "disease from wuhan seafood market virus", "2019 sars corona virus2 acute syndrome", "illness caused by wuhan virus", "severe acute respiratory syndrome coronavirus 2 disorder", "human coronavirus 2019 acute severe syndrome", "sars coronavirus 2 2019 acute infection", "novel corona virus acute severe illness", "2019 new wuhan corona virus severe acute respiratory infection", "2019 novel sars coronavirus2 acute respiratory syndrome", "2019 new sarscov2 acute severe respiratory syndrome", "illness due to 2019 new wuhan pneumonia virus", "2019novel seafood market coronavirus respiratory syndrome", "seafood market corona virus acute severe illness", "disorder from 2019 new severe acute respiratory syndromerelated coronavirus2", "2019 novel chinese corona virus severe respiratory illness", "2019 novel wuhan seafood market pneumonia virus disease", "2019novel sars coronavirus2 severe acute respiratory illness", "wuhan corona virus 2019 severe illness", "syndrome due to 2019 new sars coronavirus 2", "seafood market corona virus syndrome", "2019 novel coronavirus severe disorder", "wuhan corona virus 2019 acute pneumonia", "2019new sars cov 2 disease", "wuhan severe respiratory syndrome", "covid19 virus severe acute respiratory illness", "2019 new chinese corona virus acute pneumonia", "disorder due to 2019 severe acute respiratory syndrome corona virus 2", "2019 new sars coronavirus2 acute syndrome", "2019novel chinese coronavirus acute severe disorder", "2019 novel coronavirus acute respiratory illness", "illness from 2019new sars cov 2", "2019new chinese coronavirus severe respiratory illness", "2019 new chinese corona virus illness", "2019 new chinese coronavirus severe acute syndrome", "hcov 19 respiratory infection", "2019novel wuhancorona virus acute severe pneumonia", "pneumonia from 2019 new sars coronavirus 2", "wuhan corona virus 2019 acute syndrome", "seafood market coronavirus 2019 severe respiratory syndrome", "2019 new seafood market virus disease", "sars cov 2 acute severe respiratory disorder", "infection caused by wuhancorona virus", "illness due to 2019 novel severe acute respiratory syndromerelated coronavirus 2", "disorder from novel corona virus", "disorder caused by 2019novel severe acute respiratory syndrome corona virus 2", "2019 new corona virus acute severe disease", "sars corona virus 2 acute severe syndrome", "2019 human corona virus severe acute syndrome", "sars coronavirus 2 2019 severe acute respiratory syndrome", "sars coronavirus2 2019 acute respiratory disease", "2019 novel corona virus severe acute respiratory illness", "2019novel seafood market coronavirus severe acute infection", "sars coronavirus 2 2019 severe acute respiratory illness", "2019novel sars coronavirus 2 severe acute syndrome", "illness caused by 2019 new severe acute respiratory syndromerelated corona virus2", "2019novel wuhan virus severe acute respiratory syndrome", "2019 new coronavirus severe respiratory disease", "2019 new sars corona virus2 acute pneumonia", "2019ncov acute severe respiratory illness", "disease due to 2019new wuhan coronavirus", "seafood market corona virus 2019 severe acute respiratory disorder", "illness due to sars coronavirus 2", "pneumonia from 2019 new sars coronavirus2", "2019 new corona virus acute severe pneumonia", "2019novel sars cov 2 syndrome", "2019new sarscov2 acute severe respiratory syndrome", "2019 novel corona virus severe acute respiratory disorder", "2019 human corona virus severe respiratory infection", "wuhancoronavirus severe infection", "2019 novel seafood market coronavirus severe acute respiratory disorder", "covid19 virus acute severe respiratory illness", "disease from wuhan virus 2019", "pneumonia due to human coronavirus 2019", "2019 new wuhancoronavirus severe acute illness", "2019 seafood market corona virus severe acute respiratory infection", "2019 novel seafood market corona virus severe acute illness", "syndrome from 2019 new seafood market coronavirus", "2019 novel chinese coronavirus severe syndrome", "illness from 2019novel sars corona virus 2", "2019 sarscov2 acute severe infection", "sarscov2 2019 acute severe pneumonia", "wuhan severe acute syndrome", "2019new sars corona virus 2 acute disease", "2019new sars cov 2 disorder", "2019 novel wuhancoronavirus acute pneumonia", "hcov 19 severe infection", "wuhan seafood market pneumonia virus acute severe disease", "covid19 virus severe acute illness", "infection caused by 2019novel sars coronavirus 2", "2019novel wuhan seafood market virus respiratory illness", "2019 novel corona virus severe acute infection", "pneumonia from 2019novel wuhan virus", "2019novel wuhancorona virus acute disease", "2019 novel wuhan virus acute respiratory syndrome", "2019 new wuhan seafood market pneumonia virus acute disease", "sars corona virus2 2019 illness", "hcov 19 acute respiratory disease", "sars coronavirus2 severe infection", "disease due to 2019 wuhan seafood market virus", "illness from 2019 sars coronavirus 2", "syndrome caused by 2019 new sarscov2", "2019 novel sars corona virus 2 acute respiratory illness", "2019 new sars corona virus2 severe respiratory syndrome", "wuhan coronavirus acute severe pneumonia", "human corona virus 2019 acute severe respiratory syndrome", "pneumonia from sarscov2 2019", "disease due to severe acute respiratory syndrome coronavirus2 2019", "2019 chinese coronavirus severe acute pneumonia", "disorder caused by 2019 severe acute respiratory syndromerelated corona virus2", "2019 novel wuhan corona virus acute severe infection", "2019 novel wuhan seafood market pneumonia virus acute severe pneumonia", "disorder from 2019novel severe acute respiratory syndromerelated corona virus 2", "syndrome caused by chinese coronavirus", "2019 novel corona virus acute respiratory illness", "2019 wuhan virus acute illness", "2019 novel chinese corona virus severe illness", "2019 new seafood market virus respiratory illness", "illness from wuhan corona virus 2019", "new corona virus acute respiratory syndrome", "illness due to 2019new sars coronavirus 2", "2019 novel wuhancorona virus acute severe respiratory infection", "2019 novel seafood market corona virus severe acute respiratory illness", "2019 new corona virus respiratory infection", "infection due to hcov 19", "2019new seafood market coronavirus acute respiratory disorder", "novel coronavirus acute respiratory syndrome", "virus responsible for covid19 acute illness", "2019 new chinese coronavirus acute severe pneumonia", "seafood market coronavirus acute severe pneumonia", "2019 new corona virus severe pneumonia", "disease from wuhancoronavirus 2019", "2019 new severe acute respiratory syndrome corona virus 2 pneumonia", "pneumonia from 2019new sars corona virus 2", "2019 new sars corona virus 2 pneumonia", "2019 sars coronavirus 2", "2019ncov acute respiratory infection", "seafood market coronavirus 2019 acute pneumonia", "wuhan seafood market virus respiratory infection", "wuhan corona virus severe respiratory disease", "infection caused by 2019novel chinese corona virus", "seafood market coronavirus 2019 acute disease", "pneumonia from 2019 novel severe acute respiratory syndrome coronavirus 2", "disorder due to 2019 novel wuhan virus", "illness due to severe acute respiratory syndromerelated coronavirus 2 2019", "disease due to 2019 novel seafood market coronavirus", "2019 novel sarscov2 acute severe respiratory infection", "2019novel wuhancorona virus acute respiratory syndrome", "2019new seafood market corona virus severe acute syndrome", "pneumonia caused by 2019 novel seafood market virus", "new coronavirus severe respiratory infection", "covid 19 virus severe respiratory disease", "2019 sars corona virus2 acute severe pneumonia", "illness due to 2019novel wuhan seafood market pneumonia virus", "pneumonia due to 2019 new coronavirus", "illness from 2019 new wuhancorona virus", "novel coronavirus 2019 acute pneumonia", "2019new sars corona virus2 acute severe illness", "2019novel wuhan coronavirus asevere acute respiratory disease", "syndrome caused by 2019 novel sarscov2", "wuhan corona virus 2019 acute respiratory disorder", "illness due to severe acute respiratory syndrome corona virus 2", "wuhan corona virus 2019 severe disorder", "syndrome from wuhan seafood market pneumonia virus 2019", "sars coronavirus2 2019 severe pneumonia", "illness due to 2019 seafood market corona virus", "pneumonia due to 2019 new chinese corona virus", "sars corona virus2 severe illness", "2019new wuhan virus acute respiratory illness", "2019 novel corona virus severe respiratory infection", "seafood market corona virus 2019 severe respiratory syndrome", "infection due to virus causing covid19", "2019 new seafood market corona virus acute respiratory illness", "hcov 19 respiratory disease", "2019 sars coronavirus2", "pneumonia caused by 2019 ncov", "illness from 2019 new wuhan seafood market virus", "sars coronavirus2 acute severe respiratory infection", "sars corona virus 2", "2019new seafood market corona virus severe acute pneumonia", "seafood market coronavirus infection", "disorder due to 2019new wuhan virus", "infection from 2019 new severe acute respiratory syndrome coronavirus 2", "wuhan corona virus severe disorder", "sars cov 2 2019 severe respiratory disorder", "2019 novel sars cov 2 acute severe respiratory infection", "2019 novel corona virus acute severe disease", "pneumonia caused by 2019 new seafood market virus", "seafood market corona virus acute respiratory syndrome", "illness due to 2019 novel sars corona virus 2", "2019novel seafood market corona virus acute severe infection", "disorder caused by sars corona virus2", "disease from 2019 severe acute respiratory syndromerelated corona virus2", "syndrome from sars corona virus 2 2019", "seafood market coronavirus acute severe respiratory infection", "wuhan seafood market pneumonia virus", "disorder caused by 2019 wuhan corona virus", "severe acute respiratory syndromerelated coronavirus2 2019 infection", "disorder from 2019 novel seafood market virus", "2019new wuhan pneumonia virus severe acute syndrome", "illness caused by seafood market corona virus", "2019 wuhan coronavirus severe illness", "2019new sars corona virus2 acute severe respiratory illness", "2019 novel corona virus severe respiratory disease", "disorder caused by 2019 ncov", "syndrome due to covid 19 virus", "infection caused by 2019 novel severe acute respiratory syndromerelated coronavirus 2", "2019 human coronavirus acute infection", "seafood market respiratory disorder", "2019new wuhan pneumonia virus acute severe pneumonia", "2019 novel wuhan seafood market pneumonia virus severe respiratory disorder", "infection from 2019new sars cov 2", "2019novel wuhan seafood market virus asevere acute respiratory disease", "2019 wuhan corona virus severe acute respiratory syndrome", "2019 wuhan virus severe acute illness", "disease due to 2019 chinese coronavirus", "wuhan corona virus 2019 acute respiratory disease", "2019new wuhancoronavirus respiratory syndrome", "2019 novel wuhan seafood market virus severe acute pneumonia", "2019new wuhancorona virus severe acute respiratory infection", "infection from wuhan corona virus", "2019 new wuhan seafood market virus severe syndrome", "disease from 2019 novel severe acute respiratory syndromerelated corona virus 2", "2019novel severe acute respiratory syndrome coronavirus2 illness", "2019 new wuhan seafood market virus severe respiratory disorder", "pneumonia caused by 2019 novel severe acute respiratory syndrome corona virus 2", "disease due to 2019new sarscov2", "2019 novel sarscov2 severe illness", "2019novel chinese corona virus acute severe respiratory disorder", "pneumonia from 2019 new wuhan seafood market virus", "2019new wuhan coronavirus acute infection", "pneumonia caused by 2019 novel severe acute respiratory syndrome corona virus2", "human corona virus 2019 acute disease", "2019novel seafood market coronavirus acute infection", "sars coronavirus2 2019 severe acute respiratory syndrome", "sars corona virus2 2019 severe acute respiratory infection", "disease due to 2019 sars coronavirus2", "2019 novel sars coronavirus2 acute syndrome", "infection caused by sars coronavirus 2 2019", "illness caused by 2019 novel seafood market coronavirus", "2019new sars corona virus2 severe acute respiratory syndrome", "2019 new wuhan seafood market virus severe acute pneumonia", "sarscov2 2019 severe disease", "2019 sars corona virus2 acute severe illness", "disease caused by seafood market coronavirus 2019", "infection caused by 2019 novel severe acute respiratory syndrome coronavirus2", "2019 new seafood market virus severe acute infection", "2019new chinese coronavirus severe acute disease", "2019new sars corona virus 2 severe acute syndrome", "2019new chinese coronavirus acute pneumonia", "sars cov 2 severe acute respiratory illness", "seafood market respiratory disease", "sars coronavirus2 2019 disorder", "2019new wuhancorona virus acute illness", "sars corona virus2 2019 infection", "2019 new severe acute respiratory syndrome coronavirus2 disease", "infection due to 2019 wuhancoronavirus", "2019novel wuhan coronavirus acute severe respiratory disorder", "2019novel sars corona virus2 acute severe infection", "2019 novel wuhancoronavirus disorder", "pneumonia from 2019 sars cov 2", "2019 new sars corona virus 2 acute severe respiratory infection", "2019 novel wuhan coronavirus severe disorder", "2019novel wuhan seafood market pneumonia virus acute severe syndrome", "illness due to wuhan pneumonia virus 2019", "2019novel seafood market corona virus severe acute respiratory disorder", "new coronavirus 2019 acute respiratory syndrome", "disease caused by 2019novel severe acute respiratory syndromerelated corona virus 2", "2019new wuhan seafood market virus respiratory illness", "novel corona virus infection", "2019new seafood market coronavirus severe acute infection", "wuhan seafood market pneumonia virus infection", "illness caused by 2019 new severe acute respiratory syndromerelated corona virus 2", "disorder from 2019novel wuhancoronavirus", "2019 seafood market corona virus respiratory illness", "2019 novel seafood market coronavirus acute severe respiratory syndrome", "illness due to 2019 new coronavirus", "2019 sars coronavirus 2 severe respiratory disorder", "2019new chinese corona virus acute severe disorder", "2019 novel sars coronavirus 2 severe illness", "2019 wuhancoronavirus severe respiratory infection", "2019 new coronavirus severe syndrome", "pneumonia from wuhancoronavirus", "2019 seafood market corona virus acute respiratory disease", "2019new seafood market corona virus severe disorder", "2019new wuhan coronavirus severe disease", "illness from seafood market virus", "seafood market virus 2019", "2019 new sars corona virus2 severe respiratory illness", "wuhan coronavirus 2019 acute severe respiratory illness", "2019 chinese coronavirus acute severe respiratory disorder", "2019novel severe acute respiratory syndromerelated coronavirus2", "2019 new wuhan pneumonia virus respiratory disease", "2019 new wuhan corona virus severe disease", "infection due to wuhancorona virus 2019", "2019new wuhan seafood market pneumonia virus severe acute respiratory disorder", "2019 novel seafood market corona virus severe acute respiratory infection", "illness due to 2019 severe acute respiratory syndrome coronavirus 2", "2019novel sars corona virus 2 acute respiratory disease", "sars corona virus 2 2019 acute infection", "pneumonia from 2019novel wuhan seafood market pneumonia virus", "2019 novel chinese coronavirus acute disorder", "seafood market corona virus acute respiratory disease", "illness due to hcov 19", "2019new wuhancoronavirus severe acute respiratory infection", "hcov 19 acute disease", "2019novel seafood market coronavirus acute syndrome", "2019new sars corona virus2 acute respiratory disorder", "2019 novel sars corona virus 2 pneumonia", "illness due to 2019novel severe acute respiratory syndrome coronavirus2", "syndrome caused by wuhan virus", "2019 wuhancorona virus acute severe respiratory syndrome", "disease due to 2019novel sars coronavirus 2", "2019 novel wuhan corona virus severe syndrome", "2019 human corona virus acute respiratory disorder", "hcov 19 severe acute infection", "pneumonia from 2019new wuhan pneumonia virus", "2019new wuhan corona virus respiratory infection", "illness due to 2019novel sars cov 2", "disease caused by 2019ncov", "2019novel wuhan coronavirus severe infection", "pneumonia from chinese coronavirus 2019", "infection caused by 2019 new sars coronavirus2", "2019new sars coronavirus2 acute pneumonia", "2019ncov infection", "2019new wuhan coronavirus acute severe disorder", "2019 sars coronavirus 2 pneumonia", "2019novel wuhancorona virus acute severe respiratory syndrome", "seafood market corona virus severe acute respiratory illness", "syndrome due to 2019 new sars coronavirus2", "disease due to 2019novel wuhancoronavirus", "new corona virus acute severe respiratory illness", "illness from 2019new severe acute respiratory syndromerelated corona virus 2", "disease from 2019 sars corona virus 2", "2019 novel sars corona virus2 respiratory syndrome", "2019 new corona virus severe disease", "disease from seafood market coronavirus 2019", "2019 novel wuhan pneumonia virus disease", "2019novel wuhan seafood market pneumonia virus acute respiratory syndrome", "2019novel sars corona virus 2 respiratory infection", "2019 new wuhan corona virus severe respiratory disorder", "2019 wuhan seafood market virus respiratory syndrome", "sars cov 2 respiratory disease", "severe acute respiratory syndromerelated coronavirus2 2019", "2019novel chinese coronavirus severe syndrome", "pneumonia from severe acute respiratory syndromerelated corona virus 2", "2019 new wuhan coronavirus severe infection", "disease due to 2019new sars corona virus 2", "2019novel wuhancoronavirus respiratory infection", "wuhan corona virus 2019 severe acute syndrome", "2019new wuhan corona virus", "2019new sars coronavirus 2 severe disorder", "2019 new sars corona virus2 severe syndrome", "2019 new sars corona virus2 severe respiratory disease", "2019 new sarscov2 severe acute respiratory infection", "chinese corona virus respiratory disease", "wuhan seafood market pneumonia virus 2019 acute pneumonia", "pneumonia caused by new corona virus", "pneumonia due to 2019new sars coronavirus 2", "seafood market virus 2019 acute illness", "2019 novel seafood market corona virus acute respiratory disorder", "2019new wuhan pneumonia virus acute respiratory illness", "sars coronavirus 2 2019 acute severe syndrome", "2019 new severe acute respiratory syndrome coronavirus 2 pneumonia", "disease from 2019novel wuhan seafood market virus", "human coronavirus 2019 acute syndrome", "2019 novel wuhan seafood market pneumonia virus severe acute disorder", "2019new wuhancoronavirus acute severe illness", "2019 sars coronavirus2 acute severe illness", "2019 new wuhan coronavirus severe acute respiratory illness", "infection caused by chinese corona virus", "2019 sars coronavirus2 syndrome", "2019new wuhan corona virus severe respiratory illness", "2019 wuhan coronavirus acute severe pneumonia", "chinese coronavirus 2019 severe respiratory syndrome", "wuhan pneumonia virus 2019 respiratory illness", "2019novel wuhan corona virus severe acute disease", "2019 novel wuhan seafood market virus acute severe respiratory disease", "2019 corona virus severe acute respiratory syndrome", "2019 novel wuhan virus respiratory disease", "infection from 2019new severe acute respiratory syndrome coronavirus 2", "2019new wuhan pneumonia virus respiratory infection", "wuhan seafood market pneumonia virus respiratory syndrome", "2019 novel wuhan pneumonia virus acute severe syndrome", "new corona virus 2019 disorder", "sars cov 2 2019 severe pneumonia", "pneumonia from novel coronavirus 2019", "2019 corona virus acute severe respiratory syndrome", "disorder from wuhan seafood market virus", "2019 wuhan seafood market virus acute severe disease", "pneumonia caused by 2019 chinese corona virus", "illness due to 2019 novel severe acute respiratory syndrome coronavirus2", "2019novel chinese corona virus acute severe respiratory disease", "2019new wuhan coronavirus acute severe respiratory infection", "2019 novel seafood market coronavirus severe acute respiratory syndrome", "human coronavirus 2019 severe respiratory infection", "2019novel sars corona virus2 severe acute respiratory syndrome", "2019 new sarscov2 acute respiratory disease", "chinese coronavirus 2019 acute infection", "2019new wuhan corona virus acute syndrome", "2019new sars corona virus 2 pneumonia", "2019new wuhan coronavirus acute syndrome", "sars corona virus2 acute illness", "2019 chinese corona virus acute respiratory disease", "syndrome due to 2019 novel wuhan coronavirus", "disorder from sars coronavirus 2 2019", "2019 new seafood market corona virus illness", "2019 sars coronavirus 2 severe disorder", "2019new sars coronavirus2 acute severe pneumonia", "wuhan seafood market virus severe illness", "2019 ncov acute disorder", "pneumonia caused by 2019 new severe acute respiratory syndrome coronavirus2", "disorder due to 2019new severe acute respiratory syndromerelated corona virus2", "illness caused by 2019 new severe acute respiratory syndrome corona virus2", "wuhan seafood market virus", "syndrome caused by 2019new wuhan coronavirus", "2019novel wuhan coronavirus respiratory illness", "disorder caused by 2019 novel sarscov2", "2019 novel coronavirus illness", "2019novel sars cov 2 acute respiratory infection", "2019new seafood market coronavirus severe acute syndrome", "2019 new wuhan corona virus acute severe respiratory disease", "sars corona virus2 2019 severe acute respiratory disorder", "2019 novel sars coronavirus2 asevere acute respiratory disease", "virus causing covid19 severe acute respiratory illness", "2019 novel sars cov 2 disease", "wuhan pneumonia virus 2019 pneumonia", "2019 corona virus respiratory infection", "2019novel severe acute respiratory syndrome corona virus 2 infection", "novel corona virus acute severe syndrome", "disorder from 2019 novel wuhan corona virus", "2019 wuhan corona virus respiratory disease", "syndrome from 2019 wuhan pneumonia virus", "2019 novel sars coronavirus 2 pneumonia", "2019 new seafood market virus severe respiratory disease", "2019 human coronavirus acute illness", "novel coronavirus 2019 acute severe respiratory disease", "2019 novel sars cov 2 acute severe respiratory disorder", "hubei acute severe disease", "2019new seafood market corona virus severe acute disease", "2019new wuhancoronavirus infection", "wuhancoronavirus acute severe disease", "pneumonia from 2019 severe acute respiratory syndromerelated coronavirus 2", "2019 novel seafood market corona virus illness", "novel corona virus severe acute respiratory disorder", "pneumonia from severe acute respiratory syndromerelated corona virus 2 2019", "virus responsible for covid19 acute severe infection", "2019 sars coronavirus 2 infection", "new coronavirus 2019 severe acute pneumonia", "infection from sars coronavirus 2 2019", "wuhancoronavirus 2019 acute severe respiratory disorder", "chinese coronavirus acute respiratory infection", "2019novel sars cov 2 severe respiratory disease", "sars corona virus 2 acute disorder", "pneumonia caused by 2019new wuhancoronavirus", "syndrome from 2019 new sars coronavirus 2", "disorder from 2019 new wuhan pneumonia virus", "syndrome caused by 2019new sars coronavirus 2", "novel coronavirus", "disorder from wuhan corona virus", "2019new wuhancorona virus syndrome", "wuhancoronavirus 2019 severe respiratory disorder", "pneumonia caused by severe acute respiratory syndromerelated coronavirus2", "wuhan pneumonia virus severe acute respiratory disorder", "chinese corona virus severe respiratory disorder", "illness caused by 2019novel severe acute respiratory syndromerelated coronavirus2", "pneumonia caused by severe acute respiratory syndrome coronavirus2", "illness from severe acute respiratory syndromerelated coronavirus2 2019", "severe acute respiratory syndromerelated coronavirus 2 2019", "infection from 2019 wuhancoronavirus", "severe acute respiratory syndrome type 2", "2019 new coronavirus acute respiratory syndrome", "2019 new sars coronavirus 2 acute severe illness", "2019new wuhancorona virus severe acute disorder", "2019 human coronavirus severe acute respiratory disorder", "wuhan pneumonia virus 2019 syndrome", "wuhan coronavirus acute severe syndrome", "syndrome due to 2019 novel seafood market virus", "2019 new sars cov 2 pneumonia", "disease from wuhancorona virus 2019", "disorder due to 2019 severe acute respiratory syndromerelated coronavirus 2", "2019 novel coronavirus severe respiratory illness", "illness due to 2019new severe acute respiratory syndromerelated corona virus2", "disease from 2019new severe acute respiratory syndrome corona virus 2", "syndrome due to 2019 novel corona virus", "seafood market corona virus respiratory illness", "pneumonia due to 2019new wuhan seafood market virus", "covid19 virus acute severe respiratory disorder", "2019 seafood market coronavirus severe acute disorder", "2019 corona virus severe disease", "2019 novel seafood market corona virus respiratory disorder", "virus responsible for covid19 disorder", "2019 new wuhan corona virus acute severe disorder", "2019novel sars corona virus2 acute severe disorder", "disease due to 2019new wuhan seafood market virus", "2019 human coronavirus", "disorder caused by 2019new severe acute respiratory syndromerelated coronavirus2", "2019novel wuhancoronavirus severe disease", "syndrome from 2019ncov", "2019new seafood market coronavirus severe acute disease", "2019 novel chinese corona virus severe acute disorder", "2019 corona virus severe acute illness", "2019 sars coronavirus2 respiratory infection", "new coronavirus 2019 acute respiratory infection", "new corona virus acute severe disorder", "2019new seafood market corona virus asevere acute respiratory disease", "2019 novel corona virus severe acute respiratory infection", "illness caused by 2019 new severe acute respiratory syndrome coronavirus 2", "infection due to sars cov 2", "2019new sarscov2 severe respiratory disorder", "syndrome due to wuhan corona virus 2019", "disorder due to 2019 human corona virus", "disorder caused by wuhancorona virus 2019", "sars coronavirus 2 asevere acute respiratory disease", "2019new severe acute respiratory syndrome corona virus 2 disorder", "2019novel wuhan corona virus pneumonia", "disorder from 2019new wuhan coronavirus", "2019 new corona virus severe acute disease", "sars corona virus 2 2019 acute severe pneumonia", "2019 novel wuhan seafood market virus severe acute respiratory syndrome", "2019 new wuhancorona virus severe acute disease", "novel coronavirus respiratory illness", "seafood market corona virus severe syndrome", "sars coronavirus 2 2019 acute respiratory disorder", "infection caused by 2019 severe acute respiratory syndrome corona virus2", "pneumonia from 2019novel severe acute respiratory syndromerelated corona virus 2", "wuhan seafood market pneumonia virus severe acute disease", "2019 human coronavirus acute severe infection", "2019new sars coronavirus2 acute infection", "wuhancorona virus 2019 severe disease", "disease from 2019new severe acute respiratory syndromerelated corona virus 2", "2019novel sars coronavirus 2 acute severe respiratory syndrome", "covid 19 virus acute infection", "2019 novel wuhan virus severe acute disorder", "2019novel wuhan coronavirus respiratory disease", "illness from 2019 new seafood market corona virus", "novel coronavirus severe acute syndrome", "syndrome due to novel corona virus 2019", "wuhan seafood market pneumonia virus 2019 acute severe illness", "2019 sars coronavirus 2 acute respiratory syndrome", "2019new wuhan coronavirus severe acute disorder", "wuhan severe acute respiratory illness", "hcov19 disease", "seafood market corona virus 2019 disorder", "virus causing covid19 acute severe disorder", "novel coronavirus 2019 disease", "infection due to 2019 new seafood market corona virus", "sars coronavirus 2 acute severe disease", "2019novel wuhancorona virus acute severe respiratory disorder", "novel corona virus 2019 acute disease", "2019 novel wuhan seafood market pneumonia virus severe acute syndrome", "infection caused by 2019 new severe acute respiratory syndromerelated coronavirus 2", "2019novel chinese corona virus acute illness", "2019 seafood market corona virus acute pneumonia", "seafood market virus 2019 severe respiratory syndrome", "disorder due to 2019new sarscov2", "disorder due to 2019novel severe acute respiratory syndromerelated corona virus 2", "pneumonia from 2019novel severe acute respiratory syndrome coronavirus2", "syndrome from sarscov2", "2019 novel wuhan corona virus severe acute respiratory disorder", "covid19 virus severe disorder", "illness from 2019new wuhan coronavirus", "2019 novel seafood market coronavirus acute respiratory syndrome", "seafood market acute disease", "sars coronavirus 2 2019 acute severe respiratory disease", "2019novel wuhancoronavirus acute respiratory disorder", "disorder due to 2019 new wuhan pneumonia virus", "2019 new wuhan seafood market pneumonia virus acute severe disorder", "2019novel seafood market coronavirus asevere acute respiratory disease", "2019 wuhan seafood market virus acute severe respiratory infection", "2019novel wuhan virus severe acute respiratory illness", "illness due to 2019 human corona virus", "2019 wuhan coronavirus severe respiratory illness", "2019 new corona virus severe infection", "2019 novel seafood market corona virus severe acute infection", "infection from 2019 novel severe acute respiratory syndromerelated coronavirus2", "pneumonia due to 2019novel wuhan pneumonia virus", "seafood market severe acute infection", "human coronavirus 2019 severe respiratory syndrome", "2019new severe acute respiratory syndromerelated coronavirus2 pneumonia", "pneumonia due to wuhan seafood market virus 2019", "infection from 2019 severe acute respiratory syndromerelated coronavirus2", "2019 novel wuhan corona virus acute respiratory disorder", "illness from 2019novel severe acute respiratory syndromerelated coronavirus2", "2019new sars cov 2 severe acute syndrome", "2019 human coronavirus acute syndrome", "illness from 2019novel sars coronavirus2", "illness from virus causing covid19", "2019 ncov disorder", "2019novel wuhancoronavirus acute respiratory syndrome", "2019 wuhan seafood market virus acute infection", "2019 new corona virus severe acute syndrome", "disorder from 2019 new severe acute respiratory syndrome coronavirus2", "2019 new chinese corona virus disease", "2019 sars cov 2 asevere acute respiratory disease", "sars coronavirus 2 severe respiratory infection", "sars cov 2 acute illness", "2019 new severe acute respiratory syndrome coronavirus 2 disorder", "illness due to wuhan seafood market pneumonia virus", "syndrome from 2019 new wuhancoronavirus", "2019new sars coronavirus2 acute severe respiratory disease", "2019 new seafood market coronavirus acute severe respiratory syndrome", "infection from 2019 novel wuhan corona virus", "seafood market virus 2019 acute disease", "2019 novel severe acute respiratory syndrome coronavirus 2 infection", "2019new sars corona virus 2 acute infection", "2019 wuhan coronavirus respiratory disorder", "2019new seafood market coronavirus disease", "infection from 2019novel seafood market virus", "disease from severe acute respiratory syndromerelated corona virus2", "2019novel wuhan coronavirus severe respiratory infection", "disorder from 2019new sars coronavirus2", "wuhancoronavirus severe respiratory disorder", "sars coronavirus2 2019 acute severe respiratory syndrome", "2019 chinese coronavirus severe acute infection", "2019 novel seafood market corona virus acute severe disease", "2019novel seafood market virus acute severe pneumonia", "pneumonia due to 2019 wuhan virus", "2019novel seafood market coronavirus acute disorder", "illness from 2019new severe acute respiratory syndromerelated coronavirus 2", "sars cov 2 severe illness", "severe acute respiratory syndrome corona virus 2 2019 disease", "2019 seafood market coronavirus severe disease", "human coronavirus 2019 respiratory syndrome", "2019novel wuhan coronavirus acute respiratory illness", "disease due to severe acute respiratory syndrome corona virus 2 2019", "2019 severe acute respiratory syndrome coronavirus 2 disorder", "wuhan coronavirus 2019 acute disorder", "pneumonia from 2019 seafood market virus", "2019 coronavirus severe acute pneumonia", "2019novel sars coronavirus 2 acute respiratory disease", "syndrome from 2019 novel sars corona virus 2", "2019 corona virus severe acute respiratory infection", "2019novel sars cov 2 severe acute syndrome", "2019 novel wuhan seafood market pneumonia virus acute respiratory disease", "2019novel seafood market virus respiratory disease", "wuhan disorder", "2019new wuhan seafood market pneumonia virus severe respiratory infection", "syndrome caused by 2019 new wuhan seafood market virus", "pneumonia due to 2019 sars coronavirus 2", "disease due to 2019 severe acute respiratory syndrome coronavirus 2", "2019novel sars corona virus 2 respiratory syndrome", "2019 novel wuhan seafood market virus severe acute infection", "disease caused by 2019 severe acute respiratory syndrome coronavirus2", "chinese corona virus severe respiratory disease", "2019 new wuhan pneumonia virus acute respiratory disease", "2019 new seafood market virus severe acute respiratory disorder", "2019 seafood market virus acute severe illness", "disease caused by covid19 virus", "hcov19 severe acute syndrome", "chinese coronavirus 2019 severe disorder", "human corona virus 2019 acute respiratory infection", "wuhancoronavirus respiratory syndrome", "sars coronavirus 2 2019 respiratory syndrome", "2019 new sars corona virus2 respiratory syndrome", "wuhan seafood market pneumonia virus acute respiratory infection", "2019 new wuhan corona virus acute severe syndrome", "2019 new wuhan seafood market pneumonia virus severe acute illness", "2019novel wuhan seafood market pneumonia virus severe respiratory illness", "virus causing covid19 acute syndrome", "2019 novel wuhancoronavirus acute severe disease", "2019 new wuhan seafood market virus acute disorder", "sars corona virus 2 acute syndrome", "illness from 2019new wuhan seafood market virus", "illness caused by 2019new wuhan seafood market pneumonia virus", "2019 new sars coronavirus 2 respiratory disease", "2019 seafood market coronavirus respiratory syndrome", "2019 new wuhancorona virus asevere acute respiratory disease", "2019 new wuhan coronavirus severe respiratory infection", "2019 new seafood market coronavirus severe acute respiratory infection", "pneumonia caused by chinese coronavirus", "illness from 2019novel severe acute respiratory syndromerelated corona virus 2", "disease from 2019 sars coronavirus 2", "sars coronavirus2 2019 acute severe respiratory illness", "infection from 2019 sars corona virus 2", "hcov 19 severe acute illness", "wuhancorona virus 2019 acute severe respiratory disorder", "human coronavirus 2019 asevere acute respiratory disease", "2019new sars coronavirus2 respiratory disease", "illness due to severe acute respiratory syndromerelated coronavirus2 2019", "2019novel wuhan corona virus acute severe respiratory disorder", "2019novel seafood market coronavirus severe syndrome", "pneumonia from 2019 sars coronavirus2", "2019new sars coronavirus2 pneumonia", "2019 new wuhan coronavirus acute respiratory illness", "2019 novel seafood market corona virus acute respiratory disease", "chinese coronavirus 2019 acute severe syndrome", "pneumonia due to wuhan coronavirus 2019", "pneumonia due to 2019novel seafood market coronavirus", "syndrome from 2019 novel sarscov2", "severe acute respiratory syndromerelated coronavirus2 2019 pneumonia", "2019 new seafood market virus acute disorder", "wuhancoronavirus 2019 acute severe syndrome", "disease due to 2019 new seafood market virus", "infection from 2019 wuhan seafood market pneumonia virus", "wuhancoronavirus acute severe pneumonia", "novel coronavirus severe respiratory disorder", "disorder caused by severe acute respiratory syndrome corona virus2 2019", "sars corona virus 2 2019 acute respiratory infection", "2019 new wuhan seafood market virus severe acute syndrome", "2019novel sars corona virus2 acute disease", "syndrome due to 2019 new chinese corona virus", "2019 new sars corona virus2 severe infection", "pneumonia caused by 2019novel severe acute respiratory syndrome corona virus2", "2019 wuhan seafood market virus syndrome", "2019 novel wuhan pneumonia virus severe respiratory disorder", "sars2", "2019 novel wuhan corona virus acute respiratory syndrome", "infection due to 2019 novel sars coronavirus 2", "2019new sars coronavirus 2 acute pneumonia", "2019 new seafood market corona virus respiratory syndrome", "syndrome caused by 2019 wuhan seafood market virus", "wuhan seafood market pneumonia virus 2019 infection", "2019 novel sars corona virus 2 acute disease", "human corona virus 2019 acute syndrome", "infection caused by hcov 19", "disease from 2019new sars coronavirus2", "2019new sarscov2 acute respiratory disease", "sarscov2 2019 acute severe illness", "2019 novel wuhan pneumonia virus severe respiratory disease", "2019 novel wuhan pneumonia virus severe acute syndrome", "2019 new wuhan seafood market virus acute severe respiratory infection", "2019 new wuhan coronavirus severe acute respiratory syndrome", "pneumonia due to wuhan corona virus 2019", "2019novel wuhan seafood market pneumonia virus acute severe respiratory disease", "pneumonia from 2019 novel chinese coronavirus", "2019 coronavirus acute severe respiratory infection", "wuhancoronavirus acute disease", "2019new wuhan coronavirus severe acute infection", "severe acute respiratory syndrome coronavirus2 disorder", "2019 novel sars coronavirus2 acute severe respiratory illness", "2019new sars coronavirus2 acute syndrome", "syndrome from 2019 sars corona virus2", "disorder caused by 2019novel sars coronavirus2", "seafood market corona virus 2019 acute severe respiratory syndrome", "disease due to chinese coronavirus 2019", "2019novel sars corona virus2 severe respiratory infection", "2019 novel wuhancorona virus acute severe syndrome", "wuhan virus 2019 acute severe pneumonia", "2019 new seafood market corona virus respiratory illness", "sars coronavirus2 acute severe respiratory illness", "infection due to 2019novel sars cov 2", "2019 new sars corona virus 2 respiratory illness", "pneumonia caused by 2019 new severe acute respiratory syndrome corona virus 2", "2019novel wuhan seafood market pneumonia virus severe pneumonia", "sarscov2 2019 severe acute respiratory infection", "illness due to wuhancoronavirus 2019", "2019 wuhan seafood market virus acute respiratory syndrome", "syndrome due to novel coronavirus", "2019 human corona virus asevere acute respiratory disease", "2019 novel sars cov 2 acute illness", "2019 novel wuhan corona virus severe disease", "2019novel sars coronavirus 2 respiratory disorder", "sars coronavirus 2 2019 acute respiratory syndrome", "new coronavirus 2019 illness", "wuhancoronavirus illness", "pneumonia caused by severe acute respiratory syndrome corona virus2 2019", "illness due to 2019new wuhan coronavirus", "2019 wuhan coronavirus acute severe disorder", "disorder from 2019 new severe acute respiratory syndrome coronavirus 2", "severe acute respiratory syndrome coronavirus2 2019 disease", "2019novel chinese coronavirus acute respiratory disease", "2019 sars corona virus 2 acute severe disorder", "2019 new chinese corona virus pneumonia", "2019 wuhan virus severe disease", "2019 new chinese coronavirus acute severe respiratory infection", "illness caused by severe acute respiratory syndrome coronavirus 2 2019", "chinese coronavirus 2019 severe respiratory infection", "2019new wuhancorona virus disorder", "2019 seafood market corona virus disorder", "2019 new sars coronavirus2 severe respiratory syndrome", "2019 new seafood market coronavirus respiratory disease", "infection caused by novel corona virus", "2019 wuhan pneumonia virus severe respiratory infection", "human coronavirus 2019 acute respiratory illness", "seafood market coronavirus acute illness", "2019new sars cov 2 severe acute respiratory disorder", "pneumonia from 2019novel wuhancoronavirus", "2019new wuhan corona virus severe acute disease", "pneumonia caused by sars corona virus 2 2019", "2019 wuhan seafood market virus severe respiratory syndrome", "2019 new wuhan seafood market virus disease", "2019new seafood market virus severe acute respiratory disorder", "2019 sars coronavirus2 severe acute illness", "2019 new wuhancorona virus infection", "covid19 virus acute severe infection", "disorder from severe acute respiratory syndrome coronavirus2 2019", "wuhan seafood market virus acute respiratory syndrome", "2019new wuhan coronavirus severe illness", "wuhan virus acute severe pneumonia", "syndrome from 2019 novel sars cov 2", "2019 novel wuhan pneumonia virus acute respiratory disease", "2019 new sars corona virus2 severe acute respiratory infection", "infection due to 2019 ncov", "sars coronavirus2 2019 acute respiratory syndrome", "syndrome due to wuhan seafood market virus 2019", "2019 new wuhan corona virus acute disorder", "2019 novel sars coronavirus 2 acute pneumonia", "2019 novel chinese corona virus severe respiratory disease", "chinese corona virus 2019 severe disease", "2019 wuhan corona virus acute severe respiratory infection", "2019 seafood market coronavirus severe pneumonia", "2019new sars corona virus2 acute severe disorder", "infection caused by covid 19 virus", "2019new sars corona virus 2 acute respiratory illness", "pneumonia caused by 2019new wuhan corona virus", "new corona virus syndrome", "virus causing covid19 severe pneumonia", "2019new wuhan seafood market virus asevere acute respiratory disease", "pneumonia from wuhan seafood market virus", "syndrome due to 2019 novel wuhan pneumonia virus", "disease due to 2019novel severe acute respiratory syndrome corona virus2", "2019 novel wuhan coronavirus illness", "2019new wuhan seafood market virus severe illness", "illness caused by 2019 severe acute respiratory syndromerelated coronavirus 2", "disorder due to 2019 new severe acute respiratory syndromerelated coronavirus 2", "2019 wuhan pneumonia virus severe acute pneumonia", "wuhan seafood market pneumonia virus 2019 severe respiratory disease", "2019 chinese coronavirus acute severe disorder", "pneumonia from sars cov 2 2019", "2019 seafood market virus severe respiratory disease", "wuhan seafood market virus 2019 acute severe pneumonia", "2019 novel wuhan virus severe pneumonia", "wuhan severe respiratory infection", "2019new wuhancoronavirus respiratory illness", "disorder caused by 2019new chinese coronavirus", "severe acute respiratory syndromerelated coronavirus2", "disorder due to sars coronavirus2 2019", "wuhan seafood market virus respiratory syndrome", "2019novel sars corona virus2 syndrome", "2019 new sarscov2 acute respiratory infection", "wuhancoronavirus 2019 acute respiratory syndrome", "illness due to 2019 severe acute respiratory syndromerelated coronavirus2", "disorder due to 2019 new wuhancoronavirus", "syndrome from sars coronavirus 2 2019", "chinese corona virus pneumonia", "syndrome due to wuhancoronavirus", "wuhan virus syndrome", "pneumonia due to 2019 new severe acute respiratory syndromerelated coronavirus2", "disease caused by 2019 new wuhan corona virus", "2019novel chinese corona virus", "2019new sarscov2 acute respiratory disorder", "2019 novel chinese coronavirus acute infection", "disease from wuhan seafood market virus 2019", "2019 novel wuhan seafood market virus severe respiratory disorder", "2019 new sars corona virus 2 respiratory syndrome", "infection due to 2019 novel wuhan corona virus", "2019 novel seafood market virus syndrome", "disorder due to 2019novel wuhan seafood market pneumonia virus", "disorder caused by 2019 novel coronavirus", "2019novel sarscov2 respiratory illness", "wuhancorona virus acute disorder", "sarscov2 severe pneumonia", "hcov 19 acute disorder", "2019 new sars coronavirus2 acute severe syndrome", "virus causing covid19 acute severe respiratory disease", "disorder caused by 2019new sars corona virus2", "infection from 2019novel sars corona virus2", "syndrome due to 2019 chinese coronavirus", "2019novel wuhan seafood market pneumonia virus respiratory infection", "disorder from 2019novel severe acute respiratory syndromerelated coronavirus 2", "2019new wuhan virus respiratory syndrome", "seafood market coronavirus severe acute disease", "new corona virus severe acute syndrome", "2019 new wuhancorona virus acute severe respiratory disorder", "chinese coronavirus 2019 disease", "disease due to 2019 new coronavirus", "disease due to 2019new sars coronavirus2", "illness caused by severe acute respiratory syndromerelated coronavirus 2 2019", "2019new seafood market virus acute severe pneumonia", "2019novel sarscov2 acute disorder", "2019ncov severe disorder", "2019 new sarscov2 acute severe respiratory infection", "2019 new sars coronavirus2 pneumonia", "2019 chinese coronavirus syndrome", "2019new sars coronavirus2 respiratory disorder", "2019novel wuhan pneumonia virus severe respiratory disorder", "2019 wuhan pneumonia virus severe acute illness", "novel corona virus acute severe pneumonia", "2019 novel wuhan seafood market virus respiratory illness", "seafood market corona virus 2019 respiratory disorder", "human coronavirus 2019 acute severe respiratory disease", "2019 new wuhan seafood market pneumonia virus severe illness", "2019 new chinese corona virus acute severe disorder", "2019 new sars corona virus 2 severe respiratory infection", "sars cov 2 respiratory infection", "2019 novel coronavirus acute severe syndrome", "wuhan seafood market pneumonia virus acute pneumonia", "2019 wuhan pneumonia virus severe acute syndrome", "infection caused by 2019 severe acute respiratory syndrome coronavirus 2", "2019ncov severe acute respiratory illness", "2019 new chinese coronavirus acute syndrome", "disease from 2019 new coronavirus", "disease caused by 2019novel sars corona virus 2", "chinese coronavirus severe pneumonia", "sars corona virus2 severe acute illness", "2019 sars corona virus 2 disease", "infection from seafood market coronavirus 2019", "pneumonia from 2019 sarscov2", "wuhancoronavirus 2019 severe infection", "2019 new wuhan seafood market virus severe acute respiratory syndrome", "2019 new sars coronavirus2 acute severe respiratory disease", "2019 novel sarscov2", "sars corona virus 2 2019 severe acute respiratory disorder", "2019new severe acute respiratory syndrome corona virus 2 disease", "wuhan seafood market pneumonia virus acute respiratory disorder", "disease from 2019novel severe acute respiratory syndromerelated coronavirus2", "pneumonia caused by wuhan virus", "2019 wuhan corona virus severe acute disorder", "2019new wuhancorona virus severe acute respiratory illness", "2019novel sars coronavirus 2 acute pneumonia", "2019 new corona virus respiratory syndrome", "2019 new wuhan virus severe respiratory disease", "chinese coronavirus 2019 severe acute respiratory infection", "disorder due to 2019 novel severe acute respiratory syndrome coronavirus 2", "2019 novel wuhan virus respiratory disorder", "2019new wuhan seafood market virus severe respiratory illness", "2019novel seafood market coronavirus severe acute respiratory infection", "pneumonia from sars coronavirus 2", "seafood market corona virus disease", "2019 novel sars cov 2 respiratory infection", "2019 severe acute respiratory syndromerelated corona virus 2 pneumonia", "illness due to 2019 coronavirus", "2019 new wuhancorona virus acute severe disorder", "2019 new chinese corona virus acute severe respiratory infection", "human corona virus 2019 pneumonia", "syndrome caused by seafood market virus 2019", "seafood market acute disorder", "disease caused by 2019 new wuhan seafood market pneumonia virus", "novel corona virus severe respiratory infection", "pneumonia due to 2019 novel wuhan seafood market pneumonia virus", "syndrome from 2019 new corona virus", "2019 novel sars corona virus2 severe pneumonia", "new corona virus 2019 severe respiratory illness", "2019 wuhan seafood market virus acute pneumonia", "2019novel severe acute respiratory syndrome coronavirus2 pneumonia", "syndrome from new coronavirus", "2019 novel seafood market virus severe acute illness", "wuhan seafood market virus 2019 acute severe syndrome", "2019novel sars cov 2 severe acute respiratory illness", "2019new sars coronavirus2 respiratory illness", "wuhan pneumonia virus 2019 acute pneumonia", "2019new sarscov2 acute disease", "2019novel sars cov 2 illness", "2019 sars corona virus2 disease", "sars coronavirus 2 acute severe syndrome", "new coronavirus 2019 severe syndrome", "disorder from 2019 novel sarscov2", "2019 wuhancoronavirus acute respiratory illness", "2019 chinese coronavirus severe acute illness", "2019 new sars cov 2 acute respiratory infection", "2019novel chinese corona virus severe respiratory syndrome", "disorder due to 2019novel severe acute respiratory syndromerelated coronavirus2", "chinese coronavirus 2019 acute syndrome", "novel coronavirus severe acute respiratory disorder", "pneumonia due to wuhan seafood market pneumonia virus", "2019 new chinese coronavirus severe acute pneumonia", "2019 chinese coronavirus acute severe respiratory illness", "2019novel sars corona virus 2 acute severe infection", "seafood market corona virus 2019 acute respiratory disorder", "2019 human corona virus acute infection", "novel coronavirus 2019 severe respiratory illness", "2019 new sars coronavirus 2 acute severe respiratory disease", "sars corona virus2 acute severe syndrome", "2019 new sars cov 2 syndrome", "disease from 2019 sars cov 2", "new coronavirus 2019 severe acute respiratory disorder", "wuhan severe acute respiratory infection", "2019 new sars cov 2 severe acute respiratory infection", "pneumonia due to 2019 novel wuhan seafood market virus", "2019new severe acute respiratory syndrome coronavirus2 illness", "infection due to 2019 severe acute respiratory syndromerelated corona virus2", "2019novel sars coronavirus2 acute severe respiratory illness", "syndrome caused by 2019new sars coronavirus2", "2019 human corona virus severe disorder", "2019 human coronavirus acute disorder", "2019 seafood market coronavirus illness", "2019 novel sars cov 2 severe respiratory disorder", "2019novel wuhan seafood market pneumonia virus acute respiratory infection", "infection caused by 2019 wuhan pneumonia virus", "illness from 2019 new wuhan seafood market pneumonia virus", "2019 ncov acute disease", "human coronavirus 2019 respiratory illness", "infection from 2019new severe acute respiratory syndromerelated coronavirus2", "wuhan seafood market virus 2019 respiratory infection", "2019new seafood market coronavirus acute respiratory infection", "2019 coronavirus severe respiratory illness", "novel corona virus acute respiratory disorder", "2019 wuhan virus asevere acute respiratory disease", "infection due to 2019 novel sarscov2", "disease from virus responsible for covid19", "seafood market virus severe acute respiratory infection", "severe acute respiratory syndromerelated coronavirus2 2019 illness", "sars cov 2 acute severe infection", "2019 novel sarscov2 acute severe syndrome", "2019new wuhancoronavirus acute respiratory illness", "2019novel chinese coronavirus acute severe respiratory infection", "wuhan seafood market virus 2019 acute infection", "sarscov2 acute pneumonia", "2019novel seafood market coronavirus acute severe illness", "2019 novel wuhan seafood market virus acute severe disease", "disease due to 2019 new wuhan seafood market virus", "2019 novel wuhancoronavirus severe acute illness", "2019 novel sars coronavirus2 infection", "2019novel seafood market corona virus acute severe disease", "infection caused by seafood market virus 2019", "syndrome due to sars coronavirus 2 2019", "2019novel wuhan corona virus severe respiratory disorder", "2019 wuhan virus acute respiratory syndrome", "2019 novel seafood market virus acute respiratory disorder", "sars corona virus2 2019 acute severe disease", "illness due to 2019new severe acute respiratory syndrome coronavirus 2", "pneumonia due to 2019 new severe acute respiratory syndrome coronavirus2", "seafood market coronavirus acute respiratory illness", "wuhan coronavirus 2019 respiratory disease", "2019 new wuhan coronavirus acute respiratory disorder", "pneumonia caused by 2019novel wuhan corona virus", "2019 coronavirus severe respiratory disease", "2019 new sarscov2 acute syndrome", "disorder due to 2019novel wuhancoronavirus", "2019novel sars coronavirus2 acute syndrome", "2019new sars corona virus 2 acute severe respiratory disorder", "disease from 2019 wuhan seafood market pneumonia virus", "pneumonia due to severe acute respiratory syndromerelated corona virus2", "2019 novel wuhancoronavirus severe respiratory infection", "disease caused by 2019 novel corona virus", "hcov19 severe infection", "2019 new seafood market coronavirus severe acute syndrome", "2019 new wuhancorona virus severe syndrome", "2019 seafood market corona virus severe pneumonia", "hcov 19 severe pneumonia", "2019new wuhan seafood market pneumonia virus acute respiratory disease", "sarscov2 acute illness", "illness caused by 2019 wuhan seafood market virus", "2019 new wuhan pneumonia virus severe acute respiratory syndrome", "illness caused by 2019 sarscov2", "pneumonia caused by severe acute respiratory syndromerelated coronavirus 2", "disorder from 2019new severe acute respiratory syndrome corona virus 2", "2019 new wuhan corona virus acute respiratory infection", "novel corona virus 2019 acute severe respiratory disorder", "disease from 2019new wuhan corona virus", "syndrome due to chinese corona virus", "syndrome from 2019novel sars coronavirus 2", "novel corona virus 2019 respiratory infection", "2019 new wuhancorona virus acute severe respiratory syndrome", "2019 novel sarscov2 acute pneumonia", "2019 novel seafood market virus severe acute respiratory syndrome", "2019 new chinese corona virus severe syndrome", "2019new sars coronavirus2 acute severe disorder", "2019new sars corona virus2 acute respiratory syndrome", "wuhan coronavirus severe infection", "2019 sarscov2 respiratory disorder", "2019novel wuhancoronavirus acute severe infection", "wuhancoronavirus 2019 acute respiratory disease", "chinese corona virus severe acute syndrome", "illness due to 2019 new severe acute respiratory syndrome corona virus2", "disease due to severe acute respiratory syndrome coronavirus2", "wuhan seafood market virus 2019 severe acute illness", "2019new sarscov2 acute severe respiratory infection", "illness caused by 2019novel sars coronavirus2", "2019novel wuhancoronavirus acute illness", "disorder due to 2019 novel severe acute respiratory syndrome corona virus 2", "infection caused by 2019 new wuhan virus", "sarscov2 severe respiratory syndrome", "wuhancoronavirus 2019 acute severe pneumonia", "2019 novel seafood market virus acute disorder", "2019 new severe acute respiratory syndromerelated coronavirus 2 disorder", "wuhancoronavirus acute severe respiratory illness", "disease due to 2019 severe acute respiratory syndrome coronavirus2", "2019 novel corona virus acute pneumonia", "2019new wuhancoronavirus acute pneumonia", "2019 novel sars corona virus 2 respiratory disease", "sars corona virus 2 acute severe illness", "pneumonia caused by 2019 novel severe acute respiratory syndromerelated corona virus2", "2019 novel seafood market virus acute syndrome", "2019novel sars corona virus 2 severe respiratory disease", "novel coronavirus severe respiratory syndrome", "disease caused by sars corona virus 2 2019", "infection due to novel corona virus", "disease from 2019novel wuhan coronavirus", "2019 new chinese corona virus severe acute respiratory syndrome", "2019novel wuhan coronavirus pneumonia", "wuhan seafood market pneumonia virus 2019 acute disease", "wuhan corona virus 2019 respiratory infection", "sarscov2 severe acute disease", "disorder caused by 2019 wuhan seafood market pneumonia virus", "2019new wuhan corona virus severe acute infection", "disorder caused by 2019 new severe acute respiratory syndromerelated coronavirus2", "wuhan coronavirus acute severe respiratory illness", "seafood market corona virus acute severe pneumonia", "2019new sars corona virus 2 severe respiratory syndrome", "disorder caused by 2019 novel wuhan seafood market virus", "illness from 2019new wuhan seafood market pneumonia virus", "2019 sars corona virus2 acute severe infection", "illness caused by wuhan coronavirus 2019", "2019 novel chinese coronavirus syndrome", "wuhan pneumonia virus 2019 acute illness", "2019 new seafood market virus severe respiratory syndrome", "2019 corona virus acute respiratory syndrome", "2019 novel sars cov 2 acute pneumonia", "2019 new wuhancorona virus acute disorder", "2019novel wuhan corona virus acute severe infection", "2019novel chinese coronavirus acute disorder", "disorder caused by 2019 wuhan pneumonia virus", "disorder due to 2019new severe acute respiratory syndrome corona virus2", "2019new wuhan seafood market pneumonia virus asevere acute respiratory disease", "virus responsible for covid19 acute severe respiratory illness", "2019 novel seafood market virus severe disorder", "2019 new wuhancoronavirus respiratory disorder", "2019 novel wuhan coronavirus acute syndrome", "wuhancoronavirus 2019 respiratory illness", "2019 wuhan seafood market pneumonia virus acute respiratory illness", "new coronavirus severe acute respiratory syndrome", "2019 seafood market corona virus acute severe respiratory syndrome", "2019new wuhan pneumonia virus respiratory disorder", "sars corona virus 2 2019 acute respiratory disease", "2019 novel sarscov2 severe respiratory illness", "novel corona virus severe respiratory disease", "2019new wuhan virus severe acute syndrome", "sars corona virus2 2019 acute severe respiratory syndrome", "2019 novel seafood market virus acute respiratory illness", "2019 sars corona virus2 acute disorder", "2019 new wuhan pneumonia virus severe respiratory syndrome", "syndrome caused by seafood market virus", "2019 seafood market coronavirus acute severe disease", "2019new sars corona virus 2 respiratory infection", "2019 new wuhan seafood market virus acute severe respiratory illness", "pneumonia caused by 2019 novel wuhan corona virus", "novel coronavirus 2019 acute severe disease", "seafood market virus 2019 asevere acute respiratory disease", "wuhan seafood market pneumonia virus severe pneumonia", "2019new sars corona virus 2 respiratory disease", "disease due to wuhan corona virus", "2019 novel chinese corona virus infection", "2019new sarscov2 syndrome", "disease caused by 2019novel wuhan virus", "infection from wuhan seafood market pneumonia virus", "illness from 2019new seafood market coronavirus", "2019 novel wuhan corona virus acute respiratory infection", "2019 sars corona virus2 severe acute respiratory illness", "2019 novel seafood market virus acute severe illness", "illness caused by sars cov 2 2019", "2019 novel sars coronavirus 2 severe acute respiratory illness", "2019novel wuhancoronavirus pneumonia", "2019novel sars cov 2 severe disorder", "wuhan virus 2019 acute severe respiratory disorder", "disorder due to new coronavirus", "wuhan seafood market pneumonia virus acute illness", "infection caused by 2019 new wuhan corona virus", "new coronavirus disorder", "2019 novel wuhan seafood market virus severe acute illness", "2019 new coronavirus severe acute disorder", "disease from 2019 new sars cov 2", "disease due to severe acute respiratory syndrome corona virus2", "wuhan seafood market pneumonia virus 2019 severe disorder", "infection due to sars coronavirus2", "disorder caused by wuhan seafood market virus", "2019 wuhan seafood market pneumonia virus acute severe respiratory disease", "2019 novel wuhan pneumonia virus acute syndrome", "wuhan virus acute respiratory syndrome", "severe acute respiratory syndrome coronavirus2 pneumonia", "2019novel seafood market coronavirus severe illness", "2019novel wuhan pneumonia virus acute severe respiratory infection", "illness from 2019new sars corona virus2", "wuhan virus 2019 acute severe respiratory infection", "new coronavirus severe disease", "2019 new sars corona virus2 severe disorder", "2019new seafood market corona virus acute severe respiratory disease", "2019 new sars cov 2 severe acute pneumonia", "2019new wuhan pneumonia virus asevere acute respiratory disease", "disorder caused by 2019novel wuhancoronavirus", "pneumonia from 2019 novel severe acute respiratory syndromerelated corona virus 2", "2019 sarscov2 severe acute respiratory illness", "illness caused by 2019 new coronavirus", "2019new sars coronavirus 2 severe acute respiratory infection", "disease caused by 2019 new severe acute respiratory syndrome corona virus2", "pneumonia due to 2019 new seafood market virus", "2019new chinese coronavirus severe respiratory disorder", "hubei severe acute respiratory disorder", "2019ncov acute severe respiratory syndrome", "disorder from 2019novel seafood market corona virus", "disease due to 2019new severe acute respiratory syndromerelated coronavirus 2", "2019 sars coronavirus 2 disorder", "2019 coronavirus acute severe disease", "disorder due to chinese corona virus", "2019 novel wuhan corona virus acute illness", "2019novel seafood market corona virus severe respiratory syndrome", "hcov19 acute infection", "seafood market corona virus acute syndrome", "hcov19 severe acute illness", "2019 new wuhan seafood market virus severe respiratory syndrome", "wuhan pneumonia virus respiratory illness", "illness due to severe acute respiratory syndromerelated corona virus2", "pneumonia due to severe acute respiratory syndromerelated corona virus 2 2019", "2019 chinese coronavirus severe disorder", "chinese corona virus acute severe respiratory illness", "illness due to 2019 new seafood market coronavirus", "illness from 2019new sars coronavirus2", "2019 new sars coronavirus 2 acute respiratory illness", "illness caused by sars corona virus2 2019", "sars corona virus2 acute respiratory disease", "2019 new seafood market virus acute infection", "pneumonia from 2019 novel sars coronavirus2", "new corona virus 2019 severe illness", "illness caused by 2019 seafood market corona virus", "2019 sars cov 2 respiratory infection", "2019new wuhan seafood market virus acute severe pneumonia", "pneumonia caused by 2019 novel sars coronavirus2", "2019 wuhan corona virus disease", "syndrome caused by 2019 new corona virus", "2019novel wuhan pneumonia virus severe disorder", "2019novel seafood market coronavirus severe acute respiratory syndrome", "infection caused by new coronavirus 2019", "sars corona virus 2 2019 severe acute pneumonia", "illness from 2019 wuhancoronavirus", "wuhan corona virus pneumonia", "covid19 virus pneumonia", "2019 human coronavirus acute severe respiratory infection", "pneumonia from 2019new wuhan corona virus", "infection due to wuhan virus 2019", "2019ncov severe acute respiratory disorder", "2019 sars cov 2 severe acute respiratory syndrome", "disorder due to sars cov 2", "wuhan virus acute respiratory illness", "illness due to 2019 sars corona virus2", "syndrome from virus responsible for covid19", "2019 new seafood market coronavirus acute respiratory syndrome", "2019 corona virus acute severe disorder", "pneumonia caused by 2019novel chinese coronavirus", "disorder caused by 2019 sarscov2", "2019ncov acute severe respiratory disorder", "disease from 2019 new wuhan coronavirus", "2019 novel wuhan pneumonia virus severe respiratory infection", "sars cov 2 2019 respiratory syndrome", "sars coronavirus 2 2019 severe disorder", "2019 human coronavirus acute severe disorder", "covid 19 virus respiratory disorder", "2019 novel sars coronavirus2 acute disorder", "sars corona virus 2 severe acute disorder", "illness due to novel corona virus", "2019 novel sars coronavirus2 respiratory syndrome", "2019 novel sars corona virus 2 severe disorder", "2019 novel wuhan virus acute disorder", "2019 novel sars coronavirus 2 acute severe disease", "2019 novel wuhancoronavirus acute severe respiratory disorder", "2019new sars cov 2 acute severe respiratory disease", "disorder caused by sars coronavirus 2", "2019 sars corona virus2 severe infection", "2019 wuhan pneumonia virus acute severe respiratory disorder", "sars corona virus2 illness", "infection from 2019 ncov", "covid 19 virus pneumonia", "2019new sars cov 2 severe respiratory infection", "wuhancorona virus 2019 acute respiratory disorder", "2019new seafood market coronavirus severe respiratory illness", "2019new wuhan corona virus syndrome", "wuhan coronavirus illness", "new corona virus 2019 severe infection", "2019novel wuhan virus severe disease", "chinese corona virus 2019 asevere acute respiratory disease", "illness from 2019 new sars corona virus2", "2019 new wuhan seafood market pneumonia virus severe acute infection", "2019 sars corona virus2 severe pneumonia", "wuhan coronavirus 2019 illness", "2019 novel wuhan seafood market virus severe illness", "disorder due to severe acute respiratory syndromerelated corona virus 2 2019", "2019novel sarscov2 asevere acute respiratory disease", "2019novel sars corona virus 2 severe acute disorder", "pneumonia caused by 2019 wuhan coronavirus", "infection due to 2019 wuhan seafood market virus", "sars coronavirus2 severe disease", "disorder caused by 2019 novel wuhancorona virus", "sars corona virus 2 2019 severe respiratory syndrome", "2019new seafood market corona virus severe acute disorder", "sars coronavirus 2 severe acute respiratory illness", "disorder due to 2019 new severe acute respiratory syndromerelated coronavirus2", "2019new wuhan virus severe respiratory syndrome", "severe acute respiratory syndrome corona virus 2 2019", "hcov19 infection", "wuhan coronavirus acute respiratory infection", "2019new wuhan pneumonia virus severe acute respiratory disorder", "wuhan pneumonia virus acute disease", "wuhan seafood market pneumonia virus severe acute respiratory disorder", "2019new seafood market virus severe acute respiratory infection", "wuhan coronavirus acute severe illness", "disease from 2019 sars corona virus2", "disorder from wuhan virus", "2019 chinese corona virus illness", "2019novel wuhan pneumonia virus acute syndrome", "syndrome from novel corona virus", "illness from 2019new wuhancoronavirus", "2019 novel wuhan seafood market virus acute severe respiratory syndrome", "infection from 2019new sars corona virus2", "covid19 virus severe respiratory disorder", "2019 new wuhan virus acute severe respiratory disorder", "2019 new wuhancoronavirus severe infection", "2019 sars corona virus2 pneumonia", "pneumonia from wuhan virus 2019", "disease due to 2019novel severe acute respiratory syndrome coronavirus 2", "2019new sars coronavirus2 severe illness", "pneumonia due to wuhan virus", "disorder from severe acute respiratory syndromerelated coronavirus 2 2019", "disorder from 2019 new seafood market virus", "illness caused by wuhancoronavirus", "2019 wuhancorona virus severe acute pneumonia", "2019novel sars coronavirus2 severe acute illness", "infection caused by 2019novel severe acute respiratory syndromerelated corona virus2", "infection due to 2019 novel severe acute respiratory syndrome corona virus2", "disorder from severe acute respiratory syndrome coronavirus2", "hcov 19 respiratory illness", "wuhancorona virus acute respiratory disorder", "2019novel sars coronavirus2 acute severe illness", "2019novel seafood market corona virus disease", "2019 seafood market coronavirus severe disorder", "2019 wuhan corona virus respiratory illness", "2019 new chinese coronavirus acute severe respiratory disease", "infection from sarscov2 2019", "wuhancorona virus 2019 disease", "2019new sarscov2 severe syndrome", "2019new wuhancoronavirus severe acute illness", "virus causing covid19 acute respiratory syndrome", "2019 new sarscov2 asevere acute respiratory disease", "pneumonia from 2019 novel chinese corona virus", "2019 new wuhan virus acute severe respiratory syndrome", "syndrome caused by 2019 sars coronavirus 2", "2019 corona virus acute pneumonia", "chinese corona virus acute pneumonia", "2019 novel seafood market corona virus acute severe syndrome", "wuhan acute severe respiratory infection", "chinese corona virus 2019 severe respiratory disorder", "illness caused by 2019 corona virus", "2019 wuhan virus severe respiratory infection", "2019novel wuhancoronavirus acute severe respiratory illness", "pneumonia from 2019 new severe acute respiratory syndromerelated coronavirus2", "chinese coronavirus severe syndrome", "wuhan coronavirus acute respiratory disorder", "2019 novel wuhancorona virus acute respiratory disorder", "wuhan corona virus disorder", "disorder caused by 2019new wuhancorona virus", "infection due to virus responsible for covid19", "pneumonia from 2019 human coronavirus", "2019 new wuhan pneumonia virus asevere acute respiratory disease", "illness due to 2019 sarscov2", "illness due to 2019 new wuhan seafood market pneumonia virus", "wuhan corona virus severe acute respiratory infection", "wuhan virus acute respiratory disorder", "2019 novel wuhan virus severe disease", "2019 wuhan coronavirus acute respiratory syndrome", "2019new wuhan seafood market pneumonia virus severe acute infection", "illness caused by seafood market virus", "2019new wuhan coronavirus severe acute respiratory syndrome", "illness due to 2019 novel chinese coronavirus", "pneumonia due to 2019novel wuhancorona virus", "disorder from sars cov 2", "2019novel chinese corona virus acute respiratory disorder", "pneumonia caused by 2019 new wuhan pneumonia virus", "2019novel wuhan seafood market virus acute syndrome", "novel corona virus 2019 severe respiratory syndrome", "disease caused by 2019new sarscov2", "novel corona virus 2019 respiratory disease", "chinese corona virus", "2019 wuhancorona virus severe respiratory infection", "disease due to sars coronavirus2 2019", "sarscov2 severe acute respiratory infection", "pneumonia caused by 2019 human corona virus", "2019 wuhan corona virus disorder", "2019 seafood market virus respiratory syndrome", "severe acute respiratory syndrome corona virus 2", "disease from 2019new severe acute respiratory syndromerelated coronavirus 2", "2019 novel sars coronavirus2 acute severe illness", "2019 novel coronavirus acute severe disorder", "wuhan virus acute respiratory disease", "2019 chinese coronavirus acute severe pneumonia", "sars corona virus 2 asevere acute respiratory disease", "2019novel wuhan seafood market pneumonia virus acute severe pneumonia", "2019new sarscov2 acute severe respiratory disease", "syndrome due to wuhancorona virus 2019", "pneumonia caused by sars coronavirus 2 2019", "wuhancoronavirus 2019 respiratory disorder", "syndrome from 2019novel sars corona virus 2", "novel coronavirus 2019 acute respiratory illness", "2019new wuhan seafood market pneumonia virus acute severe respiratory disease", "2019 wuhancorona virus disorder", "infection caused by seafood market corona virus", "2019new sars corona virus2 acute respiratory disease", "2019novel seafood market corona virus severe syndrome", "2019 novel coronavirus respiratory disorder", "2019new wuhancoronavirus severe syndrome", "2019 human coronavirus acute respiratory infection", "sarscov2 severe acute respiratory disorder", "2019novel wuhancorona virus severe infection", "2019 wuhan pneumonia virus acute respiratory disorder", "2019 new sars coronavirus2 acute respiratory disease", "illness due to hcov19", "disease from wuhancorona virus", "seafood market virus acute severe illness", "wuhan corona virus acute respiratory syndrome", "2019 sars cov 2 acute respiratory illness", "2019 new seafood market corona virus acute respiratory disorder", "seafood market coronavirus severe acute respiratory disorder", "2019novel sars coronavirus 2 acute disease", "2019 human corona virus severe acute infection", "covid 19 virus acute respiratory infection", "2019 new seafood market coronavirus acute severe respiratory disease", "novel coronavirus 2019 severe disorder", "2019novel wuhan coronavirus acute severe illness", "disorder from 2019 novel sars coronavirus2", "human corona virus 2019 severe acute syndrome", "2019new chinese corona virus disorder", "2019novel wuhan virus severe syndrome", "2019 sarscov2 respiratory illness", "disorder due to severe acute respiratory syndrome coronavirus2 2019", "2019 wuhancorona virus acute severe disease", "2019 novel wuhan coronavirus severe syndrome", "2019 new wuhan pneumonia virus acute disease", "2019 wuhan virus severe acute disease", "new coronavirus acute severe respiratory illness", "2019 novel wuhancoronavirus acute severe respiratory syndrome", "2019novel seafood market corona virus asevere acute respiratory disease", "illness caused by severe acute respiratory syndrome corona virus2", "2019novel wuhan seafood market pneumonia virus severe acute respiratory infection", "2019 new wuhan virus acute syndrome", "disease caused by wuhan virus 2019", "wuhan pneumonia virus 2019 severe disease", "illness due to covid19 virus", "hcov19 acute respiratory disease", "2019 sarscov2 severe acute disorder", "sars corona virus 2 2019 severe acute respiratory illness", "2019new severe acute respiratory syndromerelated corona virus2 disease", "illness caused by 2019 new wuhan seafood market virus", "pneumonia caused by sars cov 2", "wuhan seafood market virus 2019 severe pneumonia", "syndrome due to 2019novel wuhan virus", "wuhan corona virus 2019 illness", "new coronavirus severe infection", "2019 ncov illness", "illness due to severe acute respiratory syndromerelated corona virus2 2019", "2019novel sars corona virus2 respiratory disease", "seafood market coronavirus 2019 severe respiratory infection", "wuhancorona virus 2019 acute severe respiratory syndrome", "sars coronavirus 2 disorder", "2019 sars coronavirus2 acute respiratory illness", "2019 novel sars coronavirus2 acute illness", "wuhan seafood market pneumonia virus severe disorder", "covid 19 virus acute disease", "hcov19 severe syndrome", "2019 sars cov 2 acute disease", "2019 new wuhan seafood market virus severe disorder", "2019 sars corona virus2 severe acute respiratory disorder", "wuhan seafood market pneumonia virus 2019 pneumonia", "illness from 2019 wuhancorona virus", "disorder from 2019 new sars coronavirus2", "pneumonia from 2019novel sarscov2", "new corona virus 2019 acute syndrome", "infection due to 2019 sars coronavirus 2", "illness from seafood market virus 2019", "disorder from 2019 seafood market corona virus", "ICD11: RA01.0", "human coronavirus 2019 acute respiratory syndrome", "infection due to chinese coronavirus", "2019new wuhan seafood market pneumonia virus severe acute respiratory illness", "2019novel sars coronavirus 2 acute infection", "2019 novel corona virus acute severe respiratory disease", "2019 new sars corona virus 2 severe respiratory disease", "2019new chinese corona virus acute severe pneumonia", "2019 new seafood market coronavirus severe infection", "2019ncov acute severe pneumonia", "wuhan seafood market pneumonia virus disorder", "2019 novel wuhan virus acute syndrome", "2019 novel wuhan corona virus severe acute pneumonia", "wuhancoronavirus 2019 acute pneumonia", "infection due to 2019 sars corona virus2", "hcov19 severe acute disorder", "wuhan virus severe acute disease", "virus causing covid19 severe acute syndrome", "2019 novel sarscov2 syndrome", "sars coronavirus2 2019 severe respiratory syndrome", "2019 new sars coronavirus2 severe acute disorder", "virus responsible for covid19 severe syndrome", "sars corona virus2", "2019 new wuhan virus acute severe disease", "disease from 2019novel wuhancorona virus", "illness caused by wuhancorona virus", "2019 novel coronavirus severe respiratory infection", "disease caused by seafood market corona virus 2019", "2019novel sars cov 2 acute severe syndrome", "2019 sars coronavirus2 severe acute pneumonia", "2019 wuhan virus severe acute infection", "disease caused by wuhancoronavirus", "syndrome caused by 2019novel wuhancoronavirus", "illness from 2019 severe acute respiratory syndromerelated coronavirus2", "infection due to sars coronavirus2 2019", "2019 new seafood market corona virus pneumonia", "illness caused by 2019novel sars coronavirus 2", "severe acute respiratory syndrome corona virus2 infection", "seafood market severe syndrome", "wuhan seafood market virus severe respiratory infection", "2019 seafood market corona virus acute severe infection", "2019new wuhan seafood market pneumonia virus disorder", "hcov 19 severe acute disorder", "2019new wuhan corona virus severe respiratory disease", "2019 chinese corona virus acute severe respiratory disease", "syndrome from sars corona virus2", "disorder from 2019 novel severe acute respiratory syndrome coronavirus2", "new corona virus 2019 acute pneumonia", "seafood market acute respiratory infection", "2019novel wuhan pneumonia virus acute respiratory infection", "2019novel sars corona virus2 severe respiratory disease", "2019 new wuhan coronavirus acute severe infection", "disease from 2019new wuhan pneumonia virus", "syndrome due to 2019novel wuhancorona virus", "2019new sars cov 2 severe respiratory illness", "covid19 virus acute severe illness", "2019 new sars coronavirus2 severe syndrome", "2019 seafood market coronavirus acute severe respiratory infection", "virus responsible for covid19 respiratory disorder", "2019 wuhancoronavirus respiratory illness", "2019novel sarscov2 severe acute pneumonia", "virus causing covid19 severe acute infection", "infection caused by 2019new chinese coronavirus", "wuhan seafood market virus acute respiratory infection", "2019 novel wuhan virus respiratory syndrome", "2019new wuhan pneumonia virus acute respiratory disorder", "infection from severe acute respiratory syndrome corona virus 2 2019", "2019novel severe acute respiratory syndrome coronavirus2 disease", "2019new wuhan coronavirus acute pneumonia", "disorder caused by 2019 severe acute respiratory syndromerelated coronavirus2", "pneumonia from 2019new severe acute respiratory syndrome corona virus 2", "syndrome from sars corona virus2 2019", "hcov 19 respiratory disorder", "virus causing covid19", "illness due to 2019new wuhan corona virus", "sars cov 2 2019 acute illness", "2019 wuhan seafood market pneumonia virus acute severe respiratory infection", "human coronavirus 2019 severe acute disease", "2019novel seafood market virus acute disease", "illness due to 2019new sars cov 2", "wuhancoronavirus 2019 pneumonia", "new coronavirus 2019 disorder", "2019 wuhancorona virus severe respiratory disorder", "2019new wuhancoronavirus pneumonia", "novel corona virus 2019 severe disorder", "sars cov 2 2019 respiratory disorder", "2019 wuhan virus acute severe syndrome", "2019novel wuhan seafood market virus severe acute syndrome", "2019 novel wuhan seafood market pneumonia virus severe acute respiratory infection", "2019 novel chinese coronavirus acute respiratory disorder", "human coronavirus 2019 severe acute pneumonia", "2019 novel seafood market coronavirus acute severe disorder", "2019 sars coronavirus 2 severe respiratory illness", "2019ncov severe acute disorder", "novel corona virus 2019 severe acute illness", "2019 new sars coronavirus 2 acute severe syndrome", "2019new sarscov2 severe acute infection", "infection caused by 2019novel wuhan corona virus", "infection due to 2019novel seafood market corona virus", "syndrome due to sars cov 2", "illness from 2019 novel severe acute respiratory syndrome corona virus2", "wuhancoronavirus acute respiratory infection", "2019 human corona virus acute illness", "illness caused by 2019 severe acute respiratory syndromerelated corona virus 2", "disease due to hcov19", "sars cov 2 2019 acute respiratory infection", "2019 new chinese corona virus acute severe pneumonia", "2019 novel wuhan corona virus acute syndrome", "2019 seafood market coronavirus severe respiratory syndrome", "syndrome due to 2019novel wuhancoronavirus", "pneumonia from 2019novel seafood market corona virus", "new coronavirus respiratory syndrome", "chinese corona virus 2019 severe acute respiratory infection", "sars coronavirus2 pneumonia", "wuhan severe respiratory illness", "seafood market virus acute severe respiratory illness", "disease due to 2019novel wuhan seafood market pneumonia virus", "2019 chinese corona virus severe infection", "disease from 2019new sars corona virus2", "2019 new wuhan seafood market virus acute pneumonia", "wuhancoronavirus acute respiratory syndrome", "2019 sars coronavirus 2 acute severe disease", "sars coronavirus 2 2019 severe acute disease", "new coronavirus acute disorder", "2019new seafood market corona virus illness", "2019 new sarscov2 respiratory disorder", "2019 new coronavirus acute severe respiratory disease", "syndrome from 2019 sars coronavirus2", "pneumonia from 2019new chinese corona virus", "2019 coronavirus severe acute syndrome", "2019novel sars coronavirus 2 respiratory infection", "virus causing covid19 illness", "2019novel chinese coronavirus acute pneumonia", "wuhan seafood market virus pneumonia", "2019 severe acute respiratory syndromerelated corona virus2 pneumonia", "new corona virus acute severe syndrome", "2019novel severe acute respiratory syndrome coronavirus 2 illness", "2019 new wuhan seafood market pneumonia virus respiratory infection", "2019 novel sars corona virus2 severe acute illness", "2019novel wuhan seafood market virus acute severe respiratory disease", "2019 coronavirus severe disease", "novel corona virus asevere acute respiratory disease", "syndrome caused by 2019novel wuhan seafood market pneumonia virus", "2019 new wuhan corona virus severe infection", "2019 chinese corona virus acute respiratory illness", "2019 seafood market virus acute disorder", "2019 new sars cov 2 acute syndrome", "2019new chinese coronavirus respiratory syndrome", "sarscov2 pneumonia", "2019 new sars corona virus 2 severe acute respiratory disorder", "2019 human corona virus severe infection", "wuhan coronavirus 2019 acute respiratory illness", "2019new wuhan corona virus acute respiratory disorder", "illness caused by sarscov2", "illness from 2019 new severe acute respiratory syndromerelated coronavirus2", "2019new sarscov2 respiratory infection", "wuhancoronavirus severe disease", "2019 new corona virus disorder", "2019new wuhan seafood market pneumonia virus severe syndrome", "2019 novel chinese corona virus acute severe syndrome", "2019 novel sars coronavirus 2 severe acute pneumonia", "sars corona virus2 2019 acute respiratory infection", "chinese corona virus 2019 disorder", "2019new chinese coronavirus severe acute respiratory disorder", "human corona virus 2019 acute severe infection", "2019 new wuhan virus acute severe infection", "wuhan pneumonia virus 2019 acute syndrome", "2019novel sars corona virus 2 severe acute respiratory illness", "wuhan acute respiratory syndrome", "2019 novel wuhan seafood market pneumonia virus severe illness", "2019 novel sars coronavirus 2 asevere acute respiratory disease", "2019new wuhan corona virus severe acute respiratory disorder", "wuhan seafood market pneumonia virus 2019 acute severe respiratory infection", "illness caused by 2019 human coronavirus", "sars coronavirus2 2019 disease", "2019novel wuhan virus severe pneumonia", "2019 wuhancoronavirus respiratory syndrome", "2019 new sars coronavirus 2 acute respiratory disorder", "2019 novel coronavirus acute severe pneumonia", "seafood market corona virus acute severe respiratory infection", "syndrome due to 2019novel wuhan corona virus", "disease due to severe acute respiratory syndrome corona virus2 2019", "pneumonia caused by 2019 wuhan seafood market virus", "infection due to new coronavirus", "seafood market corona virus 2019 acute respiratory disease", "2019 novel wuhan seafood market pneumonia virus severe infection", "novel corona virus severe disease", "2019 novel wuhan seafood market pneumonia virus severe syndrome", "2019novel wuhan virus acute severe pneumonia", "covid 19 virus illness", "2019 seafood market corona virus severe acute pneumonia", "pneumonia from novel coronavirus", "2019novel sars coronavirus2 acute pneumonia", "illness caused by severe acute respiratory syndrome corona virus 2 2019", "seafood market virus acute disease", "2019 novel seafood market coronavirus severe respiratory illness", "wuhan corona virus acute respiratory disease", "hcov19 respiratory disorder", "2019 novel chinese corona virus acute severe disease", "2019new sarscov2 severe respiratory illness", "2019new wuhancorona virus acute severe respiratory syndrome", "disease due to wuhancoronavirus 2019", "2019 new seafood market corona virus acute illness", "2019 novel seafood market virus acute respiratory disease", "sars coronavirus 2 2019 severe acute syndrome", "disorder caused by 2019new sars cov 2", "sars coronavirus 2 2019 acute severe illness", "disorder caused by 2019 wuhancoronavirus", "covid 19 virus acute respiratory disease", "sars corona virus2 severe pneumonia", "2019new wuhancoronavirus severe acute respiratory illness", "sars cov 2 2019 severe infection", "2019 sars corona virus 2 acute severe infection", "2019 severe acute respiratory syndrome corona virus2 pneumonia", "2019novel chinese coronavirus acute severe respiratory disease", "illness due to 2019new seafood market coronavirus", "wuhancorona virus", "2019 new wuhancoronavirus acute severe respiratory disease", "seafood market virus respiratory disorder", "novel corona virus acute respiratory infection", "2019 severe acute respiratory syndromerelated coronavirus 2 illness", "2019 novel wuhancorona virus severe acute respiratory illness", "wuhan seafood market pneumonia virus 2019 acute severe infection", "2019 new corona virus respiratory illness", "illness due to 2019 seafood market virus", "pneumonia due to 2019 novel severe acute respiratory syndromerelated coronavirus2", "pneumonia from severe acute respiratory syndrome corona virus 2 2019", "2019 novel chinese corona virus severe acute respiratory syndrome", "wuhan seafood market virus 2019 acute severe respiratory disease", "2019 wuhan seafood market pneumonia virus acute severe respiratory syndrome", "2019novel seafood market virus severe acute disorder", "syndrome caused by 2019 new wuhan corona virus", "2019novel chinese coronavirus", "infection from 2019 novel wuhan seafood market pneumonia virus", "sars corona virus 2 2019 acute severe illness", "disorder caused by sars corona virus 2", "2019 wuhan seafood market pneumonia virus severe acute infection", "2019novel wuhan pneumonia virus acute illness", "2019 new wuhancorona virus illness", "seafood market corona virus 2019 acute respiratory infection", "2019 novel wuhancoronavirus severe respiratory illness", "2019new sars coronavirus2 severe respiratory disorder", "wuhan pneumonia virus 2019 severe respiratory disorder", "covid 19 virus acute severe illness", "pneumonia due to 2019 corona virus", "2019 seafood market virus severe acute syndrome", "2019 new seafood market corona virus acute severe syndrome", "2019 novel chinese corona virus acute severe respiratory illness", "disease caused by 2019 severe acute respiratory syndromerelated corona virus 2", "2019new wuhancorona virus severe respiratory illness", "2019 wuhan corona virus respiratory infection", "wuhancoronavirus 2019 severe syndrome", "new corona virus respiratory disorder", "chinese coronavirus 2019 severe syndrome", "syndrome due to 2019 novel wuhan seafood market pneumonia virus", "pneumonia due to 2019 novel wuhancoronavirus", "2019new seafood market corona virus acute illness", "2019new severe acute respiratory syndrome coronavirus2 pneumonia", "2019 wuhancorona virus severe acute respiratory infection", "2019 novel sarscov2 acute severe illness", "illness from severe acute respiratory syndrome corona virus2", "wuhancorona virus 2019 respiratory syndrome", "sars corona virus2 respiratory infection", "2019 new wuhan seafood market virus acute severe respiratory disease", "2019novel sarscov2 acute respiratory disease", "2019 novel sars corona virus 2 asevere acute respiratory disease", "2019novel sars corona virus 2 acute severe disorder", "chinese coronavirus acute severe respiratory disorder", "disease from seafood market virus 2019", "wuhancorona virus severe syndrome", "2019new sars corona virus2 infection", "2019 severe acute respiratory syndromerelated coronavirus 2", "2019 new sarscov2 infection", "2019 sarscov2 acute respiratory illness", "wuhan seafood market pneumonia virus disease", "2019new wuhan corona virus severe acute respiratory syndrome", "2019novel wuhan pneumonia virus severe disease", "2019 wuhan seafood market virus severe respiratory illness", "2019 novel sars cov 2 respiratory disorder", "2019 seafood market corona virus severe acute disease", "disease caused by 2019novel severe acute respiratory syndrome corona virus2", "2019 novel sars corona virus2 respiratory infection", "2019 sars coronavirus 2 acute disease", "chinese corona virus disorder", "2019 coronavirus disease", "wuhancoronavirus 2019 severe acute illness", "2019 new sars corona virus2 pneumonia", "sars coronavirus2 2019 acute severe disorder", "wuhan corona virus 2019 acute severe disease", "2019 novel seafood market coronavirus", "seafood market corona virus 2019 acute severe illness", "wuhancoronavirus severe respiratory illness", "new corona virus 2019 severe respiratory disorder", "2019novel wuhan pneumonia virus respiratory infection", "sars coronavirus2 acute severe respiratory disorder", "2019 new sars cov 2 severe disease", "illness from 2019 novel severe acute respiratory syndromerelated corona virus 2", "covid19 virus respiratory disease", "sars corona virus 2 acute infection", "2019new wuhancoronavirus respiratory infection", "syndrome from seafood market corona virus 2019", "2019 severe acute respiratory syndrome coronavirus2 disease", "2019 novel wuhan pneumonia virus pneumonia", "disorder caused by 2019 novel severe acute respiratory syndromerelated coronavirus 2", "2019novel chinese corona virus severe acute pneumonia", "2019novel sars coronavirus2 respiratory infection", "2019novel seafood market virus severe disorder", "chinese corona virus acute respiratory disorder", "infection caused by 2019novel sars corona virus2", "disease caused by new corona virus", "wuhan pneumonia virus 2019 severe respiratory disease", "2019 novel severe acute respiratory syndrome coronavirus2 disease", "disorder from 2019new severe acute respiratory syndromerelated corona virus 2", "disease due to 2019 novel wuhan seafood market virus", "seafood market corona virus 2019 disease", "infection from 2019 new coronavirus", "2019 new chinese coronavirus pneumonia", "2019 new wuhan seafood market pneumonia virus respiratory illness", "2019 novel wuhancorona virus respiratory disorder", "disease caused by sars coronavirus2", "wuhancorona virus severe acute respiratory illness", "2019new wuhan corona virus acute respiratory disease", "novel corona virus disease", "2019new chinese coronavirus acute severe respiratory disorder", "infection caused by 2019new sars corona virus2", "2019 novel wuhan corona virus severe acute disorder", "2019new seafood market coronavirus respiratory syndrome", "wuhan virus infection", "pneumonia from 2019 new wuhan corona virus", "2019 novel seafood market corona virus severe illness", "2019 sars corona virus 2 acute respiratory infection", "2019 new sars coronavirus 2 acute illness", "sars coronavirus2 severe illness", "seafood market corona virus acute severe disease", "sars corona virus2 acute severe pneumonia", "2019 novel seafood market virus acute severe respiratory infection", "infection due to 2019novel severe acute respiratory syndrome coronavirus 2", "novel corona virus acute pneumonia", "2019 new chinese corona virus acute syndrome", "novel coronavirus severe respiratory infection", "wuhan acute severe respiratory illness", "2019 novel wuhan corona virus acute severe respiratory illness", "2019 novel wuhan coronavirus respiratory disease", "2019 sarscov2 disease", "wuhan seafood market pneumonia virus 2019 acute severe respiratory illness", "2019 new chinese coronavirus respiratory syndrome", "2019novel chinese corona virus acute severe syndrome", "2019 new wuhan seafood market pneumonia virus disorder", "2019novel sarscov2 severe acute respiratory infection", "disorder from 2019 new corona virus", "disorder caused by severe acute respiratory syndromerelated coronavirus2", "disease from 2019 new chinese corona virus", "seafood market coronavirus acute severe illness", "2019novel sars coronavirus 2 pneumonia", "wuhancoronavirus severe respiratory infection", "disease from chinese corona virus 2019", "disorder due to 2019new wuhan seafood market virus", "2019 sars corona virus2 severe respiratory infection", "seafood market coronavirus", "2019novel wuhan seafood market virus acute severe disease", "2019 coronavirus asevere acute respiratory disease", "disorder caused by severe acute respiratory syndrome coronavirus 2", "illness due to virus causing covid19", "disease due to 2019 sarscov2", "syndrome caused by 2019new sars corona virus 2", "syndrome due to 2019 human corona virus", "2019 new sars cov 2 acute illness", "2019new wuhan coronavirus respiratory illness", "2019 new seafood market corona virus severe pneumonia", "disease caused by covid 19 virus", "novel corona virus acute severe respiratory illness", "wuhan virus 2019 acute disease", "2019 novel sars coronavirus 2 acute severe respiratory illness", "disease from 2019 novel sars coronavirus 2", "disease from 2019 novel wuhancorona virus", "2019novel wuhan coronavirus severe acute respiratory syndrome", "wuhancorona virus 2019 severe acute respiratory disorder", "2019 novel wuhan seafood market virus severe disorder", "novel corona virus severe infection", "disease from 2019novel wuhancoronavirus", "2019 wuhan seafood market virus severe syndrome", "2019 new sars coronavirus2 acute severe pneumonia", "infection due to 2019 new wuhan coronavirus", "infection due to 2019 human corona virus", "pneumonia due to severe acute respiratory syndromerelated coronavirus 2 2019", "infection due to 2019 wuhan virus", "2019 novel seafood market corona virus severe acute syndrome", "2019 new corona virus asevere acute respiratory disease", "pneumonia due to 2019 novel sars coronavirus 2", "pneumonia caused by 2019new sars cov 2", "2019novel sarscov2 disorder", "seafood market coronavirus 2019 acute severe disease", "2019 new severe acute respiratory syndrome corona virus 2 disorder", "2019 sars corona virus2 acute respiratory illness", "sars cov 2 2019 acute severe infection", "2019new chinese corona virus severe acute disease", "2019 novel sars corona virus2 syndrome", "syndrome due to human coronavirus 2019", "2019new wuhan pneumonia virus severe acute disease", "2019 new wuhan coronavirus disease", "2019novel sars coronavirus 2 acute respiratory syndrome", "2019 wuhan pneumonia virus severe disease", "illness from 2019 wuhan virus", "2019 novel wuhan coronavirus acute respiratory syndrome", "2019novel sarscov2 severe acute infection", "syndrome caused by sars corona virus2", "infection caused by 2019novel wuhan seafood market pneumonia virus", "2019 corona virus severe acute disorder", "2019 wuhan coronavirus severe disease", "2019 sars cov 2 respiratory syndrome", "infection caused by 2019 new sars corona virus 2", "2019new sars corona virus2 respiratory disorder", "novel coronavirus acute syndrome", "pneumonia caused by 2019 seafood market coronavirus", "disorder due to wuhan virus", "2019 new wuhan corona virus respiratory disorder", "illness caused by 2019 novel severe acute respiratory syndrome corona virus2", "2019 novel seafood market virus severe acute respiratory disorder", "disease caused by 2019novel wuhan seafood market pneumonia virus", "seafood market virus severe acute disease", "2019 wuhan seafood market virus severe acute respiratory infection", "hcov 19 acute severe respiratory disorder", "wuhan seafood market virus acute severe respiratory illness", "2019 sarscov2 severe disorder", "pneumonia due to new coronavirus 2019", "wuhan pneumonia virus acute severe disease", "severe acute respiratory syndrome corona virus 2 pneumonia", "infection caused by 2019 corona virus", "wuhan virus 2019 respiratory disorder", "illness caused by 2019 novel sarscov2", "pneumonia due to 2019 novel sarscov2", "wuhan pneumonia virus 2019 acute respiratory disorder", "2019novel wuhan seafood market virus severe disorder", "pneumonia caused by 2019novel severe acute respiratory syndrome corona virus 2", "syndrome due to wuhancorona virus", "infection from 2019 new severe acute respiratory syndromerelated corona virus 2", "disorder due to 2019 novel wuhan corona virus", "2019new wuhan virus acute severe illness", "2019 novel severe acute respiratory syndromerelated corona virus 2 illness", "2019novel chinese coronavirus infection", "novel coronavirus 2019 illness", "sarscov2 2019 acute respiratory disease", "2019 coronavirus severe acute respiratory illness", "2019 novel seafood market corona virus acute severe respiratory disease", "disease due to 2019 human corona virus", "human coronavirus 2019 acute respiratory infection", "2019 chinese corona virus infection", "2019 chinese coronavirus acute severe disease", "2019new seafood market coronavirus severe acute pneumonia", "disease due to severe acute respiratory syndromerelated corona virus 2", "new coronavirus 2019 severe acute illness", "syndrome caused by sars coronavirus 2 2019", "infection from 2019 severe acute respiratory syndrome coronavirus 2", "sars cov 2 2019 severe illness", "wuhan seafood market virus 2019 acute severe respiratory infection", "wuhan corona virus 2019 severe syndrome", "2019 novel wuhan coronavirus severe respiratory disease", "2019novel sars corona virus 2 acute respiratory syndrome", "infection caused by wuhan corona virus", "disorder due to 2019 novel wuhancoronavirus", "2019new sars coronavirus 2 respiratory disease", "syndrome due to 2019novel wuhan coronavirus", "2019ncov acute infection", "novel corona virus acute disease", "wuhancoronavirus acute respiratory illness", "2019novel seafood market virus severe acute syndrome", "disease caused by severe acute respiratory syndromerelated corona virus 2 2019", "disorder caused by severe acute respiratory syndromerelated corona virus2", "2019 new wuhan seafood market virus respiratory syndrome", "2019new wuhancorona virus acute severe infection", "2019 novel sars coronavirus 2 acute infection", "2019novel chinese corona virus severe respiratory infection", "2019novel wuhan corona virus severe acute respiratory illness", "disorder from novel coronavirus 2019", "2019novel wuhan seafood market virus acute severe respiratory infection", "2019 new sars coronavirus2 respiratory illness", "2019novel chinese corona virus acute severe respiratory syndrome", "disease caused by seafood market coronavirus", "hubei respiratory disorder", "chinese coronavirus 2019 severe respiratory disorder", "2019new wuhan coronavirus severe syndrome", "illness caused by 2019 new severe acute respiratory syndromerelated coronavirus2", "2019novel seafood market coronavirus severe infection", "disease due to 2019new sars corona virus2", "disorder caused by 2019 novel wuhan coronavirus", "2019new severe acute respiratory syndromerelated coronavirus 2 disorder", "2019new wuhan seafood market virus severe acute respiratory infection", "disease from 2019novel wuhan corona virus", "new coronavirus 2019 acute disorder", "virus causing covid19 severe infection", "covid 19 virus severe acute respiratory syndrome", "2019novel severe acute respiratory syndromerelated coronavirus 2 infection", "sars cov 2 2019 severe acute respiratory syndrome", "pneumonia due to new coronavirus", "infection from 2019 seafood market corona virus", "wuhan pneumonia virus severe respiratory syndrome", "2019novel seafood market virus acute respiratory illness", "illness from sars corona virus2 2019", "pneumonia caused by 2019 novel severe acute respiratory syndrome coronavirus 2", "disease from 2019new wuhan seafood market pneumonia virus", "2019 seafood market corona virus severe respiratory syndrome", "2019 novel seafood market virus acute severe infection", "new corona virus 2019 acute severe respiratory disease", "illness from severe acute respiratory syndrome corona virus2 2019", "wuhan coronavirus severe illness", "2019new chinese corona virus acute severe illness", "2019novel seafood market virus acute severe respiratory disease", "infection caused by 2019 novel chinese corona virus", "disease due to 2019 new corona virus", "sars corona virus 2 2019 acute severe respiratory syndrome", "disorder due to virus responsible for covid19", "2019 novel wuhan coronavirus infection", "chinese corona virus acute severe illness", "infection from 2019 new chinese corona virus", "2019 wuhan pneumonia virus acute respiratory illness", "illness caused by 2019novel severe acute respiratory syndromerelated corona virus2", "2019 new coronavirus severe acute illness", "2019 novel seafood market corona virus", "chinese corona virus acute disorder", "2019 novel seafood market coronavirus acute respiratory infection", "pneumonia due to 2019 novel seafood market coronavirus", "disorder due to 2019 corona virus", "2019new chinese corona virus severe pneumonia", "2019 novel corona virus acute syndrome", "2019 human coronavirus severe syndrome", "virus causing covid19 respiratory disease", "2019 new sars cov 2 acute severe respiratory disease", "novel corona virus severe acute disorder", "disease caused by 2019 sars cov 2", "disorder caused by 2019 new corona virus", "infection due to 2019 sars corona virus 2", "wuhancoronavirus acute respiratory disease", "sars coronavirus2 severe respiratory disorder", "pneumonia caused by 2019 new wuhan virus", "chinese coronavirus acute severe syndrome", "2019 new sars coronavirus 2 severe acute illness", "2019 novel seafood market corona virus respiratory illness", "new coronavirus 2019 acute severe respiratory infection", "2019novel wuhan coronavirus severe disorder", "seafood market acute severe illness", "disease due to chinese corona virus", "wuhan pneumonia", "2019 new seafood market virus syndrome", "2019new sars coronavirus 2 acute severe disease", "2019novel wuhan seafood market virus severe respiratory syndrome", "2019 new wuhan seafood market pneumonia virus acute severe syndrome", "disorder caused by chinese coronavirus 2019", "disorder from 2019 new severe acute respiratory syndromerelated coronavirus 2", "2019 new wuhan seafood market pneumonia virus acute severe respiratory syndrome", "seafood market coronavirus 2019 respiratory disorder", "syndrome due to 2019 seafood market virus", "sars coronavirus 2 2019", "2019 seafood market coronavirus severe respiratory illness", "hcov 19 acute severe syndrome", "disorder from 2019 novel seafood market corona virus", "disorder due to 2019 new wuhan seafood market pneumonia virus", "sars coronavirus2 2019 severe respiratory disease", "sars corona virus2 2019 severe acute disorder", "illness caused by sars coronavirus2", "illness from 2019novel seafood market coronavirus", "pneumonia due to 2019new sars cov 2", "2019 novel wuhancoronavirus respiratory disorder", "2019 novel sars coronavirus 2 acute respiratory syndrome", "2019 novel seafood market virus severe pneumonia", "2019 novel seafood market corona virus acute severe respiratory disorder", "virus responsible for covid19 acute severe respiratory disease", "2019novel sars cov 2 acute disease", "2019 novel wuhan virus respiratory infection", "wuhan coronavirus 2019 severe illness", "2019 seafood market virus severe syndrome", "sars corona virus2 acute severe respiratory disorder", "sars coronavirus 2 severe syndrome", "infection from 2019new severe acute respiratory syndromerelated corona virus 2", "wuhancoronavirus severe acute syndrome", "disorder from new corona virus", "wuhan seafood market virus severe acute respiratory illness", "2019new wuhancorona virus acute disease", "new corona virus 2019 acute illness", "sars corona virus2 2019 severe acute respiratory syndrome", "wuhan acute respiratory infection", "2019 novel seafood market corona virus acute respiratory infection", "pneumonia caused by sars corona virus2", "2019new sars coronavirus2 severe respiratory infection", "2019 novel wuhancoronavirus syndrome", "2019novel wuhan coronavirus acute severe syndrome", "2019 new wuhan pneumonia virus pneumonia", "severe acute respiratory syndromerelated coronavirus 2 infection", "2019ncov severe acute disease", "syndrome caused by 2019novel wuhan corona virus", "2019 novel seafood market corona virus acute severe respiratory syndrome", "2019novel seafood market corona virus severe disorder", "2019novel sars corona virus 2 severe respiratory disorder", "pneumonia caused by 2019new wuhancorona virus", "seafood market coronavirus disease", "disease caused by wuhan coronavirus", "infection due to 2019 new severe acute respiratory syndromerelated corona virus2", "infection due to 2019 new severe acute respiratory syndrome coronavirus 2", "disease due to 2019novel severe acute respiratory syndromerelated corona virus2", "syndrome due to 2019new seafood market virus", "disease from 2019 chinese corona virus", "2019new wuhan corona virus disorder", "2019novel wuhan corona virus acute respiratory illness", "2019novel sars coronavirus2 severe acute respiratory infection", "2019 new seafood market virus respiratory infection", "sars coronavirus 2 respiratory syndrome", "pneumonia from chinese coronavirus", "2019 new sars cov 2 respiratory disorder", "2019novel wuhan seafood market pneumonia virus acute pneumonia", "disease due to 2019new severe acute respiratory syndrome coronavirus2", "infection caused by 2019 ncov", "human coronavirus 2019 respiratory disorder", "disease from 2019novel seafood market corona virus", "wuhancoronavirus severe acute disease", "2019novel sars coronavirus 2 severe acute respiratory syndrome", "2019novel chinese corona virus acute respiratory disease", "2019novel severe acute respiratory syndromerelated coronavirus 2 disease", "wuhancorona virus 2019", "syndrome from 2019 wuhancorona virus", "virus responsible for covid19 severe acute illness", "2019new wuhan coronavirus acute respiratory disorder", "disorder from 2019new severe acute respiratory syndromerelated coronavirus 2", "illness caused by 2019 chinese corona virus", "wuhan seafood market pneumonia virus 2019 acute respiratory infection", "2019 new seafood market corona virus acute syndrome", "sars cov 2 2019 severe respiratory infection", "2019 novel wuhancoronavirus severe respiratory disease", "sars corona virus2 2019 severe respiratory infection", "illness caused by 2019novel sars cov 2", "disorder caused by 2019new wuhan corona virus", "infection from 2019 novel sars coronavirus2", "2019 novel wuhan corona virus severe respiratory illness", "hubei acute severe syndrome", "illness caused by new corona virus", "disease from 2019 ncov", "chinese coronavirus disease", "MeSH:C000657245", "2019 sars coronavirus 2 respiratory illness", "2019 new sars coronavirus 2 respiratory illness", "2019 new seafood market virus acute severe pneumonia", "illness from 2019 new coronavirus", "2019novel sars cov 2 severe acute infection", "wuhancoronavirus 2019 acute severe respiratory illness", "syndrome from 2019novel wuhancorona virus", "sars coronavirus2 illness", "2019 new sars coronavirus 2 acute severe respiratory illness", "2019 novel sars corona virus2 severe acute respiratory syndrome", "infection caused by 2019 sars coronavirus2", "2019 new wuhan seafood market pneumonia virus severe acute respiratory disorder", "syndrome caused by 2019 novel wuhan corona virus", "2019 sars corona virus 2 acute pneumonia", "wuhancoronavirus 2019 acute severe disorder", "2019 novel sars cov 2 acute disease", "sars corona virus2 severe acute disease", "infection caused by 2019novel severe acute respiratory syndrome coronavirus2", "disease due to 2019 novel sars coronavirus 2", "2019 novel wuhan coronavirus acute severe disorder", "disorder from 2019novel chinese coronavirus", "wuhan seafood market virus 2019 acute respiratory disorder", "disease caused by wuhan pneumonia virus", "2019novel wuhan pneumonia virus acute respiratory illness", "2019novel wuhan seafood market pneumonia virus severe respiratory disease", "2019 chinese coronavirus acute severe infection", "covid 19 virus acute severe respiratory disease", "disorder due to chinese coronavirus 2019", "sars corona virus 2 2019 severe acute disease", "2019 wuhan seafood market pneumonia virus acute severe infection", "2019 novel sars cov 2 severe acute respiratory disorder", "illness caused by severe acute respiratory syndromerelated corona virus 2", "2019novel seafood market virus acute severe respiratory disorder", "wuhan seafood market pneumonia virus asevere acute respiratory disease", "2019 new sars corona virus2 disease", "infection from 2019new severe acute respiratory syndrome coronavirus2", "pneumonia from 2019new chinese coronavirus", "2019 new sars coronavirus2", "pneumonia due to 2019 new severe acute respiratory syndromerelated corona virus 2", "syndrome due to 2019 new sars cov 2", "2019new wuhan corona virus severe acute respiratory infection", "wuhancorona virus acute infection", "2019 novel sars coronavirus2 disorder", "2019 new coronavirus severe disease", "2019new chinese coronavirus acute severe respiratory infection", "infection due to 2019 sars cov 2", "2019novel wuhan seafood market virus severe acute respiratory syndrome", "syndrome caused by 2019novel sars corona virus 2", "2019 new wuhancorona virus acute syndrome", "2019new wuhan coronavirus disease", "2019 human corona virus acute severe respiratory syndrome", "2019 new sars corona virus2 disorder", "infection due to 2019 severe acute respiratory syndrome corona virus 2", "covid19 virus acute infection", "disorder caused by 2019novel wuhan seafood market virus", "2019 chinese coronavirus severe acute respiratory disorder", "2019 severe acute respiratory syndrome corona virus2 illness", "2019 novel chinese coronavirus severe acute respiratory illness", "illness due to 2019novel severe acute respiratory syndromerelated corona virus 2", "syndrome due to 2019new wuhan corona virus", "2019 wuhan coronavirus respiratory illness", "chinese coronavirus pneumonia", "2019 new seafood market virus acute respiratory illness", "infection from 2019new wuhan coronavirus", "illness from 2019 novel wuhancorona virus", "2019ncov severe acute respiratory infection", "wuhancorona virus 2019 syndrome", "wuhan pneumonia virus severe respiratory disorder", "wuhan corona virus severe acute pneumonia", "2019 novel sars coronavirus2 acute severe disorder", "wuhan seafood market virus 2019 severe respiratory infection", "hcov 19 severe acute disease", "pneumonia from novel corona virus", "2019new chinese corona virus acute severe respiratory infection", "pneumonia due to 2019 severe acute respiratory syndrome coronavirus2", "wuhancoronavirus syndrome", "2019 wuhan virus acute severe respiratory infection", "infection due to 2019 new sars coronavirus 2", "disorder due to 2019 novel seafood market corona virus", "2019new sars corona virus 2 acute severe disease", "2019new sars coronavirus 2 severe pneumonia", "2019 new sarscov2 illness", "2019 novel wuhancorona virus acute severe disorder", "2019 seafood market corona virus acute disease", "2019new seafood market corona virus acute respiratory infection", "2019new wuhancorona virus acute severe pneumonia", "disorder due to 2019 novel sarscov2", "2019 new severe acute respiratory syndromerelated coronavirus2 illness", "2019 new wuhan pneumonia virus disease", "2019new seafood market corona virus infection", "syndrome from 2019 wuhan coronavirus", "2019 novel coronavirus severe illness", "2019 coronavirus severe illness", "2019 novel wuhan pneumonia virus acute severe illness", "illness due to 2019 new sars corona virus 2", "syndrome due to wuhan corona virus", "2019 novel wuhan coronavirus acute pneumonia", "2019 new chinese corona virus syndrome", "2019 novel wuhancorona virus respiratory illness", "seafood market virus pneumonia", "sars corona virus2 2019 respiratory syndrome", "syndrome from 2019new wuhan seafood market pneumonia virus", "2019 wuhan coronavirus severe respiratory disorder", "2019novel seafood market coronavirus acute severe respiratory disease", "2019 new sarscov2 severe infection", "2019novel sarscov2 acute severe respiratory disorder", "2019 new seafood market corona virus severe acute respiratory infection", "2019 novel wuhan pneumonia virus severe infection", "2019novel wuhan virus acute severe respiratory infection", "covid 19 virus severe respiratory illness", "2019 novel seafood market virus acute severe pneumonia", "2019 new chinese corona virus acute severe syndrome", "wuhan seafood market pneumonia virus 2019 acute respiratory disorder", "2019 sars coronavirus 2 acute severe pneumonia", "2019 new sars coronavirus2 acute illness", "2019novel wuhan virus respiratory disease", "2019 wuhan pneumonia virus acute severe respiratory syndrome", "hcov19 severe pneumonia", "syndrome due to 2019novel sars corona virus2", "pneumonia from wuhan coronavirus 2019", "infection from 2019 new seafood market corona virus", "sars corona virus 2 severe respiratory syndrome", "2019novel chinese coronavirus severe acute disorder", "wuhan virus 2019 acute severe respiratory syndrome", "2019novel wuhan virus pneumonia", "disorder from 2019 new sars corona virus 2", "disease caused by 2019 novel severe acute respiratory syndrome corona virus2", "disease due to 2019 novel chinese corona virus", "illness due to 2019 novel wuhan pneumonia virus", "disorder from 2019 severe acute respiratory syndromerelated coronavirus2", "wuhan infection", "2019novel wuhan coronavirus severe pneumonia", "virus causing covid19 acute infection", "2019novel wuhan virus acute respiratory disorder", "sarscov2 infection", "2019new wuhan pneumonia virus acute pneumonia", "disorder caused by seafood market virus 2019", "human corona virus 2019 disorder", "2019 wuhan seafood market virus severe infection", "2019 wuhan virus acute pneumonia", "disease from 2019new sarscov2", "wuhan coronavirus 2019 severe acute disorder", "2019novel wuhancoronavirus severe acute respiratory infection", "syndrome caused by wuhan virus 2019", "2019 new wuhan seafood market pneumonia virus severe disease", "2019novel wuhan virus severe acute disorder", "2019new sars corona virus2 illness", "2019 novel seafood market corona virus severe acute disorder", "2019 new severe acute respiratory syndromerelated corona virus 2 infection", "wuhan coronavirus 2019 acute severe infection", "sars cov 2 2019 severe respiratory syndrome", "2019novel sars coronavirus 2 acute severe respiratory disease", "illness due to 2019new wuhan seafood market pneumonia virus", "2019new seafood market virus acute disorder", "disorder caused by wuhan seafood market pneumonia virus 2019", "novel coronavirus 2019 acute respiratory infection", "2019novel sarscov2 severe acute syndrome", "novel coronavirus respiratory disease", "2019 sars coronavirus2 severe respiratory illness", "2019new seafood market virus acute severe disease", "chinese coronavirus 2019 severe acute pneumonia", "novel coronavirus 2019 acute syndrome", "sarscov2 acute severe disease", "2019 seafood market coronavirus acute severe respiratory disease", "2019 sars coronavirus 2 acute respiratory illness", "2019 seafood market virus severe disease", "disease due to 2019 novel severe acute respiratory syndrome corona virus2", "new corona virus acute severe respiratory disorder", "wuhancorona virus 2019 severe syndrome", "sars corona virus 2 2019 acute severe disorder", "virus causing covid19 syndrome", "illness caused by 2019 new wuhan virus", "2019 human corona virus acute syndrome", "2019 new seafood market coronavirus respiratory infection", "2019 seafood market coronavirus respiratory infection", "disorder due to 2019 new severe acute respiratory syndrome corona virus 2", "2019 wuhan virus acute respiratory disease", "2019 new sars coronavirus 2 severe infection", "2019novel wuhan corona virus acute illness", "2019 novel seafood market coronavirus severe acute pneumonia", "illness caused by 2019 novel wuhan coronavirus", "2019 wuhancorona virus disease", "2019 novel sars corona virus2 severe acute respiratory infection", "2019 wuhan seafood market virus severe respiratory infection", "pneumonia due to 2019 novel wuhan corona virus", "2019novel sarscov2 severe respiratory infection", "2019 human corona virus acute severe pneumonia", "2019 seafood market coronavirus acute severe respiratory disorder", "2019new sars cov 2 acute disorder", "pneumonia caused by 2019new wuhan seafood market virus", "virus responsible for covid19 acute severe respiratory infection", "2019 new sarscov2 severe acute illness", "2019 novel wuhancorona virus severe respiratory disease", "pneumonia due to 2019new wuhancorona virus", "disorder from 2019novel sars cov 2", "wuhan seafood market virus acute disease", "2019 new corona virus severe acute respiratory disorder", "2019 wuhan seafood market virus pneumonia", "wuhan pneumonia virus severe acute respiratory illness", "2019 new wuhancorona virus respiratory infection", "wuhan seafood market pneumonia virus 2019 illness", "wuhan virus 2019 acute syndrome", "disorder due to 2019 severe acute respiratory syndrome coronavirus2", "disease caused by 2019new severe acute respiratory syndrome coronavirus2", "wuhan seafood market virus 2019 acute pneumonia", "2019 new wuhan virus acute severe pneumonia", "2019new sars corona virus2", "2019 wuhan pneumonia virus respiratory infection", "syndrome caused by 2019new chinese corona virus", "2019 chinese corona virus respiratory infection", "2019 new wuhan coronavirus respiratory illness", "illness from wuhancoronavirus 2019", "2019 new wuhan pneumonia virus acute respiratory infection", "2019 seafood market coronavirus acute severe disorder", "2019new severe acute respiratory syndromerelated coronavirus2 disease", "syndrome caused by 2019new seafood market corona virus", "2019 novel seafood market corona virus acute illness", "illness caused by 2019 novel sars corona virus2", "syndrome due to 2019novel sars coronavirus2", "illness due to 2019novel seafood market virus", "2019 new wuhan seafood market virus severe pneumonia", "2019new wuhancorona virus severe disease", "seafood market corona virus illness", "2019novel sars coronavirus2 severe acute syndrome", "2019 new sars cov 2 acute severe illness", "2019 wuhan corona virus severe acute respiratory infection", "wuhan seafood market pneumonia virus severe acute respiratory infection", "illness from human corona virus 2019", "disease caused by sars corona virus2", "2019 wuhancoronavirus respiratory disorder", "2019 sarscov2 severe acute respiratory infection", "infection caused by 2019 wuhancoronavirus", "2019new sars corona virus 2 severe respiratory disease", "2019new wuhan seafood market pneumonia virus acute syndrome", "sars corona virus 2 respiratory disorder", "2019 novel seafood market virus infection", "2019novel sars corona virus 2 acute respiratory illness", "sarscov2 2019 respiratory illness", "wuhan coronavirus acute severe respiratory infection", "disorder from sars coronavirus 2", "seafood market coronavirus 2019 severe acute respiratory illness", "illness due to virus responsible for covid19", "2019novel sarscov2 severe pneumonia", "2019 novel sars cov 2 severe respiratory disease", "infection from wuhan corona virus 2019", "syndrome caused by 2019 sars cov 2", "wuhan pneumonia virus severe acute disease", "2019 wuhan seafood market virus respiratory infection", "2019 sarscov2 severe respiratory disease", "hubei disease", "seafood market corona virus acute respiratory illness", "wuhan seafood market virus 2019 severe respiratory disorder", "disorder due to 2019new sars corona virus 2", "illness from 2019 new corona virus", "wuhan coronavirus 2019 acute respiratory disease", "2019novel wuhan virus severe respiratory infection", "2019novel sars cov 2 acute respiratory disease", "infection from 2019 new sars coronavirus2", "2019new seafood market virus respiratory disorder", "2019 human coronavirus acute severe pneumonia", "syndrome from wuhan corona virus 2019", "sars coronavirus2 acute severe illness", "2019new seafood market virus severe acute disorder", "2019 new wuhan virus severe respiratory infection", "syndrome due to 2019ncov", "2019 novel wuhan seafood market virus acute respiratory illness", "infection caused by 2019 severe acute respiratory syndrome corona virus 2", "2019 novel chinese corona virus severe acute pneumonia", "sars coronavirus 2 pneumonia", "2019novel sars corona virus2 infection", "2019 new coronavirus acute severe respiratory infection", "syndrome caused by wuhan seafood market pneumonia virus", "wuhancorona virus 2019 severe respiratory illness", "2019 novel chinese coronavirus severe acute infection", "disorder caused by 2019novel sars coronavirus 2", "sars coronavirus2 2019 acute severe syndrome", "infection due to wuhan coronavirus 2019", "infection from 2019novel severe acute respiratory syndromerelated coronavirus2", "2019 sars coronavirus 2 acute severe disorder", "sarscov2 acute severe respiratory infection", "pneumonia from 2019novel severe acute respiratory syndrome coronavirus 2", "2019 new wuhan virus severe acute respiratory disorder", "2019new wuhan virus severe acute disorder", "2019novel sarscov2 severe acute illness", "disorder from severe acute respiratory syndromerelated corona virus2", "2019 new severe acute respiratory syndrome coronavirus2", "2019 wuhan pneumonia virus acute respiratory syndrome", "syndrome due to virus responsible for covid19", "sars coronavirus2 2019 acute severe pneumonia", "seafood market virus disease", "infection caused by 2019novel wuhan pneumonia virus", "wuhan seafood market pneumonia virus 2019 acute disorder", "2019 novel chinese corona virus respiratory syndrome", "2019novel seafood market virus infection", "syndrome due to new corona virus", "2019 new sars cov 2 acute disease", "sars coronavirus2", "novel coronavirus 2019 severe infection", "new corona virus 2019 acute respiratory disease", "disease from new corona virus 2019", "disease caused by 2019 severe acute respiratory syndrome corona virus2", "2019 wuhan pneumonia virus acute severe respiratory disease", "infection caused by sars corona virus2", "2019 sars corona virus 2 acute severe pneumonia", "wuhan pneumonia virus respiratory disorder", "seafood market coronavirus 2019 acute severe respiratory infection", "2019 new sars corona virus 2 asevere acute respiratory disease", "wuhan seafood market pneumonia virus 2019 severe syndrome", "2019 new chinese corona virus acute infection", "2019novel sars cov 2 severe respiratory disorder", "2019 corona virus acute severe illness", "wuhancorona virus severe acute respiratory syndrome", "2019 novel sars coronavirus2 severe acute respiratory infection", "infection caused by severe acute respiratory syndromerelated corona virus2 2019", "2019new wuhancorona virus acute respiratory disease", "2019novel seafood market virus acute respiratory disease", "2019new wuhan coronavirus acute respiratory illness", "2019new seafood market coronavirus acute severe disease", "severe acute respiratory syndromerelated coronavirus2 illness", "virus causing covid19 acute respiratory disorder", "wuhancorona virus 2019 acute respiratory disease", "2019 sarscov2 acute severe pneumonia", "disease due to 2019 novel chinese coronavirus", "2019 wuhan coronavirus acute syndrome", "covid19 virus severe respiratory infection", "2019 wuhan corona virus severe infection", "pneumonia caused by 2019 sars corona virus 2", "illness from severe acute respiratory syndromerelated corona virus2", "2019 new wuhancorona virus acute severe syndrome", "2019 new sars corona virus 2 syndrome", "2019novel wuhancoronavirus acute respiratory illness", "2019new wuhan virus severe acute illness", "new corona virus acute severe pneumonia", "2019novel seafood market coronavirus acute severe infection", "syndrome from 2019new wuhancorona virus", "pneumonia from wuhancoronavirus 2019", "illness caused by wuhancoronavirus 2019", "pneumonia from chinese corona virus 2019", "disorder caused by 2019new wuhan seafood market virus", "2019 new wuhan virus", "syndrome caused by 2019 novel seafood market virus", "disease due to 2019 sars corona virus2", "disease from chinese corona virus", "pneumonia from 2019 novel wuhan seafood market pneumonia virus", "wuhan pneumonia virus infection", "wuhan pneumonia virus respiratory syndrome", "disease caused by 2019 coronavirus", "2019 novel wuhan seafood market pneumonia virus syndrome", "illness caused by severe acute respiratory syndromerelated coronavirus2", "hubei syndrome", "wuhancorona virus 2019 severe acute respiratory infection", "2019 new wuhan seafood market pneumonia virus acute respiratory disorder", "sars corona virus 2 2019 severe acute disorder", "2019 novel chinese corona virus acute illness", "2019 wuhan seafood market pneumonia virus respiratory disease", "disease due to 2019 novel seafood market corona virus", "2019 sars coronavirus2 severe acute infection", "infection from sars cov 2 2019", "2019 novel chinese corona virus illness", "illness from 2019novel severe acute respiratory syndrome coronavirus 2", "seafood market coronavirus severe acute infection", "2019 novel sars coronavirus 2 severe pneumonia", "2019novel wuhan corona virus severe respiratory disease", "2019novel wuhan corona virus acute severe respiratory disease", "infection due to 2019new severe acute respiratory syndromerelated corona virus 2", "syndrome due to 2019new sars coronavirus2", "illness from severe acute respiratory syndrome coronavirus 2", "2019 wuhan corona virus acute severe syndrome", "2019 new sars coronavirus 2 acute severe respiratory syndrome", "covid 19 virus acute severe respiratory disorder", "severe acute respiratory syndromerelated corona virus2 pneumonia", "2019 novel chinese corona virus acute respiratory illness", "virus responsible for covid19 severe respiratory illness", "2019novel chinese corona virus acute severe illness", "wuhan coronavirus 2019 acute severe respiratory disorder", "2019novel sars coronavirus 2 severe acute respiratory infection", "2019novel sars corona virus 2 severe infection", "disease caused by 2019novel wuhancoronavirus", "infection due to 2019new severe acute respiratory syndrome corona virus2", "covid19 virus acute syndrome", "syndrome due to 2019novel seafood market coronavirus", "new coronavirus 2019 severe disease", "pneumonia due to wuhan coronavirus", "wuhancoronavirus 2019 acute illness", "2019 human corona virus disorder", "infection due to 2019 new sars coronavirus2", "2019 wuhan virus severe infection", "2019 new wuhan pneumonia virus disorder", "2019 new chinese coronavirus respiratory illness", "disorder due to 2019 wuhancorona virus", "hubei acute severe respiratory syndrome", "2019new wuhan seafood market pneumonia virus acute severe syndrome", "2019new seafood market coronavirus severe disease", "2019new seafood market corona virus acute respiratory syndrome", "2019novel seafood market corona virus respiratory syndrome", "hubei severe acute disease", "sarscov2 acute severe respiratory disorder", "disease caused by 2019novel wuhan seafood market virus", "2019 sarscov2 pneumonia", "disorder due to 2019new severe acute respiratory syndromerelated coronavirus 2", "sars cov 2 2019 acute disease", "wuhan seafood market pneumonia virus 2019 respiratory illness", "2019 new sars coronavirus 2 severe acute respiratory illness", "illness from 2019novel wuhancorona virus", "seafood market corona virus 2019 respiratory infection", "2019new wuhancoronavirus respiratory disorder", "2019new wuhan coronavirus acute severe respiratory syndrome", "disease from novel corona virus", "illness due to seafood market corona virus", "wuhan seafood market virus severe acute disease", "disorder caused by severe acute respiratory syndromerelated corona virus 2 2019", "illness due to 2019ncov", "2019 chinese coronavirus severe disease", "disease from 2019novel severe acute respiratory syndromerelated coronavirus 2", "syndrome from 2019novel sars corona virus2", "2019 new wuhan seafood market virus acute severe disorder", "2019 new coronavirus pneumonia", "2019novel sars coronavirus 2 acute severe syndrome", "2019 novel wuhan coronavirus", "2019new sars corona virus 2 severe respiratory infection", "infection caused by 2019new severe acute respiratory syndromerelated coronavirus2", "infection caused by wuhancorona virus 2019", "sarscov2 illness", "covid19 virus infection", "2019 coronavirus severe acute infection", "2019 new severe acute respiratory syndromerelated corona virus 2 pneumonia", "syndrome from 2019new wuhan virus", "2019 new sars cov 2 severe respiratory syndrome", "human coronavirus 2019 acute severe pneumonia", "2019 new wuhan virus illness", "2019 new severe acute respiratory syndromerelated corona virus2 illness", "2019novel sars corona virus2 severe illness", "2019new sarscov2 respiratory disease", "2019 coronavirus acute respiratory disease", "disorder from 2019 novel wuhan pneumonia virus", "2019novel chinese coronavirus severe illness", "disease caused by hcov 19", "seafood market coronavirus 2019 acute infection", "covid 19 virus severe disorder", "disease from wuhan virus", "2019novel wuhan seafood market virus acute infection", "2019 new severe acute respiratory syndrome corona virus 2", "2019new sarscov2", "2019 new seafood market corona virus severe acute syndrome", "pneumonia from hcov19", "sars corona virus2 2019 disorder", "2019novel sars corona virus2 acute respiratory illness", "2019new wuhan seafood market pneumonia virus severe respiratory disorder", "2019 novel wuhan corona virus infection", "2019 new wuhan pneumonia virus severe respiratory illness", "2019 new chinese corona virus severe acute illness", "infection due to 2019 novel wuhan virus", "novel coronavirus acute severe respiratory syndrome", "2019new severe acute respiratory syndrome corona virus 2 illness", "illness caused by covid 19 virus", "infection due to 2019new wuhan corona virus", "disease due to 2019 severe acute respiratory syndromerelated coronavirus 2", "2019 sars cov 2 respiratory disease", "sars coronavirus 2 2019 respiratory disease", "2019 wuhan corona virus acute disorder", "2019 new wuhancoronavirus severe acute disease", "2019 new wuhan virus disorder", "2019new wuhan seafood market pneumonia virus acute disorder", "2019 novel wuhancoronavirus severe respiratory syndrome", "wuhan seafood market virus 2019 severe disorder", "illness due to 2019 new chinese corona virus", "2019 novel chinese coronavirus acute severe disease", "syndrome due to 2019novel sars coronavirus 2", "2019new wuhancorona virus acute severe respiratory infection", "2019new sars coronavirus 2 acute respiratory syndrome", "2019new wuhancoronavirus disorder", "hcov 19 severe disease", "seafood market virus acute severe respiratory disorder", "2019 sars coronavirus 2 acute severe respiratory disorder", "disease caused by new coronavirus 2019", "new coronavirus acute severe disease", "disorder caused by 2019 novel severe acute respiratory syndromerelated corona virus 2", "2019 novel wuhan coronavirus acute severe respiratory illness", "2019new sarscov2 disease", "2019 new chinese corona virus disorder", "illness due to new coronavirus", "2019 human corona virus severe respiratory disease", "new corona virus infection", "chinese corona virus 2019 acute severe respiratory disorder", "syndrome caused by 2019new sars corona virus2", "covid19 virus acute disease", "disease due to 2019 new sars corona virus 2", "2019 novel severe acute respiratory syndromerelated coronavirus 2 illness", "pneumonia caused by 2019 chinese coronavirus", "infection due to sarscov2", "2019 new chinese corona virus severe disorder", "novel corona virus 2019 severe respiratory illness", "disorder caused by 2019new sarscov2", "2019novel chinese corona virus severe infection", "wuhancoronavirus 2019 respiratory disease", "disorder from 2019 novel severe acute respiratory syndromerelated coronavirus 2", "wuhan pneumonia virus 2019 acute infection", "disease due to hcov 19", "2019 coronavirus severe respiratory syndrome", "chinese corona virus 2019 severe acute respiratory illness", "2019 new sars cov 2 infection", "sarscov2 respiratory syndrome", "2019new chinese coronavirus severe respiratory disease", "hubei severe respiratory syndrome", "disorder from 2019novel severe acute respiratory syndrome corona virus 2", "covid19 virus severe acute disorder", "2019 wuhancoronavirus severe illness", "novel coronavirus 2019 severe acute illness", "2019 new seafood market coronavirus acute infection", "sars cov 2 2019 acute severe pneumonia", "2019 novel wuhan seafood market virus illness", "wuhan corona virus 2019 acute disease", "2019new seafood market coronavirus acute disease", "new corona virus 2019 severe acute respiratory infection", "wuhan corona virus 2019 severe disease", "infection due to wuhancoronavirus", "2019 wuhan seafood market virus severe acute respiratory illness", "sarscov2 2019 pneumonia", "wuhan pneumonia virus 2019 acute severe infection", "pneumonia from 2019 new wuhancoronavirus", "2019 new wuhan virus severe respiratory illness", "2019 new wuhan coronavirus severe acute infection", "infection due to 2019new sars coronavirus 2", "infection due to 2019novel severe acute respiratory syndromerelated corona virus 2", "2019 new severe acute respiratory syndromerelated coronavirus 2", "2019novel sars coronavirus 2 syndrome", "sars corona virus 2 severe acute infection", "wuhan pneumonia virus acute severe respiratory disorder", "chinese corona virus severe disorder", "2019 seafood market virus respiratory illness", "2019 human coronavirus severe acute respiratory infection", "seafood market coronavirus 2019 severe acute respiratory syndrome", "wuhancorona virus illness", "pneumonia caused by 2019novel sars cov 2", "2019new sars corona virus 2 acute severe respiratory disease", "seafood market virus severe acute disorder", "2019 novel sars corona virus 2 severe infection", "2019 new chinese coronavirus respiratory disease", "infection due to novel coronavirus 2019", "disease from 2019novel chinese corona virus", "2019 new seafood market virus acute severe respiratory disorder", "2019novel seafood market coronavirus respiratory disease", "2019novel wuhancoronavirus severe acute respiratory disorder", "disorder from 2019 corona virus", "2019novel wuhan seafood market pneumonia virus severe acute illness", "2019new seafood market virus severe acute disease", "seafood market severe respiratory illness", "wuhan corona virus disease", "infection caused by 2019 new sars corona virus2", "illness from 2019 new sars coronavirus2", "sars corona virus 2 severe respiratory disorder", "sars coronavirus2 2019 respiratory illness", "2019 novel coronavirus acute severe respiratory infection", "2019 new wuhancoronavirus severe disease", "2019 new sars coronavirus2 acute respiratory syndrome", "2019 novel seafood market virus respiratory syndrome", "2019 seafood market virus severe respiratory infection", "2019 novel coronavirus severe respiratory disorder", "disease caused by severe acute respiratory syndrome corona virus2", "2019 new wuhan seafood market virus acute illness", "pneumonia caused by 2019novel seafood market virus", "infection from 2019 novel chinese corona virus", "2019 novel wuhan virus severe acute respiratory disorder", "infection due to severe acute respiratory syndromerelated coronavirus2 2019", "2019 new wuhan corona virus acute severe infection", "chinese corona virus acute disease", "2019 new sars coronavirus 2 severe acute disorder", "2019 chinese coronavirus severe infection", "wuhan corona virus acute severe respiratory disease", "new coronavirus 2019 acute syndrome", "2019 novel sars coronavirus 2 respiratory disorder", "2019new chinese corona virus", "2019novel severe acute respiratory syndromerelated coronavirus2 infection", "2019ncov acute respiratory illness", "2019 novel wuhan virus pneumonia", "2019 novel wuhan corona virus pneumonia", "covid 19 virus acute pneumonia", "pneumonia caused by 2019 novel wuhan pneumonia virus", "2019novel sars coronavirus 2 acute severe respiratory illness", "2019novel sars cov 2 severe acute disorder", "disease due to 2019novel severe acute respiratory syndromerelated corona virus 2", "2019 new sars corona virus 2 acute pneumonia", "pneumonia due to sars cov 2", "2019 sars corona virus2 severe disorder", "2019 novel seafood market coronavirus severe disease", "2019 sarscov2 respiratory syndrome", "2019 wuhan seafood market pneumonia virus severe pneumonia", "2019 human coronavirus acute severe syndrome", "2019 new wuhancoronavirus severe acute syndrome", "2019 new wuhan seafood market virus acute severe infection", "infection caused by 2019 human coronavirus", "disorder from 2019novel severe acute respiratory syndrome coronavirus 2", "wuhan virus 2019 severe acute respiratory syndrome", "2019 new wuhancoronavirus acute severe illness", "2019 new sars coronavirus2 acute respiratory illness", "2019new severe acute respiratory syndromerelated corona virus2 illness", "syndrome from seafood market coronavirus", "corona virus disease 2019", "disorder due to sars coronavirus2", "2019new sars corona virus 2 disease", "2019 seafood market virus acute severe respiratory disorder", "2019new wuhan seafood market pneumonia virus acute illness", "2019 new chinese corona virus severe acute infection", "2019 novel wuhan coronavirus acute severe respiratory syndrome", "2019novel wuhan coronavirus acute respiratory disorder", "novel coronavirus acute severe respiratory disorder", "2019new sars corona virus 2 infection", "2019 human coronavirus severe acute respiratory syndrome", "2019 wuhan corona virus infection", "disease due to sarscov2", "illness caused by 2019 new sars cov 2", "2019 wuhancoronavirus severe syndrome", "2019novel sars cov 2 acute severe illness", "illness caused by 2019new chinese corona virus", "hcov 19 acute severe respiratory illness", "wuhan coronavirus 2019 acute pneumonia", "wuhan acute pneumonia", "wuhancorona virus respiratory disorder", "2019 novel sars corona virus2 acute severe respiratory infection", "syndrome from 2019 sars cov 2", "wuhancorona virus severe respiratory syndrome", "wuhan seafood market pneumonia virus 2019 severe acute disorder", "human corona virus 2019 severe acute disorder", "sarscov2 2019 respiratory disorder", "2019 novel chinese coronavirus severe acute illness", "2019 coronavirus severe respiratory disorder", "syndrome from 2019novel wuhancoronavirus", "covid 19 virus severe illness", "disease due to wuhan coronavirus 2019", "2019new wuhan corona virus acute severe syndrome", "illness caused by 2019 novel chinese coronavirus", "2019new wuhan seafood market pneumonia virus acute severe respiratory infection", "2019 novel seafood market coronavirus severe acute respiratory illness", "covid 19 virus disease", "2019 seafood market corona virus severe acute respiratory disorder", "pneumonia caused by 2019new severe acute respiratory syndromerelated coronavirus 2", "disorder from 2019novel wuhan corona virus", "2019 new seafood market virus severe infection", "2019new sarscov2 severe respiratory disease", "novel coronavirus severe acute infection", "novel coronavirus severe acute disorder", "human corona virus 2019 acute severe disorder", "infection from 2019novel wuhan seafood market pneumonia virus", "2019 new wuhancorona virus severe acute disorder", "2019 novel seafood market corona virus acute syndrome", "infection due to 2019 severe acute respiratory syndromerelated coronavirus 2", "2019 human corona virus acute severe respiratory disease", "infection from 2019 new wuhan seafood market virus", "2019 novel corona virus severe illness", "sars coronavirus 2 severe respiratory disorder", "syndrome due to hcov19", "sars corona virus2 severe respiratory syndrome", "2019 new sarscov2 respiratory infection", "2019 new sars coronavirus 2 acute infection", "2019 ncov severe acute disease", "2019 novel coronavirus severe acute respiratory infection", "pneumonia caused by 2019new severe acute respiratory syndromerelated coronavirus2", "sars corona virus2 severe syndrome", "2019new wuhancoronavirus acute severe pneumonia", "2019 new sars coronavirus 2 severe respiratory illness", "2019 novel wuhancorona virus acute severe disease", "infection due to severe acute respiratory syndrome corona virus 2 2019", "wuhancorona virus 2019 severe acute disease", "illness caused by 2019 novel severe acute respiratory syndrome coronavirus2", "2019new sars cov 2 acute disease", "new coronavirus acute syndrome", "disorder caused by 2019 novel corona virus", "infection from 2019 new sars corona virus2", "disease due to 2019 coronavirus", "virus responsible for covid19 infection", "2019 new wuhan virus respiratory syndrome", "2019new wuhan coronavirus acute respiratory disease", "human corona virus 2019 severe acute disease", "disease due to wuhan virus 2019", "2019novel sars corona virus 2 respiratory illness", "sars coronavirus 2 severe acute disorder", "2019 novel wuhancorona virus acute illness", "disease from new corona virus", "2019 seafood market corona virus acute respiratory infection", "2019 sarscov2 acute infection", "2019 novel wuhan coronavirus acute respiratory infection", "sars cov 2 acute severe respiratory syndrome", "syndrome from sars cov 2", "novel corona virus acute syndrome", "human coronavirus 2019 severe acute syndrome", "infection from 2019 novel severe acute respiratory syndrome corona virus2", "2019 novel severe acute respiratory syndromerelated coronavirus 2 infection", "disease from severe acute respiratory syndrome corona virus 2 2019", "illness caused by 2019 novel sars coronavirus 2", "disorder from 2019 new wuhancorona virus", "new coronavirus 2019 acute illness", "2019novel seafood market coronavirus acute pneumonia", "2019 novel wuhan seafood market virus acute respiratory syndrome", "disorder from 2019new wuhancorona virus", "pneumonia from 2019 severe acute respiratory syndrome coronavirus 2", "2019 sars corona virus2 severe acute disorder", "seafood market coronavirus 2019 acute severe respiratory disease", "wuhan seafood market pneumonia virus 2019 disorder", "2019 seafood market virus severe acute respiratory syndrome", "syndrome caused by 2019novel seafood market coronavirus", "novel corona virus 2019 severe acute respiratory disorder", "2019 coronavirus acute respiratory disorder", "2019 new wuhan pneumonia virus severe respiratory disorder", "2019 new sars cov 2", "covid 19 virus severe acute disorder", "disorder caused by 2019 new seafood market coronavirus", "2019ncov severe syndrome", "wuhan corona virus respiratory disorder", "2019 new coronavirus respiratory disease", "wuhan corona virus", "2019 novel chinese corona virus acute severe respiratory disorder", "disorder caused by 2019novel wuhancorona virus", "2019 novel sars corona virus 2 acute severe syndrome", "pneumonia caused by 2019 new sars corona virus2", "hcov19 syndrome", "sars corona virus 2 2019 acute illness", "syndrome caused by 2019 wuhancorona virus", "severe acute respiratory syndrome coronavirus2 disease", "pneumonia from new corona virus", "2019 sars corona virus2 acute pneumonia", "disorder due to covid 19 virus", "2019 sars coronavirus 2 acute severe respiratory illness", "2019 novel sars coronavirus2 severe acute respiratory syndrome", "2019 novel wuhan seafood market virus respiratory syndrome", "2019new wuhan coronavirus acute respiratory infection", "novel coronavirus 2019 severe disease", "illness due to 2019novel wuhan pneumonia virus", "disorder due to wuhancoronavirus", "pneumonia caused by 2019 severe acute respiratory syndromerelated coronavirus2", "2019 novel seafood market coronavirus severe respiratory infection", "2019new sars corona virus 2 acute severe illness", "disorder due to 2019 sars coronavirus 2", "sars coronavirus2 2019 severe disease", "new corona virus acute severe disease", "2019novel wuhancorona virus severe acute infection", "2019 human corona virus severe acute respiratory illness", "disorder from hcov 19", "2019 ncov acute severe respiratory syndrome", "2019 seafood market virus", "illness due to 2019new wuhan pneumonia virus", "illness from 2019novel sars cov 2", "illness caused by virus responsible for covid19", "disease from 2019 new sars corona virus 2", "sars corona virus2 severe acute syndrome", "disease from severe acute respiratory syndrome corona virus 2", "2019new wuhan virus infection", "sars corona virus 2 respiratory infection", "2019new chinese corona virus acute syndrome", "seafood market coronavirus respiratory illness", "2019 sars corona virus2 severe acute pneumonia", "new corona virus 2019 pneumonia", "wuhan coronavirus 2019 disease", "hcov19 acute disease", "2019novel chinese coronavirus acute severe illness", "2019 novel wuhancoronavirus severe acute disease", "2019novel sars cov 2 severe acute pneumonia", "infection due to new coronavirus 2019", "illness from severe acute respiratory syndrome coronavirus 2 2019", "2019 novel seafood market corona virus syndrome", "disorder caused by 2019 novel severe acute respiratory syndromerelated coronavirus2", "pneumonia caused by seafood market coronavirus", "2019new wuhan seafood market pneumonia virus acute severe respiratory disorder", "2019new wuhan seafood market virus severe acute disease", "2019novel sarscov2 respiratory disorder", "wuhan pneumonia virus 2019 severe syndrome", "illness due to 2019 novel sarscov2", "2019new sars coronavirus2", "wuhan respiratory infection", "2019 novel corona virus asevere acute respiratory disease", "wuhan seafood market pneumonia virus severe respiratory disorder", "2019 new wuhancoronavirus acute pneumonia", "2019novel wuhan coronavirus acute respiratory syndrome", "disease caused by 2019new severe acute respiratory syndrome corona virus 2", "sars corona virus 2 2019 respiratory infection", "2019 novel wuhan corona virus syndrome", "2019 sars coronavirus 2 acute infection", "2019 wuhancorona virus respiratory disorder", "wuhan virus 2019 illness", "infection caused by seafood market coronavirus 2019", "disease caused by severe acute respiratory syndromerelated coronavirus 2 2019", "2019new severe acute respiratory syndrome coronavirus 2 disease", "pneumonia from 2019 new severe acute respiratory syndrome coronavirus 2", "wuhan corona virus 2019 acute illness", "2019 sars coronavirus2 severe infection", "wuhancorona virus 2019 acute respiratory infection", "2019 severe acute respiratory syndrome coronavirus2 pneumonia", "2019new chinese corona virus acute severe respiratory disorder", "corona virus disease19", "illness due to 2019 novel seafood market corona virus", "illness from wuhan corona virus", "sars corona virus 2 infection", "infection caused by 2019 human corona virus", "illness from 2019 new severe acute respiratory syndromerelated coronavirus 2", "seafood market coronavirus acute severe respiratory illness", "disease due to severe acute respiratory syndromerelated coronavirus 2 2019", "2019novel wuhan pneumonia virus acute severe disease", "2019 sars coronavirus2 severe acute respiratory syndrome", "2019new seafood market virus severe respiratory disorder", "2019new seafood market coronavirus acute severe illness", "infection caused by 2019novel severe acute respiratory syndromerelated coronavirus 2", "sars corona virus 2 acute severe pneumonia", "2019 sarscov2 acute severe respiratory disease", "2019new wuhancorona virus acute respiratory disorder", "wuhan coronavirus 2019 severe acute syndrome", "2019 new wuhancorona virus syndrome", "2019new wuhancorona virus severe pneumonia", "2019 sars corona virus2 disorder", "2019 novel wuhan seafood market virus severe acute respiratory infection", "2019novel sars coronavirus2 severe disease", "wuhan virus 2019 acute pneumonia", "illness caused by 2019new sars cov 2", "disease caused by severe acute respiratory syndrome corona virus 2", "2019novel wuhan seafood market virus severe acute disease", "2019 new seafood market corona virus severe acute illness", "2019 novel wuhancorona virus severe acute respiratory disorder", "sars cov 2 acute severe disorder", "2019new wuhancoronavirus severe respiratory infection", "2019 novel sars coronavirus 2 infection", "illness from 2019new seafood market corona virus", "wuhan pneumonia virus acute syndrome", "2019novel wuhancorona virus severe acute pneumonia", "illness due to wuhan corona virus", "syndrome from hcov 19", "2019 novel sarscov2 respiratory disease", "2019new chinese coronavirus severe pneumonia", "2019novel wuhan pneumonia virus acute severe pneumonia", "2019novel wuhancorona virus severe syndrome", "illness caused by 2019 sars coronavirus 2", "infection from 2019 novel severe acute respiratory syndrome corona virus 2", "2019novel sars corona virus2 severe acute syndrome", "illness due to 2019 novel wuhan coronavirus", "disorder caused by 2019novel chinese coronavirus", "2019 wuhan coronavirus acute severe syndrome", "disease due to 2019new wuhan seafood market pneumonia virus", "illness caused by 2019 novel severe acute respiratory syndromerelated coronavirus2", "hubei severe respiratory illness", "illness due to 2019novel sars corona virus2", "pneumonia caused by 2019new sarscov2", "2019novel chinese coronavirus asevere acute respiratory disease", "2019 new seafood market corona virus severe acute respiratory disorder", "2019 novel chinese coronavirus acute respiratory infection", "2019 sars corona virus2 acute respiratory disease", "disorder due to 2019 sars coronavirus2", "2019novel severe acute respiratory syndromerelated coronavirus 2 illness", "2019new sars coronavirus 2 severe respiratory syndrome", "2019 novel sars corona virus2 acute infection", "2019 new wuhancorona virus acute illness", "wuhan pneumonia virus 2019", "2019new sars corona virus2 acute severe respiratory syndrome", "2019new sars cov 2 severe pneumonia", "sars coronavirus2 2019 illness", "2019 novel severe acute respiratory syndromerelated corona virus2 pneumonia", "seafood market corona virus 2019 severe acute respiratory infection", "disorder due to 2019new sars coronavirus2", "2019novel chinese corona virus severe respiratory disorder", "2019 human corona virus acute severe illness", "syndrome caused by sars coronavirus2 2019", "disease from 2019novel severe acute respiratory syndromerelated corona virus 2", "2019 novel wuhan coronavirus severe acute respiratory syndrome", "2019novel chinese corona virus respiratory illness", "2019 novel wuhan virus severe acute disease", "2019 wuhan corona virus acute severe respiratory syndrome", "pneumonia from 2019 novel seafood market coronavirus", "wuhan virus severe acute respiratory infection", "infection from 2019 human corona virus", "syndrome from 2019novel wuhan pneumonia virus", "new corona virus 2019 acute severe respiratory infection", "sars coronavirus 2 infection", "2019 new chinese corona virus severe respiratory illness", "illness caused by 2019 novel wuhan seafood market pneumonia virus", "syndrome due to 2019 sars cov 2", "2019novel sars corona virus2 acute illness", "2019 new wuhan corona virus disorder", "wuhan seafood market virus severe acute respiratory infection", "sarscov2 2019 severe syndrome", "seafood market virus 2019 acute pneumonia", "disease caused by 2019new severe acute respiratory syndromerelated coronavirus 2", "pneumonia from 2019 sars coronavirus 2", "2019novel seafood market virus severe illness", "wuhan corona virus 2019 severe respiratory illness", "2019 new seafood market virus acute severe disease", "pneumonia caused by 2019new seafood market corona virus", "2019 corona virus acute severe pneumonia", "chinese coronavirus 2019 acute disorder", "illness due to chinese corona virus 2019", "severe acute respiratory syndromerelated corona virus2 2019 disorder", "covid19 virus severe syndrome", "2019novel sarscov2 severe respiratory disorder", "2019 sars coronavirus 2 respiratory syndrome", "disease caused by 2019novel sars corona virus2", "disorder from 2019 new wuhan coronavirus", "2019new sars coronavirus2 acute respiratory disease", "2019 new corona virus acute illness", "wuhan pneumonia virus respiratory infection", "wuhan acute severe disorder", "2019 novel sars coronavirus2 acute respiratory illness", "2019novel wuhan pneumonia virus severe acute respiratory syndrome", "2019 new chinese corona virus acute disease", "2019novel sars corona virus 2 severe acute disease", "2019 new chinese coronavirus acute pneumonia", "pneumonia caused by 2019 novel severe acute respiratory syndromerelated coronavirus 2", "infection caused by 2019new severe acute respiratory syndromerelated coronavirus 2", "seafood market corona virus severe illness", "sars coronavirus 2 severe infection", "2019novel wuhan pneumonia virus severe acute infection", "2019novel wuhan virus severe respiratory illness", "2019novel sars coronavirus2 respiratory syndrome", "illness due to severe acute respiratory syndrome coronavirus 2 2019", "seafood market severe acute respiratory infection", "2019new chinese coronavirus acute severe infection", "disease from wuhan pneumonia virus", "2019 sars corona virus 2 severe acute pneumonia", "2019new wuhan seafood market virus severe respiratory disease", "disorder from 2019 new seafood market coronavirus", "2019 novel wuhancoronavirus asevere acute respiratory disease", "2019 wuhan virus severe respiratory disorder", "2019new seafood market virus illness", "2019 new sars cov 2 illness", "pneumonia due to 2019 sars corona virus2", "2019novel seafood market coronavirus severe acute respiratory disorder", "disease due to 2019new chinese coronavirus", "wuhan corona virus 2019 severe acute respiratory disorder", "disorder caused by 2019 novel wuhan seafood market pneumonia virus", "illness caused by 2019 novel seafood market virus", "2019 novel sars coronavirus2 severe respiratory disease", "2019 novel sars coronavirus2 acute severe respiratory infection", "pneumonia due to wuhan pneumonia virus 2019", "2019novel wuhan seafood market pneumonia virus acute infection", "new coronavirus 2019 severe pneumonia", "disease due to 2019new seafood market corona virus", "seafood market coronavirus 2019", "infection due to 2019 novel sars coronavirus2", "infection from 2019 wuhan corona virus", "2019 new wuhan seafood market pneumonia virus respiratory disorder", "chinese coronavirus severe acute pneumonia", "hcov19 acute severe pneumonia", "2019novel chinese coronavirus severe respiratory disorder", "hubei severe acute disorder", "2019new sars cov 2 respiratory infection", "illness due to severe acute respiratory syndrome coronavirus2", "novel corona virus 2019 acute severe disease", "2019 sarscov2 acute respiratory disorder", "virus responsible for covid19 acute respiratory disease", "2019new sars coronavirus 2 severe syndrome", "2019 wuhan seafood market pneumonia virus acute severe respiratory disorder", "pneumonia due to 2019 ncov", "seafood market corona virus 2019 severe pneumonia", "2019 new sars corona virus2 acute respiratory infection", "2019 novel wuhan coronavirus acute severe disease", "sars coronavirus2 acute respiratory disorder", "wuhan corona virus severe syndrome", "2019 novel wuhan virus acute severe respiratory disorder", "sars coronavirus 2 2019 acute severe disease", "2019 seafood market coronavirus acute respiratory disorder", "2019 new wuhan corona virus severe pneumonia", "2019new wuhan pneumonia virus infection", "illness from 2019novel sars coronavirus 2", "disorder from 2019novel severe acute respiratory syndromerelated corona virus2", "2019 wuhancorona virus acute respiratory illness", "2019novel chinese corona virus syndrome", "pneumonia caused by severe acute respiratory syndrome coronavirus 2 2019", "new corona virus 2019 acute severe respiratory illness", "2019 new sarscov2 respiratory syndrome", "2019 wuhan corona virus severe pneumonia", "2019 sars corona virus2 severe respiratory disorder", "2019 wuhan pneumonia virus acute severe disorder", "illness caused by wuhan coronavirus", "disease caused by 2019 new severe acute respiratory syndrome corona virus 2", "2019 new wuhancoronavirus acute infection", "severe acute respiratory syndromerelated coronavirus 2 2019 disease", "2019novel seafood market virus severe respiratory disease", "2019new sarscov2 pneumonia", "pneumonia from 2019new sars cov 2", "syndrome caused by sars cov 2", "wuhancoronavirus 2019 acute severe respiratory syndrome", "2019 novel chinese corona virus severe disease", "2019 novel seafood market virus respiratory disease", "2019 new wuhancorona virus severe illness", "2019 novel seafood market coronavirus severe respiratory disease", "2019 novel wuhan pneumonia virus severe acute illness", "disease due to 2019new seafood market virus", "2019novel seafood market virus severe respiratory illness", "wuhan pneumonia virus 2019 severe acute respiratory syndrome", "novel coronavirus 2019 severe acute infection", "2019ncov asevere acute respiratory disease", "2019 new wuhan virus pneumonia", "2019 new chinese coronavirus disorder", "wuhan coronavirus acute severe disorder", "2019 wuhan corona virus severe respiratory syndrome", "pneumonia from 2019novel severe acute respiratory syndrome corona virus2", "hcov 19 acute severe respiratory infection", "2019new seafood market corona virus respiratory infection", "pneumonia from severe acute respiratory syndrome coronavirus 2 2019", "2019 new seafood market coronavirus acute severe respiratory disorder", "2019 new wuhan corona virus severe acute respiratory disorder", "wuhancorona virus 2019 acute disease", "2019 novel chinese corona virus severe acute respiratory illness", "2019 novel wuhan pneumonia virus acute disorder", "chinese corona virus illness", "wuhancoronavirus 2019 severe acute respiratory infection", "2019 sars coronavirus2 asevere acute respiratory disease", "2019 sars corona virus 2 acute severe respiratory illness", "infection from sars corona virus2", "chinese coronavirus severe acute disease", "2019 sars corona virus 2 acute severe respiratory disease", "pneumonia caused by 2019new sars coronavirus 2", "disorder due to wuhan seafood market pneumonia virus 2019", "pneumonia from wuhan seafood market virus 2019", "2019novel chinese corona virus illness", "novel coronavirus asevere acute respiratory disease", "2019 wuhan virus severe illness", "disorder caused by 2019 severe acute respiratory syndrome corona virus 2", "2019new seafood market virus infection", "wuhan virus 2019 disease", "2019 sars corona virus 2 severe acute illness", "2019 novel severe acute respiratory syndrome coronavirus2 pneumonia", "sars cov 2 severe acute respiratory infection", "2019novel wuhancorona virus acute severe disease", "2019 novel wuhan seafood market virus severe respiratory infection", "disease from 2019 novel severe acute respiratory syndrome coronavirus2", "2019 wuhan pneumonia virus acute severe respiratory infection", "2019 novel chinese corona virus severe acute infection", "2019new severe acute respiratory syndromerelated coronavirus2 illness", "syndrome due to 2019new wuhancorona virus", "2019novel wuhan pneumonia virus respiratory syndrome", "syndrome caused by 2019new seafood market coronavirus", "2019 new wuhan pneumonia virus acute illness", "2019 novel severe acute respiratory syndromerelated coronavirus2 illness", "2019 sars coronavirus2 acute severe respiratory infection", "disorder due to wuhan coronavirus 2019", "2019 new seafood market coronavirus acute severe illness", "wuhan virus severe infection", "2019 novel sars corona virus 2 severe acute respiratory syndrome", "2019 new coronavirus acute respiratory illness", "2019new sars corona virus2 severe respiratory illness", "virus causing covid19 severe acute pneumonia", "illness from 2019new seafood market virus", "seafood market coronavirus 2019 asevere acute respiratory disease", "infection from virus causing covid19", "2019 wuhan virus severe pneumonia", "illness due to 2019 novel seafood market coronavirus", "disorder caused by virus responsible for covid19", "2019new wuhan pneumonia virus acute severe disease", "infection due to 2019new wuhan seafood market virus", "2019novel wuhan corona virus disease", "disorder caused by 2019 new wuhan seafood market virus", "pneumonia due to 2019 novel seafood market virus", "2019 new sarscov2 severe acute pneumonia", "2019ncov disease", "2019 new wuhan coronavirus acute severe respiratory disease", "NCBI Taxonomy ID: 2697049", "2019 sars cov 2 acute severe disorder", "illness from 2019 novel seafood market virus", "disease from wuhan seafood market pneumonia virus", "wuhancoronavirus 2019 illness", "2019 novel wuhan virus acute respiratory illness", "infection caused by wuhancoronavirus 2019", "disorder caused by 2019 novel sars coronavirus2", "2019novel chinese corona virus severe acute respiratory disorder", "2019 sars cov 2 acute severe respiratory syndrome", "2019 novel wuhancorona virus", "covid 19 virus acute severe respiratory illness", "2019 novel coronavirus", "sars corona virus2 severe acute respiratory infection", "2019 sars cov 2 respiratory illness", "2019 new sars corona virus 2 severe acute disease", "illness from novel corona virus 2019", "2019 novel wuhan seafood market virus severe respiratory disease", "2019novel wuhan coronavirus illness", "2019novel sars coronavirus 2 acute disorder", "2019 sars cov 2 acute respiratory infection", "infection caused by 2019 new wuhan seafood market pneumonia virus", "disorder due to 2019 new wuhan seafood market virus", "2019 chinese corona virus respiratory illness", "disease from wuhancoronavirus", "wuhan seafood market virus acute severe respiratory syndrome", "2019novel sars corona virus2", "2019new sars coronavirus2 infection", "pneumonia from 2019new seafood market virus", "wuhan acute disease", "novel corona virus 2019 pneumonia", "infection from wuhancorona virus", "2019new wuhan coronavirus severe acute respiratory infection", "disease from 2019 novel severe acute respiratory syndrome corona virus2", "2019ncov respiratory disease", "disease caused by 2019 novel sars cov 2", "2019 new corona virus severe acute illness", "syndrome from new corona virus", "seafood market coronavirus respiratory infection", "wuhan corona virus respiratory syndrome", "covid 19 virus respiratory syndrome", "2019 seafood market corona virus acute respiratory disorder", "2019 sars corona virus 2 severe respiratory illness", "2019new sars coronavirus 2 acute respiratory disease", "infection due to severe acute respiratory syndromerelated coronavirus 2 2019", "infection from 2019 new severe acute respiratory syndromerelated coronavirus 2", "2019 new corona virus severe acute respiratory syndrome", "2019 sars cov 2 severe acute illness", "2019 ncov severe respiratory infection", "new coronavirus 2019 severe acute respiratory syndrome", "2019novel sars corona virus2 acute severe respiratory infection", "infection due to 2019new wuhan virus", "illness from 2019 corona virus", "disease from sars cov 2", "pneumonia caused by 2019 severe acute respiratory syndrome corona virus2", "pneumonia from wuhan seafood market pneumonia virus", "2019 sars cov 2 acute respiratory disease", "2019 novel wuhan corona virus acute severe disorder", "sarscov2 acute severe pneumonia", "2019novel chinese coronavirus pneumonia", "2019ncov acute severe disorder", "pneumonia from 2019new wuhan seafood market pneumonia virus", "disease due to 2019 novel severe acute respiratory syndrome coronavirus 2", "2019new chinese corona virus acute disorder", "2019new sars coronavirus 2 disease", "new corona virus disorder", "2019 novel corona virus pneumonia", "wuhan corona virus 2019 severe acute disease", "illness from wuhan seafood market pneumonia virus", "2019 novel corona virus severe disorder", "pneumonia from 2019 wuhancorona virus", "wuhan seafood market virus severe respiratory disorder", "2019new chinese coronavirus acute disorder", "2019 novel severe acute respiratory syndrome coronavirus2 disorder", "seafood market virus acute respiratory disorder", "2019new wuhan seafood market pneumonia virus acute infection", "2019new sarscov2 illness", "human coronavirus 2019 acute illness", "2019 novel wuhancoronavirus acute disease", "2019 chinese corona virus acute disorder", "pneumonia due to 2019new severe acute respiratory syndromerelated coronavirus2", "illness from 2019 novel severe acute respiratory syndrome corona virus 2", "2019 novel chinese corona virus severe acute syndrome", "infection due to 2019 severe acute respiratory syndromerelated coronavirus2", "chinese coronavirus 2019 syndrome", "pneumonia due to new corona virus 2019", "2019new sars corona virus 2 severe illness", "syndrome caused by 2019new wuhan corona virus", "2019 new corona virus acute respiratory disorder", "disease caused by 2019 seafood market corona virus", "new corona virus 2019 respiratory disorder", "wuhan pneumonia virus 2019 acute severe pneumonia", "2019new chinese corona virus severe acute illness", "human corona virus 2019 acute severe respiratory infection", "2019new wuhan coronavirus acute severe respiratory illness", "syndrome caused by 2019 new wuhancorona virus", "2019 wuhan coronavirus severe disorder", "2019 sars corona virus 2 acute severe syndrome", "2019new wuhan corona virus severe disease", "2019 sars cov 2 acute respiratory disorder", "2019novel sars corona virus2 acute severe respiratory disease", "disorder due to 2019 novel severe acute respiratory syndromerelated corona virus2", "chinese corona virus 2019 acute severe respiratory syndrome", "infection due to severe acute respiratory syndromerelated corona virus 2 2019", "2019 seafood market corona virus respiratory disorder", "2019 new sars coronavirus2 syndrome", "2019 novel sars coronavirus2 acute severe pneumonia", "2019 new sars coronavirus 2 acute severe disease", "2019 seafood market coronavirus acute respiratory syndrome", "disease from 2019 new corona virus", "2019 new wuhan pneumonia virus severe acute respiratory disorder", "2019 novel corona virus disease", "2019new wuhan seafood market pneumonia virus pneumonia", "illness from 2019 coronavirus", "chinese coronavirus 2019 severe respiratory disease", "2019 wuhan coronavirus severe acute illness", "disorder due to severe acute respiratory syndromerelated coronavirus 2 2019", "seafood market coronavirus 2019 severe respiratory illness", "disease due to 2019new wuhan virus", "pneumonia from 2019novel chinese corona virus", "pneumonia from 2019 novel coronavirus", "2019novel wuhan virus infection", "2019 new chinese corona virus severe illness", "sars corona virus2 2019 severe acute illness", "2019 novel seafood market corona virus severe disease", "2019new sarscov2 severe pneumonia", "2019 novel sars corona virus2 acute severe respiratory syndrome", "illness due to 2019new sars corona virus 2", "2019new chinese corona virus acute severe syndrome", "disease due to 2019 wuhan virus", "wuhan coronavirus 2019 respiratory syndrome", "2019 new wuhancoronavirus acute severe respiratory disorder", "illness from 2019novel sars corona virus2", "pneumonia caused by 2019 new chinese coronavirus", "illness due to 2019 ncov", "2019 wuhan corona virus severe respiratory infection", "infection caused by new corona virus 2019", "2019novel chinese corona virus acute severe pneumonia", "wuhan pneumonia virus 2019 acute respiratory disease", "disease from 2019new sars coronavirus 2", "disease from 2019 novel sars corona virus2", "2019new wuhan virus acute respiratory disorder", "wuhan seafood market pneumonia virus 2019 acute severe respiratory disorder", "pneumonia caused by 2019new wuhan virus", "2019novel wuhancorona virus severe respiratory illness", "illness due to 2019 novel wuhan seafood market virus", "2019 sarscov2 syndrome", "sars cov 2 severe respiratory syndrome", "illness caused by 2019 new sars corona virus2", "2019 chinese coronavirus severe respiratory disorder", "wuhancorona virus 2019 severe respiratory disease", "2019 corona virus acute illness", "2019 wuhan seafood market pneumonia virus asevere acute respiratory disease", "2019novel wuhan virus severe respiratory syndrome", "human corona virus 2019 severe disorder", "virus responsible for covid19 acute severe respiratory syndrome", "2019 new chinese corona virus severe respiratory disorder", "hubei severe acute syndrome", "wuhan coronavirus 2019 acute severe illness", "2019new wuhan seafood market pneumonia virus severe acute disorder", "illness caused by chinese corona virus 2019", "syndrome caused by 2019 new seafood market corona virus", "2019novel sars coronavirus2 severe acute respiratory disorder", "disease due to seafood market virus", "syndrome due to 2019new sars corona virus 2", "2019 wuhan seafood market virus illness", "virus responsible for covid19 acute severe disorder", "2019 new seafood market virus acute severe respiratory illness", "2019 novel corona virus acute severe respiratory syndrome", "2019novel wuhan pneumonia virus severe acute illness", "sars corona virus2 2019 severe acute respiratory illness", "2019 wuhan seafood market pneumonia virus infection", "2019new seafood market corona virus acute severe illness", "wuhan corona virus 2019 infection", "human coronavirus 2019", "illness due to wuhan corona virus 2019", "2019 new seafood market corona virus acute severe respiratory illness", "illness due to severe acute respiratory syndrome corona virus2", "2019 new corona virus severe respiratory disorder", "2019novel seafood market corona virus acute severe respiratory syndrome", "2019 novel seafood market corona virus respiratory disease", "infection due to 2019 novel sars corona virus 2", "illness due to 2019 wuhancoronavirus", "seafood market coronavirus severe disorder", "wuhan pneumonia virus 2019 severe acute syndrome", "infection from 2019 novel wuhan virus", "disease from virus causing covid19", "2019new sarscov2 respiratory disorder", "2019 novel sars corona virus 2 severe respiratory infection", "novel coronavirus acute respiratory infection", "2019 wuhancoronavirus severe respiratory disorder", "2019 wuhan virus disease", "covid 19 virus severe acute syndrome", "new coronavirus acute infection", "2019 new wuhan virus respiratory infection", "new coronavirus acute severe disorder", "syndrome due to 2019 seafood market coronavirus", "seafood market coronavirus acute severe respiratory disease", "sars coronavirus2 2019 acute severe respiratory disease", "seafood market corona virus 2019 acute severe infection", "2019new sars cov 2 acute infection", "disorder from 2019 novel severe acute respiratory syndrome coronavirus 2", "pneumonia due to 2019novel sars coronavirus 2", "2019 new sars cov 2 acute severe respiratory infection", "seafood market corona virus 2019 acute severe respiratory infection", "disease due to 2019 novel severe acute respiratory syndromerelated corona virus2", "disorder due to 2019new wuhan corona virus", "2019 novel sarscov2 severe acute respiratory infection", "2019new seafood market virus severe acute illness", "2019new chinese coronavirus acute severe disease", "2019 wuhan corona virus acute severe disease", "2019new wuhan virus acute infection", "2019 wuhan corona virus severe illness", "seafood market corona virus 2019 acute respiratory syndrome", "2019ncov severe disease", "pneumonia from 2019 ncov", "novel corona virus 2019 acute severe pneumonia", "2019ncov severe pneumonia", "infection due to 2019novel wuhan coronavirus", "infection due to 2019 wuhan pneumonia virus", "virus responsible for covid19", "infection due to sars cov 2 2019", "2019 chinese corona virus severe acute disorder", "2019 novel sars corona virus 2 acute severe infection", "wuhan coronavirus infection", "pneumonia due to new corona virus", "novel coronavirus acute severe respiratory illness", "2019 new sars coronavirus2 severe acute respiratory illness", "2019 new sars cov 2 severe acute infection", "wuhancorona virus 2019 severe respiratory disorder", "human coronavirus 2019 severe acute respiratory illness", "disease due to 2019 new sars cov 2", "2019 new corona virus", "wuhan seafood market pneumonia virus 2019 respiratory disorder", "2019 new sars corona virus 2 acute severe disease", "sars cov 2 2019 acute severe disorder", "disease due to sars corona virus 2", "sars coronavirus2 severe acute respiratory disorder", "syndrome due to 2019 new wuhancoronavirus", "2019new sars coronavirus2 severe disease", "2019new sarscov2 severe disorder", "seafood market acute severe respiratory disorder", "syndrome caused by 2019new wuhan seafood market virus", "2019 new severe acute respiratory syndrome coronavirus 2 infection", "2019 new severe acute respiratory syndrome corona virus 2 disease", "infection due to 2019 novel seafood market virus", "2019 novel coronavirus disease", "illness from 2019 new seafood market virus", "disorder from 2019new chinese corona virus", "syndrome from human corona virus 2019", "disorder from 2019novel wuhan pneumonia virus", "illness caused by 2019 novel wuhan corona virus", "2019ncov acute respiratory disease", "seafood market corona virus acute severe respiratory syndrome", "2019 new coronavirus severe acute disease", "2019 novel coronavirus severe acute respiratory disorder", "illness caused by novel corona virus", "illness from 2019 novel sars cov 2", "disease from 2019novel wuhan seafood market pneumonia virus", "2019 ncov acute severe respiratory infection", "covid19 virus severe acute respiratory disorder", "2019novel wuhan pneumonia virus acute respiratory syndrome", "hubei severe respiratory disease", "2019 sarscov2 severe acute disease", "2019 chinese corona virus severe respiratory disease", "pneumonia caused by 2019 sars cov 2", "sarscov2 2019 acute severe respiratory infection", "new corona virus acute pneumonia", "2019new wuhan coronavirus severe respiratory disorder", "disorder from 2019 severe acute respiratory syndrome corona virus2", "2019 novel seafood market corona virus acute disease", "chinese coronavirus severe acute respiratory illness", "infection from sars coronavirus 2", "2019 human coronavirus severe pneumonia", "wuhan virus 2019 severe acute syndrome", "2019 new corona virus acute severe illness", "2019 wuhan virus severe respiratory syndrome", "2019 novel wuhancoronavirus severe respiratory disorder", "syndrome caused by 2019 seafood market virus", "wuhan pneumonia virus 2019 disorder", "wuhan pneumonia virus 2019 severe acute illness", "2019new chinese coronavirus acute severe pneumonia", "covid 19 virus severe acute respiratory infection", "2019 new sars corona virus2 acute severe disease", "wuhan virus acute severe disorder", "2019 new corona virus respiratory disease", "seafood market corona virus 2019 acute severe disorder", "disorder caused by 2019 wuhan seafood market virus", "sarscov2 severe respiratory disorder", "2019new wuhancorona virus acute severe respiratory disorder", "wuhan pneumonia virus disease", "new coronavirus 2019 severe acute respiratory illness", "seafood market coronavirus acute syndrome", "2019novel wuhancorona virus severe illness", "2019new sars corona virus2 acute severe disease", "disease caused by 2019 novel wuhan pneumonia virus", "2019novel wuhancoronavirus severe acute pneumonia", "virus responsible for covid19 acute respiratory illness", "virus responsible for covid19 acute disorder", "wuhan virus 2019 severe respiratory illness", "syndrome caused by 2019 sars corona virus2", "2019novel seafood market corona virus severe respiratory disease", "novel corona virus acute disorder", "wuhan seafood market pneumonia virus severe illness", "2019 coronavirus syndrome", "seafood market syndrome", "2019 wuhancoronavirus severe acute disease", "2019 novel wuhan virus acute respiratory disorder", "infection caused by 2019 new sars cov 2", "human coronavirus 2019 severe syndrome", "infection from wuhan coronavirus 2019", "2019 seafood market virus respiratory disease", "2019 novel wuhan coronavirus acute disease", "hcov19 acute severe disease", "seafood market virus 2019 severe acute disorder", "wuhan pneumonia virus acute respiratory disorder", "seafood market virus acute severe respiratory syndrome", "disease caused by 2019 novel sars corona virus 2", "disorder caused by wuhan coronavirus", "disease from 2019 novel chinese corona virus", "2019 wuhan virus severe acute pneumonia", "2019novel seafood market coronavirus acute severe respiratory disorder", "2019 sarscov2 acute pneumonia", "pneumonia from virus responsible for covid19", "disease caused by 2019novel chinese coronavirus", "illness due to new corona virus 2019", "2019new sars coronavirus 2 acute severe respiratory infection", "2019novel wuhan seafood market pneumonia virus illness", "disease from sars coronavirus2 2019", "disorder caused by 2019 sars coronavirus2", "2019 novel wuhancoronavirus respiratory syndrome", "2019novel seafood market corona virus acute severe respiratory illness", "covid19 virus acute disorder", "hcov 19 acute severe illness", "pneumonia from 2019 novel sars coronavirus 2", "disorder from 2019new sarscov2", "2019novel severe acute respiratory syndrome coronavirus2", "2019 sarscov2 severe respiratory illness", "2019 new sars coronavirus2 acute severe infection", "hubei acute severe disorder", "wuhan virus 2019 severe acute disease", "syndrome due to wuhan coronavirus 2019", "novel corona virus severe acute disease", "virus responsible for covid19 syndrome", "2019 new corona virus acute syndrome", "wuhancoronavirus acute severe respiratory disorder", "2019 novel seafood market corona virus acute severe illness", "pneumonia caused by 2019new wuhan coronavirus", "pneumonia due to 2019new wuhan seafood market pneumonia virus", "2019novel wuhan coronavirus acute disease", "2019novel sars corona virus2 acute syndrome", "novel corona virus severe acute respiratory infection", "2019novel seafood market virus syndrome", "disorder from 2019 wuhan seafood market virus", "2019 novel wuhan virus severe acute syndrome", "wuhan seafood market pneumonia virus 2019 severe illness", "covid 19 virus severe acute pneumonia", "2019new wuhan virus acute respiratory syndrome", "wuhan coronavirus disorder", "disorder from 2019 severe acute respiratory syndromerelated corona virus2", "2019novel wuhan seafood market pneumonia virus severe disease", "2019 sars corona virus 2 respiratory infection", "2019novel wuhan virus acute disorder", "disorder due to 2019 sars corona virus 2", "pneumonia due to 2019 novel corona virus", "illness due to 2019new severe acute respiratory syndromerelated coronavirus 2", "disease due to 2019novel wuhan seafood market virus", "2019 novel wuhan seafood market virus acute severe respiratory disorder", "pneumonia due to wuhancoronavirus 2019", "pneumonia due to virus responsible for covid19", "syndrome from 2019 ncov", "new corona virus 2019 acute severe infection", "illness caused by severe acute respiratory syndromerelated corona virus2", "2019 wuhan seafood market pneumonia virus severe syndrome", "wuhan pneumonia virus severe acute infection", "illness from 2019 severe acute respiratory syndromerelated corona virus 2", "2019 chinese coronavirus respiratory illness", "2019new wuhan virus syndrome", "2019new wuhan seafood market virus respiratory infection", "pneumonia due to 2019new severe acute respiratory syndrome corona virus 2", "illness caused by severe acute respiratory syndrome coronavirus 2", "syndrome from 2019 corona virus", "infection due to 2019novel wuhancorona virus", "wuhan seafood market virus 2019 acute severe illness", "illness caused by 2019 novel corona virus", "wuhan pneumonia virus 2019 severe disorder", "disorder caused by 2019novel sars cov 2", "new coronavirus 2019 pneumonia", "illness caused by 2019ncov", "disease caused by 2019 new seafood market coronavirus", "2019 new seafood market corona virus acute infection", "2019novel sars cov 2 acute severe respiratory infection", "2019novel sars coronavirus2 severe respiratory syndrome", "2019 corona virus severe acute infection", "2019 ncov acute respiratory disorder", "2019 novel seafood market virus severe acute respiratory infection", "2019 novel chinese coronavirus illness", "syndrome from 2019 new seafood market corona virus", "2019ncov acute syndrome", "syndrome caused by human coronavirus 2019", "new corona virus pneumonia", "infection from 2019novel wuhan coronavirus", "wuhan seafood market pneumonia virus 2019 acute respiratory disease", "2019 sars coronavirus 2 severe acute disease", "2019new wuhancorona virus severe acute disease", "virus causing covid19 asevere acute respiratory disease", "2019novel wuhan pneumonia virus illness", "2019 wuhan coronavirus severe respiratory syndrome", "2019new chinese corona virus acute respiratory disease", "2019new wuhan seafood market virus acute severe illness", "covid 19 virus acute syndrome", "sars corona virus 2 2019 asevere acute respiratory disease", "infection from 2019 novel severe acute respiratory syndrome coronavirus 2", "2019 novel coronavirus respiratory syndrome", "2019 new sars cov 2 acute severe respiratory syndrome", "2019novel wuhan seafood market virus severe respiratory illness", "2019new wuhan seafood market pneumonia virus respiratory infection", "2019novel sarscov2 severe syndrome", "new coronavirus severe acute respiratory infection", "wuhan virus 2019 severe acute respiratory disorder", "2019new chinese corona virus acute severe respiratory disease", "pneumonia due to 2019new seafood market corona virus", "2019 new severe acute respiratory syndrome coronavirus2 infection", "2019 novel wuhan seafood market virus acute respiratory disease", "disorder due to 2019new wuhancorona virus", "2019 new wuhancorona virus severe respiratory infection", "2019novel wuhancorona virus severe disorder", "2019novel seafood market coronavirus acute severe respiratory infection", "2019 novel wuhan virus severe acute illness", "2019 new sars coronavirus2 severe respiratory illness", "seafood market virus 2019 respiratory syndrome", "illness from sars corona virus2", "chinese corona virus 2019 acute severe illness", "wuhan coronavirus 2019", "2019 human corona virus respiratory disorder", "2019 new seafood market virus acute illness", "2019 new wuhancoronavirus severe acute pneumonia", "severe acute respiratory syndromerelated corona virus2 2019 pneumonia", "pneumonia due to sars corona virus 2 2019", "2019novel sars coronavirus 2 severe acute respiratory illness", "2019novel wuhan corona virus disorder", "2019new wuhan seafood market pneumonia virus illness", "disorder caused by 2019new wuhan virus", "2019 seafood market corona virus acute syndrome", "2019 new sars corona virus2 acute disorder", "sars coronavirus 2 2019 severe respiratory infection", "2019 new sars cov 2 disorder", "2019new sarscov2 acute respiratory syndrome", "2019 new sars coronavirus2 acute severe respiratory infection", "disorder from 2019 severe acute respiratory syndromerelated coronavirus 2", "new coronavirus 2019 infection", "disease caused by 2019new wuhancorona virus", "2019 novel sars corona virus 2 acute severe illness", "new corona virus 2019 acute disease", "wuhan virus acute syndrome", "2019new wuhan pneumonia virus severe respiratory illness", "2019new wuhan virus severe illness", "2019 wuhancorona virus severe acute disorder", "2019 novel chinese corona virus acute severe respiratory disease", "2019 wuhan pneumonia virus acute respiratory disease", "wuhan pneumonia virus 2019 acute severe respiratory disease", "disorder caused by 2019 new chinese coronavirus", "2019 novel seafood market virus acute respiratory infection", "wuhan corona virus acute respiratory illness", "novel corona virus severe acute infection", "2019 novel wuhan pneumonia virus respiratory infection", "2019novel seafood market virus severe respiratory disorder", "seafood market corona virus 2019 severe acute respiratory syndrome", "2019 novel severe acute respiratory syndrome coronavirus 2", "2019 wuhan coronavirus respiratory infection", "2019novel wuhan virus acute severe respiratory disease", "2019 novel sars coronavirus 2 acute respiratory disorder", "illness from 2019 new sars cov 2", "2019 new sars coronavirus 2 respiratory infection", "2019novel sars corona virus2 severe acute pneumonia", "2019 novel seafood market coronavirus acute severe respiratory disorder", "wuhan virus 2019 acute severe respiratory illness", "2019 novel severe acute respiratory syndromerelated corona virus2 disease", "sars cov 2 2019 acute pneumonia", "2019 new sars corona virus 2", "2019 wuhan corona virus acute syndrome", "2019 human corona virus severe acute disease", "new corona virus severe pneumonia", "syndrome due to 2019 wuhan seafood market pneumonia virus", "2019new seafood market corona virus severe acute respiratory illness", "sars corona virus2 severe acute respiratory syndrome", "2019 novel sarscov2 respiratory infection", "syndrome caused by 2019 sars coronavirus2", "sars coronavirus 2 acute severe disorder", "infection from seafood market virus 2019", "2019new seafood market corona virus syndrome", "seafood market corona virus 2019 acute severe respiratory disease", "disease due to 2019 novel severe acute respiratory syndrome corona virus 2", "disease due to 2019 wuhan seafood market pneumonia virus", "2019novel seafood market coronavirus acute respiratory illness", "2019 new sarscov2 disorder", "2019 novel sars cov 2 severe infection", "2019 sarscov2 infection", "2019 sarscov2 severe acute syndrome", "2019 sarscov2 severe acute pneumonia", "2019new chinese corona virus respiratory infection", "sarscov2 severe respiratory illness", "wuhan seafood market pneumonia virus 2019 severe acute pneumonia", "2019 novel corona virus severe acute disease", "2019 severe acute respiratory syndromerelated coronavirus2 infection", "2019 novel seafood market corona virus respiratory infection", "hubei acute severe respiratory disorder", "disease from 2019novel sarscov2", "2019 new seafood market virus acute respiratory disease", "2019 sars coronavirus 2 severe acute respiratory syndrome", "infection due to 2019 new severe acute respiratory syndrome corona virus 2", "2019 new wuhan seafood market pneumonia virus respiratory syndrome", "2019new severe acute respiratory syndrome coronavirus2 disease", "disorder due to 2019ncov", "2019 novel sars corona virus 2 acute severe respiratory illness", "hubei acute disease", "2019new wuhancoronavirus severe acute respiratory syndrome", "2019 new seafood market coronavirus severe respiratory disorder", "hcov19 severe acute respiratory infection", "severe acute respiratory syndrome corona virus 2 disorder", "2019 wuhan coronavirus severe acute disease", "pneumonia from 2019 severe acute respiratory syndrome corona virus2", "2019 new wuhan seafood market virus pneumonia", "2019 new wuhan seafood market pneumonia virus severe acute pneumonia", "infection due to 2019novel seafood market coronavirus", "disease due to new corona virus", "2019 new wuhancorona virus respiratory disease", "2019 novel wuhan seafood market pneumonia virus severe acute respiratory illness", "sarscov2 severe infection", "infection caused by novel coronavirus 2019", "2019 new wuhancoronavirus acute severe respiratory infection", "2019novel sars coronavirus 2 respiratory disease", "2019new sars cov 2 severe acute respiratory syndrome", "disorder from 2019 new sars coronavirus 2", "2019novel chinese coronavirus severe respiratory disease", "2019 new wuhancorona virus acute severe respiratory illness", "2019 novel wuhancorona virus disease", "2019 novel seafood market corona virus severe acute respiratory disorder", "syndrome due to 2019novel sarscov2", "wuhan corona virus 2019 severe respiratory infection", "2019new wuhancoronavirus acute severe respiratory illness", "2019 new wuhan seafood market virus severe acute respiratory illness", "2019novel wuhan seafood market virus acute respiratory illness", "2019 new severe acute respiratory syndrome coronavirus2 illness", "novel corona virus 2019 severe acute syndrome", "2019new wuhancoronavirus acute severe respiratory syndrome", "infection due to wuhan corona virus 2019", "sars corona virus 2 disorder", "2019 wuhan seafood market virus acute respiratory infection", "2019 novel severe acute respiratory syndromerelated coronavirus2 infection", "2019 novel wuhan corona virus severe respiratory infection", "covid 19 virus disorder", "illness caused by 2019 new chinese corona virus", "2019 sars corona virus 2 infection", "2019 novel chinese corona virus acute severe respiratory syndrome", "infection from chinese corona virus", "virus causing covid19 respiratory disorder", "2019 new sars corona virus 2 acute respiratory syndrome", "2019new sars coronavirus2 severe pneumonia", "2019new wuhan coronavirus disorder", "2019novel severe acute respiratory syndrome coronavirus 2 infection", "chinese corona virus 2019 acute severe respiratory illness", "wuhancorona virus respiratory infection", "syndrome from 2019 novel chinese corona virus", "2019 new wuhan corona virus acute pneumonia", "disorder from severe acute respiratory syndrome corona virus 2", "sars coronavirus2 disease", "2019 new sars cov 2 acute severe disease", "2019novel sars cov 2 severe illness", "2019 wuhancorona virus severe acute respiratory syndrome", "2019novel wuhan corona virus severe acute illness", "2019new sars coronavirus 2 severe respiratory illness", "pneumonia caused by 2019 wuhan pneumonia virus", "illness from 2019 novel sarscov2", "2019new chinese corona virus acute severe respiratory syndrome", "2019 new wuhan corona virus acute illness", "novel corona virus 2019 acute severe illness", "sarscov2 2019 respiratory infection", "sars coronavirus2 acute disease", "pneumonia caused by 2019 severe acute respiratory syndrome coronavirus 2", "sarscov2 2019", "2019 novel seafood market coronavirus acute severe respiratory illness", "covid 19 virus", "disease due to wuhan seafood market virus", "2019 novel wuhan virus illness", "pneumonia due to 2019novel severe acute respiratory syndromerelated coronavirus 2", "2019novel seafood market coronavirus severe acute pneumonia", "sars corona virus2 acute disease", "2019 novel seafood market virus acute respiratory syndrome", "seafood market virus acute syndrome", "2019novel sars corona virus 2 severe respiratory infection", "2019 novel wuhancorona virus acute infection", "2019 wuhan coronavirus acute severe infection", "2019novel sars corona virus 2 severe syndrome", "infection from severe acute respiratory syndromerelated coronavirus2 2019", "pneumonia caused by 2019new severe acute respiratory syndromerelated corona virus2", "sars coronavirus2 2019 severe acute disease", "wuhan coronavirus severe pneumonia", "2019 severe acute respiratory syndromerelated coronavirus2 disease", "illness from 2019 sarscov2", "2019 novel sars cov 2 severe acute respiratory syndrome", "pneumonia caused by 2019 wuhan corona virus", "novel corona virus 2019 acute severe syndrome", "2019 new sars corona virus 2 severe respiratory syndrome", "seafood market coronavirus 2019 acute illness", "illness caused by 2019novel severe acute respiratory syndrome corona virus2", "hcov19 pneumonia", "illness caused by 2019 new wuhan coronavirus", "infection due to 2019 new corona virus", "wuhan virus 2019", "2019 chinese corona virus disorder", "wuhancoronavirus severe acute respiratory disorder", "seafood market coronavirus severe respiratory infection", "illness from 2019 sars cov 2", "2019new wuhan corona virus acute severe respiratory disease", "seafood market coronavirus severe infection", "wuhan pneumonia virus 2019 severe acute disorder", "disease due to sars corona virus2", "2019new seafood market coronavirus acute pneumonia", "seafood market coronavirus 2019 severe pneumonia", "sars coronavirus2 2019 acute respiratory disorder", "2019 novel wuhan coronavirus severe acute respiratory disorder", "infection caused by wuhan virus 2019", "infection due to 2019 new sars corona virus2", "disease due to 2019 sars coronavirus 2", "2019novel wuhan seafood market virus severe acute respiratory infection", "2019 wuhan pneumonia virus severe respiratory syndrome", "wuhan corona virus 2019 severe acute pneumonia", "2019novel wuhan corona virus acute severe respiratory infection", "2019novel sars cov 2 severe disease", "2019 chinese corona virus acute disease", "2019 novel wuhan pneumonia virus acute severe infection", "syndrome due to new coronavirus 2019", "wuhan seafood market pneumonia virus respiratory disorder", "2019novel chinese corona virus acute severe respiratory infection", "2019 wuhan pneumonia virus acute severe disease", "disease caused by sars cov 2", "new coronavirus acute respiratory syndrome", "2019 novel sars corona virus2 severe infection", "sars cov 2 2019 severe acute disorder", "infection from 2019 sars corona virus2", "2019 wuhan corona virus respiratory syndrome", "2019novel seafood market coronavirus severe respiratory illness", "2019 new seafood market corona virus acute severe infection", "2019 novel seafood market virus severe acute infection", "sarscov2 respiratory infection", "illness due to 2019 new wuhancoronavirus", "2019 severe acute respiratory syndrome coronavirus2", "virus responsible for covid19 severe acute syndrome", "sars corona virus 2 severe infection", "wuhan pneumonia virus severe acute disorder", "2019novel seafood market coronavirus acute severe respiratory illness", "syndrome due to covid19 virus", "2019 wuhancoronavirus asevere acute respiratory disease", "wuhan coronavirus 2019 respiratory disorder", "2019 new wuhan coronavirus acute severe respiratory illness", "2019 ncov acute severe respiratory disease", "2019 new sars coronavirus 2 severe illness", "2019new wuhan coronavirus severe respiratory illness", "2019new wuhan pneumonia virus acute severe illness", "sars corona virus 2 severe acute illness", "novel coronavirus acute severe illness", "illness from wuhan coronavirus", "2019new wuhancorona virus acute respiratory syndrome", "2019new wuhan coronavirus severe pneumonia", "sars corona virus 2 2019 acute pneumonia", "2019novel wuhan corona virus severe acute respiratory syndrome", "disease caused by hcov19", "2019 seafood market virus acute respiratory illness", "2019 novel wuhan pneumonia virus acute severe disease", "hcov19 severe illness", "chinese coronavirus 2019 acute severe respiratory syndrome", "2019 novel wuhan corona virus severe infection", "2019new seafood market coronavirus severe disorder", "2019 novel wuhan seafood market virus severe pneumonia", "2019new sars coronavirus 2 severe acute infection", "seafood market virus severe acute respiratory disorder", "pneumonia due to 2019new wuhancoronavirus", "syndrome from 2019new wuhan corona virus", "2019 sars corona virus 2 acute respiratory disorder", "wuhan coronavirus respiratory syndrome", "2019new sars corona virus 2 acute severe respiratory syndrome", "2019 seafood market corona virus infection", "infection caused by 2019novel wuhan seafood market virus", "illness due to 2019novel wuhan virus", "2019 novel wuhancoronavirus", "2019 ncov severe pneumonia", "disease due to 2019 new severe acute respiratory syndromerelated coronavirus 2", "disease due to severe acute respiratory syndromerelated coronavirus2", "2019novel wuhan virus acute severe syndrome", "2019new sarscov2 severe acute syndrome", "infection caused by sars corona virus 2", "2019new sars cov 2 severe acute pneumonia", "seafood market virus severe disorder", "2019new sars coronavirus 2 acute respiratory infection", "2019 new sars coronavirus2 acute severe illness", "disorder from 2019novel wuhancorona virus", "covid19 virus disease", "novel coronavirus 2019 severe respiratory disease", "infection due to 2019 novel severe acute respiratory syndromerelated corona virus2", "2019 seafood market coronavirus acute disorder", "2019 sars coronavirus 2 severe respiratory disease", "2019 novel sars coronavirus 2 respiratory infection", "disease from 2019 new severe acute respiratory syndromerelated corona virus2", "2019 coronavirus respiratory syndrome", "disorder due to hcov 19", "novel coronavirus acute respiratory disorder", "seafood market virus 2019 severe acute respiratory infection", "covid 19 virus acute respiratory disorder", "pneumonia caused by human corona virus 2019", "pneumonia due to 2019novel severe acute respiratory syndromerelated coronavirus2", "wuhan virus acute severe syndrome", "disease from severe acute respiratory syndromerelated corona virus 2 2019", "wuhan virus severe respiratory disease", "2019new severe acute respiratory syndromerelated coronavirus2 infection", "2019 new sars corona virus 2 respiratory disease", "2019 sars coronavirus 2 acute respiratory infection", "2019 chinese coronavirus acute severe respiratory infection", "sars corona virus 2 2019 severe respiratory illness", "2019 novel sars corona virus 2 illness", "seafood market corona virus 2019 severe acute infection", "2019new wuhan pneumonia virus severe disease", "wuhancoronavirus 2019 severe pneumonia", "chinese coronavirus 2019 acute severe pneumonia", "infection from 2019novel seafood market corona virus", "2019 novel sars corona virus2 severe disease", "sars corona virus 2 severe disorder", "2019 novel coronavirus acute severe respiratory illness", "2019 novel wuhan seafood market pneumonia virus acute syndrome", "disease caused by novel coronavirus", "hcov 19 infection", "2019new wuhan corona virus acute respiratory syndrome", "pneumonia due to 2019novel sars cov 2", "2019 new sarscov2 severe acute disease", "2019 ncov acute infection", "illness due to human coronavirus 2019", "disease from 2019novel severe acute respiratory syndromerelated corona virus2", "pneumonia from severe acute respiratory syndrome corona virus2", "infection due to 2019 new severe acute respiratory syndromerelated corona virus 2", "illness from 2019 novel wuhan pneumonia virus", "illness from 2019 novel severe acute respiratory syndrome coronavirus2", "infection due to 2019 novel severe acute respiratory syndrome coronavirus 2", "2019 new seafood market corona virus disease", "novel coronavirus 2019 severe respiratory syndrome", "human corona virus 2019 asevere acute respiratory disease", "2019novel wuhan seafood market virus acute severe infection", "illness due to 2019new chinese corona virus", "2019 new chinese coronavirus acute respiratory illness", "seafood market coronavirus severe acute pneumonia", "wuhancorona virus severe respiratory disorder", "wuhancoronavirus acute severe disorder", "new coronavirus 2019 acute disease", "disorder due to 2019 wuhan coronavirus", "2019 novel sars coronavirus2 severe acute respiratory disorder", "illness from 2019new chinese corona virus", "2019new wuhan virus acute severe respiratory syndrome", "infection from covid19 virus", "pneumonia from 2019novel severe acute respiratory syndromerelated coronavirus 2", "2019 human corona virus severe acute respiratory infection", "chinese corona virus 2019 acute pneumonia", "disorder due to 2019 chinese coronavirus", "wuhancorona virus 2019 severe pneumonia", "disease due to 2019 novel severe acute respiratory syndromerelated coronavirus2", "wuhan seafood market pneumonia virus 2019 severe respiratory syndrome", "syndrome due to 2019new chinese coronavirus", "pneumonia from 2019new severe acute respiratory syndrome coronavirus 2", "disorder from wuhan coronavirus", "wuhancoronavirus acute respiratory disorder", "seafood market coronavirus 2019 acute severe respiratory illness", "sars corona virus 2 2019 infection", "2019 wuhan pneumonia virus acute disorder", "wuhancorona virus severe acute infection", "novel corona virus 2019 severe respiratory infection", "2019 novel sars coronavirus 2 disorder", "2019 seafood market corona virus severe acute respiratory illness", "2019 novel chinese coronavirus infection", "disease due to 2019new seafood market coronavirus", "wuhancoronavirus 2019 syndrome", "infection from 2019 wuhan pneumonia virus", "pneumonia due to 2019 severe acute respiratory syndrome corona virus2", "2019new sars cov 2 acute respiratory syndrome", "2019 novel sarscov2 severe acute respiratory illness", "wuhan coronavirus 2019 asevere acute respiratory disease", "chinese corona virus 2019 acute respiratory illness", "disorder from 2019 wuhancorona virus", "2019 novel chinese coronavirus respiratory disorder", "2019new wuhancorona virus acute severe disorder", "2019 novel seafood market coronavirus respiratory disease", "disease due to 2019 novel corona virus", "pneumonia from hcov 19", "2019 novel seafood market coronavirus acute illness", "2019new wuhancorona virus severe acute pneumonia", "severe acute respiratory syndromerelated corona virus2 2019", "illness caused by 2019new severe acute respiratory syndromerelated corona virus 2", "chinese coronavirus 2019 acute severe respiratory disorder", "2019 wuhan seafood market pneumonia virus acute respiratory syndrome", "2019 novel wuhan seafood market pneumonia virus acute severe respiratory disease", "disorder from 2019 new severe acute respiratory syndromerelated corona virus2", "2019 novel sars corona virus 2 severe acute infection", "wuhancorona virus 2019 acute severe syndrome", "2019novel sars cov 2 asevere acute respiratory disease", "disorder from novel corona virus 2019", "2019 new sarscov2 acute illness", "2019novel wuhan seafood market virus severe respiratory disease", "syndrome caused by 2019 new seafood market virus", "illness from 2019 novel sars coronavirus 2", "sars corona virus2 severe respiratory infection", "syndrome caused by 2019 novel sars corona virus 2", "2019 sars coronavirus2 acute pneumonia", "syndrome from 2019 seafood market virus", "2019 sars corona virus2 severe syndrome", "severe acute respiratory syndrome coronavirus 2 disease", "wuhan seafood market pneumonia virus acute severe pneumonia", "infection from human coronavirus 2019", "2019 new wuhancorona virus severe respiratory illness", "2019new wuhan pneumonia virus acute severe respiratory disease", "illness from 2019 novel seafood market corona virus", "2019new wuhan pneumonia virus acute severe respiratory infection", "pneumonia caused by 2019new sars corona virus 2", "infection due to wuhan coronavirus", "wuhan seafood market pneumonia virus 2019 severe acute respiratory disorder", "pneumonia due to 2019 seafood market corona virus", "novel corona virus 2019 disease", "2019 sars coronavirus2 severe respiratory syndrome", "wuhan coronavirus 2019 infection", "chinese corona virus acute respiratory syndrome", "2019 new coronavirus severe acute respiratory disorder", "2019 wuhan seafood market virus acute disease", "2019 novel wuhan pneumonia virus severe disease", "infection from 2019 seafood market virus", "sars coronavirus2 acute respiratory infection", "2019 wuhan pneumonia virus severe acute disorder", "wuhan coronavirus 2019 severe respiratory disease", "2019 novel severe acute respiratory syndromerelated corona virus 2 infection", "2019 human corona virus acute pneumonia", "2019novel wuhan pneumonia virus acute infection", "2019 novel seafood market virus disease", "infection caused by covid19 virus", "disorder from 2019 novel chinese corona virus", "virus causing covid19 infection", "virus responsible for covid19 respiratory infection", "seafood market corona virus 2019 acute infection", "2019novel sars corona virus 2 acute respiratory infection", "2019 new chinese corona virus acute severe infection", "2019novel wuhan pneumonia virus acute respiratory disease", "2019 novel sars corona virus 2 severe disease", "2019 seafood market coronavirus severe acute illness", "disease from human coronavirus 2019", "2019novel severe acute respiratory syndromerelated corona virus2 infection", "infection caused by 2019 novel severe acute respiratory syndromerelated coronavirus2", "wuhancoronavirus severe acute pneumonia", "illness caused by wuhancorona virus 2019", "disorder caused by wuhan corona virus 2019", "disease from sars cov 2 2019", "wuhan seafood market virus 2019 acute severe respiratory disorder", "2019 novel wuhancoronavirus severe disorder", "2019 novel wuhan seafood market pneumonia virus severe respiratory syndrome", "coronavirus disease 2019", "2019 novel seafood market coronavirus severe acute syndrome", "wuhancorona virus severe acute disease", "illness from 2019 new wuhan virus", "2019 seafood market virus infection", "disorder caused by human corona virus 2019", "chinese corona virus 2019 severe respiratory syndrome", "2019 new corona virus acute respiratory disease", "wuhan pneumonia virus acute severe respiratory syndrome", "disease caused by 2019 new severe acute respiratory syndromerelated coronavirus2", "infection due to 2019 new wuhan corona virus", "infection from chinese coronavirus 2019", "disorder from 2019new wuhan seafood market pneumonia virus", "sars corona virus 2 acute disease", "2019 new sarscov2 acute severe respiratory disease", "illness from 2019 severe acute respiratory syndrome corona virus2", "disease caused by 2019 novel sars corona virus2", "disorder from 2019 novel coronavirus", "2019 wuhancoronavirus severe respiratory syndrome", "2019 novel wuhan pneumonia virus respiratory syndrome", "2019novel sars corona virus2 severe disorder", "human coronavirus 2019 acute disorder", "2019 new wuhancoronavirus severe disorder", "2019 wuhan seafood market pneumonia virus acute severe disorder", "2019 severe acute respiratory syndrome coronavirus 2 illness", "covid19", "2019 new wuhan corona virus asevere acute respiratory disease", "2019novel wuhan seafood market pneumonia virus respiratory illness", "2019 wuhan seafood market pneumonia virus severe respiratory disorder", "2019 human corona virus", "novel corona virus severe pneumonia", "severe acute respiratory syndrome corona virus 2 infection", "wuhan coronavirus 2019 acute severe respiratory disease", "infection caused by 2019 novel chinese coronavirus", "illness from sars corona virus 2 2019", "pneumonia caused by 2019 seafood market corona virus", "2019new wuhan pneumonia virus", "2019novel seafood market virus severe acute respiratory illness", "syndrome caused by 2019 novel sars cov 2", "hcov19 acute disorder", "wuhan corona virus 2019 respiratory disorder", "infection from 2019novel severe acute respiratory syndromerelated coronavirus 2", "wuhan coronavirus severe acute syndrome", "disease due to novel corona virus 2019", "2019novel sars corona virus 2 acute pneumonia", "disease due to 2019 sars cov 2", "wuhan seafood market virus acute severe illness", "NCBI:txid2697049", "2019 new severe acute respiratory syndrome corona virus 2 illness", "2019 novel sars coronavirus2 severe illness", "2019 new seafood market coronavirus acute disease", "2019 novel sars corona virus 2 severe respiratory disorder", "syndrome due to sars cov 2 2019", "2019new wuhancoronavirus severe acute pneumonia", "syndrome due to new coronavirus", "pneumonia from severe acute respiratory syndromerelated coronavirus2", "2019 new sars corona virus 2 severe pneumonia", "2019new chinese corona virus syndrome", "illness caused by wuhan seafood market virus", "novel coronavirus 2019 respiratory infection", "2019 wuhan pneumonia virus asevere acute respiratory disease", "disorder from 2019new wuhan pneumonia virus", "seafood market coronavirus pneumonia", "hcov 19 severe acute respiratory syndrome", "new coronavirus 2019 acute severe disease", "illness due to 2019 novel severe acute respiratory syndromerelated corona virus 2", "2019 wuhan pneumonia virus acute syndrome", "2019 new sars coronavirus 2 respiratory syndrome", "illness caused by sars coronavirus 2 2019", "2019 seafood market corona virus severe acute infection", "2019new wuhan pneumonia virus severe acute pneumonia", "2019novel wuhan coronavirus severe acute syndrome", "2019 wuhancorona virus severe acute respiratory illness", "covid19 virus acute illness", "disease caused by sars coronavirus 2", "infection caused by sarscov2 2019", "2019new chinese coronavirus respiratory disorder", "syndrome caused by 2019 wuhan coronavirus", "2019 wuhan pneumonia virus respiratory disorder", "wuhancoronavirus 2019 acute severe disease", "2019 wuhan coronavirus acute severe respiratory disease", "illness caused by 2019 new sars corona virus 2", "2019 chinese corona virus acute severe syndrome", "2019 novel sars cov 2 acute severe illness", "2019novel sars cov 2 acute respiratory disorder", "pneumonia caused by 2019new sars coronavirus2", "2019 novel severe acute respiratory syndrome corona virus 2", "2019new seafood market corona virus acute disorder", "chinese coronavirus acute pneumonia", "2019novel seafood market coronavirus severe respiratory disease", "2019 sarscov2 acute severe respiratory illness", "2019new chinese coronavirus severe infection", "sarscov2 respiratory disorder", "2019novel sars cov 2 respiratory syndrome", "2019 novel sars cov 2 respiratory syndrome", "2019 wuhan coronavirus severe respiratory disease", "2019new sars coronavirus 2", "new corona virus respiratory illness", "2019 novel wuhan seafood market virus severe acute respiratory illness", "2019 seafood market coronavirus acute severe respiratory syndrome", "2019new sars corona virus 2 acute respiratory syndrome", "pneumonia caused by severe acute respiratory syndromerelated corona virus 2 2019", "2019 severe acute respiratory syndrome corona virus 2 disorder", "disease due to 2019new severe acute respiratory syndrome corona virus2", "2019 wuhan seafood market virus acute severe syndrome", "2019new wuhan seafood market pneumonia virus severe pneumonia", "2019novel seafood market virus respiratory syndrome", "2019 wuhan virus respiratory infection", "sars cov 2 2019 respiratory disease", "virus responsible for covid19 severe pneumonia", "2019 seafood market virus severe respiratory syndrome", "wuhan seafood market virus 2019 respiratory syndrome", "wuhancoronavirus 2019 acute syndrome", "wuhan coronavirus 2019 severe disorder", "2019 human coronavirus severe acute disease", "2019novel sars coronavirus 2 severe pneumonia", "illness due to 2019 new severe acute respiratory syndrome corona virus 2", "infection from 2019novel wuhancorona virus", "hubei acute respiratory infection", "pneumonia from sarscov2", "2019new wuhan seafood market virus severe acute disorder", "covid19 virus acute severe syndrome", "2019new wuhancoronavirus acute severe respiratory disorder", "illness caused by 2019novel chinese corona virus", "2019ncov severe respiratory disease", "2019novel sars coronavirus2 pneumonia", "2019new wuhancorona virus respiratory disorder", "seafood market coronavirus severe respiratory disease", "pneumonia due to 2019 severe acute respiratory syndrome coronavirus 2", "wuhancorona virus disease", "2019 novel wuhan coronavirus asevere acute respiratory disease", "2019 new seafood market coronavirus acute severe pneumonia", "2019new seafood market corona virus severe respiratory syndrome", "severe acute respiratory syndromerelated coronavirus 2 2019 illness", "illness due to chinese coronavirus", "2019 new wuhan virus severe acute pneumonia", "human coronavirus 2019 severe infection", "infection from 2019 seafood market coronavirus", "wuhan coronavirus pneumonia", "syndrome from 2019novel seafood market virus", "wuhan coronavirus severe acute pneumonia", "2019new wuhan corona virus severe illness", "illness due to 2019 novel sars coronavirus 2", "chinese coronavirus 2019 severe acute respiratory illness", "chinese corona virus 2019", "2019novel sars cov 2 respiratory disease", "new coronavirus 2019 severe respiratory infection", "2019 wuhancorona virus severe acute illness", "pneumonia caused by severe acute respiratory syndrome corona virus2", "infection from 2019new wuhan virus", "2019 human corona virus severe pneumonia", "infection due to 2019 novel severe acute respiratory syndromerelated coronavirus 2", "pneumonia due to 2019new seafood market virus", "2019novel sars corona virus2 acute severe disease", "disorder caused by 2019 wuhan coronavirus", "pneumonia from 2019 novel seafood market corona virus", "virus responsible for covid19 severe acute respiratory disorder", "2019 new sars corona virus 2 acute syndrome", "2019 novel sars corona virus2 acute severe respiratory disease", "disorder from chinese coronavirus 2019", "2019new wuhancorona virus severe illness", "disease from 2019 new severe acute respiratory syndrome coronavirus2", "disease due to 2019 new severe acute respiratory syndrome coronavirus 2", "2019 new wuhan seafood market pneumonia virus severe syndrome", "infection from 2019 severe acute respiratory syndromerelated coronavirus 2", "2019novel chinese corona virus severe acute respiratory infection", "new corona virus acute respiratory illness", "2019 new wuhan virus severe pneumonia", "2019 sarscov2 acute severe respiratory syndrome", "disorder caused by 2019 new wuhan virus", "wuhan pneumonia virus acute respiratory disease", "2019new seafood market corona virus severe infection", "2019 novel sars coronavirus 2 severe disease", "2019 novel sars corona virus2 pneumonia", "sarscov2 acute respiratory disorder", "2019 chinese corona virus acute severe illness", "2019 new wuhan corona virus respiratory syndrome", "seafood market coronavirus 2019 severe acute pneumonia", "illness caused by sars coronavirus 2", "2019 new wuhan virus severe respiratory syndrome", "2019 seafood market corona virus", "2019new wuhan coronavirus asevere acute respiratory disease", "2019novel sars cov 2", "2019 new sars coronavirus2 severe respiratory disease", "infection due to covid19 virus", "2019new wuhan pneumonia virus severe acute illness", "2019 novel wuhan pneumonia virus acute respiratory infection", "illness from 2019 novel severe acute respiratory syndrome coronavirus 2", "2019 corona virus acute severe syndrome", "infection caused by severe acute respiratory syndrome coronavirus 2", "2019 new sarscov2 severe illness", "2019 new wuhan coronavirus severe syndrome", "2019novel wuhancorona virus illness", "2019 wuhan coronavirus infection", "2019 new sarscov2 disease", "wuhan coronavirus acute respiratory illness", "2019 novel sars corona virus 2 respiratory syndrome", "pneumonia from 2019 seafood market coronavirus", "2019new seafood market corona virus acute pneumonia", "illness due to novel corona virus 2019", "2019 novel wuhan seafood market virus acute severe respiratory infection", "disease from novel coronavirus", "wuhan pneumonia virus severe disorder", "2019 wuhancorona virus acute disorder", "pneumonia caused by 2019 new sars coronavirus 2", "disease caused by 2019 novel seafood market corona virus", "disorder from severe acute respiratory syndromerelated corona virus 2", "pneumonia from 2019novel seafood market coronavirus", "sars coronavirus 2 severe illness", "wuhan virus acute severe disease", "2019 chinese coronavirus severe illness", "2019 new sars coronavirus 2 acute respiratory syndrome", "disorder due to 2019new wuhan coronavirus", "pneumonia caused by novel coronavirus", "2019new wuhan corona virus acute severe respiratory infection", "novel coronavirus acute severe respiratory infection", "2019 wuhancorona virus severe pneumonia", "2019new wuhancoronavirus", "2019 novel chinese corona virus severe syndrome", "illness caused by 2019new severe acute respiratory syndrome corona virus2", "2019novel sars corona virus2 severe infection", "pneumonia due to novel coronavirus", "seafood market severe acute respiratory disorder", "pneumonia caused by wuhancoronavirus 2019", "2019 corona virus disorder", "illness caused by 2019 novel wuhan seafood market virus", "wuhancoronavirus 2019 severe acute disorder", "illness from 2019novel severe acute respiratory syndrome coronavirus2", "2019 novel wuhan virus severe respiratory syndrome", "2019new wuhancoronavirus acute severe disease", "2019novel chinese coronavirus acute severe syndrome", "disease due to 2019 new chinese corona virus", "2019 novel sars cov 2 acute severe disorder", "wuhan corona virus 2019 acute severe respiratory syndrome", "wuhan pneumonia virus 2019 respiratory syndrome", "disease due to 2019 novel sars cov 2", "disease caused by seafood market virus 2019", "sars coronavirus2 2019 acute pneumonia", "2019new wuhancoronavirus acute infection", "seafood market corona virus 2019 severe acute disease", "2019 new sars coronavirus2 acute respiratory disorder", "2019 novel wuhan corona virus severe respiratory disorder", "sars corona virus 2 acute severe infection", "2019novel wuhancoronavirus acute infection", "2019 new wuhancorona virus acute respiratory disease", "sars coronavirus2 disorder", "2019new sars coronavirus 2 severe infection", "illness due to 2019novel sars corona virus 2", "2019 wuhan pneumonia virus acute severe illness", "2019 seafood market coronavirus pneumonia", "new corona virus severe respiratory infection", "wuhan virus 2019 acute severe syndrome", "2019 wuhancorona virus acute severe disorder", "2019 wuhan pneumonia virus severe acute respiratory illness", "new corona virus acute severe illness", "2019 new chinese coronavirus severe respiratory disease", "chinese coronavirus severe acute syndrome", "2019 novel wuhan seafood market pneumonia virus severe acute respiratory disorder", "2019 sars corona virus2 illness", "hubei acute respiratory disorder", "2019 new chinese corona virus acute severe respiratory disorder", "2019 new sars corona virus 2 illness", "2019 new wuhan seafood market pneumonia virus severe respiratory infection", "2019 novel wuhancorona virus pneumonia", "2019 new sars coronavirus2 severe disorder", "2019 sars cov 2 severe illness", "severe acute respiratory syndromerelated corona virus 2 2019 infection", "wuhan seafood market virus 2019 severe acute infection", "illness caused by 2019new wuhancorona virus", "2019 wuhan corona virus acute severe respiratory disease", "2019 new wuhancoronavirus acute disease", "pneumonia due to 2019 severe acute respiratory syndromerelated coronavirus 2", "syndrome caused by 2019 novel sars corona virus2", "pneumonia due to virus causing covid19", "2019 new wuhan virus acute severe disorder", "disorder from 2019 novel wuhan coronavirus", "2019 corona virus severe respiratory syndrome", "disease from 2019 seafood market virus", "novel coronavirus severe respiratory illness", "severe acute respiratory syndromerelated coronavirus2 2019 disorder", "disease from 2019 new sars coronavirus 2", "2019 coronavirus severe infection", "2019new severe acute respiratory syndromerelated corona virus 2 infection", "syndrome due to seafood market virus", "syndrome due to 2019new chinese corona virus", "wuhan seafood market virus 2019 severe disease", "new coronavirus acute respiratory disorder", "2019 novel sars coronavirus2 acute respiratory disorder", "2019 new chinese corona virus severe respiratory disease", "2019 human corona virus acute severe disease", "2019novel wuhan corona virus asevere acute respiratory disease", "2019novel wuhan virus respiratory illness", "infection due to 2019new sarscov2", "pneumonia due to sars coronavirus 2 2019", "severe acute respiratory syndromerelated corona virus 2 illness", "2019 new corona virus severe disorder", "pneumonia from covid19 virus", "disease from 2019new chinese corona virus", "2019 new sarscov2 acute severe syndrome", "human coronavirus 2019 disorder", "2019 novel chinese coronavirus severe disorder", "wuhan corona virus 2019 acute severe syndrome", "disorder caused by severe acute respiratory syndrome corona virus2", "2019new wuhan pneumonia virus illness", "new coronavirus severe acute respiratory disorder", "2019 sars cov 2 severe acute respiratory infection", "2019 ncov severe illness", "sars coronavirus 2 acute severe respiratory illness", "wuhan corona virus 2019 severe respiratory syndrome", "sars corona virus 2 respiratory disease", "2019 new wuhancoronavirus acute illness", "2019new wuhan seafood market pneumonia virus acute pneumonia", "sars coronavirus2 severe syndrome", "2019 coronavirus acute disease", "wuhan pneumonia virus severe pneumonia", "virus responsible for covid19 severe acute pneumonia", "wuhan corona virus severe infection", "wuhan pneumonia virus 2019 severe acute disease", "wuhan seafood market pneumonia virus 2019 severe acute respiratory infection", "pneumonia due to seafood market virus", "wuhan coronavirus acute syndrome", "2019 novel sars corona virus 2 acute severe disorder", "sarscov2 2019 acute severe respiratory disease", "new corona virus 2019 acute disorder", "wuhan pneumonia virus 2019 acute severe disorder", "hcov 19 severe respiratory illness", "2019 novel wuhancorona virus acute syndrome", "2019 chinese coronavirus severe acute respiratory syndrome", "2019new seafood market coronavirus severe respiratory disorder", "2019 new seafood market virus severe disease", "illness due to 2019 new wuhancorona virus", "2019novel wuhan virus severe acute respiratory infection", "2019new chinese coronavirus acute respiratory syndrome", "sars coronavirus 2 acute illness", "2019new sars corona virus2 acute respiratory infection", "2019 novel seafood market virus severe respiratory disease", "2019 ncov severe acute syndrome", "sars cov 2 2019 acute severe respiratory syndrome", "2019new wuhan coronavirus acute severe disease", "severe acute respiratory syndrome corona virus2 2019 infection", "wuhan pneumonia virus severe acute respiratory infection", "sars corona virus 2 2019 severe pneumonia", "infection due to 2019 novel chinese coronavirus", "wuhancorona virus severe illness", "wuhan virus 2019 acute severe infection", "disorder from 2019 new wuhan seafood market virus", "2019 new coronavirus", "disorder due to 2019 new wuhancorona virus", "sars cov 2 2019 severe acute disease", "illness from 2019 new seafood market coronavirus", "2019 novel sars corona virus2 acute disorder", "sars corona virus2 severe respiratory illness", "disorder from novel coronavirus", "wuhan seafood market virus 2019 acute severe respiratory illness", "wuhan seafood market pneumonia virus severe acute illness", "2019 severe acute respiratory syndromerelated corona virus 2 infection", "chinese coronavirus 2019 asevere acute respiratory disease", "syndrome caused by 2019 wuhan corona virus", "2019 sars corona virus 2 severe respiratory disorder", "2019novel wuhan corona virus severe syndrome", "2019new wuhan seafood market pneumonia virus acute respiratory illness", "sarscov2 2019 severe respiratory syndrome", "seafood market virus 2019 acute respiratory illness", "severe acute respiratory syndrome corona virus2 2019 illness", "wuhancorona virus 2019 acute severe infection", "sars corona virus2 2019 acute severe respiratory infection", "2019 novel sars corona virus2", "2019 wuhan corona virus acute disease", "2019new sars corona virus 2 severe acute disorder", "2019 wuhan corona virus acute respiratory infection", "2019 novel chinese coronavirus severe illness", "sars cov 2 severe acute illness", "2019novel wuhan seafood market pneumonia virus acute respiratory disorder", "2019novel wuhan seafood market pneumonia virus acute severe respiratory illness", "2019ncov acute severe respiratory disease", "2019novel seafood market coronavirus severe respiratory disorder", "disorder from 2019 novel wuhan seafood market pneumonia virus", "disorder caused by 2019 chinese coronavirus", "sars corona virus 2 disease", "2019 new corona virus acute severe respiratory infection", "2019 chinese corona virus acute respiratory disorder", "2019new sars coronavirus 2 acute respiratory disorder", "disorder from chinese coronavirus", "2019 wuhan virus respiratory syndrome", "2019 wuhan seafood market virus acute severe illness", "2019 novel wuhancoronavirus severe disease", "wuhan seafood market pneumonia virus 2019 acute severe respiratory syndrome", "new corona virus 2019 severe respiratory infection", "sars coronavirus2 severe respiratory disease", "illness from seafood market coronavirus", "2019 seafood market coronavirus acute respiratory illness", "2019new severe acute respiratory syndromerelated coronavirus 2", "pneumonia caused by 2019 new wuhan corona virus", "new coronavirus 2019", "2019 new wuhan coronavirus respiratory infection", "2019novel seafood market coronavirus acute severe pneumonia", "wuhan virus severe respiratory disorder", "novel corona virus 2019", "2019 wuhan virus acute respiratory infection", "2019 novel wuhan coronavirus severe acute illness", "2019 novel sarscov2 severe disease", "2019 seafood market corona virus severe infection", "2019new wuhancorona virus severe respiratory disease", "disease caused by 2019 wuhan seafood market virus", "syndrome caused by 2019novel wuhan seafood market virus", "2019novel sars coronavirus 2 illness", "2019novel wuhan pneumonia virus pneumonia", "2019 new wuhan corona virus acute severe disease", "2019 new sars corona virus2 acute respiratory illness", "sars coronavirus 2 2019 severe pneumonia", "2019 sars corona virus2 acute severe disease", "2019 sars coronavirus 2 severe respiratory infection", "wuhancoronavirus acute severe respiratory syndrome", "illness caused by sars corona virus 2", "2019new wuhan seafood market pneumonia virus", "2019 new wuhan seafood market pneumonia virus acute illness", "2019novel seafood market virus severe disease", "novel coronavirus 2019 acute respiratory disorder", "wuhan coronavirus 2019 respiratory infection", "2019 new coronavirus acute illness", "disease due to 2019 new wuhan pneumonia virus", "2019new chinese coronavirus acute respiratory illness", "seafood market coronavirus severe illness", "2019new wuhancoronavirus acute severe syndrome", "sars corona virus 2 2019 illness", "2019 ncov severe acute infection", "novel coronavirus severe acute respiratory infection", "2019 novel wuhan pneumonia virus severe acute respiratory disorder", "2019novel wuhan corona virus acute severe respiratory illness", "illness due to 2019 new severe acute respiratory syndromerelated coronavirus 2", "sars cov 2 2019 severe acute pneumonia", "2019 new wuhan seafood market pneumonia virus acute severe respiratory infection", "2019 new wuhan coronavirus respiratory disease", "wuhancoronavirus 2019 severe disorder", "wuhan seafood market pneumonia virus severe acute syndrome", "disease from 2019 severe acute respiratory syndromerelated corona virus 2", "new corona virus 2019 acute severe illness", "2019 novel wuhan seafood market pneumonia virus severe acute illness", "disorder from hcov19", "2019 wuhan virus acute severe respiratory disorder", "2019 new wuhan pneumonia virus severe syndrome", "2019 new wuhan seafood market pneumonia virus acute respiratory illness", "infection caused by wuhan seafood market pneumonia virus 2019", "2019 novel sars cov 2 acute disorder", "novel coronavirus 2019 severe syndrome", "disease due to sars corona virus 2 2019", "seafood market coronavirus 2019 acute respiratory infection", "sars cov 2 2019 severe respiratory illness", "hubei acute pneumonia", "infection due to human corona virus 2019", "disease due to 2019 novel sars corona virus2", "infection from wuhancoronavirus 2019", "2019 new wuhan seafood market virus acute respiratory infection", "2019 wuhan seafood market pneumonia virus respiratory illness", "wuhan seafood market pneumonia virus 2019 severe acute respiratory syndrome", "wuhancorona virus 2019 respiratory disease", "2019 wuhancorona virus acute respiratory syndrome", "2019 novel sars corona virus 2 acute illness", "hcov19 acute severe disorder", "illness due to 2019novel severe acute respiratory syndromerelated coronavirus2", "pneumonia caused by severe acute respiratory syndromerelated corona virus2", "2019new sars coronavirus 2 disorder", "novel corona virus acute infection", "sars coronavirus 2 2019 severe acute pneumonia", "syndrome from human coronavirus 2019", "2019new wuhancoronavirus disease", "wuhan corona virus 2019 syndrome", "syndrome caused by 2019novel wuhan coronavirus", "seafood market coronavirus severe pneumonia", "chinese coronavirus respiratory illness", "2019 novel wuhan seafood market pneumonia virus acute respiratory infection", "disease due to 2019new severe acute respiratory syndromerelated corona virus 2", "wuhan seafood market virus 2019 asevere acute respiratory disease", "2019 novel sarscov2 severe acute illness", "illness caused by 2019new wuhancoronavirus", "2019new wuhan virus severe infection", "novel coronavirus 2019 pneumonia", "virus responsible for covid19 respiratory disease", "chinese coronavirus acute disorder", "2019 wuhan pneumonia virus infection", "2019novel severe acute respiratory syndrome coronavirus 2 pneumonia", "new corona virus 2019 acute severe pneumonia", "virus causing covid19 severe acute disorder", "2019 wuhan seafood market pneumonia virus severe respiratory illness", "2019new chinese corona virus acute disease", "seafood market virus 2019 severe acute disease", "2019 wuhancoronavirus severe acute respiratory infection", "disease caused by 2019new severe acute respiratory syndrome coronavirus 2", "sars corona virus 2 severe acute syndrome", "2019novel sarscov2 severe respiratory illness", "2019new wuhan pneumonia virus acute severe disorder", "disease from seafood market corona virus", "2019 novel sars corona virus 2 acute syndrome", "2019novel wuhan virus", "2019 new wuhan corona virus severe syndrome", "novel corona virus 2019 asevere acute respiratory disease", "2019 sars coronavirus2 acute severe infection", "2019novel chinese corona virus severe acute syndrome", "disease from 2019 novel seafood market coronavirus", "pneumonia caused by seafood market virus", "2019 new seafood market corona virus acute pneumonia", "2019 new chinese corona virus severe acute respiratory disorder", "2019novel sars coronavirus2 severe respiratory illness", "sars coronavirus 2 2019 acute respiratory disease", "disorder caused by 2019new seafood market virus", "syndrome caused by 2019 novel wuhancoronavirus", "2019 novel sars coronavirus 2 acute severe pneumonia", "2019 wuhan virus severe acute respiratory syndrome", "disorder due to 2019 ncov", "2019novel wuhan seafood market virus severe respiratory disorder", "2019 novel wuhan coronavirus severe respiratory infection", "pneumonia from covid 19 virus", "disorder due to wuhancorona virus 2019", "2019 novel wuhan seafood market pneumonia virus asevere acute respiratory disease", "2019novel wuhan coronavirus acute severe disease", "illness caused by 2019novel sarscov2", "sarscov2 severe acute infection", "infection caused by 2019new chinese corona virus", "disorder caused by chinese corona virus", "syndrome due to seafood market coronavirus 2019", "2019 new sars corona virus2 respiratory disorder", "infection caused by wuhan coronavirus 2019", "chinese corona virus severe acute pneumonia", "2019novel seafood market virus respiratory infection", "virus responsible for covid19 acute respiratory infection", "pneumonia from 2019 novel sars cov 2", "2019new seafood market corona virus acute syndrome", "infection caused by wuhan seafood market pneumonia virus", "illness caused by 2019 new wuhancorona virus", "infection caused by 2019 new severe acute respiratory syndromerelated corona virus 2", "wuhan pneumonia virus 2019 acute severe respiratory disorder", "2019novel wuhan corona virus acute severe disorder", "disease caused by 2019 severe acute respiratory syndrome corona virus 2", "2019 seafood market corona virus syndrome", "disease caused by 2019 ncov", "pneumonia caused by sars coronavirus2 2019", "sars coronavirus2 acute severe disease", "illness due to 2019 wuhan pneumonia virus", "2019 sarscov2 severe infection", "novel corona virus acute respiratory disease", "2019 coronavirus severe respiratory infection", "2019novel seafood market corona virus pneumonia", "disease from 2019new chinese coronavirus", "wuhan seafood market pneumonia virus 2019 severe acute illness", "2019 sars cov 2 severe respiratory infection", "syndrome due to 2019 human coronavirus", "disorder from 2019 sars corona virus 2", "2019novel wuhancoronavirus acute severe pneumonia", "new coronavirus 2019 severe respiratory disease", "2019 novel wuhancoronavirus pneumonia", "syndrome due to 2019 corona virus", "wuhan seafood market virus 2019 acute disease", "2019novel wuhan coronavirus severe acute illness", "disease caused by sars coronavirus2 2019", "2019new wuhan pneumonia virus severe acute respiratory illness", "2019 new chinese coronavirus severe acute infection", "2019new sars corona virus2 severe disorder", "infection from 2019 severe acute respiratory syndromerelated corona virus2", "hcov19 acute respiratory disorder", "2019new sars coronavirus2 acute disease", "infection from 2019novel severe acute respiratory syndrome corona virus2", "sars coronavirus2 acute respiratory disease", "2019novel wuhancorona virus severe respiratory syndrome", "2019novel sarscov2 severe acute respiratory syndrome", "sarscov2 acute severe respiratory illness", "disease caused by 2019 wuhan virus", "infection caused by 2019 novel sars corona virus2", "pneumonia from 2019 novel sars corona virus2", "2019new seafood market virus severe respiratory disease", "severe acute respiratory syndromerelated coronavirus 2 disorder", "infection caused by severe acute respiratory syndrome corona virus2", "infection due to severe acute respiratory syndromerelated corona virus2", "novel coronavirus acute severe infection", "2019 novel sars coronavirus 2 acute severe respiratory infection", "2019new wuhan seafood market virus", "infection caused by new corona virus", "sars coronavirus 2 severe acute disease", "syndrome from 2019 seafood market corona virus", "2019new sars corona virus 2 acute severe disorder", "sars corona virus2 syndrome", "2019 new chinese coronavirus severe acute respiratory illness", "wuhan virus 2019 respiratory syndrome", "2019 wuhancorona virus acute infection", "disorder due to 2019 seafood market coronavirus", "2019new wuhan pneumonia virus severe illness", "pneumonia from severe acute respiratory syndrome coronavirus 2", "2019 novel seafood market virus acute severe syndrome", "2019 new sars corona virus 2 severe acute respiratory syndrome", "2019new sars corona virus 2 severe syndrome", "2019 new sars cov 2 acute respiratory disease", "2019novel seafood market coronavirus respiratory infection", "2019 novel wuhancorona virus acute severe infection", "2019 human coronavirus respiratory disorder", "infection due to 2019 severe acute respiratory syndrome coronavirus 2", "2019novel wuhancoronavirus asevere acute respiratory disease", "pneumonia from severe acute respiratory syndromerelated corona virus2 2019", "2019novel sars cov 2 acute severe respiratory illness", "disease from 2019novel sars corona virus 2", "disorder due to severe acute respiratory syndrome coronavirus2", "illness from 2019 novel wuhan seafood market virus", "hcov19 severe acute respiratory illness", "2019 new sars coronavirus 2 severe acute infection", "hcov19 severe acute disease", "2019 sars corona virus 2 illness", "2019 novel wuhancoronavirus acute respiratory illness", "2019novel wuhancoronavirus acute severe respiratory infection", "sarscov2 severe acute respiratory syndrome", "2019 sars coronavirus2 acute disease", "infection caused by 2019 new chinese corona virus", "disease from 2019new sars corona virus 2", "disorder due to 2019novel sars cov 2", "2019new seafood market coronavirus respiratory illness", "disorder caused by severe acute respiratory syndrome coronavirus2 2019", "disease caused by 2019novel seafood market virus", "2019 new seafood market virus severe respiratory infection", "2019 new wuhancorona virus severe acute respiratory disorder", "2019 novel seafood market coronavirus acute respiratory illness", "pneumonia from 2019new sarscov2", "chinese corona virus 2019 severe disorder", "wuhan seafood market virus severe acute syndrome", "new corona virus 2019 acute severe disease", "wuhan pneumonia virus respiratory disease", "2019 ncov acute severe syndrome", "2019 novel wuhan corona virus asevere acute respiratory disease", "2019 novel sars corona virus 2 severe acute respiratory disorder", "2019new wuhan seafood market virus acute respiratory disease", "2019new sarscov2 severe acute disorder", "disorder caused by 2019 new sars cov 2", "illness caused by wuhan corona virus 2019", "illness from 2019new sarscov2", "2019new chinese coronavirus acute severe syndrome", "2019new wuhan corona virus severe syndrome", "2019 novel sars coronavirus 2 severe respiratory disease", "disease caused by 2019new sars coronavirus2", "syndrome caused by covid19 virus", "disorder due to 2019 novel chinese coronavirus", "2019 wuhan seafood market virus severe illness", "2019 new chinese corona virus", "2019new sars coronavirus2 acute respiratory infection", "illness from severe acute respiratory syndromerelated coronavirus2", "2019 seafood market coronavirus asevere acute respiratory disease", "2019 human coronavirus severe acute infection", "RA01.0", "2019 new seafood market corona virus syndrome", "illness from 2019novel chinese coronavirus", "2019 novel wuhan seafood market pneumonia virus disorder", "2019 novel wuhan seafood market virus", "human corona virus 2019 acute severe disease", "2019 new seafood market corona virus severe disease", "chinese coronavirus acute severe disease", "illness from 2019 new wuhancoronavirus", "sars corona virus2 severe acute disorder", "seafood market acute pneumonia", "2019 new wuhan coronavirus severe acute pneumonia", "2019novel wuhancorona virus severe acute respiratory infection", "infection from severe acute respiratory syndrome coronavirus2", "pneumonia from 2019 wuhan coronavirus", "wuhan seafood market virus 2019 acute respiratory syndrome", "wuhan seafood market virus 2019 pneumonia", "2019 novel coronavirus acute disease", "2019 sars coronavirus2 acute severe pneumonia", "wuhan coronavirus 2019 acute respiratory disorder", "pneumonia caused by 2019 wuhancoronavirus", "2019 sarscov2 acute severe disease", "2019 new wuhan corona virus acute syndrome", "2019 novel wuhancoronavirus acute severe illness", "syndrome caused by 2019 novel sars coronavirus2", "pneumonia due to sars corona virus2 2019", "disease caused by 2019 new sarscov2", "wuhan pneumonia virus severe illness", "2019new sars cov 2 severe acute disease", "illness from 2019 novel chinese corona virus", "2019 chinese coronavirus severe acute respiratory illness", "2019 new chinese corona virus respiratory disorder", "2019 new chinese corona virus acute severe disease", "2019 novel sarscov2 acute severe respiratory illness", "wuhan corona virus severe acute illness", "hcov 19 acute severe respiratory disease", "virus causing covid19 acute respiratory illness", "syndrome due to 2019 sars coronavirus 2", "2019novel seafood market coronavirus severe acute disorder", "seafood market corona virus 2019 severe respiratory disorder", "sars coronavirus 2 2019 asevere acute respiratory disease", "infection due to 2019 new wuhan virus", "sarscov2 acute infection", "2019 novel wuhancoronavirus respiratory disease", "2019 novel sars cov 2 acute respiratory disease", "sars corona virus2 pneumonia", "disease from covid19 virus", "2019 novel chinese coronavirus severe respiratory disorder", "pneumonia caused by chinese coronavirus 2019", "2019novel wuhan virus respiratory syndrome", "2019 new wuhan virus asevere acute respiratory disease", "disease due to 2019 new seafood market corona virus", "seafood market virus 2019 acute severe pneumonia", "2019 new wuhancorona virus severe acute respiratory infection", "2019new wuhan coronavirus severe respiratory disease", "syndrome caused by wuhan coronavirus 2019", "2019 seafood market virus pneumonia", "2019 novel seafood market coronavirus acute severe syndrome", "2019new sars corona virus2 severe respiratory syndrome", "2019 novel wuhan virus acute severe infection", "2019 novel wuhan seafood market pneumonia virus severe acute pneumonia", "2019 sars corona virus2 severe acute syndrome", "2019 new sarscov2 severe disorder", "2019new wuhan coronavirus acute severe respiratory disease", "infection caused by human corona virus 2019", "2019 novel wuhancorona virus severe infection", "2019 new corona virus severe acute infection", "2019 corona virus severe respiratory disorder", "2019new wuhancoronavirus acute respiratory disorder", "infection caused by 2019new wuhan corona virus", "disorder due to 2019new severe acute respiratory syndrome coronavirus 2", "illness due to 2019new severe acute respiratory syndrome corona virus 2", "pneumonia due to 2019new severe acute respiratory syndrome corona virus2", "disease due to 2019novel sarscov2", "disease caused by 2019new seafood market virus", "wuhan pneumonia virus acute severe syndrome", "2019 novel sars coronavirus 2 acute syndrome", "seafood market severe respiratory disease", "wuhancorona virus 2019 respiratory illness", "2019new wuhancoronavirus acute severe respiratory disease", "2019novel sarscov2 severe acute disorder", "disorder from 2019 novel severe acute respiratory syndrome corona virus 2", "infection from 2019novel chinese corona virus", "2019new wuhan seafood market virus acute disease", "chinese coronavirus 2019 severe acute infection", "seafood market corona virus 2019 severe acute pneumonia", "2019new wuhan coronavirus acute severe infection", "seafood market virus acute severe respiratory infection", "2019 new sars coronavirus2 severe disease", "2019novel wuhan coronavirus acute respiratory infection", "infection due to 2019new wuhan pneumonia virus", "wuhancorona virus 2019 severe acute pneumonia", "2019new seafood market virus respiratory illness", "wuhan coronavirus severe respiratory illness", "2019 novel coronavirus infection", "2019 new seafood market virus severe acute respiratory infection", "pneumonia due to 2019 new seafood market coronavirus", "2019 sars coronavirus 2 syndrome", "sars corona virus2 2019 pneumonia", "2019 new seafood market virus illness", "2019 novel coronavirus respiratory infection", "wuhan acute severe syndrome", "novel corona virus severe acute syndrome", "pneumonia caused by 2019novel wuhan virus", "sars coronavirus2 respiratory disease", "disorder caused by 2019 chinese corona virus", "disease from 2019 novel sars coronavirus2", "2019 new wuhan corona virus severe acute respiratory syndrome", "covid 19 virus acute severe disorder", "2019new wuhan seafood market pneumonia virus severe infection", "2019 new sars coronavirus 2 severe acute respiratory infection", "2019 new wuhan virus severe acute disease", "disease caused by wuhan seafood market virus", "2019 wuhan seafood market pneumonia virus severe acute syndrome", "2019 new seafood market coronavirus illness", "2019 severe acute respiratory syndrome coronavirus 2", "2019 novel sars corona virus2 acute respiratory disease", "disorder caused by 2019 severe acute respiratory syndrome coronavirus 2", "2019 sarscov2 severe pneumonia", "2019 sars corona virus2 respiratory infection", "infection due to 2019 severe acute respiratory syndrome corona virus2", "2019 sars corona virus2 severe respiratory disease", "2019new wuhan virus severe acute pneumonia", "2019 novel chinese corona virus acute pneumonia", "2019 novel chinese coronavirus acute respiratory disease", "chinese corona virus severe acute disorder", "2019new wuhan pneumonia virus acute disease", "sars coronavirus2 acute severe disorder", "disorder due to 2019 new severe acute respiratory syndrome coronavirus 2", "infection due to wuhan seafood market virus", "2019 sars cov 2 severe acute respiratory illness", "2019 new wuhan seafood market virus severe acute disease", "2019new wuhancoronavirus severe acute disease", "human corona virus 2019 acute severe respiratory disorder", "2019 new sars corona virus 2 acute respiratory illness", "2019new wuhan pneumonia virus acute severe respiratory illness", "wuhan seafood market virus acute disorder", "novel coronavirus infection", "2019 novel sars coronavirus 2 acute severe infection", "illness caused by 2019new sarscov2", "2019 novel chinese coronavirus severe acute syndrome", "2019 new wuhan seafood market pneumonia virus acute disorder", "wuhancorona virus 2019 acute illness", "2019new sars coronavirus 2 acute disease", "2019novel sarscov2 respiratory syndrome", "2019novel sars corona virus2 severe respiratory disorder", "disorder from 2019new wuhan virus", "2019novel sarscov2 acute severe pneumonia", "hcov 19 acute severe disorder", "disease due to 2019 new wuhan virus", "2019 novel wuhan coronavirus acute respiratory disorder", "2019 new wuhan virus respiratory disorder", "2019novel sars corona virus 2 acute severe respiratory syndrome", "illness caused by 2019 new severe acute respiratory syndrome coronavirus2", "pneumonia from 2019 wuhan virus", "sars cov 2 2019 severe syndrome", "2019 new wuhan seafood market pneumonia virus acute severe disease", "pneumonia from 2019 new sarscov2", "seafood market virus severe respiratory disorder", "new corona virus 2019 severe acute illness", "2019 new wuhan coronavirus severe pneumonia", "seafood market virus acute severe disease", "seafood market acute respiratory syndrome", "disease caused by 2019 novel severe acute respiratory syndromerelated coronavirus2", "syndrome due to 2019 novel sars cov 2", "2019novel wuhan corona virus acute syndrome", "2019novel sarscov2 syndrome", "2019 new sars coronavirus2 severe acute illness", "disease caused by 2019novel severe acute respiratory syndromerelated coronavirus 2", "2019 chinese coronavirus acute respiratory syndrome", "2019novel wuhan virus acute severe respiratory syndrome", "2019novel seafood market corona virus severe acute disorder", "disease caused by 2019 novel severe acute respiratory syndrome coronavirus2", "infection caused by 2019 new severe acute respiratory syndromerelated coronavirus2", "disorder caused by wuhancoronavirus", "virus causing covid19 acute severe syndrome", "human coronavirus 2019 acute respiratory disorder", "novel coronavirus 2019 acute respiratory syndrome", "2019 new wuhan seafood market pneumonia virus severe acute respiratory syndrome", "covid 19 virus infection", "2019 new severe acute respiratory syndrome corona virus2 pneumonia", "2019 wuhan virus acute syndrome", "wuhan seafood market virus respiratory disease", "2019 novel seafood market virus acute severe disease", "hubei severe respiratory disorder", "2019 novel seafood market corona virus acute severe respiratory illness", "2019new sars corona virus 2 severe respiratory illness", "illness due to severe acute respiratory syndromerelated coronavirus 2", "2019new wuhan seafood market pneumonia virus severe disease", "novel corona virus 2019 acute severe disorder", "seafood market virus 2019 severe acute respiratory illness", "2019new sars corona virus 2 severe acute respiratory infection", "2019 novel wuhan pneumonia virus acute severe respiratory illness", "disorder caused by 2019new severe acute respiratory syndrome coronavirus2", "pneumonia due to 2019 novel severe acute respiratory syndrome coronavirus 2", "2019 seafood market virus acute respiratory syndrome", "2019 ncov severe respiratory disorder", "new coronavirus 2019 acute pneumonia", "new corona virus 2019 severe acute respiratory disorder", "illness caused by 2019new seafood market corona virus", "2019novel seafood market corona virus acute disease", "2019new sars corona virus2 respiratory disease", "syndrome caused by wuhancorona virus", "disorder due to 2019 novel severe acute respiratory syndrome coronavirus2", "2019 new chinese coronavirus severe acute respiratory infection", "2019novel wuhan corona virus acute respiratory infection", "2019new wuhan coronavirus acute severe respiratory disorder", "2019 new sars corona virus 2 severe acute syndrome", "2019 novel sars corona virus2 infection", "2019 novel wuhan seafood market pneumonia virus acute severe disease", "wuhancoronavirus acute infection", "syndrome due to 2019 seafood market corona virus", "2019novel seafood market virus", "2019 chinese corona virus pneumonia", "2019 new wuhan seafood market pneumonia virus severe acute syndrome", "wuhan seafood market virus 2019 severe acute respiratory infection", "syndrome caused by sars corona virus 2", "illness due to 2019novel chinese coronavirus", "seafood market corona virus acute respiratory disorder", "wuhan severe disorder", "2019 new wuhan corona virus respiratory infection", "2019 novel wuhancorona virus illness", "pneumonia due to wuhan seafood market pneumonia virus 2019", "illness from 2019 wuhan coronavirus", "2019 coronavirus illness", "sars coronavirus 2 2019 severe illness", "disorder caused by hcov 19", "infection from 2019 corona virus", "infection from severe acute respiratory syndromerelated corona virus 2", "pneumonia from wuhan pneumonia virus 2019", "2019 sars coronavirus2 acute syndrome", "2019 chinese corona virus acute severe respiratory disorder", "2019 new wuhan virus infection", "wuhan coronavirus acute illness", "2019novel chinese corona virus acute respiratory infection", "disease from sars corona virus2", "2019 seafood market corona virus pneumonia", "2019 novel sars coronavirus2 acute respiratory infection", "wuhan acute respiratory illness", "pneumonia caused by 2019new severe acute respiratory syndrome coronavirus 2", "illness from covid19 virus", "chinese corona virus 2019 respiratory infection", "disease due to wuhan seafood market pneumonia virus 2019", "illness caused by sarscov2 2019", "2019 sarscov2 severe respiratory disorder", "pneumonia from severe acute respiratory syndrome corona virus 2", "2019novel wuhancoronavirus acute respiratory infection", "disease caused by 2019new wuhan corona virus", "sars coronavirus 2 2019 illness", "2019 novel sars coronavirus2 severe acute syndrome", "disorder caused by 2019new severe acute respiratory syndrome corona virus2", "illness due to sars cov 2 2019", "2019 novel coronavirus acute severe respiratory syndrome", "2019new severe acute respiratory syndromerelated corona virus2", "disorder caused by 2019 new seafood market virus", "2019 novel chinese coronavirus asevere acute respiratory disease", "wuhancorona virus severe acute respiratory disorder", "wuhancoronavirus 2019 acute severe respiratory disease", "2019 new wuhan pneumonia virus acute infection", "2019new wuhan seafood market virus acute disorder", "pneumonia from 2019 new corona virus", "sars corona virus2 acute severe respiratory disease", "pneumonia caused by 2019 new sarscov2", "disease from 2019 wuhancorona virus", "2019novel sars corona virus2 acute pneumonia", "seafood market severe illness", "2019novel sars corona virus2 acute severe respiratory syndrome", "2019novel wuhan coronavirus acute pneumonia", "2019new sars cov 2 severe disease", "2019new wuhancorona virus respiratory infection", "human corona virus 2019 respiratory disorder", "2019 corona virus acute severe respiratory disorder", "2019 new sars corona virus2 asevere acute respiratory disease", "disorder due to 2019novel sars corona virus 2", "2019 new sars corona virus 2 acute severe infection", "pneumonia due to 2019new sars corona virus2", "2019 new wuhan pneumonia virus illness", "2019 sarscov2 respiratory infection", "sars corona virus2 2019 disease", "2019 new sars corona virus 2 respiratory infection", "2019new chinese corona virus acute respiratory syndrome", "disorder from 2019novel sars coronavirus2", "2019new wuhan seafood market pneumonia virus acute severe illness", "2019novel wuhancorona virus respiratory illness", "2019 wuhancoronavirus acute severe disease", "disease from severe acute respiratory syndrome coronavirus 2", "covid19 virus severe acute disease", "2019 novel seafood market virus severe acute syndrome", "wuhan seafood market pneumonia virus 2019 acute severe respiratory disease", "wuhan pneumonia virus acute severe illness", "2019 sars coronavirus2 severe respiratory disease", "2019 sars", "2019novel chinese corona virus severe acute respiratory illness", "2019 sars corona virus2 severe acute infection", "infection caused by 2019new sarscov2", "2019novel sars cov 2 severe acute disease", "pneumonia due to 2019novel chinese coronavirus", "severe acute respiratory syndromerelated coronavirus2 infection", "disease due to 2019ncov", "disease due to 2019 severe acute respiratory syndromerelated corona virus2", "pneumonia due to 2019 new wuhan seafood market virus", "new corona virus 2019 severe acute infection", "2019novel severe acute respiratory syndrome corona virus 2 disease", "disease from wuhan corona virus 2019", "2019 new chinese corona virus acute severe illness", "chinese coronavirus illness", "covid19 virus acute respiratory syndrome", "2019 corona virus severe acute disease", "infection caused by 2019 novel wuhancorona virus", "syndrome from 2019 chinese coronavirus", "wuhan virus 2019 acute respiratory disease", "2019 novel wuhancorona virus acute respiratory syndrome", "illness due to wuhan pneumonia virus", "2019novel wuhan pneumonia virus acute disorder", "2019novel wuhancorona virus severe respiratory infection", "2019novel wuhan pneumonia virus respiratory disease", "infection due to 2019novel severe acute respiratory syndrome coronavirus2", "2019 wuhan seafood market pneumonia virus", "illness caused by 2019 novel wuhan pneumonia virus", "wuhan seafood market virus 2019 acute syndrome", "infection caused by 2019 severe acute respiratory syndromerelated coronavirus 2", "2019 sarscov2 severe illness", "sars corona virus2 2019 severe infection", "2019 novel seafood market virus severe infection", "sars coronavirus 2 acute severe respiratory syndrome", "2019 new corona virus acute disease", "infection from 2019 new sars coronavirus 2", "2019new wuhan seafood market virus acute illness", "illness from 2019 new sars coronavirus 2", "2019new sars corona virus2 severe respiratory disorder", "syndrome caused by 2019 novel chinese coronavirus", "sarscov2 acute respiratory disease", "2019 sars coronavirus 2 acute severe respiratory disease", "syndrome due to 2019 coronavirus", "2019 new sars coronavirus2 acute disease", "2019 novel sars corona virus 2 acute severe respiratory disease", "pneumonia due to 2019 novel severe acute respiratory syndromerelated coronavirus 2", "2019 new severe acute respiratory syndrome corona virus2 disease", "infection caused by sars coronavirus2 2019", "2019novel seafood market virus acute respiratory infection", "2019 coronavirus acute severe syndrome", "severe acute respiratory syndromerelated coronavirus2 pneumonia", "2019new wuhan coronavirus severe acute syndrome", "2019 new corona virus severe respiratory illness", "illness caused by 2019 novel severe acute respiratory syndromerelated corona virus 2", "2019 wuhancoronavirus acute pneumonia", "illness due to 2019 new wuhan seafood market virus", "pneumonia from 2019 severe acute respiratory syndromerelated corona virus2", "2019new seafood market coronavirus acute severe infection", "2019novel wuhancoronavirus respiratory syndrome", "2019 new seafood market virus severe acute respiratory illness", "disorder caused by 2019 novel seafood market coronavirus", "syndrome caused by sarscov2", "illness due to 2019 severe acute respiratory syndrome corona virus 2", "2019novel sars cov 2 infection", "covid19 virus severe illness", "illness due to 2019novel severe acute respiratory syndrome coronavirus 2", "2019 new sars corona virus2 severe disease", "2019novel wuhan pneumonia virus severe pneumonia", "2019new sars coronavirus2 acute illness", "2019 new wuhancoronavirus acute respiratory illness", "disease caused by 2019 novel severe acute respiratory syndrome corona virus 2", "chinese coronavirus severe acute illness", "2019novel wuhan corona virus severe acute respiratory disorder", "seafood market coronavirus syndrome", "2019 new seafood market coronavirus acute severe disorder", "seafood market virus 2019 severe disorder", "2019 novel corona virus acute severe respiratory illness", "2019 novel chinese coronavirus severe respiratory syndrome", "chinese coronavirus acute respiratory syndrome", "2019 human coronavirus infection", "disorder from 2019 novel sars corona virus2", "illness due to wuhancorona virus 2019", "sars corona virus 2 severe acute respiratory disorder", "disease caused by 2019 severe acute respiratory syndrome coronavirus 2", "syndrome due to novel corona virus", "infection due to 2019 novel sars cov 2", "illness caused by wuhan seafood market pneumonia virus 2019", "disorder caused by 2019 severe acute respiratory syndromerelated coronavirus 2", "2019 novel seafood market corona virus acute respiratory syndrome", "2019 ncov acute respiratory syndrome", "sars corona virus 2 2019 acute severe respiratory disease", "pneumonia due to severe acute respiratory syndromerelated coronavirus 2", "wuhan pneumonia virus acute severe respiratory infection", "2019 new wuhan pneumonia virus severe disorder", "disorder from 2019new sars corona virus2", "wuhan coronavirus syndrome", "wuhancorona virus respiratory syndrome", "disorder due to covid19 virus", "2019 novel wuhan corona virus severe disorder", "2019 novel wuhan seafood market virus acute severe infection", "2019novel wuhan seafood market pneumonia virus severe acute syndrome", "disorder due to 2019novel chinese coronavirus", "2019 novel severe acute respiratory syndromerelated coronavirus 2", "2019 coronavirus acute illness", "severe acute respiratory syndromerelated coronavirus 2 disease", "2019 novel sars corona virus2 severe respiratory disease", "2019new sars coronavirus2 disease", "human coronavirus 2019 acute severe infection", "2019new seafood market coronavirus acute severe respiratory syndrome", "disorder due to 2019novel seafood market corona virus", "2019novel chinese coronavirus severe respiratory syndrome", "sars corona virus2 severe acute infection", "sars coronavirus 2 2019 acute severe respiratory illness", "infection from 2019novel severe acute respiratory syndromerelated corona virus2", "2019 new sars corona virus 2 acute severe disorder", "wuhan corona virus 2019 respiratory illness", "infection caused by 2019 new coronavirus", "wuhan coronavirus 2019 disorder", "2019 chinese coronavirus respiratory disease", "wuhan seafood market pneumonia virus severe infection", "2019 new corona virus acute respiratory illness", "2019novel wuhancoronavirus acute severe syndrome", "sars coronavirus2 2019 severe respiratory disorder", "disease due to 2019new wuhancoronavirus", "2019novel wuhan virus severe acute pneumonia", "2019 wuhan corona virus severe syndrome", "disorder caused by 2019 novel sars coronavirus 2", "seafood market coronavirus 2019 acute severe infection", "illness due to 2019 new sars cov 2", "2019 new seafood market virus acute pneumonia", "2019 novel seafood market corona virus severe disorder", "2019 new chinese coronavirus", "illness due to 2019novel sarscov2", "chinese coronavirus severe respiratory disease", "2019 new chinese corona virus acute severe respiratory syndrome", "infection from 2019 wuhan coronavirus", "2019 novel severe acute respiratory syndrome corona virus2 disease", "pneumonia from 2019new severe acute respiratory syndromerelated corona virus 2", "2019new chinese coronavirus acute severe respiratory syndrome", "2019 coronavirus acute severe respiratory disease", "2019 novel chinese coronavirus acute pneumonia", "2019 novel sarscov2 severe acute syndrome", "pneumonia caused by 2019 novel coronavirus", "pneumonia caused by 2019 wuhan virus", "syndrome due to 2019novel wuhan pneumonia virus", "syndrome due to 2019 new wuhan virus", "wuhan seafood market virus acute infection", "2019new seafood market corona virus severe acute respiratory infection", "syndrome from 2019 novel sars corona virus2", "disorder due to severe acute respiratory syndromerelated coronavirus2 2019", "2019 new wuhan coronavirus severe respiratory disease", "2019new sars coronavirus2 asevere acute respiratory disease", "disease due to 2019 severe acute respiratory syndromerelated coronavirus2", "2019novel wuhan seafood market pneumonia virus syndrome", "2019 novel coronavirus respiratory disease", "2019novel sars cov 2 severe respiratory infection", "infection due to severe acute respiratory syndrome corona virus 2", "2019 new severe acute respiratory syndrome coronavirus 2 disease", "2019novel seafood market virus severe respiratory syndrome", "seafood market virus respiratory syndrome", "new coronavirus syndrome", "illness caused by severe acute respiratory syndrome corona virus2 2019", "2019novel sars coronavirus 2 acute respiratory disorder", "disorder due to wuhancoronavirus 2019", "2019 new wuhancorona virus acute respiratory illness", "2019novel sars coronavirus2 severe acute disease", "2019novel seafood market corona virus severe acute respiratory infection", "2019new wuhancorona virus severe syndrome", "2019 ncov acute severe disorder", "sars coronavirus 2 severe acute respiratory disorder", "disease from 2019new wuhancorona virus", "2019novel wuhan coronavirus acute illness", "syndrome from 2019 novel corona virus", "2019 novel sars corona virus2 acute pneumonia", "2019new sarscov2 respiratory illness", "2019 wuhancorona virus severe acute infection", "2019 new corona virus respiratory disorder", "2019 wuhan virus infection", "2019 novel wuhan corona virus acute disease", "human corona virus 2019 severe acute respiratory infection", "infection caused by 2019 sars coronavirus 2", "2019new sars coronavirus 2 acute severe respiratory illness", "2019 sars coronavirus 2 severe acute infection", "chinese corona virus 2019 acute severe respiratory disease", "2019 coronavirus infection", "wuhan coronavirus severe acute infection", "2019 wuhan seafood market virus severe acute syndrome", "syndrome from chinese coronavirus", "seafood market virus 2019 severe illness", "hcov 19 severe respiratory disease", "wuhan corona virus 2019 severe acute infection", "disorder caused by 2019 new wuhancorona virus", "illness caused by 2019novel wuhan seafood market pneumonia virus", "illness from 2019 new sars corona virus 2", "pneumonia from seafood market coronavirus", "seafood market corona virus 2019 asevere acute respiratory disease", "2019 novel wuhancoronavirus acute illness", "2019 wuhan seafood market virus severe pneumonia", "2019 new sars coronavirus 2 severe acute disease", "disease due to 2019 new sars corona virus2", "disease from 2019 new wuhan pneumonia virus", "disorder due to 2019 chinese corona virus", "2019novel wuhancoronavirus acute severe respiratory disorder", "wuhan virus severe syndrome", "hubei asevere acute respiratory disease", "disorder from sars coronavirus2 2019", "2019 new sars corona virus2 severe acute disorder", "2019 novel wuhancoronavirus acute severe disorder", "2019 novel wuhan pneumonia virus acute severe respiratory infection", "syndrome from wuhan corona virus", "novel coronavirus 2019 acute severe respiratory disorder", "syndrome caused by 2019 new wuhancoronavirus", "syndrome due to 2019 chinese corona virus", "2019 new wuhan pneumonia virus acute severe illness", "2019 novel wuhan coronavirus severe acute disorder", "new coronavirus acute severe illness", "2019new wuhan coronavirus acute severe pneumonia", "seafood market coronavirus severe respiratory illness", "2019new sars coronavirus2 severe acute respiratory disorder", "wuhan pneumonia virus severe infection", "2019 seafood market corona virus respiratory infection", "2019novel sars corona virus 2 respiratory disease", "severe acute respiratory syndrome coronavirus2 2019", "2019 novel chinese corona virus acute severe infection", "2019 novel corona virus respiratory syndrome", "wuhan virus illness", "2019 human coronavirus disorder", "2019 novel corona virus severe acute disorder", "2019 novel sars coronavirus 2 severe acute respiratory disorder", "sars cov 2 severe disease", "2019 wuhan pneumonia virus severe disorder", "covid19 virus respiratory illness", "2019 novel sars coronavirus2 severe respiratory illness", "2019 new wuhancoronavirus acute severe respiratory syndrome", "2019new seafood market coronavirus acute severe pneumonia", "wuhancorona virus 2019 acute severe respiratory disease", "2019 new wuhan seafood market pneumonia virus severe infection", "sars coronavirus 2 2019 acute severe respiratory syndrome", "sars corona virus2 2019 severe acute infection", "2019novel seafood market corona virus acute respiratory syndrome", "2019 novel wuhancorona virus severe acute illness", "seafood market severe acute disorder", "2019 novel seafood market coronavirus acute respiratory disorder", "2019 wuhancorona virus severe illness", "2019 novel wuhan corona virus severe illness", "2019 new chinese coronavirus acute respiratory infection", "2019 new severe acute respiratory syndromerelated coronavirus2 infection", "2019 corona virus acute respiratory infection", "sars corona virus 2 syndrome", "2019novel wuhan seafood market virus severe acute pneumonia", "disease from sars coronavirus 2", "disease from 2019 novel chinese coronavirus", "wuhancorona virus severe acute pneumonia", "chinese corona virus 2019 severe illness", "2019novel wuhan virus acute severe illness", "2019 novel chinese coronavirus severe disease", "2019new sarscov2 acute severe respiratory disorder", "2019novel wuhancorona virus severe acute disorder", "wuhan seafood market virus 2019 severe acute respiratory disorder", "wuhan virus 2019 acute infection", "sarscov2 2019 severe pneumonia", "illness due to 2019novel wuhan seafood market virus", "pneumonia due to 2019 severe acute respiratory syndromerelated coronavirus2", "novel corona virus 2019 respiratory syndrome", "disease from severe acute respiratory syndromerelated coronavirus2 2019", "2019 new wuhan seafood market pneumonia virus severe respiratory illness", "syndrome caused by 2019new wuhancorona virus", "syndrome caused by 2019 new chinese corona virus", "2019 novel severe acute respiratory syndromerelated coronavirus 2 disease", "sarscov2 2019 respiratory syndrome", "2019novel sars corona virus 2 acute severe respiratory disorder", "2019 new sars corona virus 2 severe respiratory disorder", "syndrome from seafood market virus 2019", "syndrome caused by wuhan corona virus", "severe acute respiratory syndromerelated corona virus 2", "infection due to 2019 novel severe acute respiratory syndrome corona virus 2", "2019 novel sars cov 2 acute respiratory illness", "novel corona virus severe acute pneumonia", "syndrome due to 2019 new wuhan pneumonia virus", "pneumonia from 2019 new seafood market corona virus", "disease from 2019 new severe acute respiratory syndrome coronavirus 2", "disorder caused by 2019new severe acute respiratory syndromerelated corona virus 2", "sars cov 2 2019 acute severe illness", "2019new wuhancorona virus acute syndrome", "2019new wuhan pneumonia virus acute severe infection", "disorder caused by 2019new wuhan pneumonia virus", "2019 ncov respiratory syndrome", "human corona virus 2019 acute respiratory disorder", "disorder from 2019 wuhan pneumonia virus", "new corona virus respiratory infection", "pneumonia from 2019 novel sarscov2", "syndrome due to wuhan pneumonia virus", "pneumonia from severe acute respiratory syndrome coronavirus2", "illness from seafood market corona virus 2019", "2019 sars cov 2 severe acute infection", "2019 seafood market virus acute severe disease", "infection from human corona virus 2019", "2019 sars coronavirus2 acute severe respiratory illness", "syndrome due to seafood market corona virus", "infection caused by 2019 novel wuhan coronavirus", "sars corona virus2 severe disease"]"""

In [6]:
terms = json.loads(term_json)

In [7]:
# helper to convert char index to token index
# https://stackoverflow.com/questions/55109468/spacy-get-token-from-character-index
def get_token_for_char(doc, char_idx):
    for i, token in enumerate(doc):
        #print (i,token, char_idx, token.idx)
        if char_idx > token.idx:
            continue
        if char_idx == token.idx:
            #return (i, token)
            return i 
        if char_idx < token.idx:
        #    return (i, doc[i - 1])
            return i
    return i+1

In [8]:
# Source example: https://spacy.io/usage/rule-based-matching#phrasematcher
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp.make_doc(text) for text in tqdm(terms)]
matcher.add("CoronaPhrase", None, *patterns)

100%|██████████| 13135/13135 [00:00<00:00, 21133.00it/s]


In [9]:
doc = nlp("""Following an outbreak of pneumonia without a clear cause in the city of Wuhan in China, a novel strain of coronavirus (2019 nCoV) was detected in December 2019.

Coronaviruses were identified in the mid-1960s and are known to infect humans and a variety of animals (including birds and mammals). Since 2002, two coronaviruses infecting animals have evolved and caused outbreaks in humans: SARS CoV (Severe Acute Respiratory Syndrome) identified in southern China in 2003, and and MERS CoV (Middle East Respiratory Syndrome), identified in Saudi Arabia in 2012. Together, they have caused more than 1600 deaths.""")

matches = matcher(doc)
spans = []
for match_id, start, end in matches:
    span = doc[start:end]
    spans.append(span)
    
spans = filter_spans(spans)
ent_spans = []
for span in spans:
    ent_spans.append(Span(doc, span[0].i, span[-1].i+1, label="CoronaPhrase"))
    
doc.ents = filter_spans(list(doc.ents) + ent_spans)

In [10]:
displacy.render(doc, jupyter = True, style = "ent")

In [11]:
for ent in doc.ents:
    print('"%s"' % ent.text, ent.label_, " @ ", ent.start_char, "-", ent.end_char)

"outbreak" ENTITY  @  13 - 21
"pneumonia" ENTITY  @  25 - 34
"city" ENTITY  @  64 - 68
"Wuhan" ENTITY  @  72 - 77
"China" ENTITY  @  81 - 86
"strain" ENTITY  @  96 - 102
"coronavirus" ENTITY  @  106 - 117
"2019 nCoV" CoronaPhrase  @  119 - 128
"detected" ENTITY  @  134 - 142
"Coronaviruses" ENTITY  @  162 - 175
"identified" ENTITY  @  181 - 191
"mid-1960s" ENTITY  @  199 - 208
"infect" ENTITY  @  226 - 232
"humans" ENTITY  @  233 - 239
"animals" ENTITY  @  257 - 264
"birds" ENTITY  @  276 - 281
"mammals" ENTITY  @  286 - 293
"coronaviruses" ENTITY  @  312 - 325
"infecting animals" ENTITY  @  326 - 343
"outbreaks" ENTITY  @  368 - 377
"humans" ENTITY  @  381 - 387
"SARS" ENTITY  @  389 - 393
"CoV" ENTITY  @  394 - 397
"Severe" ENTITY  @  399 - 405
"Acute Respiratory Syndrome" ENTITY  @  406 - 432
"southern China" ENTITY  @  448 - 462
"MERS CoV" ENTITY  @  480 - 488
"Middle East Respiratory Syndrome" ENTITY  @  490 - 522
"Saudi Arabia" ENTITY  @  539 - 551
"deaths" ENTITY  @  603 - 609
